# Imaging & Analysis 



***





### Synthesis Imaging 

Chapter overview

This chapter documents CASA\'s refactored imager. These features are visible to the user via the **tclean** task.

The first five sections give an algorithm-centric view of the imager framework and are meant to convey the overall iterative reconstruction framework and how various algorithms and usage options fit into it. The other sections are more user-centric and focus on what one would need to know about specific imaging goals such as wideband imaging, mosaicking or details about spectral-cube definitions, etc. There is some overlap in content between sections, but this is meant to address the needs of readers who want to understand how the system works as well as those who want to learn how to approach their specific use case.

<div class="alert alert-info">
**NOTE**: The older **clean** task also supports most of the options described here, with a similar internal structure. However, clean will be deprecated in the near future, so we strongly encourage to use **tclean** instead. All major functionality from clean is now present in tclean via a modified interface, along with additional algorithmic options.
</div>



***





#### Introduction 

Introduction to Synthesis Imaging: Measurement Equation, Iterative $\chi^2$ minimization, Major and Minor cycles, Algorithm Options, etc.

This section describes the framework used by CASA for interferometric image reconstruction



##### Concept :

Image reconstruction in radio interferometry is the process of solving the linear system of equations $\vec{V} = [A] \vec{I}$, where $\vec{V}$ represents visibilities calibrated for direction independent effects, $\vec{I}$ is a list of parameters that model the sky brightness distribution (for example, a image of pixels) and $[A]$ is the measurement operator that encodes the process of how visibilities are generated when a telescope observes a sky brightness $\vec{I}$.  $[A]$ is generally given by $[S_{dd}][F]$ where $[F]$ represents a 2D Fourier transform, and $[S_{dd}]$ represents a 2D spatial frequency sampling function that can include direction-dependent instrumental effects. For a practical interferometer with a finite number of array elements, $[A]$ is non-invertible because of unsampled regions of the $uv$ plane. Therefore, this system of equations must be solved iteratively, applying constraints via various choices of image parameterizations and instrumental models.

 



##### Implementation ( major and minor cycles ):

Image reconstruction in CASA comprises an outer loop of *major cycles* and an inner loop of *minor cycles*. The major cycle implements transforms between the data and image spaces and the minor cycle operates purely in the image domain. Together, they implement an iterative weighted $\chi^2$ minimization process that solves the measurement equation.

 

![26ad14d4f63ff633dbd5d9e92d40a5059ab46a67](media/26ad14d4f63ff633dbd5d9e92d40a5059ab46a67.png){.image-inline width="635" height="347"}

>Iterative Image Reconstruction - Major and Minor Cycles
  

 

The data to image transform is called the *imaging* step in which a pseudo inverse of $[S_{dd}][F]$ is computed and applied to the visibilities. Operationally, weighted visibilities are convolutionally resampled onto a grid of spatial-frequency cells, inverse Fourier transformed, and normalized. This step is equivalent to calculating the normal equations as part of a least squares solution. The image to data transform is called the *prediction* step and it evaluates the measurement equation to convert a model of the sky brightness into a list of model visibilities that are later subtracted from the data to form residual visibilities. For both transforms, direction dependent instrumental effects can be accounted for via carefully constructed convolution functions.

Iterations begin with an initial guess for the image model.  Each major cycle consists of the prediction of model visibilities, the calculation of residual visibilities and the construction of a residual image. This residual image contains the effect of incomplete sampling of the spatial-frequency plane but is otherwise normalized to the correct sky flux units. In its simplest form, it can be written as a convolution of the true sky image with a point spread function. The job of the minor cycle is to iteratively build up a model of the true sky by separating it from the point spread function. This step is also called *deconvolution* and is equivalent to the process of solving the normal equations as part of a least squares solution. Different reconstruction algorithms can operate as minor cycle iterations, allowing for flexibility in (for example) how the sky brightness is parameterized. The imaging step can be approximate in that several direction dependent effects, especially baseline, frequency or time-dependent ones can sometimes  ignored, minor cycles can be approximate in that they use only PSF patches and do not try to be accurate over the entire image, but the prediction step of the major cycle must be as accurate as possible such that model components are converted to visibilities by including all possible instrumental effects.

<div class="alert alert-info">
**Basic Sequence of Imaging Logic:**

Data : Calibrated visibilities, data weights, UV sampling function
Input : Algorithm and iteration controls (stopping threshold, loop gain,...)
Output : Model Image, Restored Image, Residual Image,...

Initialize the model image
Compute the point spread function
Compute the initial residual image
While ( not reached global stopping criterion )             /* Major Cycle */
{
    While ( not reached minor-cycle stopping criterion )    /* Minor Cycle */
    {
        Find the parameters of a new flux component
        Update the model and residual images
    }
    Use current model image to predict model visibilities
    Calculate residual visibilities (data - model)
    Compute a new residual image from residual visibilities
}
Convolve the final model image with the fitted beam and add to the residual image
</div>




##### Algorithmic Options :

Within the CASA implementation, numerous choices are provided to enable the user to fine-tune the details of their image reconstruction. Images can be constructed as spectral cubes with multiple frequency channels or single-plane wideband continuum images. One or more sub images may be defined to cover a wide field of view without incurring the computational expense of very large images. The iterative framework described above is based on the Cotton-Schwab Clean algorithm [\[3\]](#Bibliography), but variants like Hogbom Clean [\[1\]](#Bibliography) and Clark Clean [\[2\]](#Bibliography) are available as subsets of this framework. The major cycle allows controls over different data weighting schemes [\[10\]](#Bibliography) and convolution functions that account for wide-field direction-dependent effects during imaging and prediction \[[\[6\]](#Bibliography), [\[7\]](#Bibliography) , [\[8\]](#Bibliography)\]. Deconvolution options include the use of point source vs multi-scale image models [\[4\]](#Bibliography) , narrow-band or wide-band models [\[5\]](#Bibliography), controls on iteration step size and stopping criteria, and external constraints such as interactive and non-interactive image masks. Mosaics may be made with data from multiple pointings, either with each pointing imaged and deconvolved separately before being combined in a final step, or via a joint imaging and deconvolution [\[9\]](#Bibliography). Options to combine single dish and interferometer data during imaging also exist. More details about these algorithms can be obtained from \[[\[10\]](#Bibliography), [\[11\]](#Bibliography), [\[12\]](#Bibliography), [\[13\]](#Bibliography)\] 

 

 



##### References :

 



##### Bibliography

1. J.\ A.\ Hogbom\ 1974\ (
2. B.\ G.\ Clark\ 1980\ (
3. F.\ R.\ Schwab,\ 1984\ (
4. T.\ J.\ Cornwell,\ 2008\ (
5. U.Rau,\ 2011\ (
6. T.\ J.\ Cornwell,\ 2008\ (
7. S.\ Bhatnagar,\ 2008\ (
8. S.Bhatnagar,\ 2013\ (
9. T.\ J.\ Cornwell,\ 1988\ (
^\ ^10.\ Briggs\ D.S.\ 1999\ ([Astron.\ Soc.\ Pac.\ Conf.\ Ser.](http://www.aspbooks.org/publications/180/127.pdf))\ [↩](#ref-cit10 "Jump back to citation 10 in the text.")^\ ^11.\ Cornwell,\ T.J\ 1999\ ([Astron.\ Soc.\ Pac.\ Conf.\ Ser.](http://www.aspbooks.org/publications/180/151.pdf))\ [↩](#ref-cit11 "Jump back to citation 11 in the text.")^\ ^12.\ Cornwell,\ T.J.,\ \"The\ Generic\ Interferometer:\ II\ Image\ Solvers\'\',\ Aips++\ note\ 184.\ Aug\ 1995\ [↩](#ref-cit12 "Jump back to citation 12 in the text.")^\ ^13.\ U.Rau,\ 2009\ ([IEEE](http://ieeexplore.ieee.org/document/5109712/))\ [↩](#ref-cit13 "Jump back to citation 13 in the text.")^



***





#### Types of images 

Ways to set up images (Cube vs MFS, single field, outliers, facets, Stokes planes ) and select data

The visibility data can be selected in many ways and imaged separately (e.g. one spectral window, one field, one channel). Data selection can also be done in the image-domain where the same data are used to create multiple image planes or multiple images (e.g. Stokes I,Q,U,V, or Taylor-polynomial coefficients or multiple-facets or outlier fields).

Parameters for data selection and image definition together define the following options.

    Data Selection                                             Imaging Definition
  ------------------- ----------------------------------------------------------------------------------------------------
     Spectral Axis     Cube (multiple channels)  or  MFS (single wideband channel) or MT-MFS (multi-term wideband images)
   Polarization axis                      Stokes Planes ( I, IV, IQUV, pseudoI, RR, LL, XX, YY, etc )
    Sky Coordinates                   Image shape, cell size, phasecenter, with or without outlier fields
    Data Selection        One pointing  vs  multiple pointings for a mosaic, data from multiple MeasurementSets, etc.

For the most part, the above axes are independent of each other and logical (and useful) combinations of them are allowed. For example, spectral cubes or wideband multi-term images can have outlier fields and/or mosaics. An example of a prohibited combination is the use of facets along with mosaics or a-projection as their algorithmic requirements contradict each other.

 



##### Types of Images

This section illustrates the mapping from visibility data to gridded visibilities and then to the image domain for different image shape and type options.

 

##### Spectral Cubes :

During gridding, N Data channels are binned onto M image channels using several optional interpolation schemes and doppler corrections to transform the data into the LSRK reference frame. When data from multiple channels are mapped to a single image channel, multi-frequency-synthesis gridding is performed within each image channel. More details are explained on the [Spectral Line Imaging](imaging.ipynb#spectral-line-imaging) page. As can be seen from the diagram, parallelization for cube imaging can be naturally done by partitioning data and image planes by frequency for both major and minor cycles.

 

![7712e396f0309a7d9309a77cfa8fc6fc2f8a0be9](media/7712e396f0309a7d9309a77cfa8fc6fc2f8a0be9.png){.image-inline width="460" height="257"}

 

##### Continuum Images

Wideband imaging involves mapping data from a wide range of frequency channels onto a single image channel.

###### Multi Frequency Synthesis (MFS) - Single Wideband Image

Data from all selected data channels are mapped to a single broadband uv-grid using appropriate uvw coordinates, and then imaged. This is accessed via the \" *specmode=\'mfs\'* \" option in the **tclean** task. Since there is only one uv grid and image, parallelization for continuum imagng is done only for the major cycle via data partitioning.

![51885b785409ea1448f61c399eb82e53f0a54729](media/51885b785409ea1448f61c399eb82e53f0a54729.png){.image-inline width="429" height="216"}

 

###### Multi-Term Multi Frequency Synthesis (MTMFS) - Taylor Coefficient Images

An improvement to standard MFS that accounts for changes in spectral index as a function of sky position is available that uses Taylor weighted averages of data from all frequencies accumulated onto NTerms uv-grids before imaging. These Taylor-weighted residual images form the input for the minor cycle of the Multi-Term MFS deconvolution algorithm which performs a linear least squares fit (see [Deconvolution Algorithms](imaging.ipynb#deconvolution-algorithms) section for more information) during deconvolution to obtain Taylor Coefficients per component (to represent sky spectra as polynomials in $I$ vs $\nu$). This option is accessed via \" *specmode=\'mfs\'* and *deconvolver*=\'mtmfs\', *nterms=2.* \" For the same data size as standard MFS (*nterms=1*), Multi-Term MFS will have $N_t$ times the gridding cost and number of images stored in memory.  Parallelization is again done only for the major cycle via data partitioning.

 ![f068a79a636e070fea341be6847ccefc61b1c6d5](media/f068a79a636e070fea341be6847ccefc61b1c6d5.png){.image-inline width="447" height="285"}

 

##### Polarization Planes

Data in the correlation basis are gridded onto separate planes per correlation, imaged, and then transformed into the Stokes basis. A special case for single plane Stokes I is implemented where data from both parallel hands are gridded onto a single uv-grid (to save memory). The point spread function is always taken from the Stokes I gridded weights. Images can be made for all Stokes parameters and correlation pairs (or all combinations possible with the selected data). This is an image-partitioning, where the same data are used to construct the different imaging products. Currently, if any correlation is flagged, all correlations for that datum are considered as flagged. An exception is the \'*pseudoI*\' option which allows Stokes I images to include data for which either of the parallel hand data are unflagged.

 

![2505ddafc4936705143e6210e997ffee48acf231](media/2505ddafc4936705143e6210e997ffee48acf231.png){.image-inline width="515" height="271"}  

##### Multiple Fields

A very large field of view can sometimes be imaged as a main field plus a series of (typically) smaller outlier fields. Imaging of fields with relatively few bright outlier sources can benefit from the overal reduction in image size that this option provides.  Instead of gridding the visibilities data onto a giant uv-grid, they are gridded onto multiple smaller images. Each sub-image is then deconvolved via separate minor cycles and their model images combined to predict model visibiliitles to subtract from the data in the next major cycle. The user must specify different phase reference centers for each image field.

Different image shapes and gridding and deconvolution algorithms can be chosen for the different outlier fields. For example, one could apply single-plane wideband imaging on the main field, but employ multi-term MFS for an outlier field to account for artificial spectral index due to the wideband primary beam at its location. One can also combine MFS and Cube shapes for different outlier fields, or choose to run Multi-Scale CLEAN on the main field and Hogbom CLEAN on a bright compact outlier.    

Overlapping fields are supported when possible (i.e. when the image types are similar enough across outliers) by always picking the \"last\" instance of that source in the list of outlier images in the order specified by the user. This convention implies that sources in the overlapping area are blanked in the \"earlier\" model images, such that those sources are not subtracted during the major cycles that clean those images.

 

![5c981ca63d45b330b41ebcd4e67b4607d47d47a5](media/5c981ca63d45b330b41ebcd4e67b4607d47d47a5.png){.image-inline width="479" height="249"}

 

##### Multiple Facets

Faceted imaging is one way of handling the w-term effect. A list of facet-centers is used to grid the data separately onto multiple adjacent sub-images. The sub images are typically simply subsets of a single large image so that the deconvolution can be performed as a joint image and a single model image is formed. The PSF to be used for deconvolution is picked from the first facet. The list of phase reference centers for all facets is automatically generated from user input of the number of facets (per side) that the image is to be divided into.

 

![08fa4167833923aa5e120ade2d66c24de84c3a4a](media/08fa4167833923aa5e120ade2d66c24de84c3a4a.png){.image-inline width="513" height="272"}

 

 



##### Mosaics

Data from multiple pointings can be combined to form a single large image. The combination can be done either before/during imaging or after deconvolution and reconstruction.

##### Stitched Mosaic

Data from multiple pointings are imaged and deconvolved separately, with the final output images being combined using a primary beam model as a weight. This is achieved by running the imaging task (**tclean**) separately per pointing, and combining them later on using the tool **im.linearmosaic**().

 ![64ef9fbb940c0eefc34e96c18d0ac726f56ee982](media/64ef9fbb940c0eefc34e96c18d0ac726f56ee982.png){.image-inline width="467" height="226"}

 

##### Joint Mosaic

Data taken with multiple pointings (and/or phase-reference centres) can be combined during imaging by selecting data from all fields together (multiple field-ids), and specifying only one output image name and one phase-reference center. If mosaic mode is enabled (*gridder=\'mosaic\'* or *\'awproject\'*) attention is paid to the pointing centers of each data-fieldID during gridding. Primary-beam models are internally used during gridding (to effectively weight the images that each pointing would produce during a combination) and one single image is passed on to the deconvolution modules. 

 

![0886eddf2dfd68343993dca5b5c02affe00ccc54](media/0886eddf2dfd68343993dca5b5c02affe00ccc54.png){.image-inline width="448" height="218"}

 

 



***





#### Imaging Algorithms 

Data weighting, gridding, fft and normalizations

Imaging is the process of converting a list of calibrated visiblities into a raw or \'dirty\' image. There are three stages to inteferometric image-formation: weighting, convolutional resampling, and a Fourier transform.

 



##### Data Weighting

During imaging, the visibilities can be weighted in different ways to alter the instrument\'s natural response function in ways that make the image-reconstruction tractable.

Data weighting during imaging allows for the improvement of the dynamic range and the ability to adjust the synthesized beam associated with the produced image. The weight given to each visibility sample can be adjusted to fit the desired output. There are several reasons to adjust the weighting, including improving sensitivity to extended sources or accounting for noise variation between samples. The user can adjust the weighting using **clean** and changing the *weighting* parameter with six options: \'natural\', \'uniform\', \'briggs\',  \'superuniform\', \'briggsabs\', and \'radial\'. Optionally, a UV taper can be applied, and various parameters can be set to further adjust the weight calculations.

 

###### Natural weighting

*Summary:* The natural weighting scheme gives equal weight to all samples. Since usually, lower spatial frequencies are sampled more often than the higher ones, the inner uv-plane will have a significantly higher density of samples and hence signal-to-noise than the outer uv-plane. The resulting \"density-upweighting\" of the inner uv-plane will produce the largest angular resolution and can sometimes result in undesirable structure in the PSF which reduces the accuracy of the minor cycle. However, at the location of a source, this method preserves the natural point-source sensitivity of the instrument.

For *weighting=\'natural\'*, visibilities are weighted only by the data weights, which are calculated during filling and calibration and should be equal to the inverse noise variance on that visibility. Imaging weight $w_i$ of sample $\dot\imath$ is given by:

$w_i = \omega_i = \frac{1}{{\sigma_i}^2}$

where the data weight $\omega_i$ is determined from $\sigma_i$, the rms noise on visibility $\dot\imath$. When data is gridded into the same uv-cell for imaging, the weights are summed, and thus a higher uv density results in higher imaging weights. No sub-parameters are linked to this mode choice. It is the default imaging weight mode, and it should produce "optimum" image with with the lowest noise (highest signal-to-noise ratio).

<div class="alert alert-info">
**NOTE**: This generally produces images with the poorest angular resolution, since the density of visibilities falls radially in the uv-plane.
</div>


###### Uniform weighting 

*Summary:* Uniform weighting gives equal weight to each measured spatial frequency irrespective of sample density. The resulting PSF has the narrowest possible main lobe (i.e. smallest possible angular resolution) and suppressed sidelobes across the entire image and is best suited for sources with high signal-to-noise ratios to minimize sidelobe contamination between sources. However, the peak sensitivity is significantly worse than optimal (typically \~20% worse for reasonably large number of antenna interferometers), since data points in densely sampled regions have been weighted down to make the weights uniform. Also, isolated measurements can get artifically high relative weights and this may introduce further artifacts into the PSF.

For *weighting=\'uniform\'*, the data weights are calculated as in \'natural\' weighting. The data is then gridded to a number of cells in the uv-plane, and after all data is gridded the uv-cells are re-weighted to have "uniform" imaging weights. This pumps up the influence on the image of data with low weights (they are multiplied up to be the same as for the highest weighted data), which sharpens resolution and reduces the sidelobe level in the field-of-view, but increases the rms image noise. No sub-parameters are linked to this mode choice.

For uniform weighting, we first grid the inverse variance $\omega_i$ for all selected data onto a grid with uv cell-size given by 2 ∕ FOV, where FOV is the specified field of view (defaults to the image field of view). This forms the gridded weights $W_k$. The weight of the $\dot\imath$-th sample is then:

$w_i = \frac{\omega_i}{W_k}$


###### Briggs weighting 

*Summary:* Briggs or Robust weighting [\[1\]](#Bibliography) creates a PSF that smoothly varies between natural and uniform weighting based on the signal-to-noise ratio of the measurements and a tunable parameter that defines a noise threshold. High signal-to-noise samples are weighted by sample density to optimize for PSF shape, and low signal-to-noise data are naturally weighted to optimize for sensitivity.

The *weighting=\'briggs\' *mode is an implementation of the flexible weighting scheme developed by Dan Briggs in his PhD thesis, which can be viewed [here](http://www.aoc.nrao.edu/dissertations/dbriggs/).

This choice brings up the sub-parameters:

```
weighting      =   'briggs'   #Weighting to apply to visibilities  
     robust    =        0.0   #Briggs robustness parameter  
     npixels   =          0   #number of pixels to determine uv-cell size 0=> field of view
```

The actual weighting scheme used is:

$w_i = \frac{\omega_i}{1 + W_k f^2}$

where

$w_i$ is the image weight for a given visibility point $i$;

$\omega_i$ is the visibility weight for a given visibility point $i$;

$W_k = \Sigma_{cell=k}\,\omega_{k}$ is the weight density of a given cell $k$ (with $\omega_{k}$ the weight of a uv point that falls in cell $k$). When using *npixels \> 0* then $\Sigma_{\omega_{k}}$ is over all weights that fall in cells in range *k ± npixels*

$f^2 = \frac{(5 \times 10^{-\text{R}})^2}{\frac{\Sigma_k W_k^2}{\Sigma_i \omega_i}}$;

R is the robust sub-parameter.

The key parameter is the *robust sub-*parameter, which sets R in the Briggs equations. The scaling of R is such that *robust=0* gives a good trade-off between resolution and sensitivity. The robust R takes value between -2.0 (close to uniform weighting) to 2.0 (close to natural).

Superuniform weighting can be combined with Briggs weighting using the *npixels *sub-parameter. This works as in 'superuniform' weighting.


###### Briggsabs weighting

*Summary:* Briggsabs is an experimental weighting scheme that is an adapted version of the Briggs weighting scheme, and is much more aggressive with respect to changes in *npixels*, the uv-cell size.

For *weighting=\'briggsabs\'*, a slightly different Briggs weighting is used, with:

$w_i = \frac{\omega_i}{W_k \text{R}^2 + 2\sigma_\text{i}^2}$

where R is the *robust* parameter and $\sigma_\text{i}$ is the *noise *parameter. In this case, R makes sense for −2.0 ≤ R ≤ 0.0 (R = 1.0 will give the same result as R = −1.0)

This choice brings up the sub-parameters:

```
weighting      = 'briggsabs'  #Weighting to apply to visibilities  
     robust    =      0.0     #Briggs robustness parameter  
     noise     =  '0.0Jy'     #noise parameter for briggs weighting when rmode='abs' 
     npixels   =        0     #number of pixels to determine uv-cell size 0=> field of view
```

 

<div class="alert alert-warning">
**WARNING:** Briggsabs weighting is experimental - use at own risk!
</div>

 

###### Superuniform weighting 

The *weighting=\'superuniform\' *mode is similar to the \'uniform\' weighting mode but there is now an additional *npixels *sub-parameter that specifies a change to the number of cells on a side (with respect to uniform weighting) to define a uv-plane patch for the weighting renormalization. If *npixels=0*, you get uniform weighting.

 

###### Radial weighting

The *weighting=\'radial\' *mode is a seldom-used option that increases the weight by the radius in the uv-plane, i.e.:

$w_i = \omega_i \times \sqrt{u_i^2 + v_i^2}$

Technically, this would be called an inverse uv-taper, since it depends on uv-coordinates and not on the data per-se. Its effect is to reduce the rms sidelobes for an east-west synthesis array. This option has limited utility.

 

###### Perchanweightdensity

When calculating weight density for Briggs style weighting in a cube, the *perchanweightdensity* parameter determines whether to calculate the weight density for each channel independently (the default, True) or a common weight density for all of the selected data. This parameter has no meaning for continuum (*specmode=\'mfs\'*) imaging but for cube imaging *perchanweightdensity=True* is a recommended  alternative option that provides more uniform sensitivity per channel for cubes, but with generally larger psfs than the *perchanweightdensity=False* option (which was also the behavior prior to CASA 5.5). When using *Briggs* style weight with *perchanweightdensity=True*, the imaging weight density calculations use only the weights of data that contribute specifically to that channel. On the other hand, when *perchanweightdensity=False*, the imaging weight density calculations sum all of the weights from all of the data channels selected whose (u,v) falls in a given uv cell on the weight density grid. Since the aggregated weights, in any given uv cell, will change depending on the number of channels included when imaging, the psf calculated for a given frequency channel will also necessarily change, resulting in variability in the psf for a given frequency channel when *perchanweightdensity=False*. In general, *perchanweightdensity=False* results in smaller psfs for the same value of robustness compared to *perchanweightdensity=True*, but the rms noise as a function of channel varies and increases toward the edge channels; *perchanweightdensity=True* provides more uniform sensitivity per channel for cubes. This may make it harder to find estimates of continuum when *perchanweightdensity=False*. If you intend to image a large cube in many smaller subcubes and subsequently concatenate, it is advisable to use *perchanweightdensity=True* to avoid surprisingly varying sensitivity and psfs across the concatenated cube.

<div class="alert alert-info">
**NOTE**: Setting *perchanweightdensity = True* only has effect when using *Briggs* (robust) or *uniform* weighting to make an image cube. It has no meaning for *natural* and *radial* weighting in data cubes, nor does it have any meaning for continuum (*specmode='mfs'*) imaging.
</div>


###### Mosweight

When doing Brigg\'s style weighting (including uniform) in **tclean**, the *mosweight* subparameter of the mosaic gridder determines whether to weight each field in a mosaic independently (*mosweight = True*), or to calculate the weight density  from the average uv distribution of all the fields combined (*mosweight = False*). The underlying issue with more uniform robust weighting is how the weight density maps onto the uv-grid, which can give high weight to areas of the uv-plane that are not actually more sensitive. The setting *mosweight = True* has long been known as potentially useful in cases where a mosaic has non-uniform sensitivity, but it was found that it is also very important for more uniform values of robust Briggs weighting in the presence of relatively poor uv-coverage. For example, snap-shot ALMA mosaics with *mosweight = False* typically show an increase in noise in the corners or in the areas furthest away from the phase-center. Therefore, as of CASA 5.4, the *mosweight* sub-parameter has been added to **tclean** with default value *mosweight = True*.

<div class="alert alert-warning">
**WARNING:** the default setting of *mosweight=True* under the mosaic gridder in **tclean** has the following disadvantages: (1) it may potentially cause memory issues for large VLA mosaics; (2) the major and minor axis of the synthesized beam may be ~10% larger than with mosweight=False. Please change to *mosweight=False* to get around these issues.
</div>


###### uvtaper

*Summary:* The effect of uvtaper this is that the clean beam becomes larger, and surface brightness sensitivity increases for extended emission.

uv-tapering applies a Gaussian taper on the weights of your UV data, in addition to the weighting scheme specified via the \'weighting\' parameter. It applies a multiplicative Gaussian taper to the spatial frequency grid, to weight down high spatial-frequency measurements relative to the rest. This means that higher spatial frequencies are weighted down relative to lower spatial frequencies, to suppress artifacts arising from poorely sampled regions near and beyond the maximum spatial frequency in the uv-plane. It is equivalent to smoothing the PSF obtained by other weighting schemes and can be specified either as a Gaussian in uv-space (eg. units of lambda or klambda) or as a Gaussian in the image domain (eg. angular units like arcsec). Because the natural PSF is smoothed out, this tunes the sensitivity of the instrument to scale sizes larger than the angular-resolution of the instrument by increasing the width of the main lobe. There are limits to how much uv-tapering is desirable, however, because the sensitiivty will decrease as more and more data is down-weighted.

<div class="alert alert-info">
**NOTE**: The on-sky FWHM in arcsec is roughly the *uvtaper* / 200 (klambda).
</div>

Examples: uv*taper=\[\'5klambda\'\]* circular taper FWHM=5 kilo-lambda, uv*taper=\[\'5klambda\',\'3klambda\',\'45.0deg\'\]*, *uvtaper=\[\'10arcsec\'\]* on-sky FWHM 10 arcseconds, *uvtaper=\[\'300.0\'\]* default units are lambda in aperture plane, uv*taper=\[\]*; no outer taper applied (default)

 

![6178646282cf25c3d316aa14c3d888e6608a49bd](media/6178646282cf25c3d316aa14c3d888e6608a49bd.png){.image-inline width="497" height="312"}




##### Gridding + FFT

Imaging weights and weighted visibilities are first resampled onto a regular uv-grid (convolutional resampling) using a prolate-spheroidal function as the gridding convolution function (GCF). The result is then Fourier-inverted and grid-corrected to remove the image-domain effect of the GCF. The PSF and residual image are then normalized by the sum-of-weights.

 

![29159db628f096f12291870d788317a84a86e15c](media/29159db628f096f12291870d788317a84a86e15c.png){.image-inline width="271" height="277"}

 

##### Direction-dependent corrections

Basic gridding methods use prolate spheroidals for gridding (*gridder=\'standard\'*) along with image-domain operations to correct for direction-dependent effects. More sophiticated, and computationally-intesitve methods (*gridder=\'wproject\',\'widefield\',\'mosaic\',\'awproject\'*) apply direction-dependent, time-variable and baseline-dependent corrections during gridding in the visibility-domain, by choosing/computing the appropriate gridding convolution kernel to use along with the imaging-weights.

The figure below shows examples of kernels used for the following gridding methods: Standard, W-Projection, and A-Projection.  Combinations of wide-field corrections are done by convolving these kernels together.  For example, AW-Projection will convolve W-kernels with baseline aperture functions and possibly include a prolate spheroidal as well for its anti-aliasing properties.   Mosaicing is implemented as a phase gradient across the gridding convolution kernel calculated at the uv-cell resolution dictated by the full mosaic image size.

In tclean, *gridder=\'mosaic\'* uses Airy disk or polynomial models to construct azimuthally symmetric beams per antenna that are transformed into baseline convolution functions and used for gridding.  *gridder=\'awproject\'* uses ray-traced models of antenna aperture illumination functions to construct GCFs per baseline and time (including azimuthal asymmetry, beam squint, and rotation with time).   More details are given in the [Wide Field Imaging](imaging.ipynb#wide-field-imaging-full-primary-beam) page.

 

![1c48e6447847e5f2e25d16da48fb67f74b5e0a70](media/1c48e6447847e5f2e25d16da48fb67f74b5e0a70.png){.image-inline width="668" height="353"}

 

Computing costs during gridding scale directly with the number of pixels needed to accurately describe each convolution kernel. The standard gridding kernel (prolate spheroid) typically has 3x3 pixels. W-Projection kernels can range from 5x5 to a few hundred pixels on a side.  A-Projection kernels typically range from 8x8 to 50x50 pixels.  When effects are combined by convolving together different kernels (for example A and W Projection), the kernel sizes increase accordingly. 

Memory (and one-time computing costs) also scale with the number of distinct kernels that one must operate with. For example, a large number of different W-Projection kernels, or an array whose antenna illumination patterns are different enough between antennas that they need to be treated separately. In the case of a heterogenous array, each baseline illumination function can be different.  Additionally, if any of these aperture illumination based kernels are rotationally asymmetric, they will need to be rotated (or recomputed at different parallactic angles) as a function of time.  

 



##### Normalization

After gridding and the FFT, images must be normalized (by the sum of weights, and optionally by some form of the primary beam weights) to ensure that the flux in the images represents sky-domain flux.

##### Sum-Of-Weights and Weight Images

The tclean task produces a number of output images used for normalization. The primary reason these are explicit images on disk (and not just internal temporary files in memory) is that for continuum paralellization, there is the need to accumulate numerators and denominators separately before the normalization step.  For the most part, end users can safely ignore the  output .weight,  .sumwt and .gridwt images.  However, their contents are documented here.

###### .sumwt 

   A single-pixel image containing the sum-of-weights (or, the peak of the PSF). For natural weighting, this is just the sum of the data weights.  For other weighting schemes it contains the effect of the weighting algorithm. For instance, uniform weighting will typically produce a smaller sum-of-weights than natural weighting.    An approximate theoretical sensitivity can be computed as  sqrt( 1/sumwt ). A more accurate calculation requires a different calculation (LINK to some docs from GM on this).   In tclean, facetted imaging will produce one value of sumwt per facet as the normalizations are to be done separately per facet.  Also, for cube imaging, .sumwt will contain one value per image channel and it can be used to visualize the relative weights across the spectrum (and therefore expected image rms). This theoretical sensitivity information is printed to the logger after the PSF generation stage.

###### .weight 

Projection gridders such as \'mosaic\' and \'awproject\' use baseline aperture illumination functions for gridding.  The quantity in the .weight image represents the square of the PB, accumulated over baselines, time and frequency.   For mosaics, it includes a combination across pointing as well (although as can be seen from the equations in the mosaicing section, this is not accurate when weights between pointings differ considerably).

###### .gridwt 

A series of temporary images for cube imaging that are stored within the parallel .workdirectory, and which accumulate binned natural weights before the calculation of imaging weights.  This is not used for normalization anywhere after the initial image weighting stage.

 

##### Normalization Steps

###### Standard Imaging

For gridders other than \'mosaic\' and \'awproject\', normalization of the image formed after gridding and the FFT is just the division by the sum of weights (read from the .sumwt image). This suffices to transform the image into units of sky brightness. This is the typical flat-noise normalization (see below).

###### Imaging with primary beams (and mosaics)

For *gridder=\'mosaic\'* and \'awproject\' that use baseline aperture illumination functions during gridding, the result is an additional instance of the PB in the images, which needs to be divided out.  Normalization involves three steps (a) division by the sum-of-weights (b) division by an average PB given by sqrt(weightimage) and (c) a scaling to move the peak of the PB = sqrt(weightimage) to 1.0.   This ensures that fluxes in the dirty image (and therefore those seen by the minor cycle) represent true sky fluxes in regions where the primary beam is at its peak value, or where the mosaic has a relatively constant flat sensitivity pattern.    The reverse operations of (b) and (c) are done before predicting a model image in the major cycle.   ( This description refers to flat-noise normalization, and corresponding changes are done for the other options ).

 

##### Types of normalization

There are multiple ways of normalizing the residual image before beginning minor cycle iterations. One is to divide out the primary beam before deconvolution and another is to divide out the primary beam from the deconvolved image. Both approaches are valid, so it is important to clarify the difference between the two. A third option is included for completeness.

For all options, the \'pblimit\' parameter controls regions in the image where PB-correction is actually computed. Regions below the pblimit cannot be normalized and are set to zero. For standard imaging, this refers only to the pb-corrected output image. For *gridder=\'mosaic\'* and *\'awproject\'* it applies to the residual, restored and pb-corrected images.  A small value (e.g. *pblimit=0.01*) can be used to increase the region of the sky actually imaged. For *gridder=\'standard\'*, there is no pb-based normalization during gridding and so the value of this parameter is ignored.The sign of the pblimit parameter is used for a different purpose. If positive, it defines a T/F pixel mask that is attached to the output residual and restored images.  If negative, this T/F pixel mask is not included.  Please note that this pixel mask is different from the deconvolution mask used to control the region where CLEAN based algorithms will search for source peaks.  In order to set a deconvolution mask based on pb level, please use the \'pbmask\' parameter.Based on the above, please note that certain values of pblimit to avoid, are 1, -1, and 0. When the pblimit is set to 1 the entire image is masked as the user is specifying that no normalization or deconvolution happens if the PB gain is lower than 1, which leads to the entire image being masked. Setting the pblimit to -1 also results in no deconvolution as mentioned in the case where pblimit is 1 but there is no masking of the image. Finally a pblimit of zero is not feasible but rather a small value such as 1e-6 is used instead to make a really large wide field image.

###### Flat-noise

The dirty image represents $I^{dirty} = I^{psf} \star \left( I^{PB} \cdot I^{sky} \right)$

Primary-beam correction is not done before the minor cycle deconvolution. The dirty image is the instrument\'s response to the product of the sky and the primary beam, and therefore the model image will represent the product of the sky brightness and the average primary beam. The noise in the image is related directly to the measurement noise due to the interferometer, and is the same all across the image. The minor cycle can give equal weight to all flux components that it finds. At the end of deconvolution, the primary beam must be divided out of the restored image. This form of normalization is useful when the primary beam is the dominant direction-dependent effect because the images going into the minor cycle satisfy a convolution equation. It is also more appropriate for single-pointing fields-of-view.

Imaging with the prolate spheroidal gridder will automatically give flat noise images.

###### Flat-sky

The dirty image represents $I^{dirty} = \frac{1}{I^{PB}} \cdot \left[I^{psf} \star \left( I^{PB} \cdot I^{sky} \right) \right]$

Approximate Primary-beam correction is done on the dirty image, before the minor cycle iterations. The amplitude of the flux components found during deconvolution will be free of the primary beam, and will represent the true sky. However, the image going into the minor cycle will not satisfy a convolution equation and the noise in the dirty image will be higher in regions where the primary-beam gain is low. Therefore, the minor cycle needs to account for this while searching for flux components (a signal-to-noise dependent CLEAN). This form of normalization is particularly useful for mosaic imaging where the sky brightness can extend across many pointings, or if there is an uneven distribution of weights across pointings. This is because joint mosaics are usually done for sources with spatial scales larger than the field-of-view of each antenna and which are not explicitly present in the measured data. In this situation, allowing the minor cycle to use flux components that span across beams of adjacent pointings is likely to provide a better constraint on the reconstruction of these unmeasured spatial frequencies, and produce smoother large-scale emission.

###### PB-square normalization

The dirty image represents $I^{dirty} = I^{PB} \cdot \left[ I^{psf} \star \left( I^{PB} \cdot I^{sky} \right) \right]$

This third option (not currenly available for use, but supported internally) is to not do any PB-based divisions after the gridding and FFT (using *gridder=\'mosaic\'* or \'awproject\', but to let the minor cycle proceed as is.  Advantages of this approach are the elimination of error-inducing divisions by the primary beam (especially in low gain regions and near PB cut-off edges).

 

 

 



##### Bibliography

1. 



***





#### Deconvolution Algorithms 

Minor cycle algorithms (Hogbom, Clark, Multi-Scale, Multi-Term)

Deconvolution refers to the process of reconstructing a model of the sky brightness distribution, given a dirty/residual image and the point-spread-function of the instrument. This process is called a deconvolution because under certain conditions, the dirty/residual image can be written as the result of a convolution of the true sky brightness and the PSF of the instrument. Deconvolution forms the minor cycle of iterative image reconstruction in CASA.

![4988e07e094ae12dd994cc53a49115f279a3ce9b](media/4988e07e094ae12dd994cc53a49115f279a3ce9b.png){.image-inline}

>The observed image (left) is the result of a convolution of the PSF (middle) and the true sky brightness distribution (right).
  

The image reconstruction framework is based on Cotton-Schwab major/minor cycles [\[1\]](#Bibliography). Within this system, the minor cycle algorithm operates largely in the image domain starting with a PSF and a residual image (i.e. the gradient of chi-square or the right hand side of the normal equations). The output is an incremental model image that defines the \'step\' taken during the chi-square minimization process. In the next major cycle, the contribution of this model image is subtracted out of the list of visibilities and the result is regridded and transformed to produce a new residual image. This approach allows for a practical trade-off between the efficiency of operating in the image domain (or simply with gridded visibilities) and the accuracy that comes from returning to the ungridded list of visibilities after every \'step\'. It also allows for minor cycle algorithms that have their own internal optimization schemes (i.e. they need not be strict chi-square minimizations) with their own control parameters. Note that any minor cycle algorithm that can operate on gridded visibilities can fit into this framework. The inputs to the minor cycle algorithm are the residual image, psf and perhaps a starting model. Outputs are a model image.

 



##### CLEAN Algorithm 

The CLEAN algorithm forms the basis for most deconvolution algorithms used in radio interferometry. The peak of the residual image gives the location and strength of a potential point source. The effect of the PSF is removed by subtracting a scaled PSF from the residual image at the location of each point source, and updating the model. Many such iterations of finding peaks and subtracting PSFs form the minor cycle.

There are several variants of the CLEAN algorithm. Some operate with a delta function sky model and others with a multi-scale sky model. In all cases, the the sky brightness is parameterized in a sparse basis such that in practice, the minor cycle algorithm needs only to search for the location and amplitude of peaks. This makes it efficient. For example, fields of compact sources are best represented by delta function locations and amplitudes. Extended emission is modeled as a linear combination of components of different scale sizes and transformed into a multi-scale basis where again, delta functions are all that are required to mark the location and amplitude of blobs of different sizes. Multi-term algorithms for wideband imaging model the sky brightness and its spectrum simultaneously, using coefficients of a Taylor polynomial as a sparse representation of a smooth spectrum. In this case, the location of each (multi-scale) component is chosen via a search and the values of the Taylor coefficients for that component are solved for via a direct linear least squares calculation.

 

##### Hogbom 

Hogbom CLEAN [\[2\]](#Bibliography) operates with a point-source model of the sky brightness distribution. The minor cycle searches for the location and amplitude of components and then subtracts a scaled and shifted version of the full PSF to update the residual image for each point source. This algorithm is efficient in that delta functions are optimal for fields of compact sources, but susceptible to errors due to inappropriate choices of imaging weights, especially if the PSF has high sidelobes. It uses the full PSF during its update step to ensure that the next residual is as accurate as possible, but this can get compute intensive.  

In its original form, the Hogbom algorithm operated just once in the image domain without new residuals being computed via a major cycle. In our CASA Imager implementation, it is treated as a minor cycle where one periodically does major cycles as well (to guard against minor cycle evolution that is not strictly constrained by the ungridded visibilities).

Since Hogbom CLEAN uses only delta functions, it is most appropriate for fields of isolated point sources. It will incur errors when imaging extended emission and this is typically seen as a mottled appearance of smooth structure and the presence of correlated residuals.

 

##### Clark 

Clark CLEAN [\[3\]](#Bibliography) also operates only in the image-domain, and uses a point-source model. There are two main differences from the Hogbom algorithm. The first is that it does its residual image updates using only a small patch of the PSF. This is an approximation that will result in a significant speed-up in the minor cycle, but could introduce errors in the image model if there are bright sources spread over a wide field-of-view where the flux estimate at a given location is affected by sidelobes from far-out sources. The second difference is designed to compensate for the above. The iterations are stopped when the brightest peak in the residual image is below the first sidelobe level of the brightest source in the initial residual image and the residual image is re-computed by subtracting the sources and their responses in the gridded Fourier domain (to eliminate aliasing errors). Image domain peak finding and approximate subtractions resume again. These two stages are iterated between until the chosen minor cycle exit criteria are satisfied (to trigger the next true major cycle that operates on ungridded visibilities).

Since Clark CLEAN also uses only delta function, it is similar in behavior to Hogbom. The main difference is that the minor cycle is expected to be much faster (for large images) because only a small fraction of the PSF is used for image-domain updates. Typically, errors due to such a truncation are controlled by transitioning to a uv-[[[[I2]{#MJXc-Node-3 .mjx-msubsup}+[Q2]{#MJXc-Node-7 .mjx-msubsup .MJXc-space2}+[U2]{#MJXc-Node-11 .mjx-msubsup .MJXc-space2}+[V2]{#MJXc-Node-15 .mjx-msubsup .MJXc-space2}]{#MJXc-Node-2 .mjx-mrow}]{#MJXc-Node-1 .mjx-math role="math"}]{#MathJax-Element-1-Frame .MJXc-processed tabindex="0" mathml="<math xmlns=\"http://www.w3.org/1998/Math/MathML\"><mi>u</mi><mi>v</mi></math>" style="font-size: 119%;"}subtraction or a major cycle when the peak residual reaches the level of the highest sidelobe for the strongest feature.

For polarization imaging, Clark searches for the peak in

$I^2 + Q^2 + U^2 + V^2$

$I^2 + Q^2 + U^2 + V^2$

 

##### Clarkstokes

In the \'*clarkstokes*\' algorithm, the Clark psf is used, but for polarization imaging the Stokes planes are cleaned sequentially for components instead of jointly as in \'*clark*\'. This means that this is the same as \'clark\' for Stokes I imaging only. This option can also be combined with *imagermode=\'csclean\'*.

 

##### Multi-Scale 

Cornwell-Holdaway Multi-Scale CLEAN (CH-MSCLEAN) [\[4\]](#Bibliography) is a scale-sensitive deconvolution algorithm designed for images with complicated spatial structure. It parameterizes the image into a collection of inverted tapered paraboloids. The minor cycle iterations use a matched-filtering technique to measure the location, amplitude and scale of the dominant flux component in each iteration, and take into account the non-orthogonality of the scale basis functions while performing updates. In other words, the minor cycle iterations consider all scales together and model components are chosen in the order of decreasing integrated flux.

MS-CLEAN can be formulated as a chi-square minimization applied to a sky model that parameterizes the sky brightness as a linear combination of flux components of different scale sizes. The figure below illustrates how a source with multi-scale features is represented by two scale sizes (for example) and how the problem reduces to one of finding the location and amplitudes of delta function components (something for which a CLEAN based approach is optimal). The top left and bottom left images show flux components of two different scale sizes. The images in the middle column show sets of delta functions that mark the locations and amplitudes of the flux components for each scale. The image on the far right is the sum of the convolutions of the first two columns of images. 

![ae3f0711d60cf1c82efb14326ff360c221a6f8c2](media/ae3f0711d60cf1c82efb14326ff360c221a6f8c2.png){.image-inline width="660" height="323"}

>A pictorial representation of how a source with structure at multiple spatial scales is modeled in MS-CLEAN.
  

###### Choosing \'scales\'

In practice, the user must specify a set of scale sizes for the algorithm to use (in units of the number of pixels). As of now, this can be done only manually with the user making guesses of what the appropriate scale sizes are. This figure illustrates how the scales can be chosen, for a given structure on the sky. 

![eee4ed5ee088da2e6ad6378a3db5fa2719e9ae58](media/eee4ed5ee088da2e6ad6378a3db5fa2719e9ae58.png){.image-inline}

>An example set of multiscale \'scale sizes\' to choose for a given source structure.
  

It is recommended that a \'0\' scale always be included to model unresolved sources. Beyond that, scale sizes should approximately follow the sizes of dominant structures in the image. For structure with very bright and sharp edges, a series of nearby scale sizes works best, often in conjunction with a mask. The largest scale size should be less than or equal to the smaller dimension of large scale features. One must also take care to avoid scale sizes that correspond to the unmeasured short spacings in the central region of uv space, as the reconstruction on these scales will see no constraints from the data and can result in arbitrary values (or divergence). For mosaics of extended emission, it is sometimes possible to use large scale sizes in the minor cycle if there are enough connected structures across pointings, but since there still is no actual short spacing uv data to constrain those scales, they should be used with caution. A reasonable starting point for setting the scales (assuming the cell size samples the mainlobe of the psf by a factor of \~5) is *scales=\[0,5,15\]*.

###### Scale Bias

By default, the optimal choice of scale per iteration is that which produces the maximum principal solution (assuming independent scales). Given this normalization, all scales supplied via the *scales* parameter are treated equally.

In addition to this base normalization, a *smallscalebias* parameter may be used to bias the solution towards smaller or larger scales. This is especially useful when very large scale emission is coupled with weak compact features. The peak from each scale\'s smoothed residual is multiplied by ( 1 - *smallscalebias* \* scale/maxscale ) to increase or decrease the amplitude relative to other scales, before the scale with the largest peak is chosen.

*smallscalebias=0.0* (default) implies equal weight to all scales (as per the natural normalization that comes with the principal solution). Increasing it from 0.0 to 1.0 biases the reconstruction towards smaller scales in the supplied range. Decreasing it from 0.0 to -1.0 biases it towards larger scales in the supplied range.  It can be useful to experiment with MS-clean in *interactive=True* mode. If you notice that bright regions of emission are overcleaned in the first few major cycles (i.e. negative regions will appear in the residual image), it suggests that too much cleaning is happening on the largest scales and it can help to increase the *smallscalebias*. Additionally, it is often necessary to clean comparatively deeply to reap the full benefit of a multi-scale CLEAN.  Note also that scale bias (*smallscalebias*) is a fine-tuning tool that will be useful only if the list of supplied scale sizes is also appropriate to the structure being deconvolved; before turning to smallscalebias, it is advisable to first ensure that the *scales* parameter is set to reasonable values.

<div class="alert alert-info">
**NOTE**: An improved *smallscalebias* paramater was implemented in CASA 5.6 for both MultiScale and MTMFS deconvolution algorithms. Details can be found in [this CASA memo](memo-series.ipynb#casa_memo9_ms_mtmfs_clean.pdf).
</div>


###### Multi-Resolution CLEAN

A related approach, called Multi-Resolution CLEAN is available in AIPS (and not in CASA). It is very similar to MS-CLEAN, although it operates on one scale size at a time. It smoothes the residual image and PSF by a particular scale size, and runs the minor cycle only on that scale. It switches scales after the next major cycle. This algorithm uses a different scale-based normalization (compared to MS-CLEAN) and has its own *scalebias* parameter which has its own formula. 

 

##### Multi-Term (with Multi-Scale) 

Multi-Scale Multi-Frequency synthesis (MSMFS) [\[5\]](#Bibliography) is a wide-band imaging algorithm that models the wide-band sky brightness distribution as a collection of inverted, tapered paraboloids of different scale sizes, whose amplitudes follow a polynomial in frequency. A linear-least squares approach is used along with standard clean-type iterations to solve for best-fit spectral and spatial parameters. A point-source version of this algorithm can be run by specifying only one scale size corresponding to a delta-function.

![b552095e2060703d930b11d1c63af3a9f32051be](media/b552095e2060703d930b11d1c63af3a9f32051be.png){.image-inline}

>A 2x2 system of equations to represent the fitting of a 2-term Taylor polynomial (Note that this is only a representative diagram using the same images shaded differently). In reality, the Hessian matrix contains different spectral PSFs.
  

The figure illustrates the set of normal equations that are to be solved in the image domain. What is usually a single convolution is now a joint convolution operator. The images on the left represent Taylor-weighted residual images, the 2x2 matrix contains spectral PSFs (the instruments\' responses to spectra given by different Taylor functions), and the model images on the right represent Taylor coefficients per component. (Note : This figure only illustrates the structure of the system of equations.)

More details about the algorithm and how to choose parameters such as the number of Taylor coefficients (nterms) and the reference frequency (reffreq) are given in the [Wideband Imaging](imaging.ipynb#wide-band-imaging) section. 

 

Multiple Scales as part of the MTMFS algorithm are treated in the same way as MS-Clean (above), with the *scales* and  *smallscalebias* parameters available for choosing a range of scales and fine-tuning which ones get preference during reconstruction.




##### Restoration 

##### Standard Restoration

The final list of flux components (or an image containing just the component delta functions) is restored by smoothing it with a Gaussian that matches the resolution of the main lobe of the PSF and adding back the residual image. This step is done in order to compensate for the unphysical nature of CLEAN based component images that include delta functions, and to include residual flux (especially for extended emission) that was not picked up as part of the model image. The need for restoration varies depending on the choice of algorithm but since all our CLEAN-based approaches include delta functions (with or without multi-scale components), this restoration step is always applied.

 

##### Multi-term restoration

Multi-term (wideband) restoration is a bit different from standard restoration in that it also modifies the residuals that are added to the smoothed model. Residuals are converted from Taylor-weighted averages of the residual data into Taylor coefficient space such that they represent the \'next higher order term\' being imaged (a standard way of represent error). Practical implications of this are a higher than expected rms in the zero-th order image because the higher order terms being fitted have more reconstruction error and are not strictly linearly independent from the zero-th order term. In the outputs of the Multi-Term algorithm, the restored images contain these modified residuals, whereas the residual images contain the unmodified residuals which conform to what astronomers typically mean by \'residual\' images. More details about the algorithm are provided in the [Wideband Imaging](https://casa.nrao.edu/casadocs-devel/stable/wide-band-imaging) section.

 



##### Clean Bias 

Clean bias, an effect noticed for decades by users of the CLEAN algorithm, is a systematic shift of reconstructed peak intensities to lower than expected values. This is usually seen in deep imaging runs with large numbers of closely-spaced weak sources, and when the PSF has sidelobes above the 0.1 level. The use of masks or clean boxes to constrain the search space alleviates the problem. A PSF with lower sidelobes (for example the PSF from MFS imaging as compared to a single channel PSF) can also prevent this type of flux bias with the CLEAN algorithm and more importantly it does so without having to invoke complicated masking procedures.

The clean bias effect can be explained by considering that the CLEAN algorithm is an L1-norm basis-pursuit method that is optimized for sparse signals that can be described with a minimal number of basis functions. For astronomical images this implies well-separated point sources whose properties can be described by single basis functions (one pixel each) and whose central peaks are minimally affected by PSF sidelobes from neighbouring sources. In a crowded field of point sources, especially with a PSF with high sidelobes, the CLEAN algorithm is more error-prone in the low SNR regime. A systematic lowering of source brightness can be explained by the algorithm constructing many artificial source components from the sidelobes of real sources.

 



##### Other Algorithms 

There are other options that are present in our code base, but not used much, could be experimental, coming in the near future, or simply untested. Information on how to add external algorithms is given below.

###### MEM

This algorithm models the sky brightness distribution as a collection of point-sources and uses a prior image along with an entropy-based penalty function to bias the solution of pixel amplitudes. The Maximum Entropy method (MEM) [\[6\]](#Bibliography) [\[7\]](#Bibliography) is a pixel-based deconvolution algorithm that performs a rigorously-constrained optimization in a basis of pixel amplitudes. MEM uses the Bayesian formulation of chi-square minimization, and applies a penalty function based on relative image entropy. This choice of penalty function biases the estimate of the true sky brightness towards a known prior image. If a flat image is chosen as the prior, the solution is biased towards being smooth, and produces a more realistic reconstruction of extended emission. Positivity and emptiness constraints can be applied on the image pixels via a penalty function.

The MEM implementation in CASA\'s imager is unstable, and is unlikely to get attention as there are better methods available now. Please use multi-scale CLEAN instead.

###### ASP

The Adaptive Scale Pixel (ASP) [\[8\]](#Bibliography) deconvolution algorithm parameterizes the sky brightness distribution into a collection of Gaussians and does a formal constrained optimization on their parameters. In the major cycle, visibilities are predicted analytically with high accuracy. In the minor cycle, the location of a flux component is chosen from the peak residual, and the parameters of the largest Gaussian that fits the image at that location are found. The total number of flux-components is also updated as the iterations proceed.

This algorithm is currently not available in CASA, but is on the mid-term implementation plan. 

 

###### Comparison between deconvolution algorithms : One example

Due to the fact that the uv-sampling is always incomplete, the result of a reconstruction algorithm can vary depending on the choice of sky model and the type of algorithm and constraints used. This figure shows a comparison between point-source CLEAN, MS-CLEAN, MEM and the ASP algorithms.

In the figure below, the top row of panels show the component images that illustrate the different sky models being used. The middle row of panels shows restored images (used for the science). It should be noted that they are all different from each other and that they are all valid images. The main difference appears to be the achievable angular resolution. The bottom panels show residual images (gradient of chi-square) which radio astronomers typically use to judge whether all the signal in the data has been modeled or not. These images show how well the different methods handle extended emission. For example, CLEAN results in significant correlated flux in the residuals. MEM does better but the error pattern has significant structure outside the source too. MS-CLEAN has lower residuals than the two previous methods but has a characteristic pattern arising from using a fixed set of scale sizes to model complicated spatial structure. The ASP method shows much more noise-like residuals owing to the fact that at each iteration it finds best-fit components. Most more recent algorithms derived using compressed-sensing theory are reported (in the literature) to produce results similar to the ASP algorithm, as they all also perform fits to parameterized basis functions.

 

![616ac63cbaf38d3c0b28e3970409e95713395ce3](media/616ac63cbaf38d3c0b28e3970409e95713395ce3.png){.image-inline width="514" height="355"} 

>A comparison between point-source CLEAN, MS-CLEAN, MEM and the ASP algorithms.
  

 

##### Adding Other Deconvolution algorithms 

External deconvolution algorithms can be connected to our imaging framework in order to access our data I/O and gridding routines (with parallelization) and avail of the option of operating within major/minor cycle loops instead of as stand-alone methods that don't often connect to the data. The only pre-requisite is that the algorithm is able to operate in the image domain on a residual image and a PSF, and produce a model image as output. 

It should be noted that although many recently developed compressed-sensing algorithms do not explicitly make this uv-domain and image-domain distinction, their practical implementations do, and in some cases it is possible to frame the algorithm within a major/minor cycle structure (with residual visibilities being computed as \'data - model\'). Another way of saying this is that our software can be used to implement the data-\>image and image-\>data transforms, while implementing an external reconstruction algorithm. The only exceptions are algorithms that require the gridding of something other than \'data - model\' and which cannot be implemented as linear combinations in the image domain.

Attempts by external algorithm developers to connect to our framework are welcome, as are suggestions for improving this interface to be more usable.

###### Task Interface

**tclean** can be used in \'only major cycle\' mode by setting *niter=0*. If *calcres=False*, *calcpsf=False* are set, then **tclean** can be also used to start directly with minor cycle algorithms that pick up .residual and .psf images from disk.

###### Tool interface

Python scripts can use our PySynthesisImager library to access each operational step of the **tclean** task, and to add or delete steps as necessary. Examples are given in the **tclean** task documentation (at the end of the examples page).

###### Within C++

For C++ programmers, it is possible to connect a new deconvolution algorithm by deriving from SDAlgorithmBase and implementing three main routines (initialization, cleanup, and a \'takeOneStep\' method that does the series of minor cycle iterations).

 

 

 

 



##### Bibliography

1. Schwab\ and\ Cotton\ 1983\ 
2. Hogbom\ 1974\ 
3. Clark\ 1980\ 
4. Cornwell\ 2008\ 
5. Rau\ &\ Cornwell\ 2011\ 
6. Cornwell\ and\ Evans\ 1985\ 
7. Narayan\ and\ Nityananda\ 1986\ 
8. Bhatnagar\ and\ Cornwell\ 2004\ 
^



***





#### Iteration Control 

Iteration controls and diagnostics



##### Major and Minor Cycles

The CASA Imager implements its iterative optimization in two nested loops, Major and Minor cycles, as described in the [Overview](imaging.ipynb#imaging-overview). 

 



##### Iteration Controls

##### loop gain

For each component selected in the CLEAN minor cycle, the response of only a fraction of the flux is subtracted out at each step. This is controlled by a loop gain $\gamma$ which is multiplied with the amplitude of the latest flux component before the residual image is updated. This fraction represents a step size used in steepest descent algorithms to counter the effect of imperfect update directions. For a point source, the residual left on the dirty image is $(1-\gamma)^{N_{CL}}$.

Loop gain is typically set at a default of 0.1. As a general rule of thumb, if the sky model being fitted to the data is a good match for the structure being imaged, a higher loop gain is tolerated. MS-Clean with appropriate scale sizes is one example. On the other hand, point source CLEAN applied to extended emission will require a very small loop gain to make adequate progress. 

 

##### Stopping Criteria

True convergence is not very easy to define or identify in practice. This is largely because of the logarithmic convergence typical of chi-square minimization or the presence of artifacts that may prevent true convergence but which would still allow imaging at a quality adequate for subsequent analysis. Imaging algorithms therefore have stopping criteria to decide when to pause a set of minor cycle iterations and to perform a major cycle as well as when to stop iterations altogether.

###### Reasons for stopping

###### Threshold

The standard stopping criterion is a threshold on the peak brightness in the residual image. When artifacts do not dominate the residuals, such a threshold is a robust way of terminating a run. A global stopping threshold is usually related to the theoretically expected rms (typically $5\sigma$). A stopping threshold to trigger a major cycle is usually related to the height of the strongest sidelobe of the strongest source. The rationale behind this choice is to expect errors in the subtraction of the contributions of flux components during the minor cycle (due to approximations such as the beam patch size) and to prevent spurious new components from being added to the model.

###### Niter

A simple stopping criterion is the total number of iterations (individual minor cycle steps). In the presence of artifacts, it is used if one wants to explicitly stop imaging early to prevent divergence or to truncate iterations once they reach the point of diminishing returns. It is usually used as an over-ride for the more natural stopping criteria of thresholding.

###### Non-Convergence

When the data do not exactly conform to what the algorithm is solving for, true convergence and theoretical noise estimates will not be reached. Some symptoms of non convergence include the residual image saturating in rms value or peak residual with no significant changes across minor and major cycle iterations. Of course, increases in absolute model flux that persist could signal divergence.

 

###### Implementation of stopping criteria in **tclean**

User Parameters :   *niter*, *cycleniter*, *threshold, nsigma*

Internally calculated controls : *cyclethreshold*

###### Minor Cycle Stopping Criteria

After a major cycle is complete, and before the minor cycle iterations begin, a *cycleniter* and *cyclethreshold* are computed and used as controls to terminate the minor cycle iterations and trigger the next major cycle. A major cycle is triggered (or minor cycle iterations are skipped) when any one of the following criteria are satisfied.

\(0\) The mask for the current plane is all False.

\(1\) Iteration limit :  *cycleniter* = min ( niter - iterdone ,  *cycleniter* )

*cyclethreshold* is internally computed and used as a major cycle trigger. It is related what fraction of the PSF can be reliably used during minor cycle updates of the residual image. By default the minor cycle iterations terminate once the peak residual reaches the first sidelobe level of the brightest source.

\(2\) Threshold :  *cyclethreshold* is computed as follows using the settings in parameters *cyclefactor*, *minpsffraction*, *maxpsffraction*, *threshold, nsigma* :

psf_fraction = max_psf_sidelobe_level \* *cyclefactor*psf_fraction = max(psf_fraction, *minpsffraction*);psf_fraction = min(psf_fraction, *maxpsffraction*);*cyclethreshold* = peak_residual \* psf_fraction         \# The peak residual is computed within the current mask.*cyclethreshold* = max( *cyclethreshold*, *threshold* )

Further, if *nsigma* (a multiplicative factor of *rms* noise) is specified (\>0.0), the n-sigma based threshold is calculated for each image plane.  The threshold per image plane is calculated using the median absolute deviation (MAD) as follows:

nsigma_threshold = *nsigma* \* robustRMS   (where robustRMS = 1.4826 \* MAD)

and then, the *cyclethreshold* calculated above is further evaulated as

*cyclethreshold* = max(*cyclethreshold*, nsigma_threshold)

\(3\) Zero iterations performed in the minor cycle.

\(4\) Divergence :  If the peak residual increases from the previous peak residual by more than 10%.

(currentPeak - prevPeak)/(prevPeak) \> 0.1

In all situations, the reason for stopping is printed in the logger, per image plane (e.g. per channel).

 

###### Global Stopping Criteria

After each major cycle, peak residuals (with and without masks) are evaluated and compared with the following criteria to decide if any more minor cycle iterations are needed or not. Any one of the following conditions will trigger termination of the imaging run.

(1)Total number of iterations \>= *niter* 

Currently iterations are counted across all image planes, including channels. In the future it will be modified to apply to one plane at a time.

\(2\) peak residual within the mask \< *threshold  * ( or the peak reasidual value differs within one part in 100 to the threshold value)**

\(3\) The mask is blank for all planes (either due to user edits or automasking)

\(4\) No change in the peak residual from the previous major cycle. This would imply that the minor cycle in between did nothing.

\(5\) peak residual within the mask \< max(nsigma thresholds across image planes)  (or the peak reasidual value differs within one part in 100 to the maximum nsigma threshold value)**

\(6\) Divergence 1 : A large relative increase of peak residual across a single major cycle. This catches sudden strong divergence.

   ( PeakRes - PrevPeakRes ) / PrevPeakRes \> 3.0    (where peak residual is computed over the entire image, ignoring the clean mask)

\(7\) Divergence 2 : A relative increase of 3 times in the peak residual from the minimum recorded so far.  This catches slow divergence.

   ( PeakRes - MinPeakRes ) / MinPeakRes \> 3.0   (where peak residual is computed over the entire image, ignoring the clean mask)

 

In all situations, the reason for stopping is printed in the logger.

When nsigma threshold is activated (nsgima\>0.0), since nsigma threshold values varies across image planes, the global exit condition that satifies in that case, can be combination of (5) and any other valid exit criteria.

(In addition to the above, a warning message is printed in the logger if the  peak residual within the clean mask increases by a factor of 2, but no actions are taken.) 

 



##### Runtime editing of Iteration Controls

When **tclean** is run with *interactive=True*, a viewer GUI opens to allow the drawing and display of masks on residual images, and also displays and allows the modification of the following iteration control parameters : *iterations left*, *cycleniter*, *cyclethreshold*, *threshold*.

Of these parameters*, iterations left,* and *cyclethreshold* are internally updated after each major cycle and then displayed in case the user wishes to edit them.

-   The field *iterations left* is auto-calculated as niter-iterdone.  If this field is hand-edited, it is taken as \'*niter*\' and the next updated value is this new niter-iterdone. 
-   The *cyclethreshold* field is auto-updated based on the peak residual at the end of the latest major cycle. If *cyclethreshold* is hand-edited, the user-set value applies to only the current set of minor cycle iterations and the auto-calculation resumes from the next major cycle. 

<div class="alert alert-info">
**Note:** Interactive **tclean** only works when a region or mask is selected in the CASA Viewer. If the entire image should be cleaned, please draw a box around the entire image. There is a known bug that when a region is first selected, and then de-selected to produce an empty mask (filled with zeros), the CASA Viewer that runs interactive tclean will still allow you to proceed, and tclean will detect an empty mask and stop. Please always mark a region/mask to continue interactive tclean, and do not forget to double-click inside the green contours to select the region.
</div>

<div class="alert alert-info">
**Note** : In casa5, the auto-calculated cyclethreshold is always displayed as 0, but hand-edited values are still honored.   In the end, the logger contains all the information about what got used, and it has been tested that iteration control and imaging proceeds as expected.
</div>

<div class="alert alert-info">
**Note**: In casa6, the auto-calculated cyclethreshold is correctly displayed in the GUI. However, hand-edited cyclethresholds do not change in the GUI until two major cycles later. However, here too, the logger contains the most accurate information about what was used, and the expected behaviour (of hand-edited cyclethresholds applying to only the current minor cycles) is seen and has been tested. Therefore, iteration control and imaging will proceed as expected.
</div>

<div class="alert alert-info">
**Note** : Threshold information via the GUI must contain units.  '0.5Jy' will work but '0.5' on its own will not.
</div>

 



##### Returned Dictionary

When the **tclean** task is run as a python command, it can produce a return value (by setting *interactive=1/0* instead of True/False).  This dictionary contains a summary of the run with information such as number of iterations done, number of major cycles, peak residual at each major cycle boundary and at which iteration count this occured, metadata to index this information for multiple image fields, channels, and stokes planes, a stopcode to indicate the reason for termination of the run (global termination criterion). This dictionary can be used for scripting.

 



***





#### Masks for Deconvolution 

Descriptions of mask types and how to create them

For the most careful imaging, you will want to restrict the region over which you allow CLEAN components to be found by using a mask. This mask is generally referred to as a clean mask.



##### Creating a **clean** mask:

There are several different ways to specify a clean mask, including:

1.  A text-based region. The [CASA region text format](imaging.ipynb#region-file-format) can be used to define clean regions either by specifying the region directly in the **tclean** command or by using an ASCII text file containing the specifications. You can use the viewer to save a region formatted according to the CRTF specification. To do this, an image must already exist to serve as a reference or template to create the mask image or the region.
2.  An image consisting of only 1 (valid) and 0 (invalid) pixel values. Such images can be generated or modified using tasks such as **makemask**. The mask has to have the same shape (number of pixels, velocity, and Stokes planes) as the output image. An exception are single velocity and/or single Stokes plane masks. They will be expanded to cover all velocity and/or Stokes planes of the output cube. 
3.  An automatically generated mask. There are several experimental algorithms available in **tclean** for automatically masking emission during the deconvolution cycle. See the [automasking section](#automasking) below for more details (does not work in the old **clean** task).
4.  A mask created by **tclean** while interactively cleaning using the viewer. You can combine this method with the options above to create an initial clean mask and modify it interactively. Please be aware that when running tclean interactively, the viewer is accessible during a major cycle, and the mask can be inadvertently by changed during the active clean cycle, although the new mask is not registered until the next major cycle. Also note that interactive **tclean** only works when a region or mask is selected in the CASA Viewer. If the entire image should be cleaned, please draw a box around the entire image. There is a known bug that when a region is first selected, and then de-selected to produce an empty mask (filled with zeros), the CASA Viewer that runs interactive tclean will still allow you to proceed, and tclean will detect an empty mask and stop. Please always mark a region/mask to continue interactive tclean, and do not forget to double-click inside the green contours to select the region.

However they are created, the masks are all converted (as necessary) and stored as CASA images consisting of the pixel values of 1 and 0. When mask files are read in and have undefined values, these values are treated as 1s by CASA. Mixed types of masks can be specified in the **tclean** task. 

<div class="alert alert-info">
In CASA, the term, 'mask' for an image is used in two different contexts. One is used for CASA images/image analysis is a T/F mask (pixel mask), which can be embedded in the parent CASA image.  The 'mask' used in imaging normally refers to a 1/0 image, which is directly used to define deconvolution region(s) (or set a 'clean mask') in the **tclean** task.
</div>



##### Automasking

The **tclean** task has an option to generate clean masks automatically during the deconvolution process by applying flux density thresholds to the residual image. Currently  \"auto-multithresh\" is the automasking algorithm available in **tclean**. Previously available experimental alogrithms, *\"*auto-thresh\" and \"auto-thresh2\" were removed in CASA 5.4. The \"auto-multithresh\" algorithm can be selected via the *usemask* parameter. For this algorithm, the mask will be updated at the beginning of a minor cycle based on the current residual image. The algorithm uses multiple thresholds based on the noise and sidelobe levels in the residual image to determine what emission to mask. It also have functionality to remove (\"prune\") small mask regions that are unlikely to be real astronomical emission. A more detailed description of the algorithm are given below and in [\[1\]](#Bibliography) .

##### *\"*auto-multithresh\"

This algorithm is intended to mimic what an experienced user would do when manually masking images while interactively cleaning. The parameters *sidelobethreshold* and *noisethreshold* control the initial masking of the image. The *sidelobethreshold* indicates the minimum sidelobe level that should be masked, while the *noisethreshold*  indicates the minimum signal-to-noise value that should be masked. The threshold used for masking is the greater of the two values calculated for each minor cycle based on the rms noise and sidelobe levels in the current residual image. 

Regions smaller than a user-specified fraction of the beam can be removed, or \"pruned\", from the mask. The size of the region is defined as the number of contiguous pixels in the region. The figure below shows an example of the pruning process.

![4d7866a3cd721cdd1848746ddc3779ecae911c8e](media/4d7866a3cd721cdd1848746ddc3779ecae911c8e.png){.image-inline}

>Figure 1 - An example of the pruning process. The image on the left shows the original threshold mask, while the image on the right shows the resulting mask after all regions smaller than a user-specified fraction of the beam area have been removed. 
  

The resulting masks are all convolved with a Gaussian that is a multiple of the synthesized beam size, which is controlled by the parameter *smoothfactor*. Only values above some fraction of the smoothed Gaussian peak are retained, which is defined via the *cutthreshold* parameter. Note that *cutthreshold* is defined as a fraction of the smoothed Gaussian peak, not as an absolute value. This procedure ensures that sources are not masked too tightly, i.e., there is a margin between the emission and the mask.  Note that *smoothfactor* and *cutthreshold* are related. A large *smoothfactor* and high *cutthreshold* can give a similar region to a lower *smoothfactor* but lower *cutthreshold*. Note that setting the cuttreshold too high (\>\~0.2) will tend to remove faint regions. 

![04de130d6a1b73abe630710354de91960c92d734](media/04de130d6a1b73abe630710354de91960c92d734.png){.image-inline}

>Figure 2 - An example of the process used to ensure that sources are not masked too tightly. The left hand image shows the initial threshold mask. The middle image shows the threshold mask convolved with a Gaussian. The right image shows the final threshold mask where only emission above some fraction of the peak in the smoothed mask is retained. The final mask is larger than the original threshold mask and better encapsulates the emission.
  

The initial threshold mask can be expanded down to lower signal-to-noise via binary dilation. This feature is particularly useful when there is significant faint extended emission. The *lownoisethreshold* parameter is used to create a mask of the low signal-to-noise emission, which we refer to as the constraint mask. Th previous total positive mask is expanded (or grown) via an operation known as binary dilation, which expands each mask region using a structuring element (also known as a kernel). Currently the structuring element is fixed with a 3x3 matrix with the diagonal element being 0. We use a constraint mask based on a low signal-to-noise threshold to limit the expansion of the mask to regions within the *lownoisethreshold*. Only the regions in the constraint mask that touch the previous mask are retained in the final constraint mask. Then the final constraint mask is pruned, smoothed, and cut using the same method as the initial threshold mask. 

The sub-parameter *growiterations* gives a maximum number of iterations used to \"grow\" the previous masking into the low signal-to-noise mask, which can speed up masking of large cubes at the expense of possibly undermasking extended emission. The sub-parameter *dogrowprune* can be used to turn off pruning for the constraint mask, which also may also speed up this process.

![250712b3a3d4d44283c4a301072959ddf78dec28](media/250712b3a3d4d44283c4a301072959ddf78dec28.png){.image-inline}

>Figure 3 - An example of how the masks are expanded into low signal-to-noise regions. The top row shows the binary dilation process. Left: The low signal-to-noise threshold mask used as a constraint mask. Middle: The final mask from the previous clean cycle. Right: The result of binary dilating the mask from the previous clean major cycle into the constraint mask. The bottom left image shows the binary dilated mask multiplied by the constraint mask to pick out only those regions in the constraint mask associated with the previous clean mask. The bottom middle image shows the final pruned, smoothed, and cut mask.
  

There is also an experimental absorption masking feature controlled by the sub-parameter *negativethreshold*, which has an analogous definition to *noisethreshold*. This feature assumes that the data has been continuum subtracted. Absorption masking can be turned off by setting the *negativethreshold* vaue to 0 (the default). Note that the negative and positive threshold masks are tracked separately and that the negative mask is not pruned or expanded into lower signal-to-noise regions.

Finally, all the masks (initial threshold mask, negative mask, low noise threshold mask) are added together with the mask from the previous major cycle to form the final mask.

All the operations described above, including obtaining image statistics, are done per spectral plane for spectral line imaging. If a channel is masked using the noise threshold and the resulting final mask is zero, then future auto-masking iterations will skip that channel. The *minpercentchange* parameter is an experimental parameter that controls whether future masks are calculated for a particular channel if the mask changes by less than n% after major cycle where the cyclethreshold is equal to the threshold for the clean. In general, we recommend *minpercentchange* to be set to -1.0 (turned off).

The *verbose* parameter records information to the log on whether a channel is included in the masking, the image noise and peak, the threshold used and it\'s value, the number of regions found in the initial mask and how many were pruned, the number of region found in the low noise threshold mask and how many of those are pruned, and the number of pixels in the negative mask. This information is helpful for optimizing parameters for different imaging cases as well as general debugging.

###### Algorithm In Detail

1.  Calculate threshold values based on the input parameters.
    a.  sidelobeThresholdValue = sidelobeThreshold \* sidelobeLevel \* peak in residual image
    b.  noiseThresholdValue =  noiseThreshold \* rms in residual image
    c.  lowNoiseThresholdValue = lowNoiseThreshold \* rms in residual image
    d.  negativeThresholdValue = negativeThreshold \* rms in residual image
2.  Create the threshold mask.
    a.  maskThreshold = max(sidelobeThresholdValue,noiseThresholdValue)
    b.  Create threshold mask by masking all emission greater than maskThreshold.
    c.  Prune regions smaller than minBeamFrac times the beam area from threshold mask.
    d.  Smooth the mask image by smoothFactor \* (beam size).
    e.  Mask everything above cutThreshold \* the peak in the smoothed mask image.
3.  Expand the mask to low signal-to-noise.
    a.  lowMaskThreshold = max(sidelobeThresholdValue,lowNoiseThresholdValue)
    b.  Create constraintMask by masking all emission greater than lowMaskThreshold.
    c.  Use binary dilation expand the previous clean cycle mask into the constraintMask.
    d.  Create the low S/N mask by retaining only the regions in the constraintMask that are connected to the previous clean cycle mask.
    e.  Prune \[can turn this off with *dogrowprune*=False\], cut, and smooth the low S/N mask the same way as was done for the threshold mask.
4.  Mask the absorption (experimental)
    a.  If negativethreshold \>0.0:
        1.  negativeMaskThreshold =  -  max(negativeThresholdValue, sidelobeThresholdValue)
        2.  mask negative pixels with values \<= negativeThresholdValue
        3.  Cut and smooth the absorption mask the same way as was done for the threshold mask.
5.  Add the threshold mask, the low S/N mask, the absorption mask, and the mask from previous clean cycle together.

 

###### Noise Estimation 

Prior to CASA 5.5, \"auto-multithresh\" estimated the noise per channel using the median absolute deviation (MAD), scaled to match a Gaussian distribution. This noise estimate is computationally fast, but may be less accurate for cases where the emission covers a large fraction (nominally 50%) of the field of view. In CASA 5.5, a new noise estimate was introduced, which uses a more complex and computationally expensive noise estimate. This estimate may yield more accurate estimates of the noise in the case where emission covers a significant fraction of the field of view. The procedure is as follows. If there is no mask, remove pixels from the noise distribution via Chauvenet\'s criterion [\[2\]](#Bibliography)  [\[3\]](#Bibliography) and then estimate the noise using the remaining pixels via the median absolute deviation. If there is a mask, then calculate the noise from the pixels outside the clean mask and inside the primary beam mask, which we refer to as the masked MAD. All MAD values are scaled to match a Gaussian distribution.

The parameter fastnoise is set to True by default. 

###### Polarization Data

As of CASA 5.6, auto-multithresh now functions with polarization data. It applies the same algorithms to the Stokes QUV images as used for the Stokes I image. This means that the full masking process is applied to the positive emission (including the prune and grow steps), but that the masking of the negative emission only includes the initial threshold mask (no prune or grow).

###### A Note on Input Parameters

The default \"auto-multithresh\" parameters have been optimized for the ALMA 12m array in its more compact configurations. The parameters may need to be modified for other input cases, e.g., ALMA 12m long baseline data, ALMA 7m array data, and  VLA data. The main parameters to explore are *noisethreshold*, *sidelobethreshold*, *lownoisethreshold*, *minbeamfrac*, and *negativethreshold* (if you have absorption). We do not recommend changing the *cutthreshold* and *smoothfactor* parameters from their default values. The *dogrowprune* and *growiterations* parameters are primarily used to improve the speed of the algorithm for large cubes.

 

 

 

<div>

</div>



##### Bibliography

1. Kepley\ et\ al.\ 2020,\ Publications\ of\ the\ Astronomical\ Society\ of\ the\ Pacific,\ 132, 024505\ 
2. Peirce,\ B.\ 1852,\ The\ Astronomical\ Journal,\ 2,\ 161.\ 
3. Chauvenet, W. A Manual of Spherical and Practical Astronomy, Volume II (London, UK: Dover; reprinted in 1960 based on fifth revised and corrected edition 1891), 558--566



***





#### Spectral Line Imaging 

Specific topics for spectral line imaging including spectral coordinates and spectral reference frames



##### Spectral coordinates and frame

In spectral line imaging, the spectral coordinates are defined by the user inputs. The relevant **tclean** parameters are the data selection parameters and image cube defining parameters:  *start, width,* *nchan,* and *outframe*.  The task attempts to adjust the inputs so that the data selection matches the defined cube.

The **tclean** tasks provide *start* and *width* parameters can be specified in channel index, frequency, or velocity. In **tclean** task, spectral mode is turned on by *specmode=\'cube\'* and its specific mode (channel, frequency, or velocity) is automatically determined from the units of the sub-parameters (*start*, *width*). Mixed specifications are currently not allowed (e.g. *start=\'5\'* and *width = \'10km/s\'* ) in **tclean**.

The underlying imaging code (FTMachine) uses to a fixed spectral reference frame (LSRK) internally. However, the user can specify an outframe so that the output cube image is relabeled to the user specified frame. If the outframe is unspecified, it defaults to LSRK. Continuum images are always labeled in LSRK. Note that tools like the Viewer can also apply on-the-fly conversions to relabel frequencies in frames other than what is in the image header. The masks (imregrid, imreframe, and exportfits) can also explicitly change the reference frame, and in some cases, regrid the channels.

##### Spectral Reference Frame

CASA (CASACORE Measures) uses the frequency frames defined in the Reference material section \"[Spectral Frames](memo-series.ipynb#spectral-frames)\". Data is typically observed in the topocentric observatory frame (TOPO) at a fixed sky frequency. Any observed line, however, will change its topocentric frequency as a function of time.  Note that for TOPO (as well as GEO) reported frequencies, the grid of output images uses the first time stamp of the observations.

When applying Doppler corrections, the data is typically regridded to the Local Standard of Rest Kinematic (LSRK, CASA default) or the sun-earth barycentric (BARY) frame, which can be specified in the *outframe* parameter. Both of  these parameters require the rest frequency of a spectral line (*restfreq* parameter).

In addition, a velocity definition (*veltype* parameter, sometimes referred to as Doppler type) is required. This parameter is typically either RADIO (CASA default) or OPTICAL. Note that those definitions are identical at $v=0$ but increasingly differ  at larger velocity values. A full list of supported velocitiy definitions is given in the Reference material section \"[Spectral Frames](memo-series.ipynb#spectral-frames)\".



##### Mapping between Data Channels and Image Channels

During the gridding process, the **tclean** task makes a choice about how to map data channels to image channels, based on the channelization encoded in the MS and the user-specified image channelization. The mapping between data channels and imager channels may vary depending on the following:

1.  If the user-specified \'start\' frequency is not at a data channel boundary, the visibility data and weights are interpolated and then evaluated at the centers of the shifted frequency grid. 
2.  When image channels are wider than data channels, visibilities and weights are binned and gridded via multi-frequency synthesis (MFS) within each image channel.
3.  On-the-fly software Doppler tracking can also be done as a time-dependent frequency relabeling of the data channels, before interpolation and binning choices are made.

Usually, a combination of these three operations are performed during gridding. There are also special cases where only some or none of them apply. This behavior is consistent between the old and new Imager code (i.e., **clean** and **tclean**).

The *interp* parameter chooses the interpolation scheme to be used during the gridding process. The choices are currently \'*nearest\'*, \'*linear\'*, and \'*cubic\'*.

-   *\'nearest\' *just picks the value of the nearest data channel.
-   \'*linear*\' will interpolate the data channel to a channel width that is an integral number of channels that fits in an image channel. For example, if the image channel is 3.14x the width of the original data channel, then interpolation will interpolate the data and weights to 3 channels with a width of 3.14/3.0 times the original width of the data channel.The linear gridder will use the 2 adjacent original data channel to interpolate.
-   \'*cubic*\' works the same as \'linear\', but with the nearest 4 instead of 2 data channels.

<div class="alert alert-warning">
**Warning:** in CASA version earlier than 5.6, the interpolated channels were ensure to be aligned with the edge of the image channel. This could cause channels to be dropped at the edges of data chuncks, causing different sensitivities at the edge of the chunkcs (which can be particularly problematic when chanchunk >1 or in parallel processing). In CASA 5.6, this has been resolved, and the interpolated channels data now align with the center of the image channel.
</div>

 



##### Software Doppler Tracking Options

For the purpose of this document, a time independent frame is a frame in which the source observed has not changed its velocity over the period of time of the observation. A time dependent frame is one in which the observed source has changed its velocity during the observation period. If datasets from different epochs are combined during imaging, the relevant period of time to consider is the entire range spanned by all datasets.

The following descriptions are specific to the task based on new imager, **tclean**.

There are three software Doppler tracking options, which will be controlled at the task level. Individual parameters are described in the parameter tab for **tclean** task page.

##### *specmode=\'cube\'*

Converts data frequencies to the time-independent spectral frame (default: LSRK).

Output image base frame : specified frame in *outframe*

In this mode, data frequencies are read from the input MS(es), and compared with image channel bin frequencies (also in LSRK) to decide which data channels go into which image channel bins. This process is a time-dependent relabeling of data frequencies. The result aligns the frequencies of spectral lines from astrophysical sources taken at different times and thus with different observed frequencies.   The relevant user parameters are: *start, width, nchan, outframe, veltype, restfreq*.

Internally, this mode converts the data to LSRK frequencies before gridding and after gridding converts them to the outframe to construct an appropriate output CASA image.   Note that for TOPO and GEO, which are actually time-dependent frames, the conversion layer encodes a specific time at which the conversion is valid. The convention in \'**clean**\' is the start epoch of the dataset, but this time can be changed via the **imreframe** task with *outframe=\'topo\'*.  

##### *specmode=\'cubedata\'*

Produces a one-to-one mapping between data channels and image channels.

Output image base frame : REST, UNDEFINED  

In this mode, no time-dependent frequency frame transformations are done during gridding/binning. In this case, data frequencies are read from the input MS(es) and compared directly with image frequencies. If the data has been observed in a time-dependent frame (e.g., TOPO), this mode will not align the frequencies of spectral lines from astrophysical sources taken at different times and thus with different observed frequencies. Only local signals  at a fixed frequency such as RFI will remain aligned, for example terrestrial RFI in case of TOPO data.

The relevant user parameters are *start, width, nchan, veltype, restfreq*.

For this mode, *outframe* is not an option as *start, veltype, restfreq* will be interpreted literally to construct the frequency grid, with no further conversions.

##### (To be implemented) *specmode=\'cubesrc\'*

Convert data frequencies to the SOURCE frame.

 Output image base frame : SOURCE  

If the FIELD table of the source being imaged contains ephemeris information, a time-dependent relabeling of the data frequencies (software Doppler tracking) is done to make spectral lines stationary in the moving source frame.  If the FIELD table of the source being imaged does not contain ephemeris information (i.e. the source is not a solar system object), the software Doppler tracking will follow a conversion to LSRK. In addition, a systemic velocity has to be specified with respect to a spectral frame, which will be recorded in the image.

 The relevant user parameters are: *start, width, nchan, frame, veltype, restfreq, sysvel, sysvelframe*. The base frame of the output image will always be SOURCE. The *sysvel* and *sysvelframe* parameters represent the systemic velocity with respect to a specific frame that will be embedded in the coordinate system. These two parameters are ignored if the ephemeris information is available. This is the only mode that allows the *start and width* parmaeters to be specified in *outframe=\'SOURCE\'* in addition to other standard frames.

##### *mode=\'mfs\'*

Multi-frequency synthesis, where there is only one large image channel. This will always be in LSRK, with the image frequency coordinates decided by the spw data selection parameter.



#####  Imaging a pre-Doppler-tracked Data Set

An MS output by **cvel** or **mstransform** will be treated the same way as any other MS observed directly in the frame that the MS is labeled with.

1.  A dataset that has been relabeled in a time-independent frame ( LSRK, LSRK, BARY, etc\.... ) using **mstransform** can use *mode=\'cube\'*. The base frame of the output image will be based the input parameters. If the MS is already in a time-independent frame, the code will detect that no additional time-dependent frequency shifts are required. A similar situation holds for datasets labeled in the SOURCE frame when *mode=\'cubesrc\'* is used.
2.  A dataset that needs channel binning/gridding with no extra time-dependent frequency transformations should use *mode=\'cubedata\'* and the output frame will be 'UNDEFINED'. For example, when an MS has already been transformed into a time-dependent frame and the user wants to image the data as is.
3.  If the MS has been relabeled \'REST\' using **mstransform**, the base frame of the output image will be \'REST\'. This method is another way to generate images with no extra time-dependent frequency transformations.



##### Parameters for Spectral-Axis Image Defination

##### *nchan*

Number of channels in output cube image.

##### *start*

The first channel of the cube image. The units of *start* will decide whether the user means \'channel index\', \'frequency\' or \'velocity\':

-   *start=3* : channel 3 of the first spw in the selected list (irrespective of channels selected using the \'spw\' parameter)  
-   *start=\'1.2GHz\'* : start frequency. The output channels are equidistant in frequency.
-   * start=\'15 km/s\'* : start velocity.   if *veltype=\'RADIO\'*, channels are equidistant in both frequency and velocity.   If *veltype=\'OPTICAL\' or  \'Z\',* the channels are equidistant in velocity but not in frequency.  Also see *veltype* section below.

##### *width*

The channel width of the resulting image cube. If *width* has units, it has to be the same units as start. If specified as an integer, it is taken as N times the width of a single data channel. For irregular channels in either frequency or velocity, a reasonable default for the width of a single data channel will be calculated.  

*If start* is* *specified in velocity and the *width* is not specified (default), then the output image will be ascending velocities (or descending frequencies) with the velocity specified in *start* as the first image channel. Also note that since the channel frequencies in the MS can be descending or ascending order, the  appropriate sign (\"+\" or \"-\" although \"+\" can be omitted) should be used for the width when the frequency or velocity specification is used to avoid any confusion.

##### *outframe*

Spectral reference frame in which to interpret *start*. This is also the frame to which the base frame of the cube image will be set for *mode=\'cube\'*. For *mode=\'cubesrc\'*, the option of specifying start in the SOURCE frame will also be allowed.

##### *veltype*

Velocity option in which to interpret start if units are \'km/s\' : 

-   RADIO: velocity in \'radio definition\': $\frac{v_{rad}}{c} = 1 - \frac{f}{f_{0}} = \frac{z}{1+z}$
-   OPTICAL: velocity in \'optical definition\': $\frac{v_{opt}}{c} = \frac{f_{0}}{f} - 1 = z$
-   Z:  the same as OPTICAL
-   RATIO: $\frac{v}{c}=\frac{f}{f_{0}}$  \* This is accepted but there will be no real interpretation of the velocity of this type.
-   BETA: relativistic definition: $\frac{v}{c}=\frac{1-\left(\frac{f}{f_{0}}\right)^2}{1+\left(\frac{f}{f_{0}}\right)^2}$
-   GAMMA: $ \frac{v}{c}=\frac{1}{\sqrt{1-BETA^2}} = 1 - \frac{1+\left(\frac{f}{f_{0}}\right)^2}{2\frac{f}{f_{0}}}$ \* This is accepted but there will be no real interpretation of the velocity of this type.

##### *restfreq*

A vector of rest frequencies, which will be encoded in the output image. If this parameter is left empty, the list of rest frequencies encoded in the SOURCE table corresponding to the field being imaged will be used. The first rest frequency in the list will be used to interpret *start* when its units indicate a velocity specification.

##### *sysvel*

Systemic velocity of a source (only for *mode=\'cubesrc\'*)

##### *sysvelframe*

Frequency frame with respect to which the systemic velocity is specified (only for *mode=\'cubesrc\'*)

##### *interpolation*

Relevant when image channel widths \> data channel widths and/or *start* is offset from the data start. This parameter is used irregardless of whether time-dependent frame conversions happen or not. It is not used **only **when *start* and *width* are aligned between the data channels and image channels and no time-dependent frequency relabeling is needed.

##### *chanchunks *

For large image cubes, the gridders can run into memory limits as they loop over all available image planes for each row of data accessed. To prevent this problem, we can grid subsets of channels in sequence so that at any given time only part of the image cube needs to be loaded into memory. This parameter controls the number of chunks to split the cube into.

##### *perchanweightdensity*

When calculating weight density for Briggs style weighting in a cube, this parameter determines whether to calculate the weight density for each channel independently or a common weight density for all of the selected data (the default). This parameter has no meaning for continuum (*specmode=\'mfs\'*) imaging but for cube imaging *perchanweightdensity=True* is a recommended alternative option that provides more uniform sensitivity per channel for cubes, but with generally larger psfs than the *perchanweightdensity=False*. See the **tclean** task pages for more information.

 

<div class="alert alert-info">
**NOTE on data selection via 'spw'**

The user should select a range larger than what the image will need, and not try to fine-tune the list of channels. The channel mapping and binning process will pick and grid only those data channels that actually map into image channels. This process is already optimized for performance.
</div>

<div class="alert alert-info">
**Note on image channel order of the output cube
**

The *start* parameter defines the spectral coordinate of the first image channel while the sign of *width* parameter controls direction of the increment along the spectral axis. If *width* is unspecified, and if *start* is defined as a velocity or frequency, the image channels will be ordered such that it always increases in value in the unit specified in *start* with increasing channel number. This is regardless of whether spectral axis order of the input visibility data is increasing or decreaseing in frequency. For example, start='-15km/s' with result in the image with channel 0 being -15km/s and becomes more positive as the image channel number increases. For *start* specified in channel (e.g. start=5)  with an unspecified *width*, image channel frequency axis order will depend on the frequency order of the input visibility data. For a full control of the spectral axis order in the output image, the user is encouraged to set *width*.
</div>




##### Using Output Images from tclean

Images from **tclean** will have LSRK or another frame specified in outframe or SOURCE or UNDEFINED or REST as the base frame. The spectral axis of the base frame is always encoded in frequency in the output images. A regularly spaced set of frequencies is represented by the *start*/*width*/*nchan* parameters are listed in the image header. An irregularly spaced set of frequencies is encoded as a tabular axis.

##### Conversion Layer of the Spectral Reference Frame

One can attach a conversion layer for a different spectral referance frame to the image using the **imreframe** task or a tool script that uses cs.setconversiontype() on top of the base frame.

The Viewer will, by default, will display in the base frame of the image if no conversion layer is attached. However, if the conversion layer is attached, it will honor the frame in the conversion layer and relabel image frequencies on-the-fly while displaying the spectral coordinate. The Viewer also has options to temporarily change the frame to any frequency frame or velocity convention with or without the conversion layer.

Note that conversion layers from LSRK to TOPO/GEO (time-independent frame to time-dependent frame) will be tied to one particular time during the observation. Our convention is the start time of the dataset being imaged.  Tool level scripts using the imageanalysis (ia) and coordinatesystem (cs) modules can be used to extract lists of frequencies or velocities in any spectral frame and velocity convention. Within a conversion layer, the commands csys = ia.coordsys(); csys.toworld( \[0,0,0,5\] ) will give the frequency of channel 5 in the frame of the conversion layer. With no conversion layer, it will list channel 5 in the base frame of the image ( i.e. LSRK ). Velocities can be read out using csys helper functions, e.g., csys.(set)restfrequency(XXX); csys.frequencytovelocity( 1.5, \'GHz\', \'RADIO\', \'km/s ) . Several other spectral axis relabeling options are possible in combination with the measured (me) module.  

CASA Images can finally be exported to the FITS format, during which frame conversions are hard-coded.

Image channels can be regridded using the imregrid task, if the user needs an explicit regrid instead of only frequency-axis relabeling.

 



##### Notes on the Frequency Frame Conversions

Conversion between the different types is done with the standard MeasConvert class (MFrequency::Convert, MRadialVelocity::Convert, MDoppler::Convert). This is what is encoded in the conversion layer of CASA Images.

Some conversions are only possible if the following frame information is available:

1.  Conversion to/from REST needs Radial Velocity information. The *sysvel* parameter in *mode=\'cubesrc\'* will be used for this. For an MS already at REST, no conversions are needed.
2.  Conversion to/from TOPO and GEO needs Epoch information. This is set in the conversion layer for *mode=\'cube\'* as the start time of the MS (after the data selections are applied) and can be modified via the **imreframe** task with *outframe=\'TOPO\'* or *\'GEO\'* and subparameter epoch.
3.  Conversion to/from TOPO needs Position information. This is read from the input MS, or Image header.
4.  All conversions need Direction information. This is the image center from the Image header.

<div class="alert alert-warning">
**Alert:** Conversion between the different frequencies can, due to relativistic effects, only be done approximately for very high (order c) radial velocities. Rather than convert between frequencies, a better approach would be to start from radial velocities and a rest frequency.
</div>

<div class="alert alert-warning">
**Alert:** For large radial velocities (of order c), the conversions are not precise, and not completely reversable, due to unknown transverse velocities, and the additive way in which corrections are done. They are correct to first order with respect to relativistic effects.
</div>



***





#### Wide Band Imaging 

MFS, MT-MFS with wide-field imaging,mosaics and wideband PB correction



##### Wide Band Imaging

**Single Pointing : **

Single pointing wideband imaging is available via the \'standard\', \'mosaic\' and \'awproject\' gridders in the tclean task.

**Joint Mosaics : **

Warning :  Joint-mosaic imaging with multi-term wideband imaging has been verified and validated only for imaging cases where the instrumental parameters do not change across the face of the mosaic (i.e. position-independent PSFs).  A series of algorithm details related to position dependent primary beam effects and point spread functions are being worked on.  Only specific modes of wideband mosaicing are currently being commissioned for the VLASS imaging pipelines. 

** **



##### Imaging at wideband sensitivity ^\[1\]^

The continuum imaging sensitivity offered by a broad band receiver is given by

$$\begin{eqnarray} \sigma_{continuum} \propto \frac{T_{sys}}{\sqrt{ N_{ant}(N_{ant}-1) ~ N_{chan}\Delta\nu~ \Delta\tau}}= \frac{\sigma_{chan}}{\sqrt{N_{chan}} } \end{eqnarray}$$

where $T_{sys}$ is the instrumental system temperature, $\Delta\nu$ is the bandwidth of each channel, $\Delta\tau$ is the integration time, $N_{chan}$ is the number of frequency channels, and $\sigma_{continuum}$ and $\sigma_{chan}$ are theoretical wideband and narrowband image noise levels.  Note that this calculation is for an ideal system whose gain is flat across the band with equally weighted channels (i.e. at the center of the primary beam). 

To take full advantage of this broadband imaging sensitivity, image re-construction algorithms need to be sensitive to the effects of combining measurements from a large range of frequencies. These include frequency-dependent angular resolution and uv-coverage, frequency-dependent array element response functions, and the spectral structure of the sky brightness distribution.

![063c6f81181136f30f7270f39f42dd89443218e7](media/063c6f81181136f30f7270f39f42dd89443218e7.png){.image-inline width="466" height="223"}

 

##### UV-Coverage

Projected baseline lengths are measured in units of the observed wavelength. Therefore the $uv$ coverage and the imaging properties of an interferometer change with frequency. As the observing frequency increases, the angular resolution for a given distribution of antennas increases (or, conversely the width of the point spread function given by $\theta_{\nu} = 1/{u}_{max}$ radians, decreases). In addition, at higher observing frequencies, the sensitivity to large spatial scales for a given distribution of antennas decreases. 

###### Bandwidth Smearing (limits of channel averaging)

The choice of frequency resolution (or channel width) at which visibilities must be measured (or can be averaged up to) for synthesis imaging depends on the $uv$ grid cell size to be used during imaging, which in turn depends on the observed frequency and the desired field of view in the image. The following condition ensures that within a field of view chosen as the half power beam width of the antenna primary beam, the image-domain bandwidth smearing is smaller than the angular resolution of the instrument:$$\begin{eqnarray} \frac{\Delta \nu}{\nu_0} < \frac{Resolution}{FoV} = \frac{\lambda/b_{max}}{\lambda/D} = \frac{D}{b_{max}} ~~~~~ \Rightarrow ~~~~~ {\Delta \nu} < {\nu_0} \frac{D}{b_{max}}  \end{eqnarray}$$

For broad-band receivers, this limit will change across the band, and the channel width should be chosen as the bandwidth smearing limit computed for $\nu_{min}$.

 

##### Sky Brightness

Stokes I continuum emission usually has smoothly varying, continuous spectral structure often following a power law functional form with curvature, steepening and turnovers at various locations in the spectrum. Power laws and polynomials are typically used to model such sky spectra. With the MT-MFS wideband imaging algorithm, a Taylor polynomial in $I$ vs $\nu$ space is fitted to the data per flux component, and the resulting coefficients used to calculate the spectral index that a power law model would provide. 

 

##### Primary Beam

At the center of the primary beam, bandpass calibration makes the gain flat across the band. Away from the pointing-direction, however, the frequency-dependence of the primary-beam introduces artificial spectral structure in the wideband flux model reconstructed from the combined measurements. This frequency dependence must be modeled and removed before or during multi-frequency synthesis imaging to recover both spatial and spectral structure of the sky brightness across a large field of view.  In general, the frequency dependence of the primary beam can be approximated by a power law.

If $\theta$ is the angular distance from the pointing center and $\theta_0$ is the primary beam FWHM at the reference frequency, then the frequency dependence of the primary beam is equivalent to a spectral index of$$\begin{eqnarray} \alpha_{\rm E} &=&-8\log(2)\left(\frac{\theta}{\theta_0}\right)^2\left(\frac{\nu}{\nu_0}\right)^2 \end{eqnarray}$$

This corresponds to an effective spectral index of -1.4 at the half power point and reference frequency.

 



##### Options in CASA for wideband imaging

<div class="alert alert-warning">
**WARNING**: Wideband mosaicing is still in its commissioning phase and not officially endorsed in CASA 5.5. With *deconvolver='mtmfs'* for multi-term imaging including wideband primary beam correction, *gridder='awproject'* has a known bug and should not be used. For *gridder='mosaic'* the uncertainties in the derived spectral index may be larger than the xxx.alpha.error images would imply, with or without the use of conjbeams, because of systematic issues that are currently being evaluated. Development/commissioning of wideband mosaicing is ongoing and will be available in a next CASA release.
</div>

 

##### (1) MFS (nterms=1)

Traditionally, multi-frequency synthesis (MFS) imaging refers to gridding visibilities from multiple frequency channels onto a single spatial-frequency grid. It assumes that the sky brightness and the primary beam are constant across the total measured bandwidth and all frequencies measure the same visibility function just at different spatial frequencies. In this case, standard imaging and deconvolution algorithms can be used to construct an accurate continuum image.

For sources with spectral structure across the observed band, this approach converts any spectral variations of the visibility function into spurious spatial structure that does not follow the standard convolution equation in the image domain and therefore will not self-correct during deconvolution.  For the VLA at L-Band, for example, a 1.0 Jy source with spectral index of -1.0 across the 1-2 GHz band will produce spectral artifacts at the $5\times10^{-3}$ level. Therefore, sources requiring dynamic ranges (peak brightness / thermal noise) less than a few hundred will not see any of these artifacts and basic MFS imaging will suffice. Detection experiments in otherwise empty fields are a good example of when this method is most appropriate.

 

##### (2) MT-MFS (nterms\>1)

To alleviate the spectral artifacts discussed above and to reconstruct the broad-band sky brightness distribution correctly, a spectral model must be folded into the reconstruction process. The advantages of such an image reconstruction are that the combined $uv$ coverage (from all channels) is used, flux components are \'tied\' across frequency by the use of an explicit spectral model or physically motivated constraints, and the angular resolution of the resulting intensity and spectral index images is not limited to that of the lowest frequency in the band. Under high signal-to-noise conditions, the angular resolution follows that of the highest frequency in the band.  Disadvantages are that the reconstruction is often tied to a specific spectral model and will work optimally only for sources whose spectral structure can be described by that model (i.e.a low order Taylor polynomial). In low signal-to-noise situations, the unnecessary fitting of higher order terms can increase the noise and error in the results.

The MTMFS algorithm models the spectrum of each flux component by a Taylor series expansion about $\nu_0$ .$$\begin{eqnarray} \vec{I}^{m}_{\nu} = \sum_{t=0}^{N_t -1} {w_{\nu}^{t}} \vec{I}^{sky}_{t} ~~~\mathrm{where}~~~ w_{\nu}^{t}&=&{ \left( \frac{\nu - \nu_0}{\nu_0} \right) }^t  \end{eqnarray}$$where $I^{sky}_t$ represents a multi-scale Taylor coefficient image,and $N_t$ is the order of the Taylor series expansion.

A Taylor expansion of a power law yields the following expressions for the first three coefficients from which the spectral index $I^{sky}_{\alpha}$ and curvature $I^{sky}_{\beta}$ images can be computed algebraically.$$\begin{equation} I^m_0 = I^{sky}_{\nu_0} ~~;~~ I^m_1 = I^{sky}_{\alpha} I^{sky}_{\nu_0} ~~;~~ I^m_2 = \left(\frac{I^{sky}_{\alpha}(I^{sky}_{\alpha}-1)}{2} + I^{sky}_{\beta}\right) I^{sky}_{\nu_0} \end{equation}$$Note that with this choice of parameterization, we are using a polynomial to model a power-law.

 

 

###### User controls

###### Reference Frequency

This is the frequency about which the Taylor expansion is done. The default is the center of the frequency range being imaged, but this is not required.  The relative weights/flags of data on either side of this frequency should be inspected to ensure that the reconstruction is not ill-conditioned. The output intensity image represents the flux at this reference frequency. Please note that the value at a specific reference frequency is different from the integrated flux across a frequency range.

###### nterms

The number of Taylor coefficients to solve for is a user parameter.  The optimal number of Taylor terms depends on the available signal-to-noise ratio, bandwidth ratio and spectral shape of the source as seen by the telescope (sky spectrum x PB spectrum). In general, *nterms*=2 is a good starting point for wideband EVLA imaging and the lower frequency bands of ALMA (when fractional bandwidth is greater than 10%) if there is at least one bright source for which a dynamic range of greater than few 100 is desired. Spectral artifacts for the VLA often look like spokes radiating out from a bright source (i.e. in the image made with standard mfs imaging).  If increasing the number of terms does not eliminate these artifacts, check the data for inadequate bandpass calibration. If the source is away from the pointing center, consider including wide-field corrections too.

The signal-to-noise ratio of the source must also be considered when choosing nterms. Note that the Taylor polynomial is in I vs $\nu$ space. This means that even for a pure power law, one may need nterms=3 or 4 in order to properly fit the data if there is adequate signal to see more spectral variation than a straight line. One should avoid trying to fit a high-order polynomial to low signal-to-noise data. 

 

 

###### Data Products

###### Taylor Coefficient Images

The basic products of the MT-MFS algorithm are a set of $N+1$ (multi-scale) Taylor coefficient images that describe the spectrum of the sky brightness at each pixel (coefficients of an $N^{th}$-order polynomial). The $0^{th}$-order coefficient image is the Stokes I intensity image at the reference frequency.

###### Multi-Term Restoration

The restoration step of the MT-MFS algorithm performs two actions in addition to the standard convolution of the model with a Gaussian beam and adding back of the residuals. First, it converts the residuals into the Taylor coefficient space before adding them to the smoothed model components (which are already Taylor coefficients). The residuals (or error) will typically be higher for higher order terms. Since the terms are not strictly independent, errors from including higher order terms may slightly increase the noise floor even on the zeroth order intensity image.  This arises because the concept of a \'residual image\' is different for a multi-term algorithm. For standard narrow-band imaging, the residual or dirty image already has sky-domain fluxes.  For multi-term imaging, the residual or dirty image must be further processed to calculate Taylor coefficients which represent sky-domain fluxes. It is this step that will provide accurate spectral indices (for example) from undeconvolved dirty images (i.e. tclean runs with niter=0 and deconvolver=\'mtmfs\').

###### Calculating Spectral Index

Spectral index is computed as $I^{sky}_{\alpha} =  I^m_1 /  I^m_0$, for all pixels above a threshold applied to the $I^m_0$. Other pixels are zeroed out and a mask is applied.  Currently this threshold is automatically calculated to be 5 x max( peak residual, user threshold ).  Right now, the spectral index calculation can be modified  in two ways (a) perform the above division oneself in a python script or (b) use the widebandpbcor task with action=\'calcalpha\'.   The ability to control this within tclean itself will be added in the future.

Spectral curvature (when possible) is also computed from the Taylor coefficients.

###### Calculating Error in Spectral Index

An estimate of spectral index error is also provided as an output image. This is an empirical error estimate derived as the result of error propagation through the division of two noisy numbers: alpha = tt1/tt0 where the \'error\' on tt1 and tt0 are just the values from the residual coefficient images at each pixel. In the limit of perfect deconvolution and noise-like residuals, this number can be accurate. However, in practice, deconvolution artifacts usually remain in the residual image (especially underneath extended emission) and they dominate the errors. In general, the spectral index error map should only be used as a guide of which regions of the image to trust relative to others, and not to use the absolute value of error for scientific analysis.  A more useful error estimate can be derived by repeating the imaging run (especially if it involves multi-scale components) with slightly different settings of scale sizes and iteration controls, to see what is true signal and what can be attributed to reconstruction uncertainty.  For high signal-to-noise compact sources, error limits of $\pm 0.05$ can be achieved. For complicated extended emission at about SNR=100 or less, typical errors are about $\pm 0.2$.  These errors are highly correlated with how appropriately the scale sizes are chosen, with errors ranging from $\pm 0.1$ or less up to $\pm 0.5$ in the limit of using delta functions to try to model extended emission.

Errors on spectral curvature are much higher than for spectral index. In one example where the M87 galaxy was imaged at L-Band, only the central bright inner lobes (at dynamic range of a few thousand) showed average spectral curvature that could be trusted.

##### (3) Cube + imcollapse.

The simplest form of wideband imaging is to treat each frequency channel independently and make an image cube. A continuum image can then be formed by first smoothing all planes to a common (lowest) angular resolution and computing the mean across frequency. Spectral structure can be modeled per pixel from this smoothed cube. The main advantage of this method is its simplicity and the fact that it does not depend on any particular spectral model. The main disadvantage is that the angular resolution of all higher frequency channels must be degraded to that of the lowest frequency before any combined analysis can be done. Also, in case of complicated spatial structure, each frequency\'s $uv$ coverage may be insufficient to guarantee reconstructions that are consistent with each other across the band.



##### Comparison of different wideband imaging methods

 

  --------------------------------------- ----------------------------------------------------------------------------------------------------------------------------------- ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ --------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                                                         Cube                                                                                                                                                                        MFS                                                                                                                                                                              MFS with a wideband model

  Angular Resolution                                                               Same angular resolution as lowest frequency data                                                                                                                           Same angular resolution as highest frequency data                                                                                                                                           Same angular resolution as highest frequency data

  Continuum Sensitivity                                                                    Narrow-band (for deconvolution)\                                                                                                                                                          Full                                                                                                                                                                                        Full
                                                                                                           \                                                                                                                                                                                                                                                                                     
                                                                                                 Full (after stacking)                                                                                                                                                                                                                                                                           

  Weak Sources                                              Low SNR sources may not be deconvolved accurately in all channels, diluting the combined result                                                               Accurate low SNR imaging, but ignores spectral variation of bright sources. Errors show up at dynamic ranges of a few 100.                                                                                             Accurate bright source modeling to allow detection of weak sources.

  Strong Sources                                                         Can handle arbitrary spectra down to the single channel sensitivity.                                                                                                                                  Ignores Spectra                                                                                                                                                             Models spectra. Most useful for strong sources.

  Extended Emission                        Fewer constraints per channel so reconstruction may not match across channels. This leads to errors when computing spectral index                                                                 Uses full spatial frequency coverage but ignores spectral. This can cause artifacts.                                                                                                   Reconstructs structure and spectra accurately but depends on the spectral model for accuracy.

  Spectral Reconstruction                                      Accurate for simple bright sources and does not depend on any predefined spectral model.                                                                                                                        Ignores spectra                                                                                                                                                    Models spectra using a wideband flux model during reconstruction.

  Primary Beam correction (and mosaics)                                    Per channel, can be done either during gridding or after imaging                                    Since an MFS image is a weighted channel average, accurate PB correction must be done per channel before combination. Post deconvolution division by a wideband primary beam is also a reasonable approximation.   Wideband PB correction must be done either during gridding or after imaging by dividing out the primary beam and its frequency dependence from the obtained model.
  --------------------------------------- ----------------------------------------------------------------------------------------------------------------------------------- ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ --------------------------------------------------------------------------------------------------------------------------------------------------------------------

 

 

 



#####  Other uses of wideband models

##### Wideband Self Calibration

The broad-band flux model generated by the MS-MFS algorithm can be used within a self-calibration loop in exactly the same manner as standard self-calibration. The purpose of such a self-calibration would be to improve the accuracy of the bandpass calibration and maintain smoothness across spectral windows or subbands that may have been treated independently.

##### Continuum Subtraction

In the case of accurate deconvolution, the wideband model may be subtracted out to study line emission on top of the continuum. The wideband model would be made by excluding channels that contain known line emission,  predicting the wideband model over the entire frequency range, and then performing a \'uvsub\' to subtract it out.



##### Example

The following images of 3C286 illustrate what wideband imaging artifacts look like and how they change with different values of nterms.  These images were made from about 15 minutes of VLA L-Band calibrator data (1-2 GHz).  Note that such clear improvements in the imaging will be visible only if there aren\'t any other sources of error (e.g. calibration errors or weak residual RFI).

 

![d3af02dfa60a934b1cff3aafac93d0b2919b89d5](media/d3af02dfa60a934b1cff3aafac93d0b2919b89d5.png){.image-inline width="549" height="341"}

 




##### Wide-Band and Wide-Field Imaging 

##### Wide-Band + W-term

W-Projection or faceted imaging can be combined with multi-term imaging (*specmode*=\'mfs\', *deconvolver*=\'mtmfs\', *gridder*=\'widefield\' or \'wproject\'). The two algorithms are distinct enough there there are no special considerations to keep in mind when combining them. 

##### Wide-Band + Full Beam

The frequency dependence of the primary beam introduces artificial spectral structure on the sky brightness distribution away from the pointing center.  Below is an example of what this spectral structure looks like, in terms of a power law spectral index.  If nothing is done to eliminate the artificial PB spectrum, it will be visible to the minor cycle during deconvolution and will be interpreted as extra sky spectral structure.   Another aspect of using a wide-band primary beam is the large shelf of continuum sensitivity outside the main lobe of the average beam. This is also a region where the PB spectrum will be varying by up to 100% in positive and negative directions, also in a time-variable way. Therefore, there is increased sensitivity to sources outside the main lobe of the average PB, but very little hope of accurately imaging them without methods that carefully incorporate time- and frequency-dependent primary beam models. 

![dcb7b7bb1677f24cffedcd93d0d1b98c2e68fa9e](media/dcb7b7bb1677f24cffedcd93d0d1b98c2e68fa9e.png){.image-inline width="505" height="361"}

 

Three methods to handle wide band primary beams are discussed below. 

###### Cube Imaging

The option of cube imaging is always present, where the primary beam is corrected per channel at the end of imaging, using appropriate frequency-dependent primary beam models.

###### Post-deconvolution Wide-band Primary Beam Correction

If primary beams are ignored during imaging (gridders other than \'awproject\' or \'mosaic\'), the artificial spectral structure will be absorbed into the sky model (to the extent that it is possible, given that the primary beams are squinted and rotating, creating a time-varying primary beam spectrum).  The output Taylor coefficient images now represent the spectral structure of (primary beam) x sky.   

Wide-band primary beam correction can be done by constructing Taylor coefficients that represent the primary beam spectrum at each pixel, and applying a polynomial division to take them out of the output images (per pixel).

Steps: (1) Compute a set of primary beams at the specified frequencies(2) Calculate Taylor-coefficient images that represent the primary beam spectrum(3) Perform a polynomial division to primary beam correct the output Taylor-coefficient images from the MT-MFS algorithm(4) Recompute spectral index (and curvature) using the corrected Taylor-coefficient images.

Currently, the widebandpbcor task performs this function, but it is scheduled to move into tclean where it will be implemented within C++, and use internally generated information about relative spectral weights.

###### Wideband AW-Projection

The use of *wbawp*=True with *gridder*=\'awproject\' and *conjbeams*=True enables conjugate beams to be used during gridding. The goal is to remove the frequency dependence of the primary beam during the gridding step so that the minor cycle sees the spectral structure of only the sky. This reduces the number of Taylor terms required to model the spectrum and removes the need for any primary beam correction on the output spectral index maps. 

Setting *wbawp=True* enables use of PB evaluated at the center frequency of each spectral window.  Setting *conjbeams=True* enables use of the PB at the \"conjugate\" frequency which effectively projects-out the scaling of the PB with frequency (see Bhatnagar et al, ApJ, [2013,Vol.770, No. 2, 91)](http://stacks.iop.org/0004-637X/770/91) .  The following plot shows the frequency dependence of a PB as a function of distance from the center of the PB.  The red curves trance the total-power response of the antenna and the blue curves show the frequency dependence of the antenna response.  The second figure below shows the effective frequency dependence when using conjugate beams duing imaging.  The blue curve is significantly flat compared to the plot in the first figure. When imaged with conjugate beams, the effects of frequency dependent PBs is effectively removed in the images fed to the minor cycle algorithms.  Image-plane based wide-band algorithms (like the MT-MFS algorithm) designed to model *only* sky frequency dependence can therefore be used without modification.

![794630806bc8bfbf48a70d1f523bda4845f764e4](media/794630806bc8bfbf48a70d1f523bda4845f764e4.png){.image-inline width="292" height="220"}![1dfad9084557b0bb6d968f7b37d6d964809f3515](media/1dfad9084557b0bb6d968f7b37d6d964809f3515.png){.image-inline width="292" height="220"}

##### Wideband + Mosaics

There are several ways of constructing wideband mosaics. The three main choices are spectral (cube vs. MT-MFS), spatial (linear vs. joint mosaics), and primary beam correction (post-deconvolution corrections vs A-Projection based approaches that account for primary beams during gridding with or without correction of the frequency dependence at that stage).  This results to a large number of options for the user.  It is important to note that all methods have trade-offs and are not likely to give identical results (especially since in our software, different algorithms currently use different PB models).

It is recommended that when possible, to use  *specmode*=\'mfs\', *deconvolver*=\'mtmfs\' with *gridder*=\'awproject\' and *wbawp*=True in order to make wideband mosaics.  For cube-based wideband mosaic imaging, it is recommended that one uses *gridder*=\'awproject\' or \'mosaic\' per channel with a post-deconvolution primary beam-correction per channel.

 

###### Wideband Mosaic Primary Beam

In a joint mosaic, one must keep in mind the spectral structure of the primary beam. In a single pointing, the spurious spectral structure is significant only away from the pointing center. Therefore, wideband options may not be required if the source of interest covers a small region at the center of the beam and if its own spectral structure isn\'t strong enough to warrant multi-term imaging.   However, in a mosaic, this primary beam spectral structure is present across the entire field of view of the mosaic, making even the imaging of flat-spectrum compact sources an exercise in wide-field and wide-band imaging.

 

![2012a00c1e7ad86ceadf38e2f3003013a18d8483](media/2012a00c1e7ad86ceadf38e2f3003013a18d8483.png){.image-inline width="454" height="175"}

 

 

 

 

 



##### Bibliography

1. Rau\ &\ Cornwell\ (2011), A&A\ 532,\ A71\ (
^



***





#### Wide-Field Imaging 

W-term, Primary Beams (models, pbcor, A-Projection)



##### **Widefield imaging **

Wide-field imaging typically refers to fields of view over which the basic 2D Fourier transform assumption of interferometric imaging does not apply and where standard on-axis calibration will not suffice. 

**[WARNING : Imaging modes with A-Projection and mosaics (*gridder=\'mosaic\'* and *\'awproject\'*) have been validated only for a few usage modes and use cases as required by the ALMA and VLASS pipelines. Please use these modes at your own discretion, and carefully read the [Known Issues](introduction.ipynb#known-issues) for CASA 5.6 when using AWproject.   Other wide-field imaging modes ( wproject, widefield, facets) work as expected. ]**

  



##### The non-coplanar baseline effect: W-term

For wide-field imaging, sky curvature and non-coplanar baselines result in a non-zero w-term. Standard 2D imaging applied to such data will produce artifacts around sources away from the phase center. CASA has two methods to correct the w-term effect.

 

##### Faceting

In this method, visibilities are gridded multiple times onto the same uv-grid, each time with a different phase-reference center. One single dirty/residual image is constructed from the resulting grid and deconvolved using a single PSF (picked from the first facet). This deconvolution is not affected by emission that crosses facet boundaries, unlike in image-domain faceting, which is an older approach where small facet images are deconvolved separately before being stitched together. [\[1\]](#Bibliography)

In **tclean**, faceting is available via *gridder=\'widefield\'* where you can specify the number of desired facets on a side.  It can be used along with W-Projection as well, for very large fields of view.

 

##### W-projection

In this method, visibilities with non-zero w-values are gridded using using a gridding convolution function (GCF) given by the Fourier transform of the Fresnel EM-wave propagator across a distance of w wavelengths. In practice, GCFs are computed for a finite set of w-values (wprojplanes) and applied during gridding. W-projection is roughly an order of magnitude faster than faceted imaging because it grids each visibility only once [\[2\].](#Bibliography)

In **tclean**, w-projection is available via *gridder=\'widefield\'* or *\'wproject\'* or *\'awproject\'*.  In all cases, the \'*wprojplanes*\' parameter must be set. It represents the number of discrete w-values to be used to quantize the range of w-values present in the dataset being imaged. An appropriate value of wprojplanes depends on whether there is a bright source far from the phase center, the desired dynamic range of an image in the presence of a bright far out source, the maximum w-value in the measurements, and the desired trade off between accuracy and computing cost. As a (rough) guide, VLA L-Band D-config may require a value of 128 for a source 30arcmin away from the phase center. A-config may require 1024 or more. To converge to an appropriate value, try starting with 128 and then increasing it if artifacts persist. W-term artifacts (for the VLA) typically look like arc-shaped smears in a synthesis image or a shift in source position between images made at different times. These artifacts are more pronounced the further the source is from the phase center. There is no harm in simply always choosing a large value (say, 1024) but there will be a significant performance cost to doing so, especially for *gridder=\'awproject\'* where it is combined with A-Projection. *wprojplanes=-1* may be used with *gridder=\'widefield\'* or *\'wproject\'* to automatically compute the number of planes.
The formula that CASA uses to calculate the number of plans when *wprojplanes=-1* is:

$N_\mathrm{wprojplanes} = 0.5\times \frac{W_\mathrm{max}}{\lambda} \times \frac{\mathrm{imsize}}{\mathrm{(radians)}}$

where $W_\mathrm{max}$ is the maximum $w$ in your $uvw$ data and imsize is the largest linear size of your image. This formula is somewhat conservative and it is possible to achieve good results by using a smaller number of planes, which can also save on speed and memory.

<div class="alert alert-warning">
**WARNING**: The algorithm that automatically calculates the number of w-bins (wprojplanes = -1) errs on the side of numerical accuracy and tends to over-estimate the number of w-bins needed. For very wide fields of view, this could result in a significant increase in required memory. It is therefore useful to point out that it is safe to manually choose a value to avoid problems associated with limited memory resources. One can do a few tests with different wprojplane values in order to find out at which values any shifts in source positions are no longer noticeable.
</div>

 

![897bc85407ab7ec0babb1e8b99481534877c8c2d](media/897bc85407ab7ec0babb1e8b99481534877c8c2d.png){.image-inline width="513" height="193"}

 



##### Antenna Voltage/Power Patterns: Primary-Beam

The aperture-illumination-function (AIF) of each antenna results in a direction-dependent complex gain that can vary with time and is usually different for each antenna. The resulting antenna power pattern is called the primary beam. There are two methods to correct for the effect of the primary beam.  

##### Image-domain PB-correction

A simple method of correcting the effect of the primary beam is a post-deconvolution image-domain division of the model image by an estimate of the average primary beam or some other model. This method ignores primary-beam variations across baselines and time, and is therefore approximate, limiting the imaging dynamic-range even within the main lobe of the beam.  This approach also cannot handle heterogenous arrays.

In **tclean**, this option is available by setting *pbcor=True*.  When used with *gridder=\'standard\'* or *\'widefield\'* or *\'wproject\'* which do not internally use any primary beam models, it will compute a model PB at the reference frequency per image channel, and divide it out of the output restored image.   If used with *gridder=\'mosaic\'* or *\'awproject\',* it will use a weighted average of the primary beam models used by the gridders per baseline and timestep.

Primary Beam correction for wide bandwidth observations is discussed in the [Wideband Imaging](imaging.ipynb#wide-band-imaging) section.

![ac1f425c3c4087e00c5de2dcda92367d7bd678e7](media/ac1f425c3c4087e00c5de2dcda92367d7bd678e7.png){.image-inline width="522" height="246"}

 

##### A-Projection

Time and baseline-dependent corrections are applied during gridding, by computing GCFs for each baseline as the convolution of the complex conjugates of two antenna aperture illumination functions. An additional image-domain normalization step is required, and can result in the image being \"flat-sky\" ( the image represents only the sky intensity) or \"flat-noise\" (the image represents the sky multiplied by the primary beam). The advantage of this method is that known time and baseline variability can be accounted for, both during gridding as well as de-gridding [\[3\]](#Bibliography).

Different primary beam effects cause artifacts at different levels in the image [\[4\]](#Bibliography). Depending on the available sensitivity of an observation or desired dynamic range, one can choose to leave out some corrections and save on computing time.  In general, the varying dish size in a heterogenous array is the dominant source of errors causing a dynamic range limit of a few 100. Next come large pointing offsets (such as beam squint or illumination offsets) and at a higher dynamic ranges ($10^4$ and beyond) are other factors such as the details about feed leg structures. On its own, parallactic angle rotation causes artifacts only at a dynamic range of around $10^5$ but if any of the other large effects (pointing offset or illumination pattern errors) are not azimuthally symmetric, then parallactic angle rotation will have an effect at much lower dynamic ranges.

###### gridder = \'awproject\'

In **tclean**, *gridder=\'awproject\'* applies the full A-Projection algorithm and uses baseline, frequency and time dependent primary beams. They are azimuthally asymmetric to account for feed leg structures. They also include beam squint, which is corrected during gridding by applying an appropriate phase gradient across the GCFs to cancel out the polarization dependent pointing offset.  The frequency dependence of the primary beam within the data being imaged is included in the calculations and can optionally also be corrected for during gridding (see [Wideband Imaging](imaging.ipynb#wide-band-imaging) section for details). 

The operations of the \'*awproject*\' gridder are controlled by three parameters: *aterm, psterm* and *wprojplanes.*  *aterm *and *psterm *controls the inclusion/exclusion of the A-term (the antenna aperture function) and the Prolate Spheroidal function (the anti-aliasing function) in the convolution functions used for gridding. *wprojplanes* controls the inclusion/exclusion of the w-term. The following table enumerates the operations for the different possible settings of these parameters. PS and PB in the table below refers to the Prolate Spheroidal and Primary Beam respectivelly and FT() referes to the Fourier transform operation. The last column also shows the mathematical content of the .pb images, which is one of the image-products on the disk in a **tclean** run. For generating a .pb image for image-plane PB correction, the gridder needs to be used with *psterm=False* and the *cfcache* parameter set to a fresh (non-existant) directory so that a fresh cfcache is generated without the PS term in it.  When *aterm=False,* the *psterm* parameter needs to be set to *True.* It can be set to *False* when *aterm=True. * However with this setting the effects of aliasing may be there in the image, particularly near the edges.

 

<table style="height: 197px; border-color: #000000;" width="539"><colgroup><col style="width: 16%" /><col style="width: 16%" /><col style="width: 16%" /><col style="width: 16%" /><col style="width: 16%" /><col style="width: 16%" /></colgroup><tbody><tr class="odd"><td style="text-align: center;"><span style="font-size: 12pt;"><strong>Operation</strong></span></td><td style="text-align: center;"><span style="font-size: 12pt;"><strong>aterm</strong></span></td><td style="text-align: center;"><span style="font-size: 12pt;"><strong>psterm</strong></span></td><td style="text-align: center;"><span style="font-size: 12pt;"><strong>wprojplanes</strong></span></td><td style="text-align: center;"><span style="font-size: 12pt;"><strong>GCF</strong></span></td><td style="text-align: center;"><span style="font-size: 12pt;"><strong>Contents of the .pb image</strong></span></td></tr><tr class="even"><td style="text-align: center;"><strong><span style="color: #ff6600;">AW-Projection</span></strong></td><td style="text-align: center;"><strong>True</strong></td><td style="text-align: center;"><p><strong>True</strong></p><p><strong>False</strong></p></td><td style="text-align: center;"><strong> &gt;1</strong></td><td style="text-align: center;"><p><strong>PS*A*W</strong></p><p><strong>A*W</strong></p></td><td style="text-align: center;"><p><strong>FT(PS) x PB</strong></p><p><strong>PB</strong></p></td></tr><tr class="odd"><td style="text-align: center;"><strong><span style="color: #ff6600;">A-Projection</span></strong></td><td style="text-align: center;"><strong>True</strong></td><td style="text-align: center;"><p><strong>True</strong></p><p><strong>False</strong></p></td><td style="text-align: center;"><strong>1</strong></td><td style="text-align: center;"><p><strong>PS*A</strong></p><p><strong>A</strong></p></td><td style="text-align: center;"><p><strong>FT(PS) x PB</strong></p><p><strong>PB</strong></p></td></tr><tr class="even"><td style="text-align: center;"><strong><span style="color: #ff6600;">W-Projection</span></strong></td><td style="text-align: center;"><strong>False</strong></td><td style="text-align: center;"><strong>True</strong></td><td style="text-align: center;"><strong>&gt;1</strong></td><td style="text-align: center;"><strong>PS*W</strong></td><td style="text-align: center;"><strong>FT(PS)</strong></td></tr><tr class="odd"><td style="text-align: center;"><strong><span style="color: #ff6600;">Standard</span></strong></td><td style="text-align: center;"><strong>False</strong></td><td style="text-align: center;"><strong>True</strong></td><td style="text-align: center;"><strong>1</strong></td><td style="text-align: center;"><strong>PS</strong></td><td style="text-align: center;"><strong>FT(PS)</strong></td></tr></tbody></table>

 

 

Full/Hybrid Mueller matrix support is being added into the system for full-polarization widefield imaging.  Currently, heterogenous arrays like ALMA are not supported, but it will be suitable for VLA widefield imaging. 

 

###### Parallel execution

The computing cost of A-Projection is larger than standard imaging, and cost of AW-Projection is higher than A-Projection.  However, since the run time scales very well with parallelization, these costs can be effectively offset with the use of parallelization (using parallel=True; see the [Parallel Processing](https://casa.nrao.edu/casadocs-devel/stable/parallel-processing) section for details about running casa in parallel mode).  The runtime scales close to linear with the number of nodes used.  We have measured this scaling for up to 200 cores, but the scaling may continue further dependening on the data size, data storage (e.g., Luster vs. standard file system), image size, algorithms used, etc. The plot below shows the measured scaling for a large EVLA L-band mosaic imaging experiment. The dark and light blue curves (legends \"Make PSF + avgPB\" and \"Make Residual\" respectively) show the measurement of the steady-state runtime as a function of the number of cores used.  The lines in black associated with both these curves show the theoratical (ideal) linear scaling curves. A memo with the details of the characterization of the runtime in parallel mode can be found [here](http://www.aoc.nrao.edu/~sbhatnag/misc/Imager_Parallelization.pdf).  **Note that parallelization is not restricted to A-Projection and can be used with any combination of *gridder *and *deconolver *setting.** 

![b9ac634819f43b993a5e2eadc2d190e899f9a258](media/b9ac634819f43b993a5e2eadc2d190e899f9a258.png){.image-inline width="554" height="388"}

There are a number of parameters to apply approximations that can reduce the computing load.

Note that current code does not work correctly for non-square mosaic images and cube imaging. Fixes for these will be included in subsequent releases.   VLA and ALMA data sets often carry the POINTING table with antenna pointing information which may not be correct.  Since by default the imaging module now uses the POINTING table, the POINTING table may need to be disabled (delete all rows of the POINTING sub-table in the MS).

 

![eb12f69b90a6c3576dd2e815a3d57c8dedb40994](media/eb12f69b90a6c3576dd2e815a3d57c8dedb40994.png){.image-inline width="583" height="349"}

###### gridder=\'mosaic\'

In **tclean**, *gridder=\'mosaic\'* applies an approximation of the A-Projection algorithm where it uses azimuthally symmetric beam models that can be different per baseline. It includes the diagonal of the Mueller matrix for multi-Stokes images, but ignores off-diagonals. The frequency dependence of the primary beam is accounted for but is not eliminated during gridding. Since time dependence is not supported by default, the computational cost is lower than A-Projection.   Since ALMA imaging typically involves small fractional bandwidths, includes data with multiple dish sizes, and needs to operate on very large cubes with many channels, this option is suitable for ALMA.  It is also possible to supply external beam models to this gridder, by setting up the vpmanager tool, and one can in principle assign beams separately for each antenna as a function of time, if needed. Note that *gridder=\'mosaic\'* can be used even on a single pointing, especially to account for effects due to a heterogenous array. 

 

##### Mosaics

Data from multiple pointings can be combined during gridding to form one single large image. Details are are described in the [Mosaicing](imaging.ipynb#mosaicing) page.  In a Linear Mosaic, data from multiple pointings are imaged (and optionally deconvolved too) before being stitched together. A Joint Mosaic is a simple extension of A-Projection in which phase gradients are applied to the gridding convolution functions to map data from each pointing to a different position on the sky.  In **tclean**, *gridder=\'mosaic\'* and *\'awproject\'* will both create joint mosaics if data from multiple pointings are selected as the input.

###### Pointing Offset Corrections

When the image phase center is chosen to be different from the observation phase center, a phase gradient is applied during gridding convolution to ensure that the image-domain primary beam is centered at the phase-reference direction. This situation is encountered for all joint mosaic imaging. By default, it is assumed that the antennas point in the same direction as the observation phase center specified in the FIELD subtable of the MS. However, entries may be supplied in the POINTING subtable of the MS and used instead of the FIELD table via the *\'usepointing\'* parameter available to *gridders=\'mosaic\'* and *\'awproject\'*. The VLASS project, for example, has time-dependent and antenna-dependent pointing offsets that are not captured in the FIELD table and which require an additional POINTING table. Note that *\'usepointing=True\'* has no meaning if there are no entries in the POINTING subtable (the default with any MS). Therefore, the default is *\'usepointing=False\'.*

-   *gridder=\'mosaic\'* reads and uses the pointing offset per timestep and baseline, but assumes that both antennas in a baseline pair are pointed in the same direction as the ANTENNA1 listed in the MS for each baseline and timestep. This has not been officially validated for CASA 5.6.
-   *gridder=\'awproject\'* reads and uses the pointing offsets for both antennas in the first baseline pair listed in the MS (per timestep) and assumes this is constant across all baselines. It applies phase gradients per timestep with the assumption that all antennas are pointed in the same direction. This has been validated on VLASS 1.2 data.

<div class="alert alert-warning">
**WARNING**: For CASA 5.6, with *'usepointing=True'*, the *gridder='mosaic'* and *'awproject'* implement slightly different solutions. For CASA 5.6, only *gridder='awproject'* has been validated for *usepointing=True*.For CASA 5.7/6.1, also the *'pointingoffsetsigdev'* parameter has been implemented for applying accurate heterogeneous pointing corrections. A few other features are expected to be implemented post 5.7/6.1, as described in the [Known Issues](introduction.ipynb#known-issues).
</div>

 



##### Primary Beam Models

##### gridder=\'standard\', \'wproject\', \'widefield\', \'mosaic\'

Default PB models :

VLA: PB polynomial fit model ([Napier and Rots, 1982) ](https://library.nrao.edu/public/memos/vla/test/VLAT_134.pdf)[\[5\]](#Bibliography)

EVLA: New EVLA beam models ([Perley 2016](https://library.nrao.edu/public/memos/evla/EVLAM_195.pdf)) [\[6\]](#Bibliography)

ALMA : Airy disks for a 10.7m dish (for 12m dishes) and  6.25m dish (for 7m dishes) each with 0.75m blockages (Hunter/Brogan 2011). Joint mosaic imaging supports heterogeneous arrays for ALMA  (Hunter/Brogan 2011)

These are all azimuthally symmetric beams. For EVLA, these models  limit the dynamic range to 10\^5 due to  beam squint with rotation and the presence of feed leg structures.  For ALMA, these models accounting only for differences in dish size, but not in any feed-leg structural differences between the different types of antennas.

 

###### Adding other PB models

Use the vpmanager tool, save its state, and supply as input to **tclean**\'s *vptable* parameter

Example : For ALMA and gridder=\'mosaic\', ray-traced (TICRA) beams are also available via the vpmanager tool. To use them, call the following before the tclean run: 

```
vp.setpbimage(telescope="ALMA", compleximage='/home/casa/data/trunk/alma/responses/ALMA_0_DV__0_0_360_0_45_90_348.5_373_373_GHz_ticra2007_VP.im',  antnames=['DV'+'%02d'%k for k in range(25)])
vp.saveastable('mypb.tab')
```

Then, supply vptable=\'mypb.tab\' to tclean.

 

##### gridder = \'awproject\'

VLA / EVLA : Uses ray traced models (VLA and EVLA) including feed leg and subreflector shadows, off-axis feed location (for beam squint and other polarization effects), and a Gaussian fit for the feed beams [\[7\].](#Bibliography)

The following figure shows an example of the ray-traced PB models.  Image on the left shows the instantaneous narrow-band PB at the lowest frequency in the band while the image on the right shows the wide-band continuum beam.  Sidelobes are at a few percent level and highly azimuthally asymmetric.  This asymmetry shows up as time-varying gains across the image as the PB rotates on the sky with Parallactic Angle.

![3b308b701fb8f3cf37868148b26c4bb2afcd291e](media/3b308b701fb8f3cf37868148b26c4bb2afcd291e.png){.image-inline width="676" height="301"}

 

 

###### External Beam models for gridder= \'awproject\'

The beam models used internally in \'awproject\' are derived from ray-traced aperture illumination functions.  However since the \'awproject\' algorithm uses the disk CF cache mechanism, a simple way to use a different beam model is to construct the disk CF cache and supply that to \'awproject\' during imaging.  The detailed documention for construcing the disk CF cache is being developed and will be released in subsequent CASA Docs release.  In the meantime, if you need to access this route sooner, please contact the CASA Helpdesk who will direct you to the related (not yet released) documentation or appropriate Algorithms R&D Group (ARDG) staff.

ALMA : Similar ray-traced model as above, but since  the correctness of its polarization properties remains un-verified, support for ALMA is not yet released for general users.

The current implementation of AW-Projection does not yet support heterogenous arrays (although the version of CASA\'s AWProjection used by LOFAR\'s LWImager does have fully heterogenous support). This, along with Full-polarization support is currently being worked on in ARDG branches.

 



##### Heterogeneous Pointing Corrections

Due to the high sensitivity of EVLA and ALMA telescopes, imaging performance can be limited by the antenna pointing errors. These pointing errors in general also vary significantly across the array and with time. Corrections to the true antenna pointing directions are contained in the POINTING sub-table, and if these corrections are present and accurate, they can be used to improve imaging of both single-pointing and mosaic fields. These heterogeneous pointing corrections are controlled by two parameters in **tclean**:

**usepointing**: When set to *True*, the antenna pointing vectors are fetched from the POINTING sub-table. When set to *False* (the default), the vectors are determined from the FIELD sub-table, effectively disabling correction of antenna pointing errors.

**pointingoffsetsigdev**: When correcting for pointing errors, the first value given in the *pointingoffsetsigdev* task is the size in arcsec of the bin used to discover antenna grouping for which phase gradients are computed. A compute for a new phase gradient is triggered for a bin if the length of the mean pointing vector of the antennas in the bin changes by more than the second value. The default value of this parameter is \[\], due a programmatic constraint. If run with this value, it will internally pick [\[600,600\]]{.error} and exercise the option of using large tolerances (10arcmin) on both axes. Please choose a setting explicitly for runs that need to use this parameter.

<div class="alert alert-warning">
**WARNING**: Heterogeneous pointing corrections have been implemented in support of the VLA Sky Survey. This option is available only for *gridder='awproject'* and has been validated primarily with VLASS on-the-fly mosaic data where POINTING subtables have been modified after the data are recorded. The use of pointing corrections is currently unverified for general VLA and ALMA data, so users should use these parameters at their discretion.
</div>

A description of the algorithm that handles the antenna pointing corrections for the AW-Projection algorithm in CASA can be found in [CASA memo 11](memo-series.ipynb#heterogeneous_pointing_corrections_memo11.pdf).The implementation of heterogeneous antenna pointing corrections was driven by requirements for the VLA Sky Survey (VLASS). Additional testing of Wideband Mosaic Imaging and Pointing Corrections can be found in this [Knowledgebase article](memo-series.ipynb#wideband-mosaic-imaging-and-pointing-corrections-for-the-vla-sky-survey).

 



##### Bibliography

1. [Sault\ et\ al,\ 1999](https://ui.adsabs.harvard.edu/#abs/1999A&AS..139..387S/abstract)\ 
2. [Cornwell\ et\ al,\ 2008](https://ui.adsabs.harvard.edu/#abs/2008ISTSP...2..647C/abstract)\ 
3. [Bhatnagar\ et\ al,\ 2008](https://ui.adsabs.harvard.edu/#abs/2008A&A...487..419B/abstract)\ 
4. [Kundert\ et\ al\ 2016](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7762834&tag=1)\ 
5. [Napier\ and\ Rots,\ 1982](https://library.nrao.edu/public/memos/vla/test/VLAT_134.pdf)\ 
6. [Perley\ 2016](https://library.nrao.edu/public/memos/evla/EVLAM_195.pdf)\ 
7. [Brisken\ 2009](https://ui.adsabs.harvard.edu/#abs/2009nsem.confE..21B/abstract)\ 
^



***





### Single-Dish Imaging 

Imaging single-dish data in CASA



***





#### Overview 

Chapter overview

This section describes imaging single-dish data.

The data should already be T$_{sys}$ and T$_{sky}$ calibrated (at least into antenna temperature units, $T_A^*$ \[K\]) , according to the process described in the single-dish calibration pages.

The CASA task used for imaging single-dish data is [sdimaging](global-task-list.ipynb#task_sdimaging).  This task will return a four-dimensional array with two position axes, one frequency or velocity axis, and one polarization axis. The output of [sdimaging](global-task-list.ipynb#task_sdimaging) is a CASA image that can be explored, analyzed, and manipulated in CASA, or exported into a versatile FITS image format via [exportfits](global-task-list.ipynb#task_exportfits).

The pages below first describe the general process of gridding single-dish data followed by the actual procedures invoked with CASA.



***





#### Theoretical Description 

A theoretical description of single-dish image generation and data gridding

For single-dish observations, ALMA uses on-the-fly mapping.  The technique is described in Mangum et al. (2007) [\[1\]](#Bibliography).

Converting single-dish observations into an image or cube is done almost entirely in the image domain. After taking and calibrating the data, the process follows three steps:

1.  Forming the image grid
2.  Populating the image grid
3.  Smoothing the image data

The fundamental parameter relevant to image quality is the sampling interval. There are a number of sampling functions that need to be considered: the sky sampling function, the image grid sampling function, and the response function of the single-dish beam. These functions all convolve against each other to yield an effective image resolution somewhat poorer than the actual theoretical FWHM of the telescope primary beam.

The dimensions and extent of the image grid are determined by the mapped area on the sky. The gridding pixel size must be at least one half the size of the theoretical beam when convolved with the sky-sampling function. Since the sky sampling function is typically 1/3 to 1/5 of the primary beam, and the effective FWHM of the telescope and sky sampling function is close to that of the telescope anyway, it\'s safe to use a pixel dimension that is 1/3th the width of the primary beam.

For example, a 30\" telescope beam with a 6\" sky sampling function has an effective FWHM of $\sim \sqrt{(30^2+6^2)}\simeq$ 30.6\". Therefore, computing an image pixel size that is 30\"/3 = 10\", is appropriately oversampling the effective beam FWHM and sampling interval.

After the coordinates of the data are transformed into sky coordinates, the image grid is formed with dimensions either consistent with the user specifications, or so that the image fully encompasses the observed sky positions.  

For each pixel in the grid (e.g. in RA-Dec space), the gridding process searches through the data for measurements taken within some cutoff radius (specified by *convsupport*). Depending on their distance from the grid coordinate, the observation is weighted according to the *kernel* type and added together in the spatial domain (i.e. entire spectra are added together). If the *clipminmax* function is invoked, the maximum AND minimium data in the ensemble (prior to weighting) are rejected before summing. This process is repeated iteratively for each element in the grid.

 



##### Bibliography

1. Mangum,\ et\ al.\ 2007,\ A&A,\ 474,\ 679-687\ (
^



***





#### Procedure SD Imaging 

The procedure for single-dish image generation and data gridding in CASA

CASA uses the [sdimaging](global-task-list.ipynb#task_sdimaging) task to grid a single-dish image or cube. While the steps are described in detail here, an example of the full single-dish calibration and imaging processes can be found in the [M100 Band 3 Single Dish CASAguide](https://casaguides.nrao.edu/index.php/M100_Band3_SingleDish_4.3).

##### Default operation

The **sdimaging** task can determine and populate almost all the gridding parameters by default.  Simply invoking **sdimaging** with the single-dish MeasurementSet and output file name will work. This will produce a single, potentially very large cube having as many channels as necessary to span the entire spectral range, with a spectral resolution equal to that of the observation.

```
sdimaging(infiles=sd_ms+'.bl',outfile=imagename)
```

The default parameter choices for imaging are selected as follows: the image pixel size is 1/3 the primary beam, the primary beam itself is computed based on the standard $\frac{\lambda}{D}$, with some empirically-validated tapering applied. The image dimensions are determined by the spatial extent of the mapped area in the observation, and by default, all channels and all spectral windows are imaged, along with all antennas and all polarizations.

##### Customized operation

Of course, users can tune their data products by specifying the image size and dimensions, the frequency/velocity characteristics, the gridding and data filtering and smoothing parameters, and so on. The defaults for **sdimaging** for image resolution (i.e. cellsize in arcsec) is determined from the rest frequency of the 0th spectral window so that there are three pixel elements across the beam, the beam being calculated with $b\times\frac{\lambda}{D}$.  See information here about tapering: [PrimaryBeamArcsec](https://safe.nrao.edu/wiki/bin/view/ALMA/PrimaryBeamArcsec). The image extent is computed by default so that the sampled area is completely encompassed in a single rectangle, and the pixel dimension follows naturally from maxsize/cellsize. The default image center (the somewhat inappropriately-named *phasecenter* parameter) is computed simply as the center of that region.

These parameters can be left to be determined by **sdimaging**, or they can be determined using CASA tools.

 

##### Image dimensions and pixel interval

The image extent can be explicitly determined using **aU.getTPSampling**:

```
xSampling, ySampling, maxsize = aU.getTPSampling(refvis, showplot=True)
```

which returns an image output showing the scans, their sky angles, and positions in RA-Dec, as shown here:

![683703caa9ae3cec3d89b02760ba73df66041b2d](media/683703caa9ae3cec3d89b02760ba73df66041b2d.png){.image-inline}

Note that **getTPSampling** MUST operate on the original MeasurementSet (i.e. one that is not split or subselected). **getTPSampling** also yields the sampling rates in the x and y (i.e. azimuth and elevation) axes, as well as the maximum size of the image, in arcseconds.

The beam size used by **sdimaging** is determined using the **aU.primaryBeamArcsec** task, though this can also be invoked by the user and used to compute, for example, a cellsize and image size. The default for **aU.primaryBeamArcsec** corresponds to a 12m antenna with normal tapering. Setting the *fwhmfactor* modifies the beam taper (see discussion in [PrimaryBeamArcsec](https://safe.nrao.edu/wiki/bin/view/ALMA/PrimaryBeamArcsec)).

```
freq=115.27e+9
fwhmfactor=1.13
diameter=12
theorybeam = aU.primaryBeamArcsec(frequency=freq*1e-9, fwhmfactor=fwhmfactor, diameter=diameter)
cell = theorybeam/9.0
imsize = int(round(maxsize/cell)*2)
```

The center of the image can be modified using the *phasecenter* parameter. Single-dish images actually have many phase centers, so the name is somewhat misleading. However it is preserved here for consistency with the interferometer terminology. In the context of single-dish data, *phasecenter* refers only to coordinates that will align with the center of the image, and this can be in J2000 or Az/El, e.g.

```
phasecenter='J2000 12h22m54.9 +15d49m15'
```

 

##### Frequency and/or velocity axis

The default rest frequency is the mean frequency of the first spectral window (i.e. that having the lowest spectral window ID). Of course it can instead be set by the user, or a different spectral window frequency can be used, extracted from the data using **msmd** tools:

```
msmd.open(vislist[0])
freq = msmd.meanfreq(spw)
msmd.close()
print "SPW %d: %.3f GHz" % (spw, freq*1e-9)
```

The third axis of the image cube can be specified using the *veltype* and *outframe* parameters. Many spectral-line observers will prefer to change these so the output has a velocity axis in the radio convention as follows:

```
veltype='radio',
outframe='lsrk',
```

and the rest frequency can be specified with:

```
restfreq='115.271204GHz'
```

The velocity extent of the image cube can be specified by selecting a spectral window (via the *spw* parameter), the channel range (via the *nchan* and *start* parameters), and the frequency/velocity resolution (via the *width* parameter). For example:

```
nchan=70,
mode='velocity',
start='1400km/s',
width='5km/s',
```

#####  Gridding parameters

The gridding kernel defaults to a box shape, but it can be specified as a spherical (\'SF\'), Primary beam (\'PB\'), Gaussian (\'GAUSS\') or Gaussian\*Jinc (GJINC). The recommended setting for ALMA data is a spherical (\'SF\') kernel. The *convsupport* parameter defines the cut-off radius for \'SF\' in units of pixels, defaulting to 3 pixels.  However, the recommended value for ALMA data is convsupport=6 (see [sdimaging](global-task-list.ipynb#task_sdimaging) and Mangum et al. 2007 [\[1\]](#Bibliography) for more details on these parameters).

The parameter *stokes* specifies the stokes product. At present, the weighting for stokes I is computed consistently with historical usage: I=XX/2+YY/2.  While this is mathematically consistent with the computation of stokes I, it is an incorrect treatment since the computation necessarily must incorporate the contributions from Q and U. Ordinarily, these terms cancel out from the computation of stokes I, but their error parameters must be incorporated, and historically, this is not respected. 

CASA development is seeking to make the computation of the weights consistent with a proper computation of stokes I, and this is done in **sdfit**, but it is not yet completed for **sdimaging. ** However, to emphasize, while the current implementation of computation for stokes I by **sdimaging** is consistent with convention, the convention is formally incorrect.

 

##### Example script

Fully specified, a call to **sdimaging** might look like the following:

```
sdimaging(infiles=sd_ms+'.bl',
    field='M42',
    spw='%s'%(spw),
    nchan=70,
    mode='velocity',
    start='1400km/s',
    width='5km/s',
    veltype='radio',
    outframe='lsrk',
    restfreq='%sGHz'%(freq/1e+9),
    gridfunction='SF',
    convsupport=6,
    stokes='I',
    phasecenter='J2000 12h22m54.9 +15d49m15',
    ephemsrcname='',
    imsize=imsize,
    cell='%sarcsec'%(cell),
    overwrite=True,
    outfile=imagename)
```

The products here are the image data, returned in the variable \'imagename\', and also a map of weights: \<imagename\>.weight. The weights indicate the robustness of the gridded data on a per-pixel basis, and are important when performing further computations and analysis with the image products.

 



##### Bibliography

1. Mangum,\ et\ al.\ 2007,\ A&A,\ 474,\ 679-687\ (
^



***





### Image Combination 

Introduction to combining images in CASA



***





#### Joint Single Dish and Interferometer Image Reconstruction 

This is a description of the joint single dish and interferometer image reconstruction algorithm within CASA 5.7/6.1.

###### Joint reconstruction of wideband single dish and interferometer data in CASA is [[experimental](casa-fundamentals.ipynb#tasks-and-tools). Please use at own discretion.]

The scope of parameters that has been tested for CASA 5.7/6.1 can be found below.

 

##### Overview

[The SDINT imaging algorithm allows joint reconstruction of wideband single dish and interferometer data. This algorithm is available in the task [**sdintimaging**](global-task-list.ipynb#task_sdintimaging)] and described in [Rau, Naik & Braun (2019)](https://iopscience.iop.org/article/10.3847/1538-3881/ab1aa7/meta).

 

###### Algorithm 

 

Interferometer data are gridded into an image cube (and corresponding PSF). The single dish image and PSF cubes are combined with the interferometer cubes in a feathering step. The joint image and PSF cubes then form inputs to any deconvolution algorithm (in either *cube* or *mfs/mtmfs* modes). Model images from the deconvolution algorithm are translated back to model image cubes prior to subtraction from both the single dish image cube as well as the interferometer data to form a new pair of residual image cubes to be feathered in the next iteration. In the case of mosaic imaging, primary beam corrections are performed per channel of the image cube, followed by a multiplication by a common primary beam, prior to deconvolution. Therefore, for mosaic imaging, this task always implements *conjbeams=True* and *normtype='flatnoise'*.

 

 ![c914c39a74a69699c2ae1d84231e2133af6d7081](media/c914c39a74a69699c2ae1d84231e2133af6d7081.png){.image-inline width="674" height="378"}

 

 

[The input single dish data are the single dish image and psf cubes. The input interferometer data is a MeasurementSet. In addition to imaging and deconvolution parameters from interferometric imaging (task **tclean**), there are controls for a feathering step to combine interferometer and single dish cubes within the imaging iterations. Note that the above diagram shows only the \'mtmfs\' variant. Cube deconvolution proceeds directly with the cubes in the green box above, without the extra conversion back and forth to the multi-term basis. Primary beam handling is also not shown in this diagram, but full details (via pseudocode) are available in the [reference publication.](https://iopscience.iop.org/article/10.3847/1538-3881/ab1aa7)]

[The parameters used for controlling the joint deconvolution are described on the [**sdintimaging**](global-task-list.ipynb#task_sdintimaging) task pages.]

 

##### Task Specification : sdintimaging 

 

The task [**sdintimaging**](global-task-list.ipynb#task_sdintimaging) contains the algorithm for joint reconstruction of wideband single dish and interferometer data. The **sdintimaging** task shares a significant number of parameters with the **tclean** task, but also contains unique parameters. A detailed overview of these parameters, and how to use them, can be found in the CASA Docs [task pages of sdintimaging](global-task-list.ipynb#task_sdintimaging).

 

##### Usage Modes

[As seen from the diagram above and described on the [**sdintimaging**](global-task-list.ipynb#task_sdintimaging) task pages, there is considerable flexibility in usage modes. One can choose between interferometer-only, singledish-only and joint interferometer-singledish imaging. Outputs are restored images and associated data products (similar to task tclean).]

The following usage modes will be released in the (experimental) sdintimaging task for CASA 6.1/5.7 . Modes being tested are all 12 combinations of :

-   Cube Imaging :  All 6 combinations of the following options.
    -   *specmode = \'cube\' *
    -   *deconvolver = \'multiscale\', \'hogbom\'        *
    -   *usedata = \'sdint\', \'sd\' , \'int\'   *
    -   *gridder = \'standard\', \'mosaic\'   *
    -   *parallel = False*

```{=html}
<!-- -->
```
-   Wideband Multi-Term Imaging :  All 6 combinations of the following options.    -   *specmode = \'mfs\' *
    -   *deconvolver = \'mtmfs\' * ( *nterms=1 * for a single-term MFS image, and *nterms\>1* for multi-term MFS image. Tests use *nterms=2* )
    -   *usedata = \'sdint\', \'sd\' , \'int\'*
    -   *gridder = \'standard\', \'mosaic\' *
    -   *parallel = False*

<div class="alert alert-info">
**NOTE**: When the INT and/or SD cubes have flagged (and therefore empty) channels, only those channels that have non-zero images in both the INT and SD cubes are used for the joint reconstruction.
</div>

<div class="alert alert-info">
**NOTE**: Single-plane joint imaging may be run with deconvolver='mtmfs' and nterms=1.
</div>

<div class="alert alert-info">
**NOTE**: All other modes allowed by the new sdintimaging task are untested as of CASA 6.1. Tests will be added in subsequent releases. Please see the Future Work section at the bottom of this page.
</div>

 

 

##### Test Results 

 

The sdintimaging task was run on a pair of simulated test datasets. Both contain a flat spectrum extended emission feature plus three point sources, two of which have spectral index=-1.0 and one which is flat-spectrum (rightmost point). The scale of the top half of the extended structure was chosen to lie within the central hole in the spatial-frequency plane at the middle frequency of the band so as to generate a situation where the interferometer-only imaging is difficult.

Please refer to the [publication](https://iopscience.iop.org/article/10.3847/1538-3881/ab1aa7/meta) for a more detailed analysis of the imaging quality and comparisons of images without and with SD data. 

Images from a run on the ALMA M100 12m+7m+TP Science Verification Data suite are also shown below.


###### Single Pointing Simulation :

Wideband Multi-Term Imaging ( deconvolver=\'mtmfs\', specmode=\'mfs\' )

<table style="height: 576px;" width="665"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td>SD + INT<br />
<p> A joint reconstruction accurately reconstructs both intensity and spectral index for the extended emission as well as the compact sources.</p></td><td><p><img src="markdown/_media/15c3a0c5553e9880865432e38820c5f165a3aa9d.png" title="fig.try_standard_mfs_mtmfs_sdint.png" class="image-inline" width="416" height="160" /></p></td></tr><tr class="even"><td><p>INT-only</p><p>The intensity has negative bowls and the spectral index is overly steep, especially for the top half of the extended component.</p></td><td><p><img src="markdown/_media/ecab604d0c63ea17708915f25dc865d123f4fb86.png" title="fig.try_standard_mfs_mtmfs_int.png" class="image-inline" width="417" height="160" /></p></td></tr><tr class="odd"><td><p>SD-only</p><p>The spectral index of the extended emission is accurate (at 0.0) and the point sources are barely visible at this SD angular resolution.</p></td><td><p><img src="markdown/_media/2ad31ce78fc986e25620147d69ab1ae858cc33ec.png" title="fig.try_standard_mfs_mtmfs_sd.png" class="image-inline" width="414" height="159" /></p></td></tr></tbody></table>

 

Cube Imaging ( deconvolver=\'multiscale\', specmode=\'cube\' )

<table style="height: 754px;" width="728"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td>SD + INT<br />
<p> A joint reconstruction has lower artifacts and more accurate intensities in all three channels, compared to the int-only reconstructions below</p></td><td><p><img src="markdown/_media/93fb162b2d84a30a1ee671e0176c6eecedb042ee.png" title="fig.try_standard_cube_multiscale_sdint.png" class="image-inline" width="614" height="236" /></p></td></tr><tr class="even"><td><p>INT-only</p><p>The intensity has negative bowls in the lower frequency channels and the extended emission is largely absent at the higher frequencies.</p></td><td><p><img src="markdown/_media/4052cd1ba39e9e48fe52f0aacafaa91fa7c140d3.png" title="fig.try_standard_cube_multiscale_int.png" class="image-inline" width="596" height="229" /></p></td></tr><tr class="odd"><td><p>SD-only</p><p>A demonstration of single-dish cube imaging with deconvolution of the SD-PSF.</p><p>In this example, iterations have not been run until full convergence, which is why the sources still contain signatures of the PSF.</p></td><td><p><img src="markdown/_media/bba19f9a48a3c11588d60f71085d9880004ff3cf.png" title="fig.try_standard_cube_multiscale_sd.png" class="image-inline" width="591" height="227" /></p></td></tr></tbody></table>

 

 

###### Mosaic Simulation 

 

An observation of the same sky brightness was simulated with 25 pointings.

 

Wideband Multi-Term Mosaic Imaging ( deconvolver=\'mtmfs\', specmode=\'mfs\' , gridder=\'mosaic\' )

<table style="height: 437px;" width="704"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td>SD + INT<br />
<p> A joint reconstruction accurately reconstructs both intensity and spectral index for the extended emission as well as the compact sources.</p><p>This is a demonstration of joint mosaicing along with wideband single-dish and interferometer combination.</p></td><td><p><img src="markdown/_media/21c249a45ca04b60e8285d94b8d4f0d161aebeb6.png" title="fig.try_mosaic_mfs_mtmfs_sdint.png" class="image-inline" width="518" height="199" /></p></td></tr><tr class="even"><td><p>INT-only</p><p>The intensity has negative bowls and the spectral index is strongly inaccurate.   Note that the errors are slightly less than the situation with the single-pointing example (where there was only one pointing's worth of uv-coverage).</p></td><td><p><img src="markdown/_media/3b1edcbf6554f3fb8842b49238e910caba4d9d87.png" title="fig.try_mosaic_mfs_mtmfs_int.png" class="image-inline" width="518" height="199" /></p></td></tr></tbody></table>

 

Cube Mosaic Imaging ( *deconvolver=\'multiscale\', specmode=\'cube\' , gridder=\'mosaic\'* )

<table style="height: 489px;" width="780"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td>SD + INT<br />
<p> A joint reconstruction produces better per-channel reconstructions compared to the INT-only situation shown below.</p><p>This is a demonstration of cube mosaic imaging along with SD+INT joint reconstruction.</p></td><td><p><img src="markdown/_media/43924cabac57bbd4b9c4e6c40686c4bc43ce8306.png" title="fig.try_mosaic_cube_multiscale_sdint.png" class="image-inline" width="631" height="242" /></p></td></tr><tr class="even"><td><p>INT-only</p><p>Cube mosaic imaging with only interferometer data. This clearly shows negative bowls and artifacts arising from the missing flux.</p></td><td><p><img src="markdown/_media/3f9258cd3d4661d3447a9e7b39a96cf8fc62a2c5.png" title="fig.try_mosaic_cube_multiscale_int.png" class="image-inline" width="640" height="246" /></p></td></tr></tbody></table>

 

 

###### Other Tests :  ALMA M100  Spectral Cube Imaging : 12m + 7m + TP 

 

The sdintimaging task was run on the [ALMA M100 Science Verification Datasets](https://almascience.nrao.edu/alma-data/science-verification).

\(1\) The single dish (TP) cube was pre-processed by adding per-plane restoringbeam information.

\(2\) Cube specification parameters were obtained from the SD Image as follows

> from sdint_helper import \*> sdintlib = SDINT_helper()> sdintlib.setup_cube_params(sdcube=\'M100_TmP\')

> Output : [Shape of SD cube : \[90 90  1 70\]]> [Coordinate ordering : \[\'Direction\', \'Direction\', \'Stokes\', \'Spectral\'\]]> nchan = 70> start = 114732899312.0Hz> width = -1922516.74324Hz> Found 70 per-plane restoring beams\#> > (For specmode=\'mfs\' in sdintimaging, please remember to set \'reffreq\' to a value within the freq range of the cube.)> > Returned Dict : {\'nchan\': 70, \'start\': \'114732899312.0Hz\', \'width\': \'-1922516.74324Hz\'}

 

\(3\) Task sdintimaging was run with automatic SD-PSF generation, n-sigma stopping thresholds, a pb-based mask at the 0.3 gain level, and no other deconvolution masks (interactive=False).

> sdintimaging(usedata=\"sdint\", sdimage=\"../M100_TP\", sdpsf=\"\",sdgain=3.0, dishdia=12.0, vis=\"../M100_12m_7m\", imagename=\"try_sdint_niter5k\", imsize=1000, cell=\"0.5arcsec\", phasecenter=\"J2000 12h22m54.936s +15d48m51.848s\", stokes=\"I\", specmode=\"cube\", reffreq=\"\", nchan=70, start=\"114732899312.0Hz\", width=\"-1922516.74324Hz\", outframe=\"LSRK\", veltype=\"radio\", restfreq=\"115.271201800GHz\", interpolation=\"linear\", chanchunks=1, perchanweightdensity=True, gridder=\"mosaic\", mosweight=True, pblimit=0.2, deconvolver=\"multiscale\", scales=\[0, 5, 10, 15, 20\], smallscalebias=0.0, pbcor=False, weighting=\"briggs\", robust=0.5, niter=5000, gain=0.1, threshold=0.0, nsigma=3.0, interactive=False, usemask=\"user\", mask=\"\", pbmask=0.3)

 

**Results from two channels are show below. **

LEFT : INT only (12m+7m)    and  RIGHT : SD+INT (12m + 7m + TP)

 

Channel 23

![18445a5ddbc066530938f1b8712e3a68bf9b8e3a](media/18445a5ddbc066530938f1b8712e3a68bf9b8e3a.png){.image-inline width="435" height="253"}

Channel 43

 

![f7c37345f62846af242938430ef9287b6b466fd4](media/f7c37345f62846af242938430ef9287b6b466fd4.png){.image-inline width="428" height="249"}

 

Moment 0 Maps :   LEFT :  INT only.        MIDDLE :   SD + INT with sdgain=1.0          RIGHT : SD + INT with sdgain=3.0

 

![d38c8835a149a2f61fcbeb77ee3d4f3eb04d6962](media/d38c8835a149a2f61fcbeb77ee3d4f3eb04d6962.png){.image-inline}

 

Moment 1 Maps :   LEFT :  INT only.        MIDDLE :   SD + INT with sdgain=1.0          RIGHT : SD + INT with sdgain=3.0

![24348b162f7e4fc3ab4b71d12f80f15f361954c6](media/24348b162f7e4fc3ab4b71d12f80f15f361954c6.png){.image-inline}

 

A comparison (shown for one channel) with and without masking is shown below.

![6e766bca3645b467ecae383e948f7e688aeee11d](media/6e766bca3645b467ecae383e948f7e688aeee11d.png){.image-inline}

 

Notes : 

-   In the reconstructed cubes, negative bowls have clearly been eliminated by using sdintimaging to combine interferometry + SD data.  Residual images are close to noise-like too (not pictured above) suggesting a well-constrained and steadily converging imaging run.  

```{=html}
<!-- -->
```
-   The source structure is visibly different from the INT-only case, with high and low resolution structure appearing more well defined.  However, the *high-resolution* peak flux in the SDINT image cube is almost a factor of 3 lower than the INT-only. While this may simply be because of deconvolution uncertainty in the ill-constrained INT-only reconstruction, it requires more investigation to evaluate absolute flux correctness.  For example, it will be useful to evaluate if the INT-only reconstructed flux changes significantly with careful hand-masking.
    -   Compare with a Feathered image : http://www.astroexplorer.org/details/apjaa60c2f1   : The reconstructed structure is consistent.

```{=html}
<!-- -->
```
-   The middle and right panels compare reconstructions with different values of sdgain (1.0 and 3.0).   The sdgain=3.0 run has a noticeable emphasis on the SD flux in the reconstructed moment maps, while the high resolution structures have the same are the same between sdgain=1 and 3.  This is consistent with expectations from the algorithm, but requires further investigation to evaluate robustness in general.

```{=html}
<!-- -->
```
-   Except for the last panel, no deconvolution masks were used (apart from a *pbmask* at the 0.3 gain level). The deconvolution quality even without masking is consistent with the expectation that when supplied with better data constraints in a joint reconstruction, the native algorithms are capable of converging on their own. In this example (same *niter* and *sdgain*), iterative cleaning with interactive and auto-masks (based mostly on interferometric peaks in the images) resulted in more artifacts compared to a run that allowed multi-scale clean to proceed on its own.

```{=html}
<!-- -->
```
-   The results using sdintimaging on these ALMA data can be compared with performance results when [using feather](https://casaguides.nrao.edu/index.php?title=M100_Band3_Combine_5.4), and when [using tp2vis](https://science.nrao.edu/facilities/alma/alma-develop-old-022217/tp2vis_final_report.pdf) (ALMA study by J. Koda and P. Teuben).

 

 

 

The following is a list of use cases that have simulation-based functional verification tests within CASA.

<table><colgroup><col style="width: 33%" /><col style="width: 33%" /><col style="width: 33%" /></colgroup><tbody><tr class="odd"><td>1</td><td>Wideband mulit-term  imaging (SD+Int)</td><td><p>Wideband data single field imaging by joint-reconstruction from single dish and interferometric data to obtain the high resolution of the interferometer while account for the zero spacing information. Use multi-term multi-frequency synthesis (MTMFS) algorithm to properly account for spectral information of the source.</p></td></tr><tr class="even"><td>2</td><td>Wideband multi-term imaging: Int only</td><td>The same as #1 except for using interferometric data only, which is useful to make a comparison with #1 (i.e. effect of missing flux). This is equivalent to running 'mtmfs' with specmode='mfs' and gridder='standard' in tclean</td></tr><tr class="odd"><td>3</td><td>Wideband multi-term imaging: SD only</td><td>The same as #1 expect for using single dish data only which is useful to make a comparison with #1 (i.e. to see how much high resolution information is missing).  Also, sometimes, the SD PSF has significant sidelobes (Airy disk) and even single dish images can benefit from deconvolution. This is a use case where wideband multi-term imaging is applied to SD data alone to make images at the highest possible resolution as well as to derive spectral index information.  </td></tr><tr class="even"><td>4</td><td>Single field cube  imaging: SD+Int</td><td><p>Spectral cube single field imaging by joint reconstruction of single dish and interferometric data to obtain single field spectral cube image.</p><p>Use multi-scale clean for deconvolution</p></td></tr><tr class="odd"><td>5</td><td>Single field cube imaging: Int only</td><td>The same as #4 except for using the interferometric data only, which is  useful to make a comparison with #4 (i.e. effect of missing flux). This is equivalent to running 'multiscale' with specmode='cube' and gridder='standard' in tclean.</td></tr><tr class="even"><td>6</td><td>Single field cube imaging: SD only</td><td><p>The same as #4 except for using the single dish data only, which is useful to make a comparison with #4</p><p>(i.e. to see how much high resolution information is missing)</p><p>Also, it addresses the use case where SD PSF sidelobes are significant and where the SD images could benefit from multiscale (or point source) deconvolution per channel.</p></td></tr><tr class="odd"><td>7</td><td><span title="">Wideband multi-term mosaic Imaging: SD+Int<br />
</span></td><td><p>Wideband data mosaic imaging by joint-reconstruction from single dish and interferometric data to obtain the high resolution of the interferometer while account for the zero spacing information.</p>Use multi-term multi-frequency synthesis (MTMFS) algorithm to properly account for spectral information of the source. Implement the concept of conjbeams (i.e. frequency dependent primary beam correction) for wideband mosaicing.</td></tr><tr class="even"><td>8</td><td><span title="">Wideband multi-term mosaic imaging: Int only<br />
</span></td><td>The same as #7 except for using interferometric data only, which is useful to make a comparison with #7 (i.e. effect of missing flux). Also, this is an alternate implementation of the concept of conjbeams ( frequency dependent primary beam correction) available via tclean, and which is likely to be more robust to uv-coverage variations  (and sumwt) across frequency. </td></tr><tr class="odd"><td>9</td><td>Wideband multi-term mosaic imaging: SD only</td><td>The same as #7 expect for using single dish data only which is useful to make a comparison with #7 (i.e. to see how much high resolution information is missing).  This is the same situation as (3) , but made on an image coordinate system that matches an interferometer mosaic mtmfs image.</td></tr><tr class="even"><td>10</td><td>Cube mosaic imaging: SD+Int</td><td><p>Spectral cube mosaic imaging by joint reconstruction of single dish and interferometric data.</p>Use multi-scale clean for deconvolution. </td></tr><tr class="odd"><td>11</td><td>Cube mosaic imaging: Int only</td><td>The same as #10 except for using the intererometric data only, which is useful to make a comparison with #10 (i.e. effect of missing flux).  This is the same use case as gridder='mosaic' and deconvolver='multiscale' in tclean for specmode='cube'.</td></tr><tr class="even"><td><p>12</p></td><td>Cube mosaic imaging: SD only</td><td><p>The same as #10 except for using the single dish data only, which is useful to make a comparison with #10 (i.e. to see how much high resolution information is missing).  This is the same situation as (6), but made on an image coordinate system that matches an interferometer mosaic cube image.</p></td></tr><tr class="odd"><td><p>13</p></td><td>Wideband MTMFS SD+INT with channel 2 flagged in INT</td><td><p>The same as #1, but with partially flagged data in the cubes. This is a practical reality with real data where the INT and SD data are likely to have gaps in the data due to radio frequency interferenece or other weight variations. </p></td></tr><tr class="even"><td> 14</td><td>Cube SD+INT with channel 2 flagged</td><td><p>The same as #4, but with partially flagged data in the cubes. This is a practical reality with real data where the INT and SD data are likely to have gaps in the data due to radio frequency interferenece or other weight variations.  </p></td></tr><tr class="odd"><td>15</td><td>Wideband MTMFS SD+INT with sdpsf=""</td><td><p>The same as #1, but with an unspecified sdpsf. This triggers the auto-calculation of the SD PSF cube using restoring beam information from the regridded input sdimage.</p></td></tr></tbody></table>


##### Future work 

 

For future work and a summary of the Code Design, please see the [\"Developer\"](global-task-list.ipynb#developer) tab of the[[ [**sdintimaging**](global-task-list.ipynb#task_sdintimaging) task.] ]

  

##### References

 [Urvashi Rau, Nikhil Naik, and [Timothy Braun 2019 [AJ 158, 1](https://iopscience.iop.org/article/10.3847/1538-3881/ab1aa7/meta).^^]{.nowrap}]{authors=""}

https://github.com/urvashirau/WidebandSDINT



***





#### Feather & CASAfeather 

Combining single dish and interferometric images

[Feathering is a technique used to combine a Single Dish (SD) image with an interferometric image of the same field.The goal of this process is to reconstruct the source emission on all spatial scales, ranging from the small spatial scales measured by the interferometer to the large-scale structure measured by the single dish.  To do this, feather combines the images in Fourier space, weighting them by the spatial frequency response of each image. This technique assumes that the spatial frequencies of the single dish and interferometric data partially overlap. The subject of interferometric and single dish data combination has a long history. See the introduction of Koda et al 2011 (and references therein) [\[1\]](#Bibliography) for a concise review, and Vogel et al 1984 [\[2\]](#Bibliography), Stanimirovic et al 1999 [\[3\]](#Bibliography), Stanimirovic 2002 [\[4\]](#Bibliography), Helfer et al 2003 [\[5\]](#Bibliography), and Weiss et al 2001 [\[6\]](#Bibliography), among other referenced papers, for other methods and discussions concerning the combination of single dish and interferometric data.]The feathering algorithm implemented in CASA is as follows: 

1.  Regrid the single dish image to match the coordinate system, image shape, and pixel size of the high resolution image.    
2.  Transform each image onto uniformly gridded spatial-frequency axes.
3.  Scale the Fourier-transformed low-resolution image by the ratio of the volumes of the two \'clean beams\' (high-res/low-res) to convert the single dish intensity (in Jy/beam) to that corresponding to the high resolution intensity (in Jy/beam). The volume of the beam is calculated as the volume under a two dimensional Gaussian with peak 1 and major and minor axes of the beam corresponding to the major and minor axes of the Gaussian.    
4.  Add the Fourier-transformed data from the  high-resolution image, scaled by $(1-wt)$ where $wt$ is the Fourier transform of the \'clean beam\' defined in the low-resolution image, to the scaled low resolution image from step 3.5.  Transform back to the image plane.

The input images for feather must have the following characteristics:

1.   Both input images must have a well-defined beam shape for this task to work, which will be a \'clean beam\' for interferometric images and a \'primary-beam\'  for a single-dish image. The beam for each image should be specified in the image header. If a beam is not defined in the header or feather cannot guess the beam based on the telescope parameter in the header, then you will need to add the beam size to the header using **imhead**.    
2.  Both input images must have the same flux density normalization scale. If necessary, the SD image should be converted from temperature units to Jy/beam. Since measuring absolute flux levels is difficult with single dishes, the single dish data is likely to be the one with the most uncertain flux calibration. The SD image flux can be scaled using the parameter *sdfactor* to place it on the same scale as the interferometer data. The casafeather task (see below) can be used to investigate the relative flux scales of the images.*    *

[Feather attemps to regrid the single dish image to the interferometric image. Given that the single dish image frequently originates from other data reduction packages, CASA may have trouble performing the necessary regridding steps. If that happens, one may try to regrid the single dish image manually to the interferometric image. CASA has a few tasks to perform individual steps, including **imregrid** for coordinate transformations, **imtrans** to swap and reverse coordinate axes, the tool **ia.adddegaxes()** for adding degenerate axes (e.g. a single Stokes axis). See the \"[Image Analysis](imaging.ipynb#image-analysis)\" chapter for additional options. If you have trouble changing image projections, you can try ][the [montage package](http://montage.ipac.caltech.edu/), which also has an [associated python wrapper](http://www.astropy.org/montage-wrapper/).]

If you are feathering large images together, set the numbers of pixels along the X and Y axes to composite (non-prime) numbers in order to improve the algorithm speed. In general, FFTs work much faster on even and composite numbers. Then use the subimage task or tool to trim the number of pixels to something desirable.

The inputs for **feather** are: 

```
#feather :: Combine two images using their Fourier transforms
imagename       = ''     #Name of output feathered image
highres         = ''     #Name of high resolution (interferometer) image
lowres          = ''     #Name of low resolution (single dish) image
sdfactor        = 1.0    #Scale factor to apply to Single Dish image
effdishdiam     = -1.0   #New effective SingleDish diameter to use in m
lowpassfiltersd = False  #Filter out the high spatial frequencies of the SD image
```

The SD data cube is specified by the *lowres* parameter and the interferometric data cube by the *highres* parameter. The combined, feathered output cube name is given by the *imagename* parameter. The parameter *sdfactor* can be used to scale the flux calibration of the SD cube. The parameter *effdishdiam* can be used to change the weighting of the single dish image.

The weighting functions for the data are usually the Fourier transform of the Single Dish beam FFT(PB~SD~) for the Single dish data, and the inverse, 1-FFT(PB~SD~), for the interferometric data. It is possible, however, to change the weighting functions by pretending that the SD is smaller in size via the *effdishdiam* parameter. This tapers the high spatial frequencies of the SD data and adds more weight to the interferometric data. The *lowpassfiltersd* can take out non-physical artifacts at very high spatial frequencies that are often present in SD data.

Note that the only inputs are for images; **feather** will attempt to regrid the images to a common shape, i.e. pixel size, pixel numbers, and spectral channels. If you are having issues with the regridding inside feather, you may consider regridding using the **imregrid** and **specsmooth** tasks.

The **feather** task does not perform any deconvolution but combines the single dish image with a presumably deconvolved interferometric image. The short spacings of the interferometric image that are extrapolated by the deconvolution process will be those that are down-weighted the most when combined with the single dish data. The single dish image must have a well-defined beam shape and the correct flux units for a model image (Jy/beam instead of Jy/pixel). Use the tasks **imhead** and **immath** first to convert if needed.

Starting with a cleaned synthesis image and a low resolution image from a single dish telescope, the following example shows how they can be feathered: 

```
feather(imagename ='feather.im',       #Create an image called feather.im
        highres   ='synth.im',         #The synthesis image is called synth.im
        lowres    ='single_dish.im')   #The SD image is called single_dish.im
```

###### Visual Interface for feather (casafeather) 

CASA also provides a visual interface to the **feather** task. The interface is run from a command line *outside* CASA by typing casafeather in a shell. An example of the interface is shown below. To start, one needs to specify a high and a low resolution image, typically an interferometric and a single dish map. Note that the single dish map needs to be in units of Jy/beam. The output image name can be specified. The non-deconvolved (dirty) interferometric image can also be specified to use as diagnostic of the relative flux scaling of the single dish and interferometer images. See below for more details. At the top of the display, the parameters *effdshdiameter* and *sdfactor* can be provided in the "Effective Dish Diameter" and "Low Resolution Scale Factor" input boxes. One you have specified the images and parameters, press the "Feather" button in the center of the GUI window to start the feathering process. The feathering process here includes regridding the low resolution image to the high resolution image.

[![c0ff299b0bd9c0afa9b65a93c6b02212362645d3](media/c0ff299b0bd9c0afa9b65a93c6b02212362645d3.png){.image-inline}]



>Figure 1:  The panel shows the "Original Data Slice", which are cuts through the u and v directions of the Fourier-transformed input images. Green is the single dish data (low resolution) and purple the interferometric data (high resolution). To bring them on the same flux scale, the low data were convolved to the high resolution beam and vice versa (selectable in color preferences). In addition, a single dish scaling of 1.2 was applied to adjust calibration differences. The weight functions are shown in yellow (for the low resolution data) and orange (for the high resolution data). The weighting functions were also applied to the green and purple slices. Image slices of the combined, feathered output image are shown in blue. The displays also show the location of the effective dish diameter by the vertical line. This value is kept at the original single dish diameter that is taken from the respective image header.
  

 

------------------------------------------------------------------------

The initial casafeather display shows two rows of plots. The panel shows the "Original Data Slice", which are either cuts through the u and v directions of the Fourier-transformed input images or a radial average. A vertical line shows the location of the effective dish diameter(s). The blue lines are the combined, feathered slices.

 

[[ ![c57e182275861b522d1e6836eab16a853d7aae7c](media/c57e182275861b522d1e6836eab16a853d7aae7c.png)]]

>Figure 2:  The casafeather "customize" window.
  

 

The \'Customize\' button (gear icon on the top menu page) allows one to set the display parameters. Options are to show the slice plot, the scatter plot, or the legend. One can also select between logarithmic and linear axes; a good option is usually to make both axes logarithmic. You can also select whether the x-axis for the slices are in the u, or v, or both directions, or, alternatively a radial average in the uv-plane. For data cubes, one can also select a particular velocity plane, or to average the data across all velocity channels. The scatter plot can display any two data sets on the two axes, selected from the \'Color Preferences\' menu. The data can be the unmodified, original data, or data that have been convolved with the high or low resolution beams. One can also select to display data that were weighted and scaled by the functions discussed above.

 ![df8181251aae5df396fe516f5befe53d616680da](media/df8181251aae5df396fe516f5befe53d616680da.png){.image-inline}

>Figure 3:  The scatter plot in casafeather. The low data, convolved with high beam, weighted and scaled is still somewhat below the equality line (plotted against high data, convolved with low beam, weighted). In this case one can try to adjust the \"low resolution scale factor\" to bring the values closer to the line of equality, ie. to adjust the calibration scales. 
  

Plotting the data as a scatter plot is a useful diagnostic tool for checking for differences in flux scaling between the high and low resolution data sets.The dirty interferometer image contains the actual flux measurements made by the telescope. Therefore, if the single dish scaling is correct, the flux in the dirty image convolved with the low resolution beam and with the appropriate weighting applied should be the same as the flux of the low-resolution data convolved with the high resolution beam once weighted and scaled. If not, the *sdfactor* parameter can be adjusted until they are the same. One may also use the cleaned high resolution image instead of the dirty image, if the latter is not available. However, note that the cleaned high resolution image already contains extrapolations to larger spatial scales that may bias the comparison.

 

 

 

 



##### Bibliography

1. Koda\ et\ al\ 2011\ (
2. Vogel\ et\ al\ 1984\ (
3. Stanimirovic\ et\ al\ 1999\ (
4. Stanimirovic\ et\ al\ 2002\ (
5. Helfer\ et\ al\ 2003\ (
6. Weiss\ et\ al\ 2001\ (
^



***





### Image Cube Visualization 

How to visualize image cubes in CASA



***





#### CARTA 

CARTA: the Cube Analysis and Rendering Tool for Astronomy

CARTA is the Cube Analysis and Rendering Tool for Astronomy, a new image visualization and analysis tool designed for the [ALMA](http://www.almaobservatory.org), the [VLA](https://science.nrao.edu/facilities/vla), and the [SKA pathfinders](https://skatelescope.org/). CARTA is expected to replace the CASA Viewer in the foreseeable future.

CARTA is an external data analysis package that is being developed by the [Academia Sinica Institute of Astronomy and Astrophysics (ASIAA)](https://www.asiaa.sinica.edu.tw) the [Inter-University Institute for Data Intensive Astronomy (IDIA)](https://idia.ac.za), the [National Radio Astronomy Observatory (NRAO)](https://science.nrao.edu), and the [Department of Physics, University of Alberta](https://www.ualberta.ca/physics)

As the image size increases drastically with modern telescopes in recent years, viewing an image with a local image viewer or with a remote image viewer via the ssh protocol becomes less efficient. The mission of CARTA is to provide usability and scalability for the future by utilizing modern web technologies and computing parallelization.

CARTA currently already has various capabilities in common the CASA Viewer, but development is still ongoing and not yet all features of the CASA Viewer are available in CARTA. Nevertheless, development of CARTA is progressing rapidly, and since development of the CASA Viewer has put on hold in favor of this, we recommend users to try CARTA for those instances where the CASA Viewer is inadequate. These include:

-   efficiently loading large data cubes
-   flexibility in assigning regions for data analysis
-   clean labeling of the axis

We expect this list to grow rapidly with upcoming CARTA releases.

For more information on CARTA, please see the [CARTA homepage](https://cartavis.github.io/).

 



***





#### Image Viewer 

Introduction to the CASA Viewer

This documentation describes how to use the CASA Viewer to display data. The Viewer can be started as a stand-alone executable or by the **viewer** task inside a CASA shell. It can display both images and MeasurementSets. The task-level access to the Viewer has been split into two tasks: **imview** for images and **msview** for MeasurementSets. These tasks offer improved scriptability, giving command line access to many of the viewer features.

[For help on the image viewer, please see the subpages shown in the tabs to the left. For the Visibility visualization, please see [2-D Visualization and Flagging of Visibility Data](calibration-and-visibility-data.ipynb#2-d-visualization-of-visibility-data-msview). ]



***





#### Viewer Basics 

An introduction to basic viewer features



##### Starting the Viewer

------------------------------------------------------------------------

![08fb3afec0b21c1c8afa30e15b80617972df1d4f](media/08fb3afec0b21c1c8afa30e15b80617972df1d4f.png){.image-inline}

>Initial Viewer Panels: The Viewer Display Panel (left) and the Data Manager Panel (right) for a regular image or data cube.
  

------------------------------------------------------------------------

Within the CASA environment, the viewer task can be used to start the CASA Viewer, displaying an image or MS. The inputs are:

```
#viewer :: View an image or visibility data set.

infile        =         ''   #(Optional)  Name of file to visualize.
displaytype   =   'raster'   #(Optional)  Type of visual rendering
                             #(raster, contour, vector or marker).
                             #lel  if an lel expression is given
                             #for infile (advanced).
```

Examples of starting the CASA Viewer:

```
CASA <1>: viewer()

CASA <2>: viewer('ngc5921.demo.ms')

CASA <3>: viewer('ngc5921.demo.cleanimg.image')

CASA <4>: viewer('ngc5921.demo.cleanimg.image', 'contour')
 
CASA <5>: viewer('"ngc5921.demo.cleanimg.image"^2', 'lel')
```

The first command creates an empty [Viewer Display Panel](#the-viewer-display-panel) and a [Load Data Window](#the-data-manager-panel---saving-and-loading-data).The second command starts the viewer with a loaded MeasurementSet. The third command starts the viewer and opens an image data cube (see [Initial Viewer Panels](#FigInitialViewerPanels)).

Examples four and five make use of the second parameter (displaytype). Example four displays the image as a contour map rather than the default raster map. Example five uses 'Lattice (Image) Expression Language' to display the square of the image data.

Note that the viewer can open FITS files, CASA image files, MeasurementSets, and saved viewer states. The viewer determines the type of file being opened automatically.

For additional scripting options when opening the viewer, see the discussion of the **imview** and **msview** tasks.

 

##### Running the Viewer Outside CASA

If you have CASA installed, then the CASA Viewer is available as a stand-alone application called casaviewer. From the operating system prompt, the following commands work the same as the casa task commands given in the previous section:

```
casaviewer &

casaviewer ms_filename &
 
casaviewer image_filename &
 
casaviewer image_filename contour &
 
casaviewer '"image_filename"^2' lel &
```




##### The Viewer Display Panel

The CASA Viewer consists of a number of graphical user interface (GUI) windows. The main [Viewer Display Panel](#the-viewer-display-panel) is used for both image and MeasurementSet viewing. It is shown in the left panels of [Initial Viewer Panels](#FigInitialViewerPanels) and [Data Display Options](#the-viewer-display-panel) and appears the same whether an image or MeasurementSet is being displayed.

 

------------------------------------------------------------------------

 

![e8cf3cee163da8e2a22ef120a841581226610667](media/e8cf3cee163da8e2a22ef120a841581226610667.png){.image-inline}

------------------------------------------------------------------------

At the top of the Viewer Display Panel are drop down menus:

-   DATA
    -   Open --- open the [Data Manager](#the-data-manager-panel---saving-and-loading-data) window.
    -   Manage Images --- open the [Image Manager](#image-manager) which contains functionality for managing the image stack.
    -   Adjust Data Display --- open the [Data Display Options](#the-viewer-display-panel) (\'Adjust\') window.
    -   Save as\... --- save/export data to a file.
    -   Print --- print the displayed image.
    -   Save Panel State --- to a 'restore' file (xml format).
    -   Restore Panel State --- from a restore file.
    -   Preferences --- manually edit the viewer configuration.
    -   Close panel--- close this Viewer Display Panel. If this is the last display panel open, this will exit the viewer.
    -   Quit Viewer --- close all display panels and exit.
-   DISPLAY PANEL    -   New Panel--- create a new, empty Viewer Display Panel.
    -   Panel Options --- open the Viewer Canvas Manager (see the [Viewing Images and Cubes](imaging.ipynb#viewing-images-and-cubes) page) window to edit margins, the number of panels, and the background.
    -   Save Panel State --- save the current state of the viewer as a file that can later be reloaded.
    -   Restore Panel State --- restore the viewer to a state previously saved as a file.
    -   Print --- print displayed image.
    -   Close Panel --- close this Viewer Display Panel. If this is the last display panel open, this will exit the viewer.
-   TOOLS
    -   Spectral Profile --- open the [Spectral Profile Browser](imaging.ipynb#spectral-profiler) window to look at intensity as a function of frequency for part of an image.
    -   Collapse/Moments --- open the [Collapse/Moments](#viewerCollapseMoments) window, which allows you to create new images from a data cube by integrating along the spectral axis.
    -   Histogram --- open the [Histogram](#viewerHistogram) inspection window, which allows you to graphically examine the distribution of pixel values in a data cube.
    -   Fit --- open the [Two-Dimensional Fitting](#viewerSpectralFit) window, which can be used to fit Gaussians to two dimensional intensity distributions.
    -   Interactive Clean --- open a window to look at ongoing interactive clean processes.
-   VIEW
    -   Main Toolbar --- show/hide the top row of icons (see [Main Toolbar (image)](#the-main-toolbar) and [Main Toolbar (usage)](#the-main-toolbar)).
    -   Mouse Toolbar --- show/hide the second row of mouse-button action selection icons (see [Mouse Toolbar (image)](#the-mouse-toolbar) and [Mouse Toolbar (usage)](#the-mouse-toolbar)).
    -   Animators--- show/hide tapedeck control panel attachment to the main [Viewer Display Panel](#the-viewer-display-panel).
    -   Cursors --- show/hide the position tracking attachment to the main [Viewer Display Panel](#the-viewer-display-panel).
    -   Regions --- show/hide the region manager attachment to the main [Viewer Display Panel](#the-viewer-display-panel).

 

##### The Main Toolbar

------------------------------------------------------------------------

 

![8b57d404c74488f1107e2ebf839c0f60faeb3c71](media/8b57d404c74488f1107e2ebf839c0f60faeb3c71.png){.image-inline}

>Main Toolbar: The Display Panel's Main Toolbar appears directly below the menus and contains \'shortcut\' buttons for most of the frequently-used menu items.
  

------------------------------------------------------------------------

 

Below the drop down menus is the Main Toolbar (see [Main Toolbar](#the-main-toolbar)). This top row of icons offers fast access to these menu items:

-   FOLDER (Data:Open shortcut) --- open the [Data Manager](#the-data-manager-panel---saving-and-loading-data) window.
-   WRENCH (Data:Adjust shortcut) --- open the [Data Display Options](#the-viewer-display-panel) ('Adjust') window.
-   MANAGE IMAGES (Data: Manage Images shortcut) - open the [Image Manager](#image-manager) window.
-   DELETE (Data:Close shortcut) --- close (unload) the selected data file. The menu expands to the right showing all loaded data.
-   SAVE DATA (Data:Save as) --- save the current data to a file.
-   NEW PANEL (Display Panel:New Panel) --- create a new, empty Viewer Display Panel.
-   PANEL WRENCH (Display Panel:Panel Options) --- open the Viewer Canvas Manager (see the [Viewing Images and Cubes](imaging.ipynb#viewing-images-and-cubes) page) window to edit margins, the number of panels, and the background.
-   SAVE PANEL (Display Panel: Save Panel State) --- save panel state to a 'restore' file.
-   RESTORE PANEL (Display Panel: Restore Panel State) --- restore panel state from a restore file.
-   SPECTRAL PROFILE (Tools: Spectral Profile) --- Open the [Spectral Profile](imaging.ipynb#spectral-profiler) window to look at intensity as a function of frequency for part of an image.
-   COLLAPSE/MOMENTS (Tools: Collapse/Moments) --- Open the [Collapse/Moments](#viewerCollapseMoments) window, which allows you to create new images from a data cube by integrating along the spectral axis.
-   HISTOGRAM (Tools:Histogram) --- Open the [Histogram](#viewerHistogram) inspection window, which allows you to graphically examine the distribution of pixel values in a data cube.
-   TWO-DIMENSIONAL FITTING (Tools:Fit) -- Open the [Two-Dimensional Fitting](#viewerSpectralFit) window, which can be used to fit Gaussians to two dimensional intensity distributions.
-   PRINT (Display Panel:Print) --- print the current display.
-   MAGNIFIER BOX --- zoom out all the way.
-   MAGNIFIER PLUS --- zoom in (by a factor of 2).
-   MAGNIFIER MINUS --- zoom out (by a factor of 2).     

##### The Mouse Toolbar

------------------------------------------------------------------------

 

![aa5ec91c272ee59cd0bef0a8d695ca7872058795](media/aa5ec91c272ee59cd0bef0a8d695ca7872058795.png){.image-inline}

>Mouse Toolbar: The \'Mouse Tool\' Bar allows you to assign how mouse buttons behave in the image display area. Initially, zooming, color adjustment, and rectangular regions are assigned to the left, middle and right mouse buttons, respectively. Click on a tool with a mouse button to assign that tool to that mouse button.
  

------------------------------------------------------------------------

Below the Main Toolbar are eleven Mouse Tool buttons (see [Mouse Toolbar](#the-mouse-toolbar)). These allow you to assign what behavior the three mouse buttons have when clicked in the display area. Clicking a mouse tool icon will \[re-\]assign the mouse button that was clicked to that tool. Black and white squares beneath the icons show which mouse button is currently assigned to which tool.The mouse tools available from the toolbar are:

-   ZOOMING (magnifying glass icon): To zoom into a selected area, press the zoom tool's mouse button (the left button by default) on one corner of the desired rectangle and drag to the desired opposite corner. Once the button is released, the zoom rectangle can still be moved or resized by dragging. To complete the zoom, double-click inside the selected rectangle. If you instead double-click outside the rectangle, you will zoom out.
-   PANNING (hand icon): Press the panning tool's mouse button on a point you wish to move, drag it to the position where you want it moved, and release. Note: The arrow keys, Page Up, Page Down, Home and End keys can also be used to pan through your data any time you are zoomed in. (Click on the main display area first, to be sure the keyboard is 'focused' there).
-   STRETCH-SHIFT COLORMAP FIDDLING (crossed arrows): This is usually the handiest color adjustment; it is assigned to the middle mouse button by default. Hold down the tool\'s mouse button and drag across the display window to adjust the stretch and color. Note that you can also adjust the color table quantitatively inside the [Data Display Options](#viewerRasterMap) window.
-   BRIGHTNESS-CONTRAST COLORMAP FIDDLING (light/dark sun): Another tool to adjust the color stretch.
-   POSITIONING (plus): This tool can place a point marker on the display to select a position. It is used to flag MeasurementSet data or to select an image position for spectral profiles. Click on the desired position with the tool's mouse button to place the point; once placed you can drag it to other locations. You can also place multiple points on the display (e.g. for different spectral profile positions) -- remove them by hovering over and hitting ESC. Double-click is not needed for this tool. See [Viewer Region Positioning](#the-region-manager-panel) for more detail.
-   RECTANGLE, ELLIPSE, and POLYGON REGION DRAWING: The rectangle region tool is assigned to the right mouse button by default. As with the zoom tool, a rectangle region is generated by dragging with the assigned mouse button; the selection is confirmed by double-clicking within the rectangle. An ellipse regions is created by dragging with the assigned mouse button. In the case of an elliptical region, both the elliptical region and its surrounding rectangle are shown on the display. The selection is confirmed by double-clicking within the ellipse. Polygon regions are created by clicking the assigned mouse button at the desired vertices and then clicking the final location twice to finish. Once created, a polygon can be moved by dragging from inside, or reshaped by dragging the handles at the vertices. See [Viewer Region Positioning](#viewerRegionPositioning) for the uses of this tool.
-   POLYLINE DRAWING: A polyline can be created by selecting this tool. It is manipulated similarly to the polygon region tool: create segments by clicking at the desired positions and then double-click to finish the line.
-   DISTANCE TOOL (ruler): After selecting the distance tool by assigning a mouse button to it, distances on the image can conveniently be measured by dragging the mouse with the assigned button pressed. The tool measures the distances along the world coordinate axes and along the hypotenuse. If the units in both axes are \[deg\], the distances are displayed in \[arcsec\].
-   POSITION-VELOCITY DIAGRAM: Use this mouse tool to drag out a position axis that can be used to generate a position velocity diagram in a new viewer panel from the region manager dock.

<div class="alert alert-info">
**NOTE**: The 'escape' key can be used to cancel any mouse tool operation that was begun but not completed, and to erase a region, point, or other tool showing in the display area.
</div>

 

##### The Main Display Area

The Main Display Area lies below the toolbars. This area shows the image or MeasurementSet currently loaded. Clicking the mouse inside the display area allows region or position selection according to the settings in the mouse toolbar.The Display Area may have up to three attached panels: the Animators panel, the Cursors panel, and the Regions panel. These may be displayed or hidden from the \"View\" dropdown menu in the main Viewer Display Panel. If one of these is missing from your viewer, check that it is checked \"on\" in that menu. The panels can also be turned off by clicking the \"X\" in the top right corner, in which case you will need to use the View menu to get them back.By default, the three panels appear attached to the main Viewer Display Panel on the right side of the image. They may be dragged to new positions. Each of the three panels can be attached to the left, top, right, or bottom of the main Viewer Display Panel or they can be entirely undocked and left as free-floating panels.

<div class="alert alert-info">
**NOTE**: Depending on your window manager, windows without focus, including detached panels and tools like the Spectral Profile Browser may sometimes display odd behavior. As a general rule, giving the window focus by clicking on it will correct the issue. If you seem to "lose" a detached panel (like the Animators Panel), then click in the main window to get it back.
</div>

<div class="alert alert-info">
**NOTE**: With all three panels turned on (and especially with several images loaded), the Main Display Panel can sometimes shrink to very small sizes as the panels grow. Try detaching the panels to get the main display panel back to a useful size.
</div>

A restart of the viewer will display all docks in the state of a previous viewer session, given that it was closed normally. In addition, the viewer docking can be changed under "Preferences" in the toolbar (for Mac OS: under the "CASA Viewer" tab on the toolbar, for Linux: under "Data"). An example is given in the [Preference Dialog](#FigViewerPreferences) figure below. Each item can be changed and the input box will only allow accepted input formats. A complete restart is required to apply the changes.

------------------------------------------------------------------------

 ![8feaa9fb0d42d24217df66390ab85c79d44fa0bd](media/8feaa9fb0d42d24217df66390ab85c79d44fa0bd.png){.image-inline}

>Preference Dialog: The Preference Dialog can be used to manually change the docking and size of the viewer panel.
  

------------------------------------------------------------------------

######  The Animators Panel

![2cfd77b4f2ced4be1a260718e231ae8110cb6d0d](media/2cfd77b4f2ced4be1a260718e231ae8110cb6d0d.png){.image-inline}

>Animators Panel: The Animators Panel allows you to scroll through the z-axis of a data cube (using the Channels tape deck) or cycle among open Images. The panel can be undocked from the main display panel.
  

------------------------------------------------------------------------

The Animators Panel allows you to scroll through the channels of a data cube and to flip through loaded images. The main features of the panel are the two "tape decks," one labeled \"Channels\" and one labeled \"Images\".

<div class="alert alert-info">
**NOTE**: You will only see the Images tape deck when multiple images are loaded.
</div>

The \"Channels\" tape deck scrolls between planes of an individual image. By default, the channel tape deck scrolls among frequency planes when Right Ascension and Declination are the displayed axes (in this case, frequency is the z-axis). From outside to inside, the buttons cause the display to jump all the way to the beginning/end of the z-axis, cause the viewer to step one plane forward or backward along the z-axis, or start a movie. The limits on the z-axis can be set manually using the windows at the end of the scroll bar. The scroll bar can also be dragged or the user can jump the display to a manually entered plane by entering the plane into the text block.When you have multiple images loaded, the Images tape deck cycles through which is image is being displayed. In the movie mode, it allows you continuously click between images. Functionally, the Images tape deck works similarly to the Channels tape deck, with the ability to step, jump, or continuously scroll through images.

<div class="alert alert-info">
**NOTE**: The check boxes next to the channel and images tabs enable or disable those panels. This doesn't have much effect when the display has only a single panel, but with multiple panels (i.e., several maps at once in the main window) it changes the nature of the display. If the "Images" box is checked then interleaved maps from different cubes are displayed. Otherwise a series of maps from a single cube are shown.
</div>

 

##### The Cursors Panel 

------------------------------------------------------------------------

 

![b58deef224d019cae4184041a4e7fd9691bfcaf6](media/b58deef224d019cae4184041a4e7fd9691bfcaf6.png){.image-inline}

>Cursors Panel: The Cursors Panel gives information about the open data cube at the current location of the cursor. Freeze the Cursors Panel using the SPACE bar.
  

------------------------------------------------------------------------

The Cursors Panel (below the Images tape deck in [Initial Viewer Panels](#FigInitialViewerPanels)) shows the intensity, position (e.g., RA and Dec), Stokes, frequency (or velocity), and pixel location for the point currently under the cursor. A separate box appears for each registered image or MeasurementSet and you can see the tracking information for each. Tracking can be \'frozen\' (and unfrozen again) by hitting the space bar when the viewer's focus is on the main display area. (To be sure that the focus is indeed the main display area, first click on the main display area.)

 

###### The Region Manager Panel 

The Region Manager Panel becomes active when regions are created. It has a large amount of functionality, from display region statistics and histograms to creating position-velocity cuts. We discuss these in [Viewer Regions](#viewerRegions). Like the Animators and Cursors Panels, the Region Manager Panel can be moved relative to the main Viewer Display Panel or entirely undocked.
##### Saving and Restoring the Display Panel State 

You can save the display panel's current state --- meaning the panel settings and the data on display --- or load a saved panel state from disk. To save the display panel state, select Save Panel State from the Display Panel drop-down menu or click the \"Save Display Panel State to File\" icon on the main toolbar (an arrow pointing from a picture to a page, see [Main Toolbar](#the-main-toolbar)). It is advisable but not required to retain the file's \'.rstr\' (\"Restore\") extension.You can restore the display panel to the saved state by loading the saved state from the Data Manager Panel, by selecting Restore Panel State from the Display Panel drop down menu, or by clicking the \"Restore Display Panel State\" icon (just to the right of the \"Save Display Panel State\" icon).It is possible to restore panel states viewing MeasurementSets, images, or panel states that have multiple layers, such as contour plots over raster images. You can also save LEL displays. You can also the save or restore the panel state with no data loaded, which is a convenient way to restore preferred initial settings such as overall panel size.Data Locations: The viewer is fairly forgiving regarding data location when restoring a saved panel state. It will find files located:

-   in the original location recorded in the restore file
-   in the current working directory (where you started the viewer)
-   in the restore file's directory
-   in the original location relative to the restore file

This means that you can generally restore a saved panel state if you move that file together with data files. The exception to this rule is that the process is less forgiving if you save the display of an LEL expression. In this case the files must be in the locations specified in the original LEL expression. If a data file is not found, restore will attempt to proceed but results may not be ideal.Manually Editing Saved Display Panel States: The saved \"Restore\" files are in ascii (xml) format, and manual edits are possible. However, these files are long and complex. Use caution, and back up restore files before editing. If you make a mistake, the viewer may not even recognize the file as a restore file. It is easier and safer to make changes on the display panel and then save the display panel state again.


##### The Data Manager Panel --- Saving and Loading Data

------------------------------------------------------------------------

 

![7d4cd267bbb4a3b269e1f22c908e1dc21a017b70](media/7d4cd267bbb4a3b269e1f22c908e1dc21a017b70.png){.image-inline}

>Data Manager Panel: The load tab of the Data Manager panel. This appears if you open the viewer without any infile specified, if you use select Open from the Data drop-down menu, or click the Open (Folder) icon. You can access the save image or save region tabs from this view or by selecting Save as\... from the Data drop down menu. The load tab shows all files in the current directory that can be loaded into the viewer --- images, MS, CASA region files, and Display Panel State files.
  

------------------------------------------------------------------------

The Data Manager Panel is used to interactively load and save images, MeasurementSets, Display Panel States, and regions. An example of the loading tab in this panel is shown in the [Data Manager Panel](#the-data-manager-panel---saving-and-loading-data) figure. This panel appears automatically if you open the viewer without specifying an input file or it can be accessed through the Data:Open menu or Open icon of the Viewer Display Panel.

 

##### Loading Data

The load tab of the Data Manager Panel allows you to interactively choose images or MeasurementSets to load into the viewer. The load tab automatically shows you the available images, MeasurementSets, and Display Panel States in the current directory that can be opened by the viewer. When you highlight an image in this view, the tab shows a brief summary of the image: pixel shape, extent of the image on the sky and in frequency/velocity, and restoring beam (if available).Selecting a file will bring up information about that file in the panel on the right of the Data Manager Panel provide options for how to display the data. Images can be displayed as:

1.  raster image
2.  contour map
3.  vector map
4.  marker map

These options area each discussed in [Viewing Images](imaging.ipynb#viewing-images-and-cubes).

Additionally, the following data types can be loaded via the Data Manager Panel:slice: a subselection of a data cube can be loaded, the start and end pixel in each spatial, polarization, and spectral dimension can be selected.LEL: Instead of only loading an image from disk, you may ask the viewer to evaluate a 'Lattice Expression Language' (LEL) expression (§ [6.1.4](https://casa.nrao.edu/casadocs-devel/docs/cookbook/casa_cookbook007.html#section%3Aanalysis.pars.lattice)). This can be entered in the box provided after you click the \"LEL\" box. The images used in the LEL expression should have the same coordinates and extents.MeasurementSets: A MeasurementSet can only be displayed as a raster. For MeasurementSets, the load tab offers options for data selection. This will reduce loading and processing times for visibility flagging.Regridding Images on Load: Optionally, you may regrid the velocity axis of an image you wish to load to match the current coordinates grid in the Display Panel. In this case, the viewer will interpolate (using the selected interpolation scheme) the cube on disk to share the same velocity gridding as the loaded coordinates. This can be used, e.g. to overlay contour maps of different spectral lines or to make synchronized movies of multiple cubes. Note that the regridding depends on the rest frequency in the image, which is used to calculate the velocities used in regridding.

 

##### Registered vs. Open Datasets

When you load data as described above, it is first opened, and then registered on all existing Display Panels. An open dataset has been prepared in memory from disk. All open datasets will have a tab in the Data Display Options window, whether currently registered or not. When a data set is registered to a Display Panel its coordinates are aligned to the master coordinate image in the panel and it is ready for drawing. If multiple Display Panels are open then a data set may be registered on one Display Panel and not on another. Only those data sets registered on a particular Display Panel show up in its Cursors Panel.Why Register More Than One Image? It is useful to have more than one image registered on a panel if you are displaying a contour image over a raster image (see [Overlay Contours on a Raster Map](#viewerContourOverlay)), to 'blink' between images (see Animators in [Viewer Display Panel](#the-viewer-display-panel)), or to compare images using the Cursors Panel.Unregistering Images: A data set can be registered or unregistered using the Image Manager in the Data drop down menu or the Image Manager icon (third from left). This icon will open the Image Manager window and the checkboxes can be used to register or unregister an image.Closing vs. Unregistering: You can close a data set that is no longer needed using the Close option in the Data drop-down menu, the \"Close\" icon (fourth from left), or right mouse button "Close" selection in the [Image Manager](#image-manager).If you close a dataset, you must reload it from disk (or recreate it if it's an LEL expression, regridded image, moment or something similar) to see it again. If you unregister a dataset, it will draw immediately if you re-register it, with its options as you have previously set them. In general, close unneeded datasets but unregister those that you intend to use again.

 

##### Image Manager

The Image Manager is used to define the master coordinate image, the sequence of images (e.g. for blinking), to register and unregister images, close images, change between raster, contour, vector, and marker displays, and to modify the properties of images. The panel can be invoked from the "Manage Images" tool, the third icon from the left (two overlapping squares).An example is shown in [Image Manager](#image-manager) figure. In this case, four images are loaded into the viewer. The sequence of images can be changed by dragging and dropping the images to new positions in the stack. The letter to the left indicates whether the image is a Raster, Contour, Vector, or Marker image. MC marks the coordinate master, in this case the second image. The checkboxes are to change the registration statuses. The Coordinate Master image can be defined by a right mouse click, and selecting the corresponding option. The right mouse menu button also offers options for quick changes between contour and raster images and to close an image.The Options button will open a drop-down box (as shown in [Image Manager](#image-manager) for the first image), in which one can again change between image type, change to a different rest frequency (or "Reset" to the value in the image header), or open the \"Display Options\" panel for that specific image with all the adjustment options explained in [Viewing a Raster Map](#viewerRasterMap) or [Viewing a Contour Map](#viewerContourMap)).

------------------------------------------------------------------------

 

![8974d9863e86c477d90bafedc2635435d5e45fbb](media/8974d9863e86c477d90bafedc2635435d5e45fbb.png){.image-inline}

>Image Manager
  

------------------------------------------------------------------------

 

 

##### Saving Data or Regions

------------------------------------------------------------------------

 

![6635be99d7c7c01aecee67542626f14a7a4fab30](media/6635be99d7c7c01aecee67542626f14a7a4fab30.png){#__mcenew .image-inline}

>Save Data Panel: The Save Data panel that appears when selecting the 'Save as\...' (see [Main Toolbar](#FigMainToolbar)).
  

 

------------------------------------------------------------------------

The viewer can create new images by carrying out velocity regridding, evaluating an LEL expression, or collapsing a data cube. You can save these images to disk using the Data Manager Panel. Select Save As under the Data drop-down menu or click the Save As (disk) icon to bring up the Data Manager Panel set to the save tabs. These tabs are shown in the figure above.From the Save Image tab of the Data Manager Panel, you can export images from the viewer to either a CASA image or FITS file on disk. Select the desired file name and click \"Save.\" The Data Manager also allows you to save your current regions to a file, either in the CASA or ds9 format. The left side of the Save Data Panel lists all images that can be exported to disk. To save an image to a file, you can either enter the new filename in the box labeled \'output name:\' followed by the save-button (alternatively the 'Enter'-key), or choose a file name from the right hand side.



***





#### Viewing Images and Cubes 

Details on viewing images and cubes



##### Viewing Images

There are several options for viewing an image. These are seen at the right of the Load Data - Viewer panel described in [Initial Viewer Panels](#initial-viewer-panels) after selecting an image. They are:

-   raster image --- a greyscale or color image,
-   contour map --- contours of intensity as a line plot,
-   vector map --- vectors (as in polarization) as a line plot,
-   marker map --- a line plot with symbols to mark positions.

The raster image is the default image display, and is what you get if you invoke the viewer with an image file and no other options. In this case, you will need to use the Open menu to bring up the Load Data panel to choose a different display.

This page discusses raster images and contour maps in detail; for an example of how to use a vector map, see the 3C286 Polarization CASAguide [here](https://casaguides.nrao.edu/index.php/3C286_Band6Pol_Imaging_for_CASA_4.3).

##### Initial Viewer Panels 

When the viewer is started, two dialogs appear. One is the Data Manager which presents two panels.

###### Data Manager

The left panel shows the files that the viewer can load while the right panel shows some statistics about the file that is selected.

![e4b6e0a334c21c09b02abeac82d22632d050643f](media/e4b6e0a334c21c09b02abeac82d22632d050643f.jpg){.image-inline width="479" height="314"}

The other panel is the Viewer Display Panel.

###### Viewer Display Panel

This panel is the main panel used to interact with the viewer. 

![1fa6b6a876aad22dcb6fc8c360d20fc4a67784ce](media/1fa6b6a876aad22dcb6fc8c360d20fc4a67784ce.jpg){.image-inline width="477" height="305"}

The image is shown on the left and image information is shown on the right. The Cursors panel displays information about the pixel at the current cursor location (as the cursor is moved around the image).

###### Animators Panel

The Animators panel allows the planes on the image cube to be displayed. This can be done by either single-stepping plane by plane or playing the planes of the image cube like a movie. The buttons are: 

<table><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td><img src="markdown/_media/2c6505666e66f7864551526a4671489e0113c673.png" title="Anim0_ToStart.png" class="image-left" /><br />
<br />
</td><td>move to the first image plane</td></tr><tr class="even"><td><img src="markdown/_media/1664330a16c2db796b734e094af3ea123c7f28e0.png" title="Anim1_RevStep.png" class="image-left" /></td><td>move back one image plane</td></tr><tr class="odd"><td><img src="markdown/_media/031fffc236c0d843392d428addac247288f0e11f.png" title="Anim2_Rev.png" class="image-left" /></td><td>play image cube in reverse</td></tr><tr class="even"><td><img src="markdown/_media/b855a1ed3836bbd79eac65b341755ec44597e88d.png" title="Anim3_Stop.png" class="image-left" /></td><td>stop playing the movie</td></tr><tr class="odd"><td><img src="markdown/_media/9778c0eb8de97fd56c1d9be55c904c1f5cabc235.png" title="Anim4_Play.png" class="image-left" /></td><td>play image cube as a movie</td></tr><tr class="even"><td><img src="markdown/_media/41a79bc85908fd3d50836936c795b9577b4dd0b7.png" title="Anim5_FwdStep.png" class="image-left" /></td><td>move forward one image plane</td></tr><tr class="odd"><td><img src="markdown/_media/b0fa85965496a6f8fbfbc38fdec6046466f4bdc8.png" title="Anim6_ToEnd.png" class="image-left" /></td><td><p>move to the last image plane</p></td></tr></tbody></table>

In addition, to these controls for moving through the image cube, there are two other areas of animation control:

  ------------------------------------------------------------------------------------------------------------------------------ ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  ![85424f67f54248c57c56336ea81bd66cf6fda3ac](media/85424f67f54248c57c56336ea81bd66cf6fda3ac.jpg){.image-left width="196" height="25"}   the rate indicates how how fast the movie should be played in terms of frames, and the second entry box (here with a zero) is for going to a particular plane of the image cube (enter a number and hit return) \[Jump doesn\'t seem to do anything\]
  ![9b03ad5ef24dbda549937b824a99bf4094af4343](media/9b03ad5ef24dbda549937b824a99bf4094af4343.jpg){.image-left width="316" height="18"}   the slider of this control allows for moving through the image cube as the slider is moved. The dialogs at the ends of the slider allows for setting the start and end points for the movie (which can be less than or equal to zero and the number of planes in the cube)
  ------------------------------------------------------------------------------------------------------------------------------ ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

###### Button Tools

These tools are designed for use with a three-button mouse. The row of boxes below the icon indicates which mouse button to which the tool is currently bound. For example, the last three icons in this table indicate that these tools are bound to the first, second, and third buttons respectively:

  ------------------------------------------------------------------------------------------------------- -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  ![ea7304b3410f36a9dccb3226349b77a36d11a998](media/ea7304b3410f36a9dccb3226349b77a36d11a998.png){.image-left}          zoom: select this tool (by clicking on this icon and pressing one of the three buttons), then click and drag out a rectangle, then double click inside the rectangle to zoom in
  ![ec32ba3d365abd638f28eab5223ff43191f45c76](media/ec32ba3d365abd638f28eab5223ff43191f45c76.png){.image-left}             panning: select this tool, then if the image is zoomed in, click and drag within the image to move the image
  ![ff0fc891b68739126b6757a6f2711f3e298bf37c](media/ff0fc891b68739126b6757a6f2711f3e298bf37c.png){.image-left}       adjust color map: select this tool, then click and drag within the image to adjust the color map
  ![30ae0891708bab0f984a47332160c32eb5a19c74](media/30ae0891708bab0f984a47332160c32eb5a19c74.png){.image-left}   contrast: select this tool, then click and drag within the image
  ![fcdf48b33278a7339b429ba55d5ca04bb3200639](media/fcdf48b33278a7339b429ba55d5ca04bb3200639.png){.image-left}           point region: select this tool, then place a point on the image, the regions panel corresponding to the dot you placed will have statistics an information about the selected point
  ![9f482145a9d4db935ea3ab16d49ee44d1641700c](media/9f482145a9d4db935ea3ab16d49ee44d1641700c.png){.image-left}       rectangular region: select this tool, then click and drag out a rectangle in the image and the regions panel corresponding to this region will have information about the rectangular region; double clicking in the region will display the statistics in to terminal window
  ![41ce877a9905c5c1496a99b09183e2095fcca245](media/41ce877a9905c5c1496a99b09183e2095fcca245.png){.image-left}        eliptical region: select this tool, then click and drag out an ellipse, the regions panel corresponding to this region will have information about the eliptical region; double clicking in the region will display the statistics in the terminal window
  ![954637995892613204657cc0f5b51c865f12a077](media/954637995892613204657cc0f5b51c865f12a077.png){.image-left}       polygon region: select this tool, then click multiple times within the image to mark out a region (one click at a time), double clicking when you have marked all of the points that denote the polygon, the regions panel corresponding to this region will have information about this region; double clicking in the region will display statistics for this region in the terminal window
  ![fdc2fd7995dbfd26ce5939da376bdbe40e2caac4](media/fdc2fd7995dbfd26ce5939da376bdbe40e2caac4.png){.image-left}         polyline region: select this tool, then click multiple times within the image to mark out a multi-segment line, the region panel for this region will display statistics about the region
  ![731bad1ac19871bd8ce249ad527d8dd85589763e](media/731bad1ac19871bd8ce249ad527d8dd85589763e.png){.image-left}            ruler: select this tool, click and drag in the image to get a display of distance along two axes
  ![c19dae88d0d5f8fc261291f1d0c2df6344b4a35e](media/c19dae88d0d5f8fc261291f1d0c2df6344b4a35e.png){.image-left}           pv diagram: select this tool, click and drag within the image to create  a to be used to create a position/velocity diagram (the diagram is created from the region panel corresponding to the P/V line that you\'ve drawn)
  ------------------------------------------------------------------------------------------------------- -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

These tools create regions that can be used to provide information about a portion of an image.

###### Regions

Regions are created with the region [Button Tools](#button-tools). For a region to be created, the Region panel (displayed on the left side of the Viewer Display Panel) must be open. If you do not see the Regions panel, it can be included in the Display Panel by selecting the Regions check box in the View menu:

![4a9aebf37ad6a0a09105e2438ffd0ba508e4734f](media/4a9aebf37ad6a0a09105e2438ffd0ba508e4734f.jpg){.image-inline width="277" height="132"}

Once the Regions dialog is in view Regions can be created and information about the regions can be viewed. For example, here one region (black rectangle) has been created and region statistics is displayed:

![f8acac9ee8c014d5338db58636ffffe4c8a181b0](media/f8acac9ee8c014d5338db58636ffffe4c8a181b0.jpg){.image-inline width="497" height="248"}

Note that in the statistics window, the brightness units are specified and the beam area is defined as the volume of the elliptical Gaussian $\frac{Π}{4ln(2)} * FWHM_{major} * FWHM_{minor}$, where ln() is the natural logarithm and $FWHM_{major}$ and $FWHM_{minor}$ are the major and minor FWHM axes of the beam, respectively. The flux density is the mean intensity multiplied by the number of beams included in the selected region.

###### Region Statistics 

If more than one region has been created, the scroll bar can be used to move from the information about one region to the next or the cursor (in the image panel) can be used to move from one region to the next and the region information will be updated as the cursor moves from region to region. Here are three regions, showing histograms for the regions:

![cfc4f812ec08c5e86f6ffac0c67aabfc4c03d72a](media/cfc4f812ec08c5e86f6ffac0c67aabfc4c03d72a.jpg){.image-inline width="464" height="218"}

###### Region Histogram 

The region which corresponds to the histogram that is shown has the corner adjustment cubes drawn.

Any regions that are created using these tools can be removed by moving the cursor over the region you would like to remove and once that region is highlighted press the escape key. Regions can also be deleted from the region panel that corresponds to the region you would like to remove.

##### Viewing a Raster Map

A raster map of an image shows pixel intensities in a two-dimensional cross-section of gridded data with colors selected from a colormap according to a scaling that can be specified by the user. Starting the casaviewer with an image as a raster map will look something like the example in [Initial Viewer Panels](#initial-viewer-panels). Once loaded, the data display can be adjusted by the user through the Data Display Options panel, which appears when you choose the Data: Adjust Data Display menu or use the wrench icon from the Main Toolbar. The Data Display Options window is shown in the right panel of [Initial Viewer Panels](#initial-viewer-panels). It consists of a tab for each image or MS loaded, under which are a cascading series of expandable categories. For an image, these are:

-   display axes
-   hidden axes
-   basic settings
-   position tracking
-   axis labels
-   axis label properties
-   beam ellipse
-   color wedge

The basic settings category is expanded by default. To expand a category to show its options, click on it with the left mouse button.

 

###### Data Display Options --- Display and Hidden Axes

In this category the physical axes (i.e. Right Ascension, Declination, Velocity, Stokes) to be displayed can be selected and assigned to the x, y, and z axes of the display. The z axis will be the axis scrolled across by the channel bar in the Animators Panel. If your image has a fourth axis (typically Stokes), then one of the axes will need to be hidden and not used in viewing. Which axis is hidden can be controlled by a slider within the hidden axes drop-down.

 

###### Data Display Options --- Basic Settings

This roll-up is open by default showing some commonly-used parameters that alter the way the image is displayed. The most frequently used of these changes how the intensity value of a pixel maps to a color on the screen. An example of this part of the panel is shown in [Mapping Intensity to Color](#FigColorMapping).

------------------------------------------------------------------------

![d2baca80c73d185c1ca04583ccba36710108d3d5](media/d2baca80c73d185c1ca04583ccba36710108d3d5.png){.image-inline width="315" height="238"}

------------------------------------------------------------------------

<div>

The options available are:

</div>

-   basic settings: *aspect ratio*

    This option controls the horizontal-vertical size ratio of data pixels on screen. \"Fixed world\" (the default) means that the *aspect ratio* of the pixels is set according to the coordinate system of the image (i.e., true to the projected sky). \"Fixed lattice\" means that data pixels will always be square on the screen. Selecting \"flexible\" allows the map to stretch independently in each direction to fill as much of the display area as possible.

-   basic settings: *pixel treatment*

    This option controls the precise alignment of the edge of the current \"zoom window\" with the data lattice. \"Edge\" (the default) means that whole data pixels are always drawn, even on the edges of the display. For most purposes, \"edge\" is recommended. \"center\" means that data pixels on the edge of the display are drawn only from their centers inwards.

    <div class="alert alert-info">
    **NOTE**: A data pixel's center is considered its "definitive" position, and corresponds to a whole number in "data pixel" or "lattice" coordinates.
    </div>

-   basic settings: *resampling mode*

    This setting controls how the data are resampled to the resolution of the screen. \"Nearest\" (the default) means that screen pixels are colored according to the intensity of the nearest data point, so that each data pixel is shown in a single color. \"Bilinear\" applies a bilinear interpolation between data pixels to produce smoother looking images when data pixels are large on the screen. \"Bicubic\" applies an even higher-order (and somewhat slower) interpolation.

-   basic settings: *data range*

    You can use the entry box provided to set the minimum and maximum data values mapped to the available range of colors as a list \[min, max\]. For very high dynamic range images, you will probably want to enter a max less than the data maximum in order to see detail in lower brightness-level pixels.

    <div class="alert alert-info">
    **NOTE**: By default you edit the scaling of a single image at once and can click between the tabs at the top of the Data Display Options window to manipulate different windows. By checking the Global Color Settings box at the bottom of this window, you will manipulate the scaling for all images at once. This can be very useful, for example, when attempting a detailed comparison between multiple reductions of the same data.
    </div>

-   basic settings: *scaling power cycles*

    This option allows logarithmic scaling of data values to colormap cells, which can be very helpful in the case of very high dynamic range. The color for a data value is determined as follows:

    1.  The value is clipped to lie within the data range \[min, max\] specified above.
    2.  This clipped value is mapped to a new value depending on the selected scaling power cycles in the following way:
        -   If the scaling power cycles is set to 0 (the default), the program considers a linear range from \[min, max\] and scales this directly onto the set of available colors.
        -   For positive scaling values, the data value (after clipping on \[min, max\] is scaled linearly to lie between 0 and p (where p is the value chosen) and 10 is raised to this power, yielding a value in the range 1 to 10^p^. That value is scaled linearly to the set of available colors.
        -   [For negative scaling values, the data value (after clipping on \[min, max\]) is scaled linearly to lie between 1 and 10^\|p\|^, where p is the power chosen. The base 10 logarithm is taken of this re-scaled data value, yielding a value in the range 0 to abs(p). That value is scaled linearly to the set of available colors. Thus the data is treated as if it had p decades of range, with an equal number of colors assigned to each decade.]
    3.  The color corresponding to a number in final range is determined by the selected colormap and its \"fiddling\" (shift/slope) and brightness/contrast settings (see Mouse Toolbar, above). Adding a color wedge to your image can help clarify the effect of the various color controls.

    See [Scaling Power Cycles](#FigScalingPowerCycles) for sample curves.

------------------------------------------------------------------------

![71082fec54dd80acd580c40814ca9ad304ba7097](media/71082fec54dd80acd580c40814ca9ad304ba7097.png){.image-inline}

------------------------------------------------------------------------

In practice, you will often manipulate the data range bringing the max down in high dynamic range images, raising the minimum to the near the noise level when using non-zero scaling cycles. It is also common to use negative power cycles when considering high dynamic range images --- this lets you bring out the faint features around the bright peaks.

-   basic settings: *colormap*

    You can select from a variety of colormaps here. Hot Metal, Rainbow and Greyscale colormaps are the ones most commonly used.

 

###### Graphical Specification of the Intensity Scale

A histogram icon next to the Data Range entry in the Data Display Options window opens the Image Color Mapping window, which allows visualization and graphical manipulation of the mapping of intensity to color. The window at the left shows a histogram of the data with a gray range showing the data range. You can use this window to select the data range graphically (with the mouse), manually (by typing into the Min and Max entry windows), or as a percentile of the data. On the right, you can select the scaling power cycles and see a visualization of the transfer function mapping intensity (x-axis) to color (y-axis).

The functionality here follows the other histogram tools, with the Display tab used to change the histogram plotting parameters. It will often be useful to use a logarithmic scaling of the y-axis and filled histograms when manipulating the color table.

 

###### Data Display Options --- Other Settings

Many of the other settings on the Data Display Options panel for raster images are self-explanatory such as those which affect beam ellipse drawing (only available if your image provides beam data), or the form of the axis labeling and position tracking information. You can also give your image a color wedge, a key to the current mapping from data values to colors.

 

###### Viewer Canvas Manager --- Panels, Margins, and Backgrounds

------------------------------------------------------------------------

![bdbf6ab7cf4f4553813d0b9ef67c336d6ca7956d](media/bdbf6ab7cf4f4553813d0b9ef67c336d6ca7956d.png){.image-inline}

------------------------------------------------------------------------

 The display area can also be manipulated from the Viewer Canvas Manager window. Use the wrench icon with a \"P\" (or the \"Display Panel\" menu) to show this window, which allows you to manipulate the infrastructure of the main display panel. You can set:

-   Margins - specify the spacing for the left, right, top, and bottom margins
-   Number of panels - specify the number of panels in x and y and the spacing between those panels.
-   Background Color - white or black (more choices to come)

[Multi-Panel Display](#FigMultiPanel) illustrates a multi-panel display along with the Viewer Canvas Manager settings which created it.

<div>

 
##### Viewing a Contour Map

Viewing a contour image is similar to viewing a raster map. A contour map shows lines of equal data value for the selected plane of gridded data (see [Viewing a Contour Map](#FigContourView)). Contour maps are particularly useful for overlaying on raster images so that two different measurements of the same part of the sky can be shown simultaneously (see [Overlay Contours on a Raster Map](#viewerContourOverlay)).

Several basic settings options control the contour levels used:

-   The contours themselves are specified by a list in the box *Relative Contour Levels*. These are defined relative to the two other parameters:
-   The *Base Contour Level* sets the zero level for the relative contour list corresponds to in units of intensity in the image.
-   The *Unit Contour Level* sets what 1 in the relative contour list corresponds to in units of intensity in the image.

Additionally, you have the option to manipulate the thickness and color of the image and to have either positive or negative contours appear dashed.

------------------------------------------------------------------------

![65b29709d5005d1f999a3482263d05da8effaea4](media/65b29709d5005d1f999a3482263d05da8effaea4.png){.image-inline}

------------------------------------------------------------------------

For example, the following settings:

       Relative Contour Levels = [0.2, 0.4, 0.6, 0.8]
       Base Contour Level = 0.0
       Unit Contour Level = <image max>

would map the maximum of the image to 1 in the relative contour levels and the base contour level to zero. So the contours will show 20%, 40%, 60%, and 80% of the peak.

Another approach is to set the unit contour to 1, so that the contours are given in intensity units (usually Jy/beam). So this setup:

       Relative Contour Levels = [0.010, 0.0.020, 0.040, 0.080, 0.160, 0.320]
       Base Contour Level = 0.0
       Unit Contour Level = 1.0

would create contours starting at 10 mJy/beam and doubling every contour.

Another useful approach is to set contours in units of the rms noise level of the image, which can be worked out from a signal free region. Then a setup like this:

       Relative Contour Levels = [-3,3,5,10,15,20]
       Base Contour Level = 0.0
       Unit Contour Level = <image rms>

Would indicate significance of the features in the image. The first two contours show emission at ± 3-sigma and so on. You can get the image rms using the [imstat task](global-task-list.ipynb#task_imstat) or using the [Region Statistics](#region-statistics) on a region of the image .

Not all images are of intensity, for example a moment-1 image ([immoments task](global-task-list.ipynb#task_immoments)) has units of velocity. In this case, absolute contours (like the last two examples) will work fine, but by default the viewer will set fractional contours but refer to the min and max of the image:

       Relative Contour Levels = [0.2, 0.4, 0.6, 0.8]
       Base Contour Level = <image min>
       Unit Contour Level = <image max>

Here we have contours spaced evenly from min to max, and this is what you get by default if you load a non-intensity image (like the moment-1 image). See [Overlaying a Contour Map](#FigContourOverlay) for an example of this.

 

###### Overlay Contours on a Raster Map

Contours of either a second data set or the same data set can be used for comparison or to enhance visualization of the data. The Data Display Options Panel will have multiple tabs (switch between them at the top of the window) that allow you to adjust each overlay individually.

<div class="alert alert-info">
**NOTE**: Axis labeling is controlled by the first-registered image overlay that has labeling turned on (whether raster or contour), so make label adjustments within that tab.
</div>

To add a Contour overlay, open the Load Data panel (Use the Data menu or click on the folder icon), select the data set and click on contour map. See [Overlaying a Contour Map](#FigContourOverlay) for an example using NGC5921.

------------------------------------------------------------------------

![f37c9cebb709797c6c3f7326d397cc7d38cb3038](media/f37c9cebb709797c6c3f7326d397cc7d38cb3038.png){.image-inline}

 

###### Creating a Position/Velocity Diagram

With an image cube loaded, it is possible to create a position/velocity diagram using the P/V Button Tool (see [Initial Viewer Panels](#initial-viewer-panels)). The first step in creating a P/V diagram is to select the tool with the mouse button to which you would like to bind the tool:

![e33db1e64044bac99a712e5e56c84163c63b8a7c](media/e33db1e64044bac99a712e5e56c84163c63b8a7c.jpg){.image-inline width="326" height="328"}

Here we have bound the P/V button tool to the first mouse button. At this point, we can drag out a P/V line, by clicking and dragging on the image:

![258d6b45ebcda4acc8293fb0be9714b062e5cb6d](media/258d6b45ebcda4acc8293fb0be9714b062e5cb6d.jpg){.image-inline width="316" height="317"}

 

After creating the P/V line, it will appear with two circles on each end. These circles can be used to adjust the line by clicking within the circle and dragging:

![0b9e0d612e604fad380a7cc40a443547806e1d81](media/0b9e0d612e604fad380a7cc40a443547806e1d81.jpg){.image-inline width="311" height="312"}

After the P/V line is properly in place, the final P/V diagram can be created from the P/V Regions panel. It is just a matter of generating the P/V diagram with the Generate P/V button:

![13f7ffec391f0f9340b6d1feb8b1c7889a12e69e](media/13f7ffec391f0f9340b6d1feb8b1c7889a12e69e.jpg){.image-inline width="538" height="279"}

The generation of the P/V diagram may take some time, but then the final diagram is displayed:

![a66c61bd7ca3a4de494cacda916c4075803c6fa7](media/a66c61bd7ca3a4de494cacda916c4075803c6fa7.jpg){.image-inline width="596" height="365"}

 

 

 

 

 

 

 

 

</div>



***





#### Regions in the Viewer 

Using CASA CRTF Regions in the Viewer

 

##### Regions and the Region Manager

 

------------------------------------------------------------------------

![b0a36b1085af0c104e3d638b0a2d23ee2a5b74d9](media/b0a36b1085af0c104e3d638b0a2d23ee2a5b74d9.png){.image-inline}

------------------------------------------------------------------------

CASA regions are following the CASA \'crtf\' standard as described in § [D](https://casa.nrao.edu/docs/cookbook/casa_cookbook015.html#chapter%3Aregionformat). CASA regions can be used in all applications, including **tclean** and [Image Analysis Tasks](imaging.ipynb#image-analysis).

<div class="alert alert-info">
**NOTE:** The CASA image analysis tasks will determine how a region is projected on a pixel image. The current CASA definition is that when the center of a pixel is inside the region, the full pixel is considered to be included in the region.  If the center of the pixel is outside the region, the full pixel will be excluded. Note that the CASA viewer behavior is not entirely consistent and for rectangles it assumes that *any* fractional pixel coverage will include the entire pixel. For other supported shapes (ellipses and polygons), however, ithe viewer adheres to the 'center of pixel' definition, consistent with the image analysis tools and tasks. 

For purely single-pixel work regions may not necessarily be the best choice and alternate methods may be preferable to using regions, eg. **ia.topixel**, **ia.toworld**, **ia.pixelvalue**.
</div>

<div class="alert alert-warning">
**ALERT:** Some region file specifications are not recognized by the viewer, the viewer only supports rectangles (box), ellipses, and polygons.
</div>

<div class="alert alert-info">
**NOTE**: A leading 'ann' (short for annotation) to a region definition indicates that it is for visual overlay purposes only.
</div>

<div class="alert alert-info">
**NOTE**: Whereas the region format is supported by all the data processing tasks, some aspects of the viewer implementation are still limited to rectangles, ellipses, and some markers. Full support for all region types is progressing with each CASA release.
</div>

Once one or more regions are created, the Region Manager Panel becomes active (see [Region Manager](#FigRegionManager)). Like the Position Tracking and Animator Panels, this can be docked or detached from the main viewer display. It contains several tabs that can be used to adjust, analyze, and save or load regions.

<div class="alert alert-info">
**NOTE**: Moving the mouse into a region will bring it into focus for the Spectral Profile or Histogram tools.
</div>

 

###### Region Creation, Selection, and Deletion

Within the display area, you can draw regions or select positions using the mouse. Regions can be created with the buttons marked as \'R\' in the mouse tool bar, which can be found on the top-left (second row of buttons) in the [Viewer Display Panel](imaging.ipynb#viewer-basics). The viewer currently supports creation of rectangles, ellipses, polygons, and the point. As usual, a mouse button can be assigned to each button as indicated by the small black square in each button (marking the left, middle, or right mouse button). An example is shown in [Region Selection](#FigRegionSelection).

Regions can be selected by SHIFT+click, de-selected by pressing SHIFT+click again. The bottom of the Region Manager Panel features a slider to switch between regions in the image. Regions can be removed by hovering over and pressing ESC or by pressing the buttons to the right side of the slider where the first button (trash can icon) deletes all regions and the far right button (red circle with a white X) deletes the region that is currently displayed in the panel.

------------------------------------------------------------------------

![9978959045808465057ed10e8fa21f4bbfeb7aa5](media/9978959045808465057ed10e8fa21f4bbfeb7aa5.png){.image-inline}

------------------------------------------------------------------------

Once regions are selected, they will feature little, skeletal squares in the corners of their boundary boxes. Selected regions can be moved by dragging with the mouse button and manually resized by grabbing the corners as handles. If more than one region is selected, all selected regions move together.

The Rectangle Region drawing tool currently enables the full functionality of the various Region Manager tabs (see below) as well as:

-   Region statistics reporting for images via double clicking (also shown in the Statistics tab of the Region Manager),
-   Defining a region to be averaged for the spectral profile tool (accessed via the Tools:Spectral Profile drop down menu or \"Open the Spectrum Profiler\" icon),
-   Flagging of MeasurementSets. Note that the Rectangle Region tool's mouse button must also be double-clicked to confirm an MS flagging edit.
-   Selecting Clean regions interactively (§ [5.3.5](https://casa.nrao.edu/docs/cookbook/casa_cookbook006.html#section%3Aim.clean.interactive))

The Polygon Region and Ellipse Region drawing have the same uses, except that polygon region flagging of a MeasurementSet is not supported.

 

###### Region Positioning

------------------------------------------------------------------------

![67074e8c42de2d0374c1c00629cdbaa989c3a1d3](media/67074e8c42de2d0374c1c00629cdbaa989c3a1d3.png){.image-inline}

------------------------------------------------------------------------

With at least one region drawn, the Region Manager becomes active. Using the Properties tab, one can manually adjust the position, annotation, and display style of the region. The entries labeled \"frames\" set which planes of the image cube the region persists through (regions can have a depth associated with them and will only appear in the frames listed in this range). One can manually adjust the width and height and the center of the box in the chosen units. The \'selected\' check box is an alternative way to the SHIFT+click to select a region. The \'annotation\' checkbox will place the \"ann\" string in front of the region ASCII output -- annotation regions are not be used for processing in, e.g. data analysis tasks. In the line and text tabs, one can set the style with which the region is displayed, the associated text, and the position and style of that text.

<div class="alert alert-info">
**NOTE**: Updating the position of a region will update the spectral profile shown if the Spectral Profile tool is open and the histogram if the Histogram tool is open. The views are linked. Dragging a region or adjusting it manually with the Properties tab is a good way to explore an image.
</div>

 

###### Region Statistics

------------------------------------------------------------------------

![31a405c0af27796c3ac3fdc1c87645e1d5649f5b](media/31a405c0af27796c3ac3fdc1c87645e1d5649f5b.png){.image-inline}

------------------------------------------------------------------------

One of the most useful features of defining a region is the ability to extract statistics characterizing the intensity distribution inside the region. You can see these in the Statistics tab of the of the Region Manager Panel (see [Region Statistics](#FigRegionStatistics)). This displays statistics for the current region in the current plane of the current image. When more than a single region is drawn, you can select them one by one and the Region Panel will update the statistics to reflect the currently selected region. All values are updated on the fly when the region is dragged across the image.

A similar functionality can be achieved by double clicking inside of a region. This will send statistics information for this region in all registered images to the terminal, looking something like this:

```python
(IRC10216.36GHzcont.image) image
          Stokes         Velocity            Frame          Doppler        Frequency
               I -2.99447e+11km/s             LSRK            RADIO      3.63499e+10
  BrightnessUnit         BeamArea             Npts              Sum             Flux
         Jy/beam          36.2521            27547     1.087686e-01     3.000336e-03
            Mean              Rms          Std dev          Minimum          Maximum
    3.948473e-06     3.723835e-04     3.723693e-04    -1.045624e-03     9.968892e-03
```

Listed parameters are Stokes, and the displayed channel Velocity with the associated Frame, Doppler and Frequency value. Sum, Mean, Rms, Std Deviation, Minimum, and Maximum value refer to those in the selected region and has the units as specified in BrightnessUnit. Npts is the number of pixels in the region, and BeamArea the beam size in pixels. FluxDensity is in Jy if the image is in Jy/beam. This is an easy way to copy and paste the statistical data to a program outside of CASA for further use.

Taking the RMS of the signal-free portion of an image or cube is a good way to estimate the noise. Contrasting this number with the maximum of the image gives an estimate of the dynamic range of the image. The FluxDensity measurement gives a way to use the viewer to do very basic photometry.

 

###### Saving and Loading Regions

------------------------------------------------------------------------

![f9d2972c79da5e3b12e60dc426b81baace780aae](media/f9d2972c79da5e3b12e60dc426b81baace780aae.png){.image-inline}

------------------------------------------------------------------------

The File tab in the Region Manager allows one to save or load selected regions, either individually or en masse. You can choose between CASA and DS9 region format. The default is a CASA region file (saved with a \'.crtf\' suffix, see § [D](https://casa.nrao.edu/docs/cookbook/casa_cookbook015.html#chapter%3Aregionformat)). The DS9 format does not offer the full flexibility and cannot capture Stokes and spectral axes. DS9 regions will only be usable as annotations in the viewer, they cannot be used for data processing in other CASA tasks. When saving regions, one can choose to save only the current region, all regions that were selected with SHIFT+click, or all regions that are visible on the screen.

<div class="alert alert-info">
**NOTE**: The load functionality for this tab will only become available once at least one region exists. To load a region when no regions exist, use the [Region Manager](#viewerData) window.
</div>

 

 

###### The Region Fit

The Viewer can attempt to fit a two-dimensional Gaussian to the emission distribution inside the currently selected region. To attempt the fit, go to the Fit tab of the Region Manager and click the \'gaussfit\' button in the bottom left of the panel. You can choose whether or not to fit a sky level (e.g., to account for a finite background, either astronomical, sky, or instrumental). After fitting the distribution, the Fit panel shows the results of the fit, the center, major and minor axis, and position angle of the Gaussian fit in pixels (I) and in world coordinates (W, RA and Dec). The detailed results of the fit will also appear in the terminal window, including a flag showing whether the fit converged.

 

###### The Region Histogram

------------------------------------------------------------------------

![71a8870c7e27d8e4226ab388d473c1f3cba13373](media/71a8870c7e27d8e4226ab388d473c1f3cba13373.png){.image-inline}

<div>

<table class="caption-table"><colgroup><col style="width: 50%" /><col style="width: 50%" /></colgroup><tbody><tr class="odd"><td style="text-align: left;">Type</td><td>Figure </td></tr><tr class="even"><td style="text-align: left;">ID</td><td>Histogram</td></tr><tr class="odd"><td style="text-align: left;">Caption</td><td>Histogram Tab: The histogram tab in the Region Manager. Right click to zoom. Hit SHIFT + Right Click to adjust the details of the histogram display.<div> </div></td></tr></tbody></table>

The viewer will automatically derive a histogram of the pixel values inside the selected region. This can be viewed using the Histogram tab of the of the Region Manager Panel. This is a pared down version of the full Histogram Tool. You can manipulate the details of the histogram plot by:

1.  Using the Right Click to zoom - either to the full range, a selected percentile, or a range that you have graphically selected by dragging the mouse.
2.  Hitting SHIFT + Right Click to open the histogram options. This lets you toggle between a logarithmic and linear y-axis, choose between a line, outline, or filled histogram, and adjust the number of bins.

The histogram will update as you change the plane of the cube or shift between regions.

</div>



***





#### Spectral Profiler 

Using the Spectral Profile GUI in the Viewer

 

##### The Spectral Profile Tool

 

------------------------------------------------------------------------

![f460a73178b5a78ab57f1664858a899137979ef2](media/f460a73178b5a78ab57f1664858a899137979ef2.png){.image-inline}

------------------------------------------------------------------------

<div>

<div class="alert alert-info">
**NOTE**: Make Sure That You Use the Radio Version! This section describes the 'Radio' version of the profiler. To be sure that you have the radio version of the tool selected (this may not be the default), click on the preferences icon (the 'gear' fourth from the left in the Spectral Profile tool) and make sure that the 'Optical' option is not checked. If you are using the Spectral Profile tool in the viewer for the very first time, you will also be prompted for a selection that will subsequently be kept for all future calls unless the preference is changed.
</div>

The Spectral Profile Tool consists of the [Spectral Profile Toolbar](#viewerSpectralProfileToolbar), a [main display area](#viewerSpectralProfile), and two associated tabs: [Spectral-Line Fitting](#viewerSpectralLineFitting) and [Line Overlays](#viewerLineOverlays).

Interaction With the Main Display Panel: For the Spectral Profile tool to work, a region or point must be specified in the main Viewer Display window. Use the mouse tools to specify a point, rectangle, ellipse, or polygon region. Alternatively, load a region file. The Spectral Profile tool will show a spectrum extracted from the region most recently highlight by the mouse in the main Viewer Display Panel. The method of extraction (i.e. mean, median, sum, or flux density) can be specified by the user using a drop down menu below the spectrum in the Spectral Profile window; the method of extraction is mean by default.

The Spectral Profile tool can also feed back to the Main Display Panel. By holding CTRL and right clicking in the spectrum, you will cause the Main Display Panel to jump to display the frequency channel corresponding to the spectral (x) coordinate of the region highlighted in the Spectral Profile tool. Holding CTRL and dragging out a spectral range while holding the right mouse button will queue a movie scrolling through images across that spectral range. You can achieve the same effect with the two-ended-arrow icon towards the right of the toolbar in the Spectral Profile window.

In both the [Spectral-Line Fitting](#viewerSpectralLineFitting) and [Line Overlays](#viewerLineOverlays) tabs, it may be useful to select a range in frequency or velocity. You can do this with the parallel lines-and-arrow icon (see below) or by holding shift, left clicking, and dragging out the range of interest. A shaded gray region should appear indicating your selection.

 

###### Spectral Profile Toolbar

------------------------------------------------------------------------

![b70f90030cd7ed87f57699da695279ded944c906](media/b70f90030cd7ed87f57699da695279ded944c906.png)

<div>

>Spectral Profile Toolbar: The toolbar for the Spectral Profile tool allows the user to save the spectrum, print or save the tool as an image, edit preferences (general, tool, legend), apply spectral smoothing, pan or zoom around the spectrum, select a range of interest, jump to a channel, or add a label.
  

The [Spectral Profile Toolbar](#FigSpectralToolbar) is the toolbar along the top of the Spectral Profile window. From left to right, the icons allow the user to:

-   (disk) export the current profile to a FITS or ASCII file
-   (printer) print the main window to a hard copy
-   (writing desk) save the panel as an image (PNG, JPG, PDF, etc.)
-   (gear) set plot preferences
-   (color wheel) set color preferences for the plot
-   (signpost) set legend preferences
-   (triangle) set the spectral smoothing method and kernel width
-   (arrows) pan the spectrum in the indicated direction NOTE: The arrow keys also allow one to pan using the keyboard.
-   (magnifying glass) zoom to the default zoom, in, and out NOTE: the +/- keys allow one to zoom with the keyboard
-   (parallel lines+arrows) drag out a range of interest in the spectrum, for use with fitting or line overlays.
-   (double-ended arrow) jump to a channel in the main viewer (single click) or define a range over which to play a movie in the viewer (with a drag).
    <div class="alert alert-info">
    **NOTE**: You can also jump to a channel with CTRL+Right Click and queue a movie by holding CTRL and dragging out a range while holding the right mouse button.
    </div>
-   (notepad and pencil) Add or edit a label on the plot. Click this icon to enter a mode where you can drag out a box to create a new annotation box or drag the corners of an existing one to resize it. You can edit the contents, color, and font of an existing annotation by right clicking on it and selecting \"Edit Annotation\" in the main Spectral Profile window.

------------------------------------------------------------------------

![3298dfa9eac7bb5c2b721bf9ea8613f647cb6b4c](media/3298dfa9eac7bb5c2b721bf9ea8613f647cb6b4c.png)

------------------------------------------------------------------------

<div>

[Spectral Profile Tool Preferences](#FigSpectralProfilePreferences) shows the setting dialogs accessible from the toolbar. This Preferences dialog opened by the \'gear\' icon allows the user to:

</div>

<div>

-   Toggle automatic scaling the x- and y-ranges of the plot.
-   Toggle the coordinate grid overlay in the background of the plot.
-   Toggle whether registered images other than the current one appear as overlays on the plot.
-   Toggle whether these profiles are plotted relative to the main profile (in development).
-   Toggle the display of tooltips (in development).
-   Toggle the plotting of a top axis.
-   Toggle between a histogram and simple line style for the plot.
-   Toggle between the radio and optical versions of the Spectral Profile tool Note: We discuss only the radio version here; this mainly impacts the Spectral Line Fitting and Collapse/Moments functionality..
-   Toggle the overplotting of a line showing the channel currently being displayed in the main Display Panel.

The Color Curve Preferences dialog opened by the \'color wheel\' icon allows the user to:

-   Select the color of the line marking the current channel shown in the main Display Panel.
-   Select the color used to overlay molecular lines from Splatalogue.
-   Select the color to plot the initial Gaussian estimate used in spectral line fitting.
-   Select the color used for the zoom rectangle.
-   Set a queue of colors used to plot the various data sets registered in the Display Panel.
-   Set a queue of colors to plot the set of Gaussian fits.
-   Set a queue of colors to plot the synthesized curve.

Two sets of preset colors, \"Traditional\" or \"Alternative\", are available or the user can define their own custom color palette.

The legend options opened by the \'signpost\' icon allow the user to toggle the plotting of a legend defining the curves shown in the main Spectral Profile window. Using a drop-down dialog, the legend can be placed in the top left corner of the plot, to the right of the plot, or below the plot. Toggling the color bar causes the color of the curve to be indicated either via a short bar or using the color of the text itself. Double click the names of the files or curves to edit the text shown for that curve by hand. A legend is only available if more than a single spectrum has been loaded.

The spectral smoothing option (triangle icon in the Spectral Profile toolbar) has two methods, \"Boxcar\" and \"Hanning\" with the selection of odd numbers for the smoothing kernel width in channels.

</div>

 

###### Main Spectral Profile Window 

------------------------------------------------------------------------

![b3606599a0ccbeb166c8d7fc12e6fd80a8c7a3ed](media/b3606599a0ccbeb166c8d7fc12e6fd80a8c7a3ed.png)

------------------------------------------------------------------------

The main window shows the spectrum extracted from the active region of the image in the main Display Panel. The spectra from the same region in any other registered images are also plotted if overlays are enabled. Menus along the bottom of the image allow the user to select how the spectrum is displayed. From left to right:

-   The units for the bottom spectral axis.
-   The units for the top spectral axis.
    <div class="alert alert-info">
    **NOTE**: Dual axes are enabled only if a single image is registered and the top axis option is enabled. In general, dual axes are not well-defined for mixed data sets. The exception is that open data cubes with matched frequency/spectral axes will allow dual axes.
    </div>
-   The units for the left intensity or flux axis
    <div class="alert alert-info">
    **NOTE**: The "Fraction of Peak" option allows for easy comparison of data with disparate intensity scales.
    </div>
-   The velocity reference frame used if a velocity axis is chosen for the top or bottom axis.
-   The method used to extract spectrum from the region --- a mean over all pixels in the region, a median, sum, or a sum converting units to get a flux density over the region (Jy).
-   Toggle the calculation and overplotting of error bars calculated from scatter in the data (rmse refers to root mean square error).

In addition to these drop-down menus, the main Spectral Profile window allows the user to do the following using keyboard and mouse inputs:

-   jump the main Display Panel window to a specified channel (CTRL+Right click): hold CTRL and right click in the spectrum. A marker will appear and the main Viewer Display Panel will jump to display that channel.
-   animate the main Display Panel in a movie across a frequency range (CTRL+Right click+drag): hold CTRL, Right click, and drag. The main Viewer Display panel will respond by showing a movie scrolling across the selected spectral channels.
-   zoom the Spectral Profile (+/-, mouse drag): Use the +/- keys to zoom in the same way as the toolbar buttons. Alternatively, press and drag the left mouse button. A yellow box is drawn onto the panel. After releasing the mouse button, the plot will zoom to the selected range.
-   pan the Spectral Profile (arrows): Use the arrow keys to pan the plot.
-   select a spectral range for analysis: hold shift, left click, and drag. A gray area will be swept out in the display. This method can be used to select a range for spectral line fitting or collapsing a data cube (in the Collapse/Moments window).

**NOTE**: If the mouse input to the Spectral Profile browser becomes confused hit the ESC key several times and it will reset.

 

###### Spectral-Line Fitting 

------------------------------------------------------------------------

![c29793490bd0d5d69c32f5485bc256ff586fd2ab](media/c29793490bd0d5d69c32f5485bc256ff586fd2ab.png)

>Spectral Line Fitting Tab: Using the Spectral Line Fitting Tab (found at the bottom left of the Spectral Profile Tool), the user can fit a combination of a polynomial and multiple Gaussian components. The range to be fit can be specified (gray region) manually or with a shift+click+drag. Initial estimates for each component may be entered by hand or specified via an initial estimates GUI. The results are output to a dialog and text file with the fit overplotted (here in blue) on the spectrum (with the possibility to save it to disk).
  

------------------------------------------------------------------------

![c648346d0ff0181b8a8984116c5357abdaed3bdf](media/c648346d0ff0181b8a8984116c5357abdaed3bdf.png)

>Specifying Initial Gaussian Estimates Graphically and Fitting Output: The top panel shows the graphical specification of initial estimates for Gaussian fitting. Slider bars specify the center, FWHM, and peak intensity for the initial estimate. The bottom panel shows the verbose output of the fitting.
  

The Spectral-Line Fitting tab, shown in [Spectral Line Fitting Tab](#FigSpectralLineFitting) and [Specifying Initial Gaussian Estimates Graphically and Fitting Output](#FigInitialGaussEstimates), allows the user to interactively fit a combination of Gaussian and polynomial profiles to the data shown in the Spectral Line Profile tool. The tool includes a number of options:

-   A drag-down menu labeled \"Curve\" at the top of the panel allows the user to pick which data set to fit.
-   The spectral range to fit can be specified by either holding shift+left click+dragging out a region in the main spectral profile window or by typing it manually into the boxes labeled Min and Max near the top left of the fitting panel.
-   Optionally multiple fits can be carried out once, fitting each spectrum in the region in turn. To enable this, check the \'MultiFit\' box.
-   Optionally a polynomial of the specified order may be fit. To do so, check the \'Polynomial\' fit check box and then specify the desired order.
-   The results may be saved to a text file. This text file should be specified before the fit is carried out. Click \'Save\' and then use the dialog to specify the file name. Note that the fit curve itself becomes a normal spectral profile data set and can be saved to disk using the toolbar (\'disk\' icon) after the fit.
-   One or more Gaussians can be fit, although results are presently most stable for one Gaussian. Specify the number of Gaussians in the box marked \"Gaussian Count\" and then enter initial estimates for the peak, center, and FWHM in the table below. Any of these values can be fixed for any of the Gaussians being fit. Initial estimates can also be manually specified by clicking \"Estimates\". This brings up an additional GUI window (see [Specifying Initial Gaussian Estimates Graphically](#FigInitialGaussEstimates)), where sliders can be used to specify initial estimates for each Gaussian to be fit.
-   For plotting purposes, one may wish to oversample the fit (i.e., plot a smooth Gaussian), you can do so by increasing the Fit Samples/Channel to a high number to finely sample the fit when plotting.

**NOTE**: Currently the tool works well for specifying a single Gaussian. Fitting multiple Gaussian components can become unstable.

 

###### Line Overlays 

------------------------------------------------------------------------

![4d513d4c6000fd2ea90d88b9a7b01d057d90114f](media/4d513d4c6000fd2ea90d88b9a7b01d057d90114f.png)

>Line Overlays Tab: The Line Overlay tab (found at the bottom left of the Spectral Profile Tool) allows users to query the CASA copy of the Splatalogue spectral line database. Enter the redshift of your source (far right of the panel), select an Astronomical Filter from the drop down menu, and use shift+click+drag to select a frequency range (or enter the range manually in the boxes marked Min and Max). The \"Search\" button will bring up a separate \"Search Results\" window, which can in turn be used to graph the candidate lines in the main Spectral Profile window (here CO v=0).
  

Each version of CASA includes a local version of the [Splatalogue](http://www.splatalogue.net) spectral line database and this can be used to identify and overplot spectral transitions. This feature, shown in [Line Overlay Tab](#FigLineOverlayTab), allows the user to search Splatalogue over the range of interest.

To overlay spectral lines:

1.  Select the Line Overlays tab in the Spectral Profiles tab.
2.  If you know it, enter the redshift or velocity of your source in the \"Doppler Shift\" panel. Otherwise, the lines will be overlaid assuming a redshift of 0.
3.  Specify a minimum and maximum frequency range to search, either by typing a range or by holding shift and left click and dragging out a range in the spectrum (you will see a gray box appear). If you don't specify a range, the tool will search over the frequency range of spectrum.
4.  Optionally, you may select an astronomical filter from the list (e.g., commonly used extragalactic lines or lines often found in hot cores, see [Splatalogue](http://www.splatalogue.net) for more information). This is usually a good idea because it pares the potentially very large list of candidate lines to a smaller set of reasonable candidates.
5.  Click \'Search\' and the Spectral Profile will search Splatalogue for a list of Spectral lines that fit the selected Astronomical Filter in the selected frequency range for the selected redshift. A \"Molecular Line Search Results\" dialog box will pop up showing the list of candidate lines.
6.  Highlight one or more of these transitions and click \'Graph Selected Lines\'. A set of vertical markers will appear in the main Spectral Profile window at the appropriate (redshifted) frequencies for the line.

NOTE: You will want to click \'Clear Lines\' between searches, especially if you update the redshift.

 

##### The Collapse/Moments Tool 

 

------------------------------------------------------------------------

![df2fa469ffe23c7fb70cba20c7821a5dfd3da869](media/df2fa469ffe23c7fb70cba20c7821a5dfd3da869.png)

------------------------------------------------------------------------

The CASA Viewer can collapse a data cube into an image, for instance allowing to look at the emission integrated along the z axis or the mean velocity of emission along the line of sight. You can access this functionality via the Collapse/Moments tool (accessed via the Tools drop down menu in the Main Display planel or via the four inward pointing arrows icon in the Main Toolbar) which is shown in [Collapse/Moments Tool](#FigMomentsTool).

The tool uses the same format as the Spectral Profile tool and will show the integrated spectrum of whatever region or point is currently selected in the Main Display Panel. To create a moment map:

1.  Select a range over which to integrate either manually using the left part of the window, by adding an interval and typing in the values into the boxes marked Min and Max or by holding SHIFT + Left Click and dragging out the range of interest.
2.  Pick the set of algorithms (listed in the box labeled \"Moment(s)\") that you will use to collapse the image along the z-axis. Clicking an option toggles that moment method, and the collapse will create a new image for each selected moment. For details on the individual collapse method, see the [immoments](global-task-list.ipynb#task_immoments) task for more details on each moment.
3.  The moment may optionally include or exclude pixels within a certain range (for example, you might include only values with signal-to-noise of three or greater when calculating the velocity dispersion). You can enter the values to include or exclude manually in the Thresholding window on the right or you can open a histogram tool to specify this range graphically by clicking Specify Graphically (before this can work, you must click \'Include\' or \'Exclude\').
4.  The results of the collapse can be saved to a file, which consists of a string specifying the specific moment tacked onto a root file name that you can specify using Select Root Output File.
5.  When you are satisfied with your chosen options, press \'Collapse\'.

**NOTE**: Even if you don't save the results of the collapse to a file, you can still save the map later using the Save as\... entry in the Data pull down menu from the Main Viewer Display Panel.

##### Interactive Position-Velocity Diagram Creation 

 

------------------------------------------------------------------------

![54c3e56ee2d8faacb841aa44931c419e06cf357e](media/54c3e56ee2d8faacb841aa44931c419e06cf357e.png)

------------------------------------------------------------------------

The route to create position-velocity cuts in the viewer is illustrated in [Position/Velocity Tool](#FigPVCutTool):

1.  Select the \'P/V cut\' tool from the Mouse Toolbar and use it to draw a line across a data cube along the axis you want to visualize.
2.  Open the Region Manager Panel and go to the pV tab. Highlight the cut you just drew. You should see the end point coordinates listed, along with information on the length and position angle of the cut. You can set the averaging width (in pixels) in a window at the bottom of the tab.
3.  When you are satisfied, hit \'Generate P/V\'. This will create a new Main Viewer Display Panel showing the position velocity cut. The axes should be Offset and velocity.

The new image can be saved to disk with the Data:Save as\... option.

</div>

</div>



***





#### Image Analysis in the Viewer 

Analysis Tools that are available in the Viewer

##### The Brightness Profile Tool

The viewer allows the user to draw multiple line segments using the \"Polyline drawing\" button, and this feature can be used to display one-dimensional brightness profiles of images, such as shown in the [Spatial Profile Tool](#FigSlicerTool). After double-clicking the last line segment, the \'Regions\' dock will then display a preview of the slice in the Spatial Profile tab and the full \"Spatial Profile Tool\" can be launched from there by clicking the \"Spatial Profile Tool\" button. This \"Spatial Profile Tool\" panel allows one to select the interpolation method to connect the pixels, and a number count for the sampled pixels in between markers. \'Automatic\' will show all pixels. The x-axis of the display can be either the distance along the slice or the X and Y coordinate projections (e.g. along RA and DEC). All segments are also listed at the bottom with their start and end coordinates, the distance and the position angles of each slice segment. The color tool can be used to give each segment a separate color.

------------------------------------------------------------------------

 

![247c9434dc6e7bfcd4508140a7efbe0653671d1d](media/247c9434dc6e7bfcd4508140a7efbe0653671d1d.png)

------------------------------------------------------------------------

 

##### The Histogram Tool

 

------------------------------------------------------------------------

![4bd76aea63562362de82fe6ddeeede1ce551317d](media/4bd76aea63562362de82fe6ddeeede1ce551317d.png)

------------------------------------------------------------------------

<div>

CASA can calculate and visualize a histogram of pixel values inside a region of interest. To examine this histogram, select Histogram from the Tools drop-down menu or the \'Histogram\' icon in the Main Toolbar (looks like a comb). This opens the full Histogram Tool; more limited versions are accessible from the Region Manager Panel, the graphical color table manipulation tool, and the Collapse/Moments tool.

The resulting Histogram Tool should look something like [Histogram Tool](#FigHistogramTool). The menus along the top (or the corresponding mouse clicks) allow one to:

-   Zoom to the full range, a selected percentile, or a graphical range.
-   Change the display of the histogram to show a log axis, display as either a line plot, an outline, or a filled histogram. Change the number of bins in the histogram, or clear the plot (to start over).
-   Configure what data are fed into the histogram. You can use this menu to tell the histogram to track the channel currently selected in the main Viewer Display Panel (click the \"Track Channel\" box) or to integrate across some range of channels (defaulting to the whole image). You can also switch the 2-D footprint used between the whole Image, the Selected Region, and All Regions.
-   Save (via the disk icon) an image of the histogram to a graphical file on disk.

The Histogram Tool also allows you to fit the distribution using either a Gaussian or a Poisson distribution, for example to estimate the noise in the image (a Gaussian will be a good choice to describe the noise in most radio data cubes). You can specify initial estimates or let the program generate initial guesses. The fit is then overplotted on the histogram (colors can be adjusted by clicking the color wheel icon in the toolbar) and details of the fit are printed to the text window below the fit button.

</div>

<div>

 

##### The Two-Dimensional Fitting Tool

 

------------------------------------------------------------------------

![df2fff474cbd464f25d5e9bb87cd7107f61a5b87](media/df2fff474cbd464f25d5e9bb87cd7107f61a5b87.png)

------------------------------------------------------------------------

 

CASA can fit two-dimensional Gaussians to an intensity distribution, and the Two-Dimensional Fitting Tool in the Viewer exposes this functionality interactively. This tool, accessed by the \'blue circles\' icon in the Main Toolbar or the Tools:Fit menu item, has an interface like that shown in [Two-Dimensional Fitting Tool](#FigTwoDFitting). The interface exposes several options:

1.  You can select whether to fit only the selected region or the whole image plane and specify which channel of the cube you want to operate on.
    <div class="alert alert-info">
    **NOTE**: The two dimensional fitter only operates on a single channel at a time.
    </div>
2.  Initial Estimates: The box in the top left corner allows the user to specify initial estimates by feeding in a file. The easiest way to make an appropriate file is to edit an existing one. Even easier, you can use the Find Sources button to automatically generate a temporary file of initial estimates.
3.  Pixel Range: You can choose to only include a certain range of pixel intensity values in the fit. For example, you might choose to only fit Gaussians to pixels a few times above the measured noise level. You can use the Specify Graphically button to bring up an interactive histogram of the region (a reduced functionality version of the full Histogram Tool).
4.  Output: You can choose to save the output of the fit as a file to the specified directory and to subtract the fit from the image and to subtract the fit from the original, creating a Residual Image that gets stored as a CASA image and automatically loaded into the viewer. This gives a way to tell how well your fit describes the total emission.
5.  Visualization: You can toggle whether the fit is displayed on the viewer or not and change the color of the marker.

Click Fit to start the fit. If the fit does not converge, try improving your initial estimates and fitting again.

</div>



***





#### Printing from the Viewer 

Printing to a file directly from the Viewer

You can select Data:Print from the drop down menu or click the printer icon in the Main Toolbar to bring up the Viewer Print Manager. From this panel, you can \"Print\" the contents of Display Panel to a hardcopy or \"Save\" them as an image in a format selected from the drop-down menu at the bottom left of the window.

<div class="alert alert-info">
**NOTE**: The save feature will overwrite the file in question without prompting.
</div>

The Viewer Print Manager allows you to adjust the DPI, orientation, and page format (Output Media) for Postscript or PDF files and to scale the image to a desired pixel size for other images.

To achieve the best output it is usually advisable to adjust the settings in the Viewer Print Manager (printer icon in the Main Toolbar), Data Display Options (Data:Adjust Data Display), and Viewer Canvas Manager (wrench+\"P\" icon in the Main Toolbar). For PDF and Postscript output, turning the DPI up all the way yields the best-looking results. For other images, a white background often makes for better looking images than the default black. It is often necessary to increase the Line Width in the Axis Label Properties (in the Data Display Options panel) to ensure that the labels will be visible when printed. Increasing from the default of 1.4 to a value around 2 often works well.

[Printing to a File](#FigPrintingImage) shows an example of printing to a file while adjusting the Data Display Options and Viewer Canvas Manager to improve the appearance of the plot.

------------------------------------------------------------------------

![173061d000f561f265692b5ea0e5a2cfa7345bfe](media/173061d000f561f265692b5ea0e5a2cfa7345bfe.png)

 



***





#### Scripting using imview 

Plotting to a file without invoking the Viewer GUI



##### Image Viewer (imview)

The **imview** task offers scriptable access to many **viewer** options. This enables the production of customized plots without invoking the GUI and allows one to open the **viewer** to a carefully selected state.

**imview** has the following inputs:

```
#imview :: View an image
raster      =    {}     #(Optional) Raster filename (string)
                        #or complete raster config
                        #dictionary. The allowed dictionary
                        #keys are file (string), scaling
                        #(numeric), range (2 element numeric
                        #vector), colormap (string), and
                        #colorwedge (bool).
contour     =    {}     #(Optional) Contour filename (string)
                        #or complete contour config
                        #dictionary. The allowed dictionary
                        #keys are file (string), levels
                        #(numeric vector), unit (float), and
                        #base (float).
zoom        =    1      #(Optional) zoom can specify
                        #intermental zoom (integer), zoom
                        #region read from a file (string) or
                        #dictionary specifying the zoom
                        #region. The dictionary can have two
                        #forms. It can be either a simple
                        #region specified with blc (2 element
                        #vector) and trc (2 element vector)
                        #[along with an optional coord key
                        #("pixel" or "world"; pixel is the
                        #default) or a complete region
                        #rectangle e.g. loaded with
                        #"rg.fromfiletorecord( )". The
                        #dictionary can also contain a
                        #channel (integer) field which
                        #indicates which channel should be
                        #displayed.
axes        =    -1     #(Optional) this can either be a
                        #three element vector (string) where
                        #each element describes what should
                        #be found on each of the x, y, and z
                        #axes or a dictionary containing
                        #fields "x", "y" and "z" (string).
out         =    {}     #(Optional) Output filename or
                        #complete output config dictionary.
                        #If a string is passed, the file
                        #extension is used to determine the
                        #output type (jpg, pdf, eps, ps, png,
                        #xbm, xpm, or ppm). If a dictionary
                        #is passed, it can contain the
                        #fields, file (string), scale
                        #(float), dpi (int), or orient
                        #(landscape or portrait). The scale
                        #field is used for the bitmap formats
                        #(i.e. not ps or pdf) and the dpi
                        #parameter is used for scalable
                        #formats (pdf or ps).
```

The *raster* and *contour* parameters specify which images to load and how these images should be displayed. These parameters take python dictionaries as inputs. The fields in these dictionaries specify how the image will be displayed.

An example call to **imview** looks like this:

```
imview(raster={'file': 'ngc5921.clean.image','range': [-0.01,0.03],'colormap': 'Hot Metal 2','scaling': -1},
      contour={'file': 'ngc5921.clean.image'},
      axes={'x':'Declination'},
      zoom={'channel': 7, 'blc': [75,75], 'trc': [175,175],'coord': 'pixel'},
      out='myout.png')
```

The argument to *raster* is enclosed in the curly braces { }. Within these braces are a number of \"key\":\"value\" pairs. Each sets an option in the **viewer**, with the GUI parameter to set defined by the \"key\" and the value to set it to defined by \"value.\" In the example above, file='ngc5921.clean.image' sets the file name of the raster image, range= \[-0.01,0.03\] sets the range of pixel values used for the scaling.

*contour* works similarly to *raster* but can accept multiple dictionaries in order to produce multiple contour overlays on a single image. To specify multiple contour overlays, simply pass multiple dictionaries (comma delimited) in to the *contour* argument:

```
contour={'file': 'file1.image', 'levels': [1,2,3] }, {'file': 'file2.image', 'levels': [0.006, 0.008, 0.010] }
```

*zoom* specifies the part of the image to be shown. The example above specifies a channel as well as the top right corner \"trc\" and the bottom left corner \"blc\" of the region of interest.

*axes* defines what axes are shown. By default, the viewer will show 'x':'Right Ascension', 'y':'Declination' but one may also view position-frequency images.

*out* defines the filename of the output, with the extension setting the file type.

Currently, the following parameters are supported:

```
raster  -- (string) image file to open
           (dict)   file (string)     => image file to open
                    scaling (float)   => scaling power cycles
                    range (float*2)   => data range
                    colormap (string) => name of colormap
                    colorwedge (bool) => show color wedge?
contour -- (string) file to load as a contour
           (dict)   file (string)     => file to load
                    levels (float*N)  => relative levels
                    base (numeric)    => zero in relative levels
                    unit (numeric)    => one in the relative levels
zoom    -- (int)    integral zoom level
           (string) region file to load as the zoom region
           (dict)   blc (numeric*2)   => bottom left corner
                    trc (numeric*2)   => top right corner
                    coord (string)    => pixel or world
                    channel (int)     => channel to display
           (dict)   <region record>   => record loaded
                                         e.g., rg.fromfiletorecord( )
axes    -- (string*3) dimension to display on the x, y, and z axes
           (dict)   x                 => dimension for x-axes
                    y                 => dimension for y-axes
                    z                 => dimension for z-axes
out     -- (string) file with a supported extension
                    [jpg, pdf, eps, ps, png, xbm, xpm, ppm]
           (dict)   file (string)     => filename
                    format (string)   => valid ext (filename ext overrides)
                    scale (numeric)   => scale for non-eps, non-ps output
                    dpi (numeric)     => dpi for eps or ps output
                    orient (string)   => portrait or landscape
```

Examples are also found in help **imview**.

 



##### Scripting using the viewer tool

The **viewer** tool may also be used to generate simple figures that can be directly saved to an output image file format (png, jpg, etc). Below is an example.

```
def dispimage(imname=''): 
    qq = viewertool() 
    qq.load(imname) 
    qq.datarange(range=[-0.01,1.1]) 
    qq.colormap(map='Rainbow 3') 
    qq.colorwedge(show=True) 
    qq.zoom(blc=[100,150], trc=[600,640]) 
    qq.output(device='fig_trial.png',format='png') 
    qq.close()
```

Note that only basic controls are available via the **viewertool** interface. For additional customization via a script, please see the following section describing \"Using Viewer state files within a script\". 

 



##### Using Viewer state files within a script 

In order to access the full flexibility of the GUI interface in customizing the **viewer** settings and display options, a hand-crafted **viewer** state can be saved, edited, and subsequently restored/rendered via a script that then allows the saving of the figure to a file on disk.

For example:

Step 1 : Customize the **viewer** by hand. For example, choose to open an image, customize the display data ranges, choose a colormap, change axis label properties, change the units of the movie axis label, edit the panel background color, adjust margins and and resize the panel window.

Step 2 : Click on the \"save viewer state\" button on the top control panel of the **viewer**. This will save a .rstr file, which is an xml file containing a complete description of the current state of the **viewer**.  

Step 3 : Edit the text xml file as required. The simplest operation is to search and replace the name of the CASA image being opened. More complex editing can be done via stand-alone editing scripts perhaps using standard python xml parser/editing packages.

Step 4 : Restore the state of the **viewer** from the edited xml .rstr file, using the **viewertool** as follows to subsequently save a .png figure to disk.

```
CASA <1>: vx = viewertool()
CASA <2>: x = vx.panel('mystate.rstr')
CASA <3>: vx.output('myfig.png',panel=x)
```

(There are two interactive ways to restore the **viewer** state as well. The first is by starting up the **viewer** with no image chosen, and then clicking on the \"restore viewer state\" button and choosing this .rstr file to open. Alternately, the **casaviewer** can itself be opened by supplying this .rstr file as the \'image\' to open.)

 



***





### Image Analysis 

How to analyze images in CASA



***





#### CASA Images 

Structure of CASA images

CASA images are stored as tables and can be accessed with CASA tasks and tools. For example, images can be visualized with the CASA **viewer **(see \'[Image Cube Visualization](imaging.ipynb#image-cube-visualization)\'), and image metadata can be listed and edited with the **imhead **task. CASA tasks and tools also enable further processing of images, including e.g. the computation of statistics including spectral indices and polarization properties, transformation onto different spatial coordinates, spatial resolutions, and spectral frames, and many other processes (see the following page, '[Dealing with Images](imaging.ipynb#dealing-with-images)' for a description of tasks that operate on CASA images).

 

###### Image Headers

Image Headers contain metadata on the observation -- e.g. the observing date, pointing position, object observed, etc., and the resulting image -- e.g. the restoring beam size, image intensity units, spatial coordinate system, spectral parameters, stokes parameters, etc.  Header metadata tells the user what is in the image, and is used by the CASA **viewer** and other tasks to set the data array on the correct spatial and spectral coordinates, assign the intensity values correctly, and otherwise properly handle the data cube. 

Image Headers can be accessed and edited via the **imhead** task and the **msmd** tool.  Header data can also be inspected with the **casabrowser**.  See the page on \'[Image Headers](imaging.ipynb#dealing-with-image-headers)\' for further details. 

 

###### Image Axes / Velocity Systems

CASA images typically have the following axis order (python indices are zero-based): Axis 0 = RA, 1 = DEC, 2 = Stokes, 3 = Frequency. The spatial axes can alternately contain GLON/GLAT or other coordinate systems.  The spectral axis of images created in CASA is always in frequency units. In addition, one or more velocity systems can be *added* to relabel the spectral axis. When images are imported into CASA from FITS files rather than generated within CASA itself, the above conventions may not apply. See the page on '[Image Import and Export](imaging.ipynb#image-and-image-value-import-and-export)' for further details on importing and exporting FITS files.

The spatial and spectral axes in CASA images can be modified using CASA tasks and tools described in the '[Reformat Images](imaging.ipynb#image-reformatting)' page. 

 

###### Image Masks

Internal Image Masks are stored as Boolean True/False cubes within the images themselves. There can be multiple masks stored in each data cube and one of them is defined to be the \'default\' mask. The default mask is the one visible when the image is displayed, e.g. in the **viewer**, and that is applied for operations on images. All masks have labels, such as mask0 etc. and they can be selected by specifying the image name followed by the mask name, separated by a colon. For example, \'mask1\' in \'image.im\' is used when specifying the image as \'image.im:mask1\'. Available masks can be listed with the task **makemask** which can also assign any mask as the default. The same task can also be used to export masks into separate CASA zero/non-zero cubes and to import such cubes as Boolean masks inside images. In addition, **makemask** enables the creation of masks from [image regions](imaging.ipynb#region-files).  More information on masks is provided on the \'[Image Masks](imaging.ipynb#image-masks)\' and \'[LEL Masks](imaging.ipynb#lel-masks)\' CASAdocs pages. 

 

###### CASA Regions

CASA Regions can be specified through simple lists in LEL (e.g. region = 'box\[\[108, 108,\], \[148, 148\]\]') or through CASA Region Text Format (CRTF) files, which are text files that contain one or more regions with specific shapes (e.g. ellipses and rectangles), sizes, and other properties. These files can be used to specify the region of an image in which to operate, and they can easily be modified by the user or converted to CASA image masks (Boolean data cubes) using the **makemask** task. More information on CRTF files is available on the \'[Region Files](imaging.ipynb#region-files)\' page.    



***





#### Dealing with Images 

CASA tasks for image analysis

Image cubes in CASA can be manipulated and analyzed in various ways mainly using tasks with an \'im\' prefix and with the **image** CASA tool. Frequently, the tasks and tools handle CASA, FITS, and MIRIAD images, but we recommend using images in the CASA format. 

In the following pages, useful image analysis tasks are introduced that span import/export tasks, image information, reformatting, mathematical operations, and spatial and spectral fitting. Available image analysis tasks include: 

-   **imhead** --- summarize and manipulate the "header" information in a CASA image
-   **imsubimage** --- Create a (sub)image from a region of the image
-   **imcontsub** --- perform continuum subtraction on a spectral-line image cube
-   **imfit** --- image plane Gaussian component fitting
-   **immath** --- perform mathematical operations on or between images
-   **immoments** --- compute the moments of an image cube
-   **impv** --- generate a position-velocity diagram along a slit
-   **imstat** --- calculate statistics on an image or part of an image
-   **imval** --- extract the data and mask values from a pixel or region of an image
-   **imtrans** --- reorder the axes of an image or cube
-   **imcollapse** --- collapse image along one or more axes by aggregating pixel values along that axis
-   **imregrid** --- regrid an image onto the coordinate system of another image
-   **imreframe** --- change the frame in which the image reports its spectral values
-   **imrebin** --- rebin an image
-   **specsmooth** --- 1-dimensional smooth images in the spectral and angular directions
-   **imsmooth** --- 2-dimensional smooth images in the spectral and angular directions
-   **specfit** --- fit 1-dimensional Gaussians, polynomial, and/or Lorentzians models to an image or image region
-   **specflux** --- Report details of an image spectrum.
-   **plotprofilemap** --- Plot spectra at their position
-   **rmfit** --- Calculation of rotation measures
-   **spxfit** --- Calculation of Spectral Indices and higher order polynomials
-   **makemask** --- image mask handling
-   **slsearch** --- query a subset of the Splatalogue spectral line catalog
-   **splattotable** --- convert a file exported from Splatalogue to a CASA table
-   **importfits** --- import a FITS image into a CASA image format table
-   **exportfits** --- write out an image in FITS format

There are other tasks which are useful during image analysis. These include:

-   **viewer** --- there are useful region statistics and image cube slice and profile capabilities in the viewer



***





#### Common Task Parameters 

Description of parameters commonly used in Image Analysis tasks

##### Common Image Analysis Task Parameters

Certain parameters are present in many image analysis tasks. These include: 

 

###### *imagename*

The *imagename* parameter is used to specify the image(s) on which a task should operate.  In most tasks, this will be a string containing the image name, but in some tasks, this can be a list of strings, as for example, in **immath**. Most image analysis tasks accept both CASA images and FITS images, although we recommend working with CASA images. 

 

###### *outfile* 

The *outfile* parameter specifies the name (in string format) of the file that the task should output.  This parameter is only present in tasks that produce processed files (typically images) as output.  It will therefore not be present for tasks that return python dictionaries, arrays, or other data types.  

 

###### *axes*

The *axes* parameter is used to specify the image axes that the task should operate on, and the user should input a list of integers for this (e.g. \"axes = \[0,1\]\").  CASA images typically have the following axis order (python indices are zero-based): Axis 0 = RA, 1 = DEC, 2 = Stokes parameter, and 3 = Frequency. The **imhead **task can be used to confirm the axis specifications in the data cube of interest, and the axes may differ from the above sequence, particularly when using FITS data cubes or CASA images that were converted from FITS files.  In the examples, we assume the above axis order. 

To obtain statistics across RA and DEC for each velocity channel, the user would run the **imstat** task (**imstat** stands for \"image statistics\") with \"axes = \[0,1\]\".  To obtain statistics over the spectral axis, one would run imstat with *axes = \[3\]*.  

  

###### *box, chans, stokes*

The box, chans, and stokes parameters are used to select parts of an image cube for the task to operate on.  If a box is applied, the task will operate only on a specific spatial region (e.g. *box = \'100,100,200,200\'* will only operate on pixels in the range (100,100) \<= (x,y) \<= (200,200) ). If specific channels are specified through chans, the task will select that segment of the spectral axis (e.g. *chans = \'30\~45\'* will operate on channels 30 through 45).  In the same way, stokes selects specific Stokes parameter axes, as e.g. s*tokes = \'I\'*.  Further detail is provided in the \'[Image Selection Parameters](imaging.ipynb#image-selection-parameters)\' page.  

 

###### *mask* 

The mask parameter tells the task to operate on specific segments of the image cube, as set by a mask.  The input for the mask parameter may be a conditional statement in LEL string format (e.g. *mask = \' \"ngc5921.im \> 0.5\'*, which selects all pixels in that image that have values larger than 0.5 and zeros out all other pixels), or may be a Boolean True/False cube or an Integer zero/non-zero cube. The task will not operate on pixels that are \"masked\", or zeroed out.  See the \'[Image Masks](imaging.ipynb#image-masks)\' page for more detail and examples of usage.  


###### *stretch* 

This parameter can be True or False, with a default value of False.  Set *stretch = True* when applying a single-plane mask to a full image cube.  As an example, if you have a mask in a single spectral channel image that you wish to apply to all spectral channels in a cube, you would \"stretch\" the mask over all of the channels. The mask can also be stretched over all Stokes parameter planes for polarization images.

 

##### Returned Python Dictionaries 

Many image analysis tasks return python dictionaries with information that is also printed to the logger. The dictionaries can be assigned to a variable and then used later for other scripting purposes. In the following the output of imstat is assigned to the python dictionary \'test_stats\':

```
CASA <20>: test_stats=imstat(imagename='test.image')

CASA <21>: test
Out[21]:
{'blc': array([0, 0, 0, 0], dtype=int32),
'blcf': '17:45:40.899, -29.00.18.780, I, 1.62457e+10Hz',
'max': array([ 0.49454519]),
'maxpos': array([32, 32, 0, 0], dtype=int32),
'maxposf': '17:45:40.655, -29.00.15.580, I, 1.62457e+10Hz',
'mean': array([ 0.00033688]),
'medabsdevmed': array([ 0.]),
'median': array([ 0.]),
'min': array([-0.0174111]),
'minpos': array([15, 42, 0, 0], dtype=int32),
'minposf': '17:45:40.785, -29.00.14.580, I, 1.62457e+10Hz',
'npts': array([ 4096.]),
'q1': array([ 0.]),
'q3': array([ 0.]),
'quartile': array([ 0.]),
'rms': array([ 0.00906393]),
'sigma': array([ 0.00905878]),
'sum': array([ 1.37985568]),
'sumsq': array([ 0.3365063]),
'trc': array([63, 63, 0, 0], dtype=int32),
'trcf': '17:45:40.419, -29.00.12.480, I, 1.62457e+10Hz'}
```

A description of how to deal with Python dictionaries is given in \'[Python and CASA](http://casa.nrao.edu/casadocs/stable/usingcasa/python-and-casa#figid-casapythondictionaries)\'.

 



***





#### Image Import/Export 

Converting images between CASA image and FITS format

The **exportfits** and **importfits** tasks enable conversion between CASA images and FITS data.  The **exportfits** task allows you to write your CASA image to a FITS file that other packages can read, and the **importfits** task converts existing FITS files into CASA images.  While many image analysis tasks can operate on FITS files, we recommend converting to CASA images for processing and analysis purposes.


##### Export CASA Image to FITS (exportfits) 

The **exportfits** task is used to export a CASA image to FITS format. The inputs are:

```
#exportfits :: Convert a CASA image to a FITS file
imagename           =         ''        #Name of input CASA image
fitsimage           =         ''        #Name of output image FITS 
                                        #file
velocity            =      False        #Use velocity (rather than 
                                        #frequency) as spectral axis
optical             =      False        #Use the optical (rather than 
                                        #radio) velocity convention
bitpix              =        -32        #Bits per pixel
minpix              =          0        #Minimum pixel value (if 
                                        #minpix > maxpix, value is 
                                        #automatically determined)
maxpix              =         -1        #Maximum pixel value (if 
                                        #minpix > maxpix, value is 
                                        #automatically determined)
overwrite           =      False        #Overwrite pre-existing
                                        #imagename
dropstokes          =      False        #Drop the Stokes axis?
stokeslast          =       True        #Put Stokes axis last in
                                        #header?
history             =       True        #Write history to the FITS
                                        #image?
dropdeg             =      False        #Drop all degenerate axes (e.g.
                                        #Stokes and/or Frequency)?
```

<div class="alert alert-warning">
**Alert:** The spectral axis of CASA images is nearly always in frequency rather than velocity. Velocities are computed only as a secondary mapping of the spectral channels with respect to a rest frequency.  If velocity units are desired and the user sets* velocity = True*, exportfits will write the spectral axis in velocity units *instead* of in frequency units.  The **exportfits** task will not output a FITS file with multiple spectral coordinate systems. 
</div>

As a simple example of an **exportfits** command, the following will write the CASA image (\'ngc5921.clean.image\') as a FITS file (\'ngc5921.clean.fits\').  In this case, the default parameter values will be adopted, so that the resulting FITS file will have the same axis order, number of pixels, etc. as the original CASA image.

```
exportfits(imagename='ngc5921.clean.image',outfile='ngc5921.clean.fits')
```

In some cases, the user may wish to use the *dropstokes,* *stokeslast*, and/or *dropdeg* parameters in order for the FITS image to be compatible with certain external applications. The *dropdeg *parameter will remove the frequency axis if it has a length of one channel, and/or it will drop the Stokes axis if that has a length of one (i.e. only one Stokes parameter is present). This would be useful, for example, for continuum data so that other programs will interpret it as a 2-D image rather than a cube.

See **[exportfits](global-task-list.ipynb#task_exportfits)** in the Global Task List for examples in which these and other parameters are specified.  

 

##### FITS Image Import (importfits)

The **importfits** task enables the user to import a FITS image into CASA image table format. It is not essential to generate a CASA image file if you intend to simply view the image, as the CASA **viewer** can read FITS images, however we recommend importing to CASA image format for analyzing images with CASA. The inputs for **importfits** are:

```
#importfits :: Convert an image FITS file into a CASA image
fitsimage           =         ''        #Name of input image FITS file
imagename           =         ''        #Name of output CASA image
whichrep            =          0        #If fits image has multiple
                                        #coordinate reps, choose one.
whichhdu            =          0        #If its file contains
                                        #multiple images, choose one.
zeroblanks          =       True        #Set blanked pixels to zero (not NaN)
overwrite           =      False        #Overwrite pre-existing imagename
defaultaxes         =      False        #Add the default 4D
                                        #coordinate axes where they are missing
defaultaxesvalues   =         []        #List of values to assign to
                                        #added degenerate axes when
                                        #defaultaxes=True (ra,dec,freq,stokes)
```

As a simple example, the following command would create a CASA image named \'ngc5921.clean.image\' from the FITS file \'ngc5921.clean.fits\':

```
importfits(fitsimage='ngc5921.clean.fits',imagename='ngc5921.clean.image')
```

See **[importfits](global-task-list.ipynb#task_importfits)** in the Global Task List for more complex examples. 

 

##### Extracting data from an image (**imval**)

The **imval** task will extract the values of the data and mask from a specified region of an image and place in the task return value as a Python dictionary. The inputs are:

```
#imval :: Get the data value(s) and/or mask value in an image.
imagename  =      ''   #Name of the input image
region     =      ''   #Image Region.  Use viewer
box        =      ''   #Select one or more box regions
chans      =      ''   #Select the channel(spectral) range
stokes     =      ''   #Stokes params to image (I,IV,IQU,IQUV)
```

Area selection using [box](#region-selection--box-) and [region](#regions--region-) is detailed above. By default, *box=\' \'* will extract the image information at the reference pixel on the direction axes. [Plane selection](#plane-selection--chans--stokes-) is controlled by *chans* and *stokes*. By default, *chans=\' \'* and *stokes=\' \'* will extract the image information in all channels and Stokes planes.For instance,

```
xval = imval('myimage', box='144,144', stokes='I' )
```

will extract the Stokes I value or spectrum at pixel 144,144, while

```
xval = imval('myimage', box='134,134.154,154', stokes='I' )
```

will extract a 21 by 21 pixel region. Extractions are returned in NumPy arrays in the return value dictionary, plus some extra elements describing the axes and selection:

```python

CASA <2>: xval = imval('ngc5921.demo.moments.integrated')

CASA <3>: xval
  Out[3]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [128, 128, 0, 0],
 'data': array([ 0.89667124]),
 'mask': array([ True], dtype=bool),
 'trc': [128, 128, 0, 0],
 'unit': 'Jy/beam.km/s'}
```

extracts the reference pixel value in this 1-plane image. Note that the \'*data\'* and \'*mask\'* elements are NumPy arrays, not Python lists. To extract a spectrum from a cube:

```python

CASA <8>: xval = imval('ngc5921.demo.clean.image',box='125,125')

CASA <9>: xval
  Out[9]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [125, 125, 0, 0],
 'data': array([  8.45717848e-04,   1.93370355e-03,   1.53750915e-03,
         2.88399984e-03,   2.38683447e-03,   2.89159478e-04,
         3.16268904e-03,   9.93389636e-03,   1.88773088e-02,
         3.01138610e-02,   3.14478502e-02,   4.03211266e-02,
         3.82498614e-02,   3.06552909e-02,   2.80734301e-02,
         1.72479432e-02,   1.20884273e-02,   6.13593217e-03,
         9.04005766e-03,   1.71429547e-03,   5.22095338e-03,
         2.49114982e-03,   5.30831399e-04,   4.80734324e-03,
         1.19265869e-05,   1.29435991e-03,   3.75700940e-04,
         2.34788167e-03,   2.72604497e-03,   1.78467855e-03,
         9.74952069e-04,   2.24676146e-03,   1.82263291e-04,
         1.98463408e-06,   2.02975096e-03,   9.65532148e-04,
         1.68218743e-03,   2.92119570e-03,   1.29359076e-03,
        -5.11484570e-04,   1.54162932e-03,   4.68662125e-04,
        -8.50282842e-04,  -7.91683051e-05,   2.95954203e-04,
        -1.30133145e-03]),
 'mask': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool),
 'trc': [125, 125, 0, 45],
 'unit': 'Jy/beam'}
```

To extract a region from the plane of a cube:

```python
CASA <13>: xval = imval('ngc5921.demo.clean.image',box='126,128,130,129',chans='23')

CASA <14>: xval
  Out[14]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [126, 128, 0, 23],
 'data': array([[ 0.00938627,  0.01487772],
       [ 0.00955847,  0.01688832],
       [ 0.00696965,  0.01501907],
       [ 0.00460964,  0.01220793],
       [ 0.00358087,  0.00990202]]),
 'mask': array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]], dtype=bool),
 'trc': [130, 129, 0, 23],
 'unit': 'Jy/beam'}

CASA <15>: print xval['data'][0][1]
0.0148777160794
```

In this example, a rectangular box was extracted, and you can see the order in the array and how to address specific elements.

 



***





#### Image Headers 

Viewing and manipulating image headers and histories

As summarized in the [CASA Images](imaging.ipynb#casa-images) page, an image header contains information on the observation -- e.g. the observing date, pointing position, object observed, etc., and the resulting image -- e.g. the restoring beam size, image intensity units, spatial coordinate system, spectral parameters, stokes parameters, etc..  Header metadata can also store notes on the observation and/or calibration and image processing.  The header tells the user what is in the image and is used by the CASA **viewer** and other tasks to set the data array on the correct spatial and spectral coordinates, assign the intensity values correctly, and otherwise properly handle the data cube. 

FITS image headers can be read in CASA using the **listfits** task, whereas CASA image headers can be read and edited using the **imhead** task.  Additionally, the **imhistory** task can be used to view the history of the image, i.e. what operations or processes have been applied to it. These three tasks are described and demonstrated below.

 



##### List the Header of a FITS image (**listfits**)

<div class="alert alert-info">
CASA can frequently read and write image FITS files directly. Nevertheless, it is advisable to convert the images to the CASA format first with **importfits** for some tasks and applications.
</div>

The task **listfits** can be used to display the Header Data Unit (HDU) of a FITS image. The input includes only the name of the of the FITS file, as follows:

```
#listfits :: List the HDU and typical data rows of a fits file:
fitsfile            =         ''        #Name of input fits file
```

The logger will output the full FITS HDU.  The example below shows the logger output for a Digital Sky Survey Image, which we have truncated somewhat due to the length of the output:

```python
###############################################Begin Task: listfits           #####listfits(fitsfile="dss.test.fits") read fitsfile=dss.test.fits d 29: DATE-OBS= '1998-11-24T11:83:00' /Observation: Date/Time                          time. Primary Array HDU ------>>> d 156: DATAMIN =                 2701 /GetImage: Minimum returned pixel value           value has wrong data type. erted to type double. d 157: DATAMAX =                22189 /GetImage: Maximum returned pixel value           value has wrong data type. erted to type double. SIMPLE  =                      T /FITS: Compliance BITPIX  =                     16 /FITS: I*2 Data NAXIS   =                      2 /FITS: 2-D Image Data NAXIS1  =                    891 /FITS: X Dimension NAXIS2  =                    893 /FITS: Y Dimension EXTEND  =                      T /FITS: File can contain extensions DATE    = '2016-11-17' /FITS: Creation Date ORIGIN  = 'STScI/MAST' /GSSS: STScI Digitized Sky Survey SURVEY  = 'POSSII-F' /GSSS: Sky Survey REGION  = 'XP061   ' /GSSS: Region Name PLATEID = 'A2U4    ' /GSSS: Plate ID SCANNUM = '01      ' /GSSS: Scan Number DSCNDNUM= '00      ' /GSSS: Descendant Number TELESCID=                      3 /GSSS: Telescope ID BANDPASS=                     35 /GSSS: Bandpass Code COPYRGHT= 'Caltech/Palomar' /GSSS: Copyright Holder SITELAT =                 33.356 /Observatory: Latitude SITELONG=                116.863 /Observatory: Longitude TELESCOP= 'Oschin Schmidt - D' /Observatory: Telescope INSTRUME= 'Photographic Plate' /Detector: Photographic Plate EMULSION= 'IIIaF   ' /Detector: Emulsion FILTER  = 'RG610   ' /Detector: Filter PLTSCALE=                   67.2 /Detector: Plate Scale arcsec per mm PLTSIZEX=                    355 /Detector: Plate X Dimension mm PLTSIZEY=                    355 /Detector: Plate Y Dimension mm PLATERA =                144.055 /Observation: Field centre RA degrees PLATEDEC=                 69.812 /Observation: Field centre Dec degrees PLTLABEL= 'SF07740 ' /Observation: Plate Label DATE-OBS= '1998-11-24T11:83:00' /Observation: Date/Time EXPOSURE=                     50 /Observation: Exposure Minutes PLTGRADE= 'A       ' /Observation: Plate Grade OBSHA   =                1.28333 /Observation: Hour Angle OBSZD   =                37.9539 /Observation: Zenith Distance AIRMASS =                1.26743 /Observation: Airmass REFBETA =                61.7761 /Observation: Refraction Coeff REFBETAP=                 -0.082 /Observation: Refraction Coeff REFK1   =               -48616.4 /Observation: Refraction Coeff REFK2   =                -148442 /Observation: Refraction Coeff CNPIX1  =                   4993 /Scan: X Corner CNPIX2  =                  10823 /Scan: Y Corner XPIXELS =                  23040 /Scan: X Dimension YPIXELS =                  23040 /Scan: Y Dimension XPIXELSZ=                15.0295 /Scan: Pixel Size microns YPIXELSZ=                     15 /Scan: Pixel Size microns ASTRMASK= 'xp.mask ' /Astrometry: GSC2 Mask WCSAXES =                      2 /GetImage: Number WCS axes WCSNAME = 'DSS     ' /GetImage: Local WCS approximation from full plat RADESYS = 'ICRS    ' /GetImage: GSC-II calibration using ICRS system CTYPE1  = 'RA---TAN' /GetImage: RA-Gnomic projection CRPIX1  =                    446 /GetImage: X reference pixel CRVAL1  =                 148.97 /GetImage: RA of reference pixel CUNIT1  = 'deg     ' /GetImage: degrees CTYPE2  = 'DEC--TAN' /GetImage: Dec-Gnomic projection CRPIX2  =                    447 /GetImage: Y reference pixel CRVAL2  =                69.6795 /GetImage: Dec of reference pixel CUNIT2  = 'deg     ' /Getimage: degrees CD1_1   =           -0.000279458 /GetImage: rotation matrix coefficient CD1_2   =            2.15165e-05 /GetImage: rotation matrix coefficient CD2_1   =            2.14552e-05 /GetImage: rotation matrix coefficient CD2_2   =             0.00027889 /GetImage: rotation matrix coefficient OBJECT  = 'data    ' /GetImage: Requested Object Name DATAMIN =                   2701 /GetImage: Minimum returned pixel value DATAMAX =                  22189 /GetImage: Maximum returned pixel value OBJCTRA = '09 55 52.730' /GetImage: Requested Right Ascension (J2000) OBJCTDEC= '+69 40 45.80' /GetImage: Requested Declination (J2000) OBJCTX  =                5438.47 /GetImage: Requested X on plate (pixels) OBJCTY  =                11269.3 /GetImage: Requested Y on plate (pixels) END (0,0) = 4058 (0,1) = 4058
```

 



##### Reading and Manipulating CASA Image Headers (**imhead**)

CASA image headers can be accessed and edited with the **imhead** task. The *imagename* and *mode* are the two primary parameters in the **imhead** task. The imhead task can be run with *mode=\'summary\'*, *\'list\'*, *\'get\'*, *\'put\'*, *\'add\'*, *\'del\'*, or *\'history\'*, and setting the mode opens up mode-specific sub-parameters. Many of these modes are described below and further documented in the **[imhead](global-task-list.ipynb#task_imhead)** page of the Global Task List. 

The default mode is *mode=\'summary\'*, which prints a summary of the image properties to the logger and terminal, and returns a dictionary containing header information. With *mode=\'summary\'*, **imhead** has the following inputs:  

```
#imhead :: List, get and put image header parameters
imagename           =         ''        #Name of the input image
mode                =  'summary'        #imhead options: add, del,
                                        #get, history, list, put, summary
     verbose        =      False        #Give a full listing of
                                        #beams or just a short summary?
                                        #Only used when the image has multiple beams
                                        #and mode='summary'.
```

Note that to capture the dictionary, it must be assigned as a Python variable, e.g. by running: 

```
header_summary = imhead('ngc5921.demo.cleanimg.image',mode='summary')
```

Setting *mode=\'list\'* prints all header keywords and values to the logger and terminal, and returns a dictionary containing the keywords and values. This mode does not have any sub-parameters.

The *mode=\'get\'* setting allows the user to retrieve the value for a specified keyword *hdkey*:

```
#imhead :: List, get and put image header parameters
imagename      =         ''        #Name of the input image
mode           =      'get'   #imhead options: list, summary, get, put
   hdkey       =         ''   #The FITS keyword
```

The *mode=\'put\'* setting allows the user to replace the current value for a given keyword *hdkey* with that specified in *hdvalue*. There are two sub-parameters that are opened by this option:

```
#imhead :: List, get and put image header parameters
imagename      =         ''        #Name of the input image
mode           =      'put'   #imhead options: list, summary, get, put
   hdkey       =         ''   #The FITS keyword
   hdvalue     =         ''   #Value of hdkey
```

<div class="alert alert-warning">
**Alert:** Be careful when using *mode='put'.* This task does not check whether the values you specify (e.g. for the axes types) are valid, and you can render your image invalid. Make sure you know what you are doing when using this option!
</div>

#####  Examples for **imhead**

In the command below, we print the header summary to the logger:

```
CASA <51>: imhead('ngc5921.demo.cleanimg.image',mode='summary')
```

The logger output is the following:

```python
#####Begin Task: imhead             #####
  Image name       : ngc5921.demo.cleanimg.image
  Object name      : N5921_2
  Image type       : PagedImage
  Image quantity   : Intensity
  Pixel mask(s)    : None
  Region(s)        : None
  Image units      : Jy/beam
  Restoring Beam   : 52.3782 arcsec, 45.7319 arcsec, -165.572 deg
 
  Direction reference : J2000
  Spectral  reference : LSRK
  Velocity  type      : RADIO
  Rest frequency      : 1.42041e+09 Hz
  Pointing center     :  15:22:00.000000  +05.04.00.000000
  Telescope           : VLA
  Observer            : TEST
  Date observation    : 1995/04/13/00:00:00
  Telescope position: [-1.60119e+06m, -5.04198e+06m, 3.55488e+06m] (ITRF)
 
  Axis Coord Type      Name             Proj Shape Tile   Coord value at pixel    Coord incr Units
  ------------------------------------------------------------------------------------------------
  0    0     Direction Right Ascension   SIN   256   64  15:22:00.000   128.00 -1.500000e+01 arcsec
  1    0     Direction Declination       SIN   256   64 +05.04.00.000   128.00  1.500000e+01 arcsec
  2    1     Stokes    Stokes                    1    1             I
  3    2     Spectral  Frequency                46    8   1.41279e+09     0.00 2.4414062e+04 Hz
                       Velocity                               1607.99     0.00 -5.152860e+00 km/s
#####End Task: imhead           
```

If the beam size per plane differs (for example, in a spectral data cube), the beam information will be displayed for the channel with the largest beam (i.e. the lowest frequency channel), the chennel with the smallest beam (i.e. the highest frequency channel), and the channel closest to the median beam size. If you set *verbose=True*, the beam information would be provided for each spectral channel (or each plane of the image).  Running **imhead** with *mode=\'summary\'* and *verbose=False* for a spectral data cube would print information on the restoring beams as follows:

```python
Restoring Beams
Pol   Type Chan      Freq   Vel
I    Max    0 9.680e+08     0   39.59 arcsec x   22.77 arcsec pa=-70.57 deg
I    Min  511 1.990e+09 -316516   20.36 arcsec x   12.05 arcsec pa=-65.67 deg
I Median  255 1.478e+09 -157949   27.11 arcsec x   15.54 arcsec pa=-70.36 deg
```

Setting mode=\'list\' prints all header keywords and values to the logger and terminal, and returns a dictionary containing the keywords and values. In the following, we capture the resulting dictionary in the variable hlist, and print the variable. 

```
CASA <52>: hlist = imhead('ngc5921.demo.cleanimg.image',mode='list')

CASA <53>: hlist
  Out[53]:
{'beammajor': 52.378242492675781,
 'beamminor': 45.731891632080078,
 'beampa': -165.5721435546875,
 'bunit': 'Jy/beam',
 'cdelt1': '-7.27220521664e-05',
 'cdelt2': '7.27220521664e-05',
 'cdelt3': '1.0',
 'cdelt4': '24414.0625',
 'crpix1': 128.0,
 'crpix2': 128.0,
 'crpix3': 0.0,
 'crpix4': 0.0,
 'crval1': '4.02298392585',
 'crval2': '0.0884300154344',
 'crval3': 'I',
 'crval4': '1412787144.08',
 'ctype1': 'Right Ascension',
 'ctype2': 'Declination',
 'ctype3': 'Stokes',
 'ctype4': 'Frequency',
 'cunit1': 'rad',
 'cunit2': 'rad',
 'cunit3': '',
 'cunit4': 'Hz',
 'datamax': ' Not Known ',
 'datamin': -0.010392956435680389,
 'date-obs': '1995/04/13/00:00:00',
 'equinox': 'J2000',
 'imtype': 'Intensity',
 'masks': ' Not Known ',
 'maxpixpos': array([134, 134,   0,  38], dtype=int32),
 'maxpos': '15:21:53.976, +05.05.29.998, I, 1.41371e+09Hz',
 'minpixpos': array([117,   0,   0,  21], dtype=int32),
 'minpos': '15:22:11.035, +04.31.59.966, I, 1.4133e+09Hz',
 'object': 'N5921_2',
 'observer': 'TEST',
 'projection': 'SIN',
 'reffreqtype': 'LSRK',
 'restfreq': [1420405752.0],
 'telescope': 'VLA'}
```

The values for these keywords can be queried using *mode=\'get\'.* In the following examples, we capture the return value:

```
CASA <53>: mybmaj = imhead('ngc5921.demo.cleanimg.image',mode='get',hdkey='beammajor')

CASA <54>: mybmaj
  Out[54]: {'unit': 'arcsec', 'value': 52.378242492699997}

CASA <55>: myobserver = imhead('ngc5921.demo.cleanimg.image',mode='get',hdkey='observer')

CASA <56>: print myobserver
{'value': 'TEST', 'unit': ''}
```

You can set the values for keywords using *mode=\'put\'*. For example:

```
CASA <57>: imhead('ngc5921.demo.cleanimg.image',mode='put',hdkey='observer',hdvalue='CASA')
  Out[57]: 'CASA'

CASA <58>: imhead('ngc5921.demo.cleanimg.image',mode='get',hdkey='observer')
  Out[58]: {'unit': '', 'value': 'CASA'}
```

 



##### Image History (imhistory)

Image headers contain records of the operations applied to them, as CASA tasks append the image header with a record of what they did. This information can be retrieved via the **imhistory** task, and new messages can be appended using the **imhistory** task as well. The primary inputs are *imagename* and *mode*, with sub-parameters arising from the selected mode. To view the history of the image, the inputs are:

```
#imhistory :: Retrieve and modify image history
imagename           =         ''        #Name of the input image
mode                =     'list'        #Mode to run in, 'list' to  
                                        #retrieve history,'append' 
                                        #to append a record to history.
     verbose        =       True        #Write history to logger if 
                                        #mode='list'?
```

With *verbose=True* (default) the image history is also reported in the CASA logger.  The **imhistory** task returns the messages in a Python list that can be captured by a variable, e.g.

```
myhistory=imhistory('image.im')
```

It is also possible to add messages to the image headers via *mode=\'append\'*.  See the **[imhistory](global-task-list.ipynb#task_imhistory)** page in the Global Task List for an example of appending messages to the image history.

 



***





#### Reformat Images 

Resizing, binning, regridding, and collapsing images

This page contains a description of various tasks that reformat images.  These include:

-   **imsubimage:** enables the user to extract a sub-image from a larger cube,
-   **imtrans:** changes the axis order in an image,
-   **imregrid:** sets the image onto a different spatial coordinate system or spectral grid,
-   **imreframe:** changes the velocity system of an image
-   **imrebin:** rebins an image in a spatial or spectral dimension 
-   **imcollapse:** collapses an image along an axis. 

 



##### Extracting sub-images (**imsubimage**)

The task **imsubimage** provides a way to extract a smaller data cube from a bigger one. The inputs are:

```

#imsubimage :: Create a (sub)image from a region of the image
imagename      =    ''    #Input image name. Default is unset.
outfile        =    ''    #Output image name. Default is unset.
box            =    ''    #Rectangular region to select in
                          #direction plane. Default is to use the
                          #entire direction plane.
region         =    ''    #Region selection. Default is to use the
                          #full image.
chans          =    ''    #Channels to use. Default is to use all
                          #channels.
stokes         =    ''    #Stokes planes to use. Default is to use
                          #all Stokes planes.
mask           =    ''    #Mask to use. Default is none.
dropdeg        =  True    #Drop degenerate axes
[     keepaxes  =    []    #If dropdeg=True, these are the]
                          #degenerate axes to keep. Nondegenerate
                          #axes are implicitly always kept.

verbose        =   True   #Post additional informative messages to
                          #the logger
```

The *region* keyword defines the size of the smaller cube and is specified via the CASA region CRTF syntax. E.g.

```
region='box [ [ 100pix , 130pix] , [120pix, 150pix ] ]'
```

will extract the portion of the image that is between pixel coordinates (100,130) and (12,150). *dropdeg=T* with selection via *keepaxes* is useful to remove axes in the data cube that are degenerate, i.e. axes with a single plane only. A single Stokes I axis is a common example.

 

 



##### Reordering the Axes of an Image Cube (**imtrans**)

Sometimes data cubes can be in axis orders that are not adequate for processing. The CASA task **imtrans** can change the ordering of the axis:

```
#imtrans :: Reorder image axes
imagename           =         ''        #Name of the input image
outfile             =         ''        #Name of output CASA image.
order               =         ''        #New zero-based axes order.
wantreturn          =       True        #Return an image tool referencing the
                                        #transposed image
```

The *order* parameter is the most important input here. It is a string of numbers that shows how axes 0, 1, 2, 3, \... are mapped onto the new cube (note that the first axis has the label 0, as typical in python). E.g. *order=\'1032\'* will reorder the input axis 0 to be axis 1 in the output, input axis 1 to be output axis 0, input axis 2 to output axis 3 (the last axis) and input axis 3 to output axis 2. Alternatively, axes can be specified by their names. E.g., to reorder an image with right ascension, declination, and frequency and reverse the first two, *order=\[''declination'', ''right ascension'', ''frequency''\]* will work. The axes names can be found typing **ia.coordsys**.**names**. Minimum match is supported, so that *order=\[\'d\', \'f\', \'r\'\]* will produce the same results.Axes can simultaneously be transposed and reversed. To reverse an axis, precede it by a \'-\'. For example, *order=\'-10-32\'* will reverse the direction of the first and third axis of the input image (the zeroth and second axes in the output image).Example (swap the stokes and spectral axes in an RA-Dec-Stokes-Frequency image):

```
imagename = 'myim.im'
outfile = 'outim.im'
order = '0132'
imtrans()
```

or

```
outfile = 'myim_2.im'
order = 132
imtrans()
```

or

```
outfile = 'myim_3.im'
order = ['r', 'd', 'f', 's']
imtrans()
```

or

```
outfile = 'myim_4.im'
order = ['rig', 'declin', 'frequ', 'stok']
imtrans()
```

If the *outfile* parameter is empty, only a temporary image is created; no output image is written to disk. The temporary image can be captured in the returned value (assuming *wantreturn*=T*rue*).

  



##### Regridding an Image (imregrid)

<div class="alert alert-info">
**Inside the Toolkit:** More complex coordinate system and image regridding operation can be carried out in the toolkit. The **coordsys** (**cs**) tool and the **ia.regrid** method are the relevant components.
</div>

It is occasionally necessary to regrid an image onto a new coordinate system. The **imregrid** task will regrid one image onto the coordinate system of another, creating an output image. In this task, the user need only specify the names of the input, template, and output images. The default inputs are:

```

#imregrid :: regrid an image onto a template image
imagename           =         ''        #Name of the source image
template            =      'get'        #A dictionary, refcode, or name of an
                                        #image that provides the output shape
                                        #and coordinate system
output              =         ''        #Name for the regridded image
asvelocity          =       True        #Regrid spectral axis in velocity space
                                        #rather than frequency space?
axes                =       [-1]        #The pixel axes to regrid. -1 => all.
interpolation       =   'linear'        #The interpolation method.  One of
                                        #'nearest', 'linear', 'cubic'.
decimate            =         10        #Decimation factor for coordinate grid
                                        #computation
replicate           =      False        #Replicate image rather than regrid?
overwrite           =      False        #Overwrite (unprompted) pre-existing
                                        #output file?
```

The output image will have the data in *imagename* regridded onto the coordinate system provided by the *template* parameter. *template* is used universally for a range of ways to define the grid of the output image: 

-   a template image: specify an image name here and the input will be regridded to the same 3-dimensional coordinate system as the one in template. Values are filled in as blanks if they do not exist in the input. Note that the input and template images must have the same coordinate structure to begin with (like 3 or 4 axes, with the same ordering)
-   a coordinate system (reference code): to convert from one coordinate frame to another one, e.g. from B1950 to J2000, the template parameter can be used to specify the output coordinate system. These following recognized keywords are supported: \'J2000\', \'B1950\' , \'B1950_VLA\', \'BMEAN\', \'ECLIPTIC\', \'GALACTIC\', \'SUPERGAL\', \'ICRS\'
-   \'*get\'*: This option returns a python dictionary in the {'csys': csys_record, 'shap': shape} format
-   a python dictionary: In turn, such a dictionary can be used as a template to define the final grid



#####  Redefining the Velocity System of an Image (**imreframe**)

**imreframe** can be used to change the velocity system of an image. It is not applying a regridding as a change from radio to optical conventions would require, but it will change the labels of the velocity axes.

```
#imreframe :: Change the frame in which the image reports its spectral values
imagename           =         ''        #Name of the input image
output              =         ''        #Name of the output image; '' => modify input image
outframe            =     'lsrk'        #Spectral frame in which the frequency or velocity
                                        #values will be reported by default
restfreq            =         ''        #restfrequency to use for velocity values (e.g.
                                        #'1.420GHz' for the HI line)
```

*outframe* defines the velocity frame (LSRK, BARY, etc.,) of the output image and a rest frequency should be specified to relabel the spectral axis in new velocity units.



#####  Rebin an Image (**imrebin**)

The task **imrebin** allows one to rebin an image in any spatial or spectral direction:

```
imrebin :: Rebin an image by the specified integer factors
imagename           =         ''        #Name of the input image
outfile             =         ''        #Output image name.
factor              =         []        #Binning factors for each axis. Use
                                        #imhead or ia.summary to determine axis
                                        #ordering.
region              =         ''        #Region selection. Default is to use the full
                                        #image.
box                 =         ''        #Rectangular region to select in
                                        #direction plane. Default is to use the entire
                                        #direction plane.
chans               =         ''        #Channels to use. Default is to use all
                                        #channels.
stokes              =         ''        #Stokes planes to use. Default is to
                                        #use all Stokes planes. Stokes planes
                                        #cannot be rebinned.
mask                =         ''        #Mask to use. Default is none.
dropdeg             =      False        #Drop degenerate axes?
crop                =       True        #Remove pixels from the end of an axis to
                                        #be rebinned if there are not enough to
                                        #form an integral bin?
```

where *factor* is a list of integers that provides the numbers of pixels to be binned for each axis. The *crop* parameters controls how pixels at the boundaries are treated if the bin values are not multiple integers of the image dimensions.Example:

```
imrebin(imagename='my.im', outfile='rebinned.im', factor=[1,2,1,4], crop=T)
```

This leaves RA untouched, bins DEC by a factor of 2, leaves Stokes as is, and bins the spectral axis by a factor of 4. If the input image has a spectral axis with a length that is not a multiple of 4, the *crop=T* setting will discard the remaining 1-3 edge pixels.

 

 



##### Collapsing an Image Along an Axis (**imcollapse**)

**imcollapse** allows to apply an aggregation function along one or more axes of an image. Functions supported are \'*max\', \'mean\', \'median\', \'min\', \'rms\', \'stdev\', \'sum\', \'variance\'* (minimum match supported). The relevant axes will then collapse to a single value or plane (i.e. they will result in a degenerate axis). The functions are specified in the *function* parameter of the **imcollapse** inputs:

```
#imcollapse :: Collapse image along one axis, aggregating pixel values along that axis.
imagename           =         ''        #Name of the input image
function            =         ''        #Function used to compute aggregation
                                        #of pixel values.
axes                =        [0]        #Zero-based axis number(s) or minimal
                                        #match strings to collapse.
outfile             =         ''        #Name of output CASA image.
box                 =         ''        #Optional direction plane box ('blcx,
                                        #blcy, trcx trcy').
     region         =         ''        #Name of optional region file to use.

chans               =         ''        #Optional zero-based contiguous
                                        #frequency channel specification.
stokes              =         ''        #Optional contiguous stokes planes
                                        #specification.
mask                =         ''        #Optional mask to use.
wantreturn          =       True        #Should an image analysis tool
                                        #referencing the collapsed image be
                                        #returned?
```

*wantreturn=True* returns an image analysis tool containing the newly created collapsed image.Example (myimage.im is a 512x512x128x4 (ra,dec,freq,stokes; i.e. in the 0-based system, frequency is labeled as axis 2) image and we want to collapse a subimage of it along its spectral axis avoiding the 8 edge channels at each end of the band, computing the mean value of the pixels (resulting image is 256x256x1x4 in size)):

```
imcollapse(imagename='myimage.im', outfile='collapse_spec_mean.im',
           function='mean', axis=2, box='127,127,383,383', chans='8~119')
```

Note that **imcollapse** will not smooth to a common beam for all planes if they differ. If this is desired, run **imsmooth** before **imcollapse**.

 



***





#### Spectral Analysis 

Moments, Spectral line fitting, Continuum subtraction etc.



#####  Continuum Subtraction on an Image Cube (imcontsub)

One method to separate line and continuum emission in an image cube is to specify a number of line-free channels in that cube, make a linear fit to the visibilities in those channels, and subtract the fit from the whole cube. Note that the task **uvcontsub** serves a similar purpose but the subtraction is performed in visibility space (see [UV Continuum Subtraction](resolveuid/2c85a22fd1004fb194cf1890672ad94a)). The **imcontsub** task will perform a polynomial baseline fit to the specified channels from an image cube and subtract it from all channels.The default inputs are:

```
#imcontsub :: Continuum subtraction on images
imagename  =      ''   #Name of the input image
linefile   =      ''   #Output line image file name
contfile   =      ''   #Output continuum image file name
fitorder   =       0   #Polynomial order for the continuum estimation
region     =      ''   #Image region or name to process see viewer
box        =      ''   #Select one or more box regions
chans      =      ''   #Select the channel(spectral) range
stokes     =      ''   #Stokes params to image (I,IV,IQU,IQUV)
```

Area selection using *box* and *region* is detailed in the [box](#region-selection--box-) and [region](#regions--region-) sections. Image cube plane selection using *chans* and *stokes* are described in the [Plane selection](#plane-selection--chans--stokes-) section.

<div class="alert alert-warning">
**ALERT:** **imcontsub** has issues when the image does not contain a spectral or stokes axis. Errors are generated when run on an image missing one or both of these axes. You will need to use the toolkit (e.g. the **ia.adddegaxes** method) to add degenerate missing axes to the image.
</div>

#####  Examples for **imcontsub**

For example, we first make a clean image from data in which no uv-plane continuum subtraction has been performed:

```
#Now clean, keeping all the channels except first and last
default('clean')
vis = 'ngc5921.demo.src.split.ms'
imagename = 'ngc5921.demo.nouvcontsub'
mode = 'channel'
nchan = 61
start = 1
width = 1
imsize = [256,256]
psfmode = 'clark'
imagermode = ''
cell = [15.,15.]
niter = 6000
threshold='8.0mJy'
weighting = 'briggs'
robust = 0.5
mask = [108,108,148,148]
interactive=False
clean()

#It will have made the image:
#-----------------------------
#ngc5921.demo.nouvcontsub.image

#You can view this image
viewer('ngc5921.demo.nouvcontsub.image')
```

Channels 0 through 4 and 50 through 60 are line-free. Continuum subtraction is then performed with:

```
default('imcontsub')
imagename = 'ngc5921.demo.nouvcontsub.image'
linefile  = 'ngc5921.demo.nouvcontsub.lineimage'
contfile  = 'ngc5921.demo.nouvcontsub.contimage'
fitorder  = 1
chans      = '0~4,50~60'
stokes    = 'I'
imcontsub()
```

 



##### Computing the Moments of an Image Cube (**immoments**)

For spectral line datasets, the output of the imaging process is an image cube, with a frequency or velocity channel axis in addition to the two sky coordinate axes. This can be most easily thought of as a series of image planes stacked along the spectral dimension. A useful product to compute is to collapse the cube into a moment image by taking a linear combination of the individual planes:

$M_m(x_i,y_i) = \sum_k^N w_m(x_i,y_i,v_k)\,I(x_i,y_i,v_k)$

for pixel i and channel k in the cube I. There are a number of choices to form the moment-m, usually approximating some polynomial expansion of the intensity distribution over velocity mean or sum, gradient, dispersion, skew, kurtosis, etc. There are other possibilities (other than a weighted sum) for calculating the image, such as median filtering, finding minima or maxima along the spectral axis, or absolute mean deviations. And the axis along which to do these calculations need not be the spectral axis (i.e. do moments along Dec for a RA-Velocity image). We will treat all of these as generalized instances of a "moment" map.The **immoments** task will compute basic moment images from a cube. The default inputs are:

```
#immoments :: Compute moments of an image cube:
imagename    =         ''   #Input image name
moments      =        [0]   #List of moments you would like to compute
axis         = 'spectral'   #The moment axis: ra, dec, lat, long, spectral, or stokes
region       =         ''   #Image Region.  Use viewer
box          =         ''   #Select one or more box regions
chans        =         ''   #Select the channel(spectral) range
stokes       =         ''   #Stokes params to image (I,IV,IQU,IQUV)
mask         =         ''   #mask used for selecting the area of the
                            #image to calculate the moments on
includepix   =         -1   #Range of pixel values to include
excludepix   =         -1   #Range of pixel values to exclude
outfile      =         ''   #Output image file name (or root for multiple moments)
```

This task will operate on the input file given by *imagename* and produce a new image or set of images based on the name given in *outfile*.The *moments* parameter chooses which moments are calculated. The choices for the operation mode are:

-   moments=-1  - mean value of the spectrum
-   moments=0   - integrated value of the spectrum
-   moments=1   - intensity weighted coordinate; traditionally used to get \'velocity fields\'
-   moments=2   - intensity weighted dispersion of the coordinate; traditionally used to get \'velocity dispersion\'
-   moments=3   - median of I
-   moments=4   - median coordinate
-   moments=5   - standard deviation about the mean of the spectrum
-   moments=6   - root mean square of the spectrum
-   moments=7   - absolute mean deviation of the spectrum
-   moments=8   - maximum value of the spectrum
-   moments=9   - coordinate of the maximum value of the spectrum
-   moments=10  - minimum value of the spectrum
-   moments=11  - coordinate of the minimum value of the spectrum

The meaning of these is described in the [CASA Toolkit Manual](https://casa.nrao.edu/docs/CasaRef/CasaRef.html), that describes the associated [image.moments](https://casa.nrao.edu/docs/CasaRef/image.moments.html#x62-620001.1.1) tool.The *axis* parameter sets the axis along which the moment is "collapsed" or calculated. Choices are: \'ra\', \'dec\', \'lat\', \'long\', \'spectral\', or \'stokes\'. A standard moment-0 or moment-1 image of a spectral cube would use the default choice 'spectral'. One could make a position-velocity map by setting \'ra\' or \'dec\'.The *includepix* and *excludepix* parameters are used to set ranges for the inclusion and exclusion of pixels based on values. For example, i*ncludepix=\[0.05,100.0\]* will include pixels with values from 50 mJy to 1000 Jy, and *excludepix=\[100.0,1000.0\]* will exclude pixels with values from 100 to 1000 Jy.If a single moment is chosen, the outfile specifies the exact name of the output image. If multiple moments are chosen, then outfile will be used as the root of the output filenames, which will get different suffixes for each moment. For image cubes that contain different beam sizes for each plane, **immoments** will smooth all planes to the largest beam size first, then collapse to the desired moment.

 

##### Hints for using **immoments**

In order to make an unbiased moment-0 image, do not put in any thresholding using *includepix* or *excludepix*. This is so that the (presumably) zero-mean noise fluctuations in off-line parts of the image cube will cancel out. If you image has large biases, like a pronounced clean bowl due to missing large-scale flux, then your moment-0 image will be biased also. It will be difficult to alleviate this with a threshold, but you can try.

To make a usable moment-1 (or higher) image, on the other hand, it is critical to set a reasonable threshold to exclude noise from being added to the moment maps. Something like a few times the rms noise level in the usable planes seems to work (put into *includepix* or *excludepix* as needed). Also use *chans* to ignore channels with bad data.

 

##### Examples using **immoments**

Below is an example for **immoments**:

```
default('immoments')
imagename = 'ngc5921.demo.cleanimg'
#Do first and second spectral moments
axis  = 'spectral'
chans = ''
moments = [0,1]
#Need to mask out noisy pixels, currently done
#using hard global limits
excludepix = [-100,0.009]
outfile = 'ngc5921.demo.moments'
 
immoments()

#It will have made the images:
#--------------------------------------
#ngc5921.demo.moments.integrated
#ngc5921.demo.moments.weighted_coord
```

Other examples of NGC2403 (a moment-0 image of a VLA line dataset) and NGC4826 (a moment-1 image of a BIMA CO line dataset) are shown in the Figure [below](http://casa.nrao.edu/casadocs/stable/image-analysis/dealing-with-images#figid-analysisfigmoments).

![0d5fd0d2ee99bce18c9c5fcce6f3439c8f9042ce](media/0d5fd0d2ee99bce18c9c5fcce6f3439c8f9042ce.png){.image-inline}

![4e1cdd67ee0996b1a1e19225e04cb383489aabd1](media/4e1cdd67ee0996b1a1e19225e04cb383489aabd1.png){.image-inline}

>NGC2403 VLA moment zero (left) and NGC4826 BIMA moment one (right) images as shown in the viewer.
  



#####  Generating Position-Velocity Diagrams (impv)

CASA can generate position-velocity (PV) diagrams via the task **impv** or directly in the **viewer** (see [Image Cube Visualization](imaging.ipynb#image-cube-visualization)**)**.  The viewer application calls the task:

```
#impv :: Construct a position-velocity image by choosing two points in the direction plane.
imagename           =         ''        #Name of the input image
outfile             =         ''        #Output image name. If empty, no image is written.
mode                =   'coords'        #If 'coords', use start and end values. If 'length', use
                                        #center, length, and pa values.
width               =          1        #Width of slice for averaging pixels perpendicular to the
                                        #slice. Must be an odd positive integer or valid
                                        #quantity. See help for details.
unit                =   'arcsec'        #Unit for the offset axis in the resulting image. Must be
                                        #a unit of angular measure.
chans               =         ''        #Channels to use. 
                                        #Channels must be contiguous. Default is to use all
                                        #channels.
     region         =         ''        #Region selection. Default is entire image. No selection
                                        #is permitted in the direction plane. 

stokes              =        'I'        #Stokes planes to use. Planes must be contiguous. Default
                                        #is to use all stokes.
mask                =         []        #Mask to use. Default is none.
     stretch        =      False        #Stretch the mask if necessary and possible? Default False
```

PV diagrams are generated by "slicing" a datacube through the RA/DEC planes. The "slit" can be defined either by start/end coordinates or by a length, center coordinate, and position angle. Averaged over the width of the 'slit' the image cube values are then stored in a new image with position and velocity as the two axes. The slit position is specified by a start and end pixel in the RA/DEC plane of the data cube. An angular unit can be set to define what is stored in the resulting PV image.

 



##### 1-dimensional Smoothing (specsmooth)

To gain higher signal-to-noise of data cubes, one can smooth the data along one dimension (for 2-dimensional smoothing, see **imsmooth** [below](#2-dimensional-smoothing--image-convolution--imsmooth-)**)**. Typically this is the spectral axis. Hanning and Boxcar smoothing kernels are available in the task **specsmooth**:

```
#specsmooth :: Smooth an image region in one dimension
imagename           =         ''        #Name of the input image
outfile             =         ''        #Output image name.
region              =         ''        #Region selection. Default is to use the full
                                        #image.
     box            =         ''        #Rectangular region to select in
                                        #direction plane. Default is to use the entire
                                        #direction plane.

mask                =         ''        #Mask to use. Default is none..
axis                =         -1        #The profile axis. Default: use the
                                        #spectral axis if one exists, axis 0
                                        #otherwise (<0).
function            =  'hanning'        #Convolution function. hanning and boxcar
                                        #are supported functions. Minimum match
                                        #is supported.
dmethod             =     'copy'        #Decimation method. '' means no
                                        #decimation, 'copy' and 'mean' are also
                                        #supported (minimum match).
```

The parameter *dmethod=\'copy\'* allows one to only keep every nth channel, if the smoothing kernel has a width of n. Leaving this parameter empty will return the same size cube as the input and setting it to 'mean' will average planes using the kernel width.

 

 



#####    Spectral Line fitting with **specfit**

**specfit** is a powerful task to perform spectral line fits in data cubes. Three types of fitting functions are currently supported, polynomials, Gaussians, and Lorentzians. **specfit** can fit these functions in two ways: over data that were averaged across a region (*multifit=False*) or on a pixel by pixel basis (*multifit=True*).

```
#specfit :: Fit 1-dimensional Gaussians and/or polynomial models to an image or image region
imagename           =         ''        #Name of the input image
box                 =         ''        #Rectangular box in direction coordinate
                                        #blc, trc. Default: entire image ('').
region              =         ''        #Region of interest. Default: Do
                                        #not use a region.
chans               =         ''        #Channels to use. Channels must be
                                        #contiguous. Default: all channels ('').
stokes              =         ''        #Stokes planes to use. Planes must be
                                        #contiguous. Default: all stokes ('').
axis                =         -1        #The profile axis. Default: use the
                                        #spectral axis if one exists, axis 0
                                        #otherwise (<0).
mask                =         ''        #Mask to use. Default is
                                        #none..
poly                =         -1        #Order of polynomial element.  Default: do
                                        #not fit a polynomial (<0).
estimates           =         ''        #Name of file containing initial estimates.
                                        #Default: No initial estimates ('').
     ngauss         =          1        #Number of Gaussian elements.  Default: 1.
     pampest        =         ''        #Initial estimate of PCF profile (gaussian
                                        #or lorentzian) amplitudes.
     pcenterest     =         ''        #Initial estimate PCF profile centers, in
                                        #pixels.
     pfwhmest       =         ''        #Initial estimate PCF profile FWHMs, in
                                        #pixels.
     pfix           =         ''        #PCF profile parameters to fix during fit.
     pfunc          =         ''        #PCF singlet functions to fit. 'gaussian'
                                        #or 'lorentzian' (minimal match
                                        #supported). Unspecified means all
                                        #gaussians.

minpts              =          0        #Minimum number of unmasked points
                                        #necessary to attempt fit.
multifit            =       True        #If true, fit a profile along the desired
                                        #axis at each pixel in the specified
                                        #region. If false, average the non-fit
                                        #axis pixels and do a single fit to that
                                        #average profile. Default False.
     amp            =         ''        #Name of amplitude solution image. Default:
                                        #do not write the image ('').
     amperr         =         ''        #Name of amplitude solution error image.
                                        #Default: do not write the image ('').
     center         =         ''        #Name of center solution image. Default: do
                                        #not write the image ('').
     centererr      =         ''        #Name of center solution error image.
                                        #Default: do not write the image ('').
     fwhm           =         ''        #Name of fwhm solution image. Default: do
                                        #not write the image ('').
     fwhmerr        =         ''        #Name of fwhm solution error image.
                                        #Default: do not write the image ('').
     integral       =         ''        #Prefix of name of integral solution image.
                                        #Name of image will have gaussian
                                        #component number appended.  Default: do
                                        #not write the image ('').
     integralerr    =         ''        #Prefix of name of integral error solution
                                        #image. Name of image will have gaussian
                                        #component number appended.  Default: do
                                        #not write the image ('').

model               =         ''        #Name of model image. Default: do not write
                                        #the model image ('').
residual            =         ''        #Name of residual image. Default: do not
                                        #write the residual image ('').
wantreturn          =       True        #Should a record summarizing the results be
                                        #returned?
logresults          =       True        #Output results to logger?
gmncomps            =          0        #Number of components in each gaussian
                                        #multiplet to fit
gmampcon            =         ''        #The amplitude ratio constraints for non-
                                        #reference components to reference
                                        #component in gaussian multiplets.
gmcentercon         =         ''        #The center offset constraints (in pixels)
                                        #for non-reference components to reference
                                        #component in gaussian multiplets.
gmfwhmcon           =         ''        #The FWHM  ratio constraints for non-
                                        #reference components to reference
                                        #component in gaussian multiplets.
gmampest            =      [0.0]        #Initial estimate of individual gaussian
                                        #amplitudes in gaussian multiplets.
gmcenterest         =      [0.0]        #Initial estimate of individual gaussian
                                        #centers in gaussian multiplets, in
                                        #pixels.
gmfwhmest           =      [0.0]        #Initial estimate of individual gaussian
                                        #FWHMss in gaussian multiplets, in pixels.
gmfix               =         ''        #Parameters of individual gaussians in
                                        #gaussian multiplets to fix during fit.
logfile             =         ''        #File in which to log results. Default is
                                        #not to write a logfile.
goodamprange        =      [0.0]        #Acceptable amplitude solution range. [0.0]
                                        #=> all amplitude solutions are
                                        #acceptable.
goodcenterrange     =      [0.0]        #Acceptable center solution range in pixels
                                        #relative to region start. [0.0] => all
                                        #center solutions are acceptable.
goodfwhmrange       =      [0.0]        #Acceptable FWHM solution range in pixels.
                                        #[0.0] => all FWHM solutions are
                                        #acceptable.
sigma               =         ''        #Standard deviation array or image name.
```

#####  Polynomial Fits

Polynomials can be fit by specifying the polynomial order in *poly*. Negative orders will not fit any polynomials.

#####  Lorentzian and Gaussian Fits

Gaussian and Lorentzian fits are very similar, they both require amplitude, center, and FWHM to be fully specified. All of the following discussion is thus valid for both functions. The parameter *pfunc* controls whether Gaussian or Lorentzian functions are to be used. Default is all Gaussians. *pfunc=\[\'L\', \'G\', \'G\', \'L\'\]* would use Lorentzian, Gaussian, Gaussian, and Lorentzian components in the order they appear in the estimates file (see below).

######  One or more single Gaussian/Lorentzian

For Gaussian and Lorentzian fits, the task will allow multiple components and **specfit** will try to find the best solution. The parameter *space*, however, is usually not uniform and to avoid local minima in the goodness-of-fit space, one can provide initial start values for the fits. This can be done either through the parameters *pampest*, *pcenterest*, and *pfwhmest* for the amplitudes, center, and FWHM estimates in image coordinates. *pfix* can take parameters that specify fixed fit values. Any combination of the characters \'*p\'* (peak), \'*c\'* (center), and \'*f\'* (fwhm) are permitted, e.g. \'*fc*\' will hold the fwhm and the center constant during the fit. Fixed parameters will have no errors associated with them in the solution. Alternatively, a file with initial values can be supplied by the *estimates* parameter (one Gaussian/Lorentzian parameter set per line). The file has the following format:

    [peak intensity], [center], [fwhm], [optional fixed parameter string]

The first three values are required and must be numerical values. The peak intensity must be expressed in map units, while the center and fwhm must be specified in pixels. The fourth value is optional and if present, represents the parameter(s) that should be held constant during the fit (see above).An example estimates file is:

    # estimates file indicating that two Gaussians should be fit
    # first guassian estimate, peak=40, center at pixel number 10.5, 
    # fwhm = 5.8 pixels, all parameters allowed to vary during
    # fit
    40, 10.5, 5.8
    # second Gaussian, peak = 4, center at pixel number 90.2, 
    # fwhm = 7.2 pixels, hold fwhm constant
    4, 90.2, 7.2, f
    # end file

and the output of a typical execution, e.g.

```python
specfit(imagename='IRC10216_HC3N.cube_r0.5.image',
region='specfit.crtf', multifit=F, estimates='', ngauss=2)

('specfit.crtf' is a CASA regions file, see Section D)
will be

Fit   :
    RA           :   09:47:57.49
    Dec          :   13.16.46.46
    Stokes       : I
    Pixel        : [146.002, 164.499, 0.000,  *]
    Attempted    : YES
    Converged    : YES
    Iterations   : 28
    Results for component 0:
        Type     : GAUSSIAN
        Peak     : 5.76 +/- 0.45 mJy/beam
        Center   : -15.96 +/- 0.32 km/s
                   40.78 +/- 0.31 pixel
        FWHM     : 7.70 +/- 0.77 km/s
                   7.48 +/- 0.74 pixel
        Integral : 47.2 +/- 6.0 mJy/beam.km/s
    Results for component 1:
        Type     : GAUSSIAN
        Peak     : 4.37 +/- 0.33 mJy/beam
        Center   : -33.51 +/- 0.58 km/s
                   23.73 +/- 0.57 pixel
        FWHM     : 15.1 +/- 1.5 km/s
                   14.7 +/- 1.5 pixel
        Integral : 70.2 +/- 8.8 mJy/beam.km/s
```

If *wantreturn=True* (the default value), the task returns a python dictionary (here captured in a variable with the inventive name of \'*fitresults\'*) :

```
fitresults=specfit(imagename='IRC10216_HC3N.cube_r0.5.image', region='specfit.rgn', multifit=F,
        estimates='', ngauss=2)
```

The values can then be used by other python code for further processing.

 

###### Gaussian Multiplets

It is possible to fit a number of Gaussians together, as multiplets with restrictions. All restrictions are relative to a reference Gaussian (the zero'th component of each multiplet). *gncomps* specifies the number of Gaussians for each multiplets, and, in fact, a number of these multiplets can be fit simultaneously. *gncomps=\[2,4,3\]*, e.g. fits a 2-component Gaussian, a 4-component Gaussian, and a 3-component Gaussian all at once. The initial parameter estimates can be specified with the *gmampest*, *gmcenterest*, and *gmfwhmest* parameters and the estimates are simply listed in the sequence of *gncomps*. E.g. if *gncomps=\[2,4,3\]* is specified with multiplet G0 consisting of 2 Gaussians a, b, multiplet G1 of 4 Gaussians c, d, e, f, and multiplet G2 of three Gaussians g, h, i, the parameter list in *gm\*est* would be like *gm\*est=\[a,b,c,d,e,f,g,h,i\]*.Restrictions can be specified via the *gmampcon* parameter for the amplitude ratio (non-reference to reference), *gmcentercon* for the offset in pixels (to a reference), and *gmfwhmcon* for the FWHM ratio (non-reference to reference). A value of 0 will not constrain anything. The reference is always the zero'th component in each multiplet, in our example, Gaussians a, c, and g. They cannot be constrained. So *gmncomps=\[2, 4, 3\]*, *gmampcon= \[ 0 , 0.2, 0 , 0.1, 4.5, 0 \]*, *gcentercon=\[24.2, 45.6, 92.7, 0 , -22.8, -33.5\],* and *gfwhmcon=\' \'* would constrain Gaussians b relative to a with a 24.2 pixel offset, Gaussian d to c with a amplitude ratio of 0.2 and a 45.6 pixel offset, Gaussian e to c with a offset of 92.7 pixels, etc. Restrictions will overrule any estimates.The parameters *goodamprange*, *goodcenterrange*, and *goodfwhmrange* can be used to limit the range of amplitude, center, and fwhm solutions for all Gaussians.

######  Pixel-by-pixel fits

As mentioned above, **specfit** can also fit spectral cubes on a pixel by pixel basis. In this case, one can choose to write none, any or all of the solution and error images for Gaussian/Lorentzian fits via the parameters *amp*, *amperr*, *center*, *centererr*, *fwhm*, and *fwhmerr*. Each Gaussian component will produce a set of images *\_0, \_1*, etc. suffixes. Writing analogous images for polynomial coefficients is not yet supported although polynomial fits when *multifit=True* is supported. Best fit coefficients are written to the logger. Pixels for which fits were not attempted or did not converge will be masked as bad.



#####  Spatial Spectral Line Properties (**specflux**)

**specflux** calculates the flux as a function of frequency and velocity over a selected spatial region. Flux densities of Jy/beam are being converted to Jy by properly integrating over the selected region.The input parameters of **specflux** are:

```
#specflux :: Report details of an image spectrum.
imagename           =         ''        #Name of the input image
box                 =         ''        #Rectangular region to select in
                                        #direction plane. Default is to use the entire
                                        #direction plane.
     region         =         ''        #Region selection.  Default is to use the full
                                        #image.

chans               =         ''        #Channels to use.  Default is to use all
                                        #channels.
stokes              =         ''        #Stokes planes to use.  Default is to
                                        #use all Stokes planes.
mask                =         ''        #Mask to use.  Default
                                        #is none.
unit                =     'km/s'        #Unit to use for the abscissa. Must be
                                        #conformant with a typical spectral axis
                                        #unit.
major               =         ''        #Major axis of overriding restoring beam.
                                        #If specified, must be a valid quantity.
minor               =         ''        #Minor axis of overriding restoring beam.
                                        #If specified, must be a valid quantity
logfile             =         ''        #File which to write details. Default is
                                        #to not write to a file.
```

The results can be written into a logfile to be plotted in other packages.

 



##### Plot Spectra on a Map (plotprofilemap)

The **profilemap** task enables plotting spectra according to their pointing directions (a.k.a. a profile map) in plots. The input should be CASA image,or FITS format cube. Spectra within the cube are averaged into a bin number specified with the *numpanels* keyword. Absent or masked data are treated according to *plotmasked* keyword setting.

```
plotprofilemap(imagename='interesting_spectralcube_casaimge.im',
figfile = 'grid_map.png',
separatepanel=F,
spectralaxis = 'velocity',
title = 'myprofilemap',
transparent = F,
showaxislabel = True,
showtick = True,
showticklabel = True,
pol=0,
numpanels='8')
```

 



##### Calculation of Rotation Measures (**rmfit**)

**rmfit** is an image domain task that accepts an input cube image containing Stokes Q and U axes and will generate the rotation measure by performing a least square fit in the image domain to obtain the best fit to the equation  $\chi = \chi_0 + RM\times \lambda^2$.

The inputs to **rmfit** are:

```
#rmfit :: Calculate rotation measure.
imagename           =         ''        #Name(s) of the input image(s). Must be specified.
rm                  =         ''        #Output rotation measure image name. If not specified, no
                                        #image is written.
rmerr               =         ''        #Output rotation measure error image name. If not
                                        #specified, no image is written.
pa0                 =         ''        #Output position angle (degrees) at zero wavelength image
                                        #name. If not specified, no image is written.
pa0err              =         ''        #Output position angle (degrees) at zero wavelength error
                                        #image name. If not specified, no image is written.
nturns              =         ''        #Output number of turns image name. If not specified, no
                                        #image is written.
chisq               =         ''        #Output reduced chi squared image name. If not specified,
                                        #no image is written.
sigma               =         ''        #Estimate of the thermal noise.  A value less than 0 means
                                        #auto estimate.
rmfg                =        0.0        #Foreground rotation measure in rad/m/m to subtract.
rmmax               =        0.0        #Maximum rotation measure in rad/m/m for which to solve.
                                        #IMPORTANT TO SPECIFY.
maxpaerr            =      1e+30        #Maximum input position angle error in degrees to allow in
                                        #solution determination.
```

This task generates the rotation measure image from stokes Q and U measurements at several different frequencies. You are required to specify the name of at least one image with a polarization axis containing stokes Q and U planes and with a frequency axis containing more than two pixels. The frequencies do not have to be equally spaced (i.e. the frequency coordinate can be a tabular coordinate). It will work out the position angle images for you. You may also specify multiple image names, in which case these images will first be concatenated along the spectral axis using **ia.imageconcat**. The requirements are that for all images, the axis order must be the same and the number of pixels along each axis must be identical, except for the spectral axis which may differ in length between images. The spectral axis need not be contiguous from one image to another. See also the i**magepol.fourierrotationmeasure** function for a new Fourier-based approach.Rotation measure algorithms that work robustly are few. The main problem is in trying to account for the $n- \pi$ ambiguity (see Leahy et al.1986 - Appendix A.1) [\[1\]](#Bibliography) and the [MIRIAD manual](http://www.cfa.harvard.edu/sma/miriad/manuals/SMAuguide/smauserhtml/imrm.html).

But as in all these algorithms, the basic process is that for each spatial pixel, the position angle vs frequency data is fit to determine the rotation measure and the position angle at zero wavelength (and associated errors). An image containing the number of $n- \pi$ turns that were added to the data at each spatial pixel and for which the best fit was found can be written. The reduced $\chi^2$ image for the fits can also be written. Any combination of output images can be written.

<div class="alert alert-info">
**NOTE**: No assessment of curvature (i.e. deviation from the simple linear position angle - $lambda^2$ functional form) is made.
</div>

The parameter *sigma* gives the thermal noise in Stokes Q and U. By default it is determined automatically using the image data. But if it proves to be inaccurate (maybe not many signal-free pixels), it may be specified. This is used for calculating the error in the position angles (via propagation of Gaussian errors).The argument *maxpaerr* specifies the maximum allowable error in the position angle that is acceptable. The default is an infinite value. From the standard propagation of errors, the error in the linearly polarized position angle is determined from the Stokes Q and U images (at each directional pixel for each frequency). If the position angle error for any pixel exceeds the specified value, the position angle at that pixel is omitted from the fit. The process generates an error for the fit and this is used to compute the errors in the output images.

<div class="alert alert-info">
**NOTE**: *maxpaerr* is not used to mask pixels in the output images.
</div>

The argument *rmfg* is used to specify a foreground RM value. For example, you may know the mean RM in some direction out of the Galaxy, then including this can improve the algorithm by reducing ambiguity. The parameter *rmmax* specifies the maximum absolute RM value that should be solved for. This quite an important parameter. If you leave it at the default, zero, no ambiguity handling will be used. So some a priori information should be supplied; this is the basic problem with rotation measure algorithms.



#####  Calculation of Spectral Indices and Higher Order Polynomials (**spxfit**)

This application fits a power logarithmic polynomial or a logarithmic transformed polynomial to pixel values along a specified axis of an image or images. These functions are most often used for fitting the spectral index and higher order terms of a spectrum. A power logarithmic polynomial has the form

$y = \frac{c_0 x}{D^{(c_1 + c_2 \ln(x/D) + c_3 \ln(x/D)^2 + c_n \ln(x/D)^{(n - 1)})}}$

and a logarithmic transformed polynomial is simply the result of this equation after taking the natural log of both sides so that it has the form

$\ln(y) = c_0 + c_1 \ln(x) + c_2 \ln(x/D)^2 +  ... + c_n \ln(x/D)^n$

Because the logarithm of the ordinate values must be taken before fitting a logarithmic transformed polynomial, all non-positive pixel values are effectively masked for the purposes of fitting. The coefficients of the two forms are equal to each other except that c0 in the second equation is equal to $\ln(c_0)$ of the first. In the case of fitting a spectral index, which is traditionally represented as $\alpha$, is equal to $c_1$. In both cases, $D$ is a normalization constant used so that abscissa values are closer to unity when they are sent to the fitter. This generally improves the probability that the fit will converge. This parameter may be specified via the *div* parameter. A value of 0 (the default) indicates that the application should determine a reasonable value for $D$, which is determined via$D = 10^{\int(\log10(\sqrt(\min(x)*\max(x)))}$where *min(x)* and *max(x)* are the minimum and maximum abscissa values, respectively.The inputs are:

```
 #spxfit :: Fit a 1-dimensional model to an image or image region
for determination of spectral index.
imagename           =                   #Name of the input image(s)
box                 =         ''        #Rectangular box in
                                        #direction coordinate blc, trc.
                                        #Default: entire image ('').
region              =         ''        #Region of interest.  Default:
                                        #Do not use a region.
chans               =         ''        #Channels to use. Channels
                                        #must be contiguous.  Default: all channels ('').
stokes              =         ''        #Stokes planes to
                                        #use. Planes must be contiguous. Default:
                                        #all stokes ('').
axis                =         -1        #The profile axis. Default:
                                        #use the spectral axis if one
                                        #exists, axis 0 otherwise (<0).
mask                =         ''        #Mask to use.  Default is none.
minpts              =          1        #Minimum number of unmasked
                                        #points necessary to attempt
                                        #fit.
multifit            =       True        #If true, fit a profile
                                        #along the desired axis at each
                                        #pixel in the specified
                                        #region. If false, average the
                                        #non-fit axis pixels and do
                                        #a single fit to that average
                                        #profile. Default False.
     spxsol         =         ''        #Name of the spectral index
                                        #function coefficient solution
                                        #image to write.
     spxerr         =         ''        #Name of the spectral index
                                        #function coefficient error
                                        #image to write.
     model          =         ''        #Name of model
                                        #image. Default: do not write the model
                                        #image ('').
     residual       =         ''        #Name of residual
                                        #image. Default: do not write the
                                        #residual image ('').
spxtype             =      'plp'        #Type of function to
                                        #fit. 'plp' => power logarithmic
                                        #polynomial, 'ltp' =>
                                        #logarithmic transformed polynomial.
spxest              =         []        #Initial estimates for the
                                        #spectral index function
                                        #coefficients.
spxfix              =         []        #Fix the corresponding spectral index function
                                        #coefficients during the fit. True=>hold fixed.
div                 =          0        #Divisor (numerical value or
                                        #quantity) to use in the
                                        #logarithmic terms of the
                                        #plp or ltp function. 0 =>
                                        #calculate a useful value on the fly.
wantreturn          =       True        #Should a record summarizing
                                        #the results be returned?
logresults          =       True        #Output results to logger?
logfile             =         ''        #File in which to log
                                        #results. Default is not to write a
                                        #logfile.
sigma               =         -1        #Standard deviation array or image name(s).
     outsigma       =         ''        #Name of output image used
                                        #for standard deviation. Ignored
                                        #if sigma is empty.
```

For more than a single input image or cube, all images must have the same dimensions along all axes other than the fit axis. *multifit* will perform a per-pixel fit, otherwise there will be a single value over the entire region.

 



##### Search for Spectral Line Rest Frequencies (**slsearch**)

The **slsearch** task allows the spectral line enthusiast to find their favorite spectral lines in subset of the [Splatalogue spectral line catalog](http://www.splatalogue.net) which is distributed with CASA. In addition, one can export custom catalogs from Splatalogue and import them to CASA using the task **splattotable** (next section) or tool method **sl.splattotable**. One can even import catalogs with lines not in Splatalogue using the same file format.The inputs to **slsearch** are as follows:

```
#slsearch :: Search a spectral line table.
tablename           =         ''        #Input spectral line table name to
                                        #search. If not specified, use the
                                        #default table in the system.
outfile             =         ''        #Results table name. Blank means do not
                                        #write the table to disk.
freqrange           =   [84, 90]        #Frequency range in GHz.
species             =       ['']        #Species to search for.
reconly             =      False        #List only NRAO recommended
                                        #frequencies.
chemnames           =       ['']        #Chemical names to search for.
qns                 =       ['']        #Resolved quantum numbers to search
                                        #for.
rrlinclude          =       True        #Include RRLs in the result set?
rrlonly             =      False        #Include only RRLs in the result set?
     intensity      =         -1        #CDMS/JPL intensity range. -1 -> do not
                                        #use an intensity range.
     smu2           =         -1        #S*mu*mu range in Debye**2. -1 -> do
                                        #not use an S*mu*mu range.
     loga           =         -1        #log(A) (Einstein coefficient) range.
                                        #-1 -> do not use a loga range.
     eu             =         -1        #Upper energy state range in Kelvin. -1
                                        #-> do not use an eu range.
     el             =         -1        #Lower energy state range in Kelvin. -1
                                        #-> do not use an el range.

verbose             =       True        #List result set to logger (and
                                        #optionally logfile)?
     logfile        =         ''        #List result set to this logfile (only
                                        #used if verbose=True).
     append         =       True        #If true, append to logfile if it
                                        #already exists, if false overwrite
                                        #logfile if it exists. Only used if
                                        #verbose=True and logfile not blank.

wantreturn          =       True        #If true, return the spectralline tool
                                        #associated with the result set.
```

The table is provided in the *tablename* parameter but if it is blank (the default), the catalog which is included with CASA will be used. Searches can be made in a parameter space with large dimensionality:

    *freqrange*             Frequency range in GHz.    *species*                Species to search for.    *reconly*                 List only NRAO recommended frequencies.    *chemnames*         Chemical names to search for.    *qns*                      Resolved quantum numbers to search for.    *intensity*               CDMS/JPL intensity range.    *smu2*                   $S\mu^{2}$ range in Debye$^2$.    *loga*                      log(A) (Einstein coefficient) range.    *el*                          Lower energy state range in Kelvin.    *eu*                         Upper energy state range in Kelvin.    *rrlinclude*              Include RRLs in the result set?    *rrlonly*                   Include only RRLs in the result set?

Notation is as found in the Splatalogue catalog.Example:Search for all lines of the species HOCN and HOCO$^+$ in the 200-300GHz range:

```
slsearch(outfile='myresults.tbl', freqrange = [200,300],
         species=['HOCN', 'HOCO+'])
```

The task can also return a python dictionary if assigned a variable like:

```
myLines = slsearch(outfile='myresults.tbl', freqrange = [200,300],
          species=['HOCN', 'HOCO+'])
```



#####  Convert Exported Splatalogue Catalogs to CASA Tables (**splattotable**)

In some cases, the internal spectral line catalog may not contain the lines in which one is interested. In that case, one can export a catalog from [Splatalogue](http://www.splatalogue.net) or even create their own \'by hand\' (be careful to get the format exactly right though!). CASA's task **splattotable** can then be used to create a CASA table that contains these lines and can be searched:

```
#splattotable :: Convert a downloaded Splatalogue spectral line list to a casa table.
filenames           =       ['']        #Files containing Splatalogue lists.
table               =         ''        #Output table name.
wantreturn          =       True        #Do you want the task to return a spectralline tool attached to the results table?
```

A search in Splatalogue will return a catalog that can be saved in a file (look for the \'Export\' section after the results on the search results page). The exported filename(s) should be entered in the *filenames* parameter of **splattotable**. The downloaded files must be in a specific format for this task to succeed. If you use the Splatalogue \'*Export CASA fields*\' feature, you should have no difficulties.

 

 



##### Bibliography

1. Leahy,\ J.\~P.,\ Pooley,\ G.\~G.,\ &\ Jagers,\ W.\~J.\ 1986,\ A&A,\ 156,\ 234\ (
^



***





#### Image Plane Analysis 

Source Fitting and Smoothing



##### Image-plane Component Fitting (**imfit**)

The inputs are:

```
#imfit :: Fit one or more elliptical Gaussian components on an image region(s)
imagename           =         ''        #Name of the input image
box                 =         ''        #Specify one or more box regions for the fit.
region              =         ''        #Region. 
chans               =         ''        #Spectral channels on which to perform fit. 
stokes              =         ''        #Stokes parameter to fit. If blank, first stokes plane is
                                        #used.
mask                =         ''        #Mask to use. Default is none.
includepix          =         []        #Range of pixel values to include for fitting.
excludepix          =         []        #Range of pixel values to exclude for fitting.
residual            =         ''        #Name of output residual image.
model               =         ''        #Name of output model image.
estimates           =         ''        #Name of file containing initial estimates of component
                                        #parameters.
logfile             =         ''        #Name of file to write fit results.
newestimates        =         ''        #File to write fit results which can be used as initial
                                        #estimates for next run.
complist            =         ''        #Name of output component list table.
dooff               =      False        #Also fit a zero level offset? Default is False
rms                 =         -1        #RMS to use in calculation of uncertainties. Numeric or
                                        #valid quantity (record or string). If numeric, it is
                                        #given units of the input image. If quantity, units must
                                        #conform to image units. If not positive, the rms of the
                                        #residual image, in the region of the fit, is used.
noisefwhm           =         ''        #Noise correlation beam FWHM. If numeric value,
                                        #interpreted as pixel widths. If quantity (dictionary,
                                        #string), it must have angular units.
```

**imfit** will return (as a Python dictionary) the results of the fit, but the results can also be written into a component list table or a logfile.

<div class="alert alert-info">
**NOTE**: To fit more than a single component, you must provide starting estimates for each component via the *estimates* file. See ''*help imfit'*' for more details on this. A noise estimate will be calculated automatically or can be provided through the *rms* and *noisefwhm* keywords.
</div>

 

##### Examples for **imfit**

```
#First fit only a single component at a time
#This is OK since the components are well-separated and not blended
#Box around component A
xfit_A_res = imfit('b1608.demo.clean2.image',box='121,121,136,136',
                   newestimates='b1608.demo.clean2.newestimate')

#Now extract the fit part of the return value
xfit_A = xfit_A_res['results']['component0']
#xfit_A
#Out[7]:
#{'flux': {'error': array([  6.73398035e-05,   0.00000000e+00,   0.00000000e+00,
#0.00000000e+00]),
#'polarisation': 'Stokes',
#'unit': 'Jy',
#'value': array([ 0.01753742,  0.        ,  0.        ,  0.        ])},
#'label': '',
#'shape': {'direction': {'error': {'latitude': {'unit': 'arcsec',
#'value': 0.00041154866279462775},
#'longitude': {'unit': 'arcsec',
#'value': 0.00046695916589535109}},
#'m0': {'unit': 'rad', 'value': -2.0541102061078207},       NOTE: 'm0' and 'm1' are the coordinates of peak/controid
#'m1': {'unit': 'rad', 'value': 1.1439131060384089},        NOTE: 'm0' and 'm1' are the coordinates of peak/controid
#'refer': 'J2000',
#'type': 'direction'},
#'majoraxis': {'unit': 'arcsec', 'value': 0.29100166137741568},
#'majoraxiserror': {'unit': 'arcsec',
#'value': 0.0011186420613222663},
#'minoraxis': {'unit': 'arcsec', 'value': 0.24738110059830495},
#'minoraxiserror': {'unit': 'arcsec',
#'value': 0.0013431999725066338},
#'positionangle': {'unit': 'deg', 'value': 19.369249322401796},
#'positionangleerror': {'unit': 'rad',
#'value': 0.016663189295782171},
#'type': 'Gaussian'},
#'spectrum': {'frequency': {'m0': {'unit': 'GHz', 'value': 1.0},
#'refer': 'LSRK',
#'type': 'frequency'},
#'type': 'Constant'}}

#Now the other components
xfit_B_res = imfit('b1608.demo.clean2.image',box='108,114,120,126',
                   newestimates='b1608.demo.clean2.newestimate',append=True)
xfit_B = xfit_B_res['results']['component0']

xfit_C_res= imfit('b1608.demo.clean2.image',box='108,84,120,96')
xfit_C = xfit_C_res['results']['component0']

xfit_D_res = imfit('b1608.demo.clean2.image',box='144,98,157,110')
xfit_D = xfit_D_res['results']['component0']

print ""
print "Imfit Results:"
print "--------------"
print "A  Flux = %6.4f Bmaj = %6.4f" % (xfit_A['flux']['value'][0],xfit_A['shape']['majoraxis']['value'])
print "B  Flux = %6.4f Bmaj = %6.4f" % (xfit_B['flux']['value'][0],xfit_B['shape']['majoraxis']['value'])
print "C  Flux = %6.4f Bmaj = %6.4f" % (xfit_C['flux']['value'][0],xfit_C['shape']['majoraxis']['value'])
print "D  Flux = %6.4f Bmaj = %6.4f" % (xfit_D['flux']['value'][0],xfit_D['shape']['majoraxis']['value'])
print ""
```

Now try fitting four components together. For this we will have to provide an estimate file. We will use the clean beam for the estimate of the component sizes:

```
estfile=open('b1608.demo.clean2.estimate','w')
print >>estfile,'#peak, x, y, bmaj, bmin, bpa'
print >>estfile,'0.017, 128, 129, 0.293arcsec, 0.238arcsec, 21.7deg'
print >>estfile,'0.008, 113, 120, 0.293arcsec, 0.238arcsec, 21.7deg'
print >>estfile,'0.008, 113,  90, 0.293arcsec, 0.238arcsec, 21.7deg'
print >>estfile,'0.002, 151, 104, 0.293arcsec, 0.238arcsec, 21.7deg'
estfile.close()
```

Then, this can be used in **imfit**:

```
fit_all_res = imfit('b1608.demo.clean2.image',
                     estimates='b1608.demo.clean2.estimate',
                     logfile='b1608.demo.clean2.imfitall.log',
                     newestimates='b1608.demo.clean2.newestimate',
                     box='121,121,136,136,108,114,120,126,108,84,120,96,144,98,157,110')
#Now extract the fit part of the return values
xfit_allA = xfit_all_res['results']['component0']
xfit_allB = xfit_all_res['results']['component1']
xfit_allC = xfit_all_res['results']['component2']
xfit_allD = xfit_all_res['results']['component3']
```

These results are almost identical to those from the individual fits. You can see a nicer printout of the fit results in the logfile.

 



##### 2-dimensional Smoothing; Image Convolution (imsmooth)

A data cube can be smoothed across spatial dimensions with **imsmooth**. The inputs are:

```
#imsmooth :: Smooth an image or portion of an image
imagename           =         ''        #Name of the input image. Must be
                                        #specified.
kernel              =    'gauss'        #Type of kernel to use. Acceptable values
                                        #are 'b', 'box', or 'boxcar' for a
                                        #boxcar kernel, 'g', 'gauss', or
                                        #'gaussian' for a gaussian kernel, 'c',
                                        #'common', or 'commonbeam' to use the
                                        #common beam of an image with multiple
                                        #beams as the gaussian to which to
                                        #convolve all the planes, 'i' or 'image'
                                        #to use an image as the kernel.
     beam           =         ''        #Alternate way of describing a Gaussian.
                                        #If specified, must be a dictionary with
                                        #keys 'major', 'minor', and 'pa' (or
                                        #'positionangle'). Do not specify beam
                                        #if specifying major, minor, and pa.
                                        #Example: Example: {'major': '5arcsec',
                                        #'minor': '2arcsec', 'pa': '20deg'}.
     targetres      =      False        #If gaussian kernel, specified parameters
                                        #are to be resolution of output image
                                        #(True) or parameters of gaussian to
                                        #convolve with input image (False).
     major          =         ''        #Major axis for the kernels. Standard
                                        #quantity representation. Must be
                                        #specified for kernel='boxcar'. Example:
                                        #'4arcsec'.
     minor          =         ''        #Minor axis. Standard quantity
                                        #representation. Must be specified for
                                        #kernel='boxcar'. Example: '2arcsec'.
     pa             =         ''        #Position angle used only for gaussian
                                        #kernel. Standard quantity
                                        #representation. Example: '40deg'.

region              =         ''        #Region selection. See Default is to use the full
                                        #image.
box                 =         ''        #Rectangular region to select in
                                        #direction plane. Default is to use the entire
                                        #direction plane.
chans               =         ''        #Channels to use. Default is to use all
                                        #channels.
stokes              =         ''        #Stokes planes to use.  Default is to
                                        #use all Stokes planes.
mask                =         ''        #Mask to use. Default
                                        #is none.
outfile             =         ''        #Output image name. Must be specified.
overwrite           =      False        #Overwrite (unprompted) pre-existing
                                        #output file?
```

where the cube/image imagename will be convolved with a kernel defined in the *kernel* keyword. Kernels \'*gauss\'* and \'*boxcar\'* need the major and minor axes sizes as input, the Gaussian kernel smoothing also requires a position angle. By default, the kernel size defines the kernel itself, i.e. the data will be smoothed with this *kernel*. If the *targetres* parameter for Gaussian kernels is set to \'*True\'*, major and minor axes will be those from the output resolution, and the kernel will be adjusted for each plane to arrive at the final resolution. The 'commonbeam' kernel is to be used when the beam shape is different as a function of frequency. This option will then smooth all planes to a single beam, defined by the largest beam in the cube. With the \'*image\'* kernel, one can specify an image that will serve as the convolution kernel. A scale factor can be applied, which defaults to flux conservation where units are Jy/beam or Jy/beam.km/s. For all other units, like K, the output will be scaled by the inverse of the convolution kernel. e.g., in the extreme case of a flat distribution the values before and after smoothing will be the same.Examples:1) Smoothing with a Gaussian kernel 20" by 10"

```
imsmooth( imagename='my.image', kernel='gauss', major='20arcsec', minor='10arcsec',targetres=T)
```

2\) Smoothing using pixel coordinates and a boxcar kernel.

```
imsmooth( imagename='new.image', major='20pix', minor='10pix', kernel='boxcar')
```


 



***





#### Image Import/Export 

Im/Exporting FITS files and dumping the data into Python



##### Image Import/Export to FITS

These tasks will allow you to write your CASA image to a FITS file that other packages can read, and to import existing FITS files into CASA as an image.

###### FITS Image Export (exportfits)

To export your images to fits format use the **exportfits** task. The inputs are:

```
#exportfits :: Convert a CASA image to a FITS file
imagename    =         ''   #Name of input CASA image
fitsimage    =         ''   #Name of output image FITS file
velocity     =      False   #Use velocity (rather than frequency) as spectral axis
optical      =      False   #Use the optical (rather than radio) velocity convention
bitpix       =        -32   #Bits per pixel
minpix       =          0   #Minimum pixel value
maxpix       =          0   #Maximum pixel value
overwrite    =      False   #Overwrite pre-existing imagename
dropstokes   =      False   #Drop the Stokes axis?
stokeslast   =       True   #Put Stokes axis last in header?
```

The *dropstokes* or *stokeslast* parameter may be needed to make the FITS image compatible with an external application.For example,

```
   exportfits('ngc5921.demo.cleanimg.image','ngc5921.demo.cleanimg.image.fits')
```

#####  FITS Image Import (importfits)

You can also use the **importfits** task to import a FITS image into CASA image table format. Note, the CASA viewer can read fits images so you don't need to do this if you just want to look at the image. The inputs for **importfits** are:

```
#importfits :: Convert an image FITS file into a CASA image
fitsimage           =         ''        #Name of input image FITS file
imagename           =         ''        #Name of output CASA image
whichrep            =          0        #If fits image has multiple
                                        #coordinate reps, choose one.
whichhdu            =          0        #If its file contains
                                        #multiple images, choose one.
zeroblanks          =       True        #Set blanked pixels to zero (not NaN)
overwrite           =      False        #Overwrite pre-existing imagename
defaultaxes         =      False        #Add the default 4D
                                        #coordinate axes where they are missing
defaultaxesvalues   =         []        #List of values to assign to
                                        #added degenerate axes when
                                        #defaultaxes=True (ra,dec,freq,stokes)
```

For example, we can read the above image back in

```
importfits('ngc5921.demo.cleanimg.image.fits','ngc5921.demo.cleanimage')
```

 




##### Extracting data from an image (**imval**)

The **imval** task will extract the values of the data and mask from a specified region of an image and place in the task return value as a Python dictionary. The inputs are:

```
#imval :: Get the data value(s) and/or mask value in an image.
imagename  =      ''   #Name of the input image
region     =      ''   #Image Region.  Use viewer
box        =      ''   #Select one or more box regions
chans      =      ''   #Select the channel(spectral) range
stokes     =      ''   #Stokes params to image (I,IV,IQU,IQUV)
```

Area selection using [box](#region-selection--box-) and [region](#regions--region-) is detailed above. By default, *box=\' \'* will extract the image information at the reference pixel on the direction axes. [Plane selection](#plane-selection--chans--stokes-) is controlled by *chans* and *stokes*. By default, *chans=\' \'* and *stokes=\' \'* will extract the image information in all channels and Stokes planes.For instance,

```
xval = imval('myimage', box='144,144', stokes='I' )
```

will extract the Stokes I value or spectrum at pixel 144,144, while

```
xval = imval('myimage', box='134,134.154,154', stokes='I' )
```

will extract a 21 by 21 pixel region. Extractions are returned in NumPy arrays in the return value dictionary, plus some extra elements describing the axes and selection:

```python

CASA <2>: xval = imval('ngc5921.demo.moments.integrated')

CASA <3>: xval
  Out[3]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [128, 128, 0, 0],
 'data': array([ 0.89667124]),
 'mask': array([ True], dtype=bool),
 'trc': [128, 128, 0, 0],
 'unit': 'Jy/beam.km/s'}
```

extracts the reference pixel value in this 1-plane image. Note that the \'*data\'* and \'*mask\'* elements are NumPy arrays, not Python lists. To extract a spectrum from a cube:

```python

CASA <8>: xval = imval('ngc5921.demo.clean.image',box='125,125')

CASA <9>: xval
  Out[9]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [125, 125, 0, 0],
 'data': array([  8.45717848e-04,   1.93370355e-03,   1.53750915e-03,
         2.88399984e-03,   2.38683447e-03,   2.89159478e-04,
         3.16268904e-03,   9.93389636e-03,   1.88773088e-02,
         3.01138610e-02,   3.14478502e-02,   4.03211266e-02,
         3.82498614e-02,   3.06552909e-02,   2.80734301e-02,
         1.72479432e-02,   1.20884273e-02,   6.13593217e-03,
         9.04005766e-03,   1.71429547e-03,   5.22095338e-03,
         2.49114982e-03,   5.30831399e-04,   4.80734324e-03,
         1.19265869e-05,   1.29435991e-03,   3.75700940e-04,
         2.34788167e-03,   2.72604497e-03,   1.78467855e-03,
         9.74952069e-04,   2.24676146e-03,   1.82263291e-04,
         1.98463408e-06,   2.02975096e-03,   9.65532148e-04,
         1.68218743e-03,   2.92119570e-03,   1.29359076e-03,
        -5.11484570e-04,   1.54162932e-03,   4.68662125e-04,
        -8.50282842e-04,  -7.91683051e-05,   2.95954203e-04,
        -1.30133145e-03]),
 'mask': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool),
 'trc': [125, 125, 0, 45],
 'unit': 'Jy/beam'}
```

To extract a region from the plane of a cube:

```python
CASA <13>: xval = imval('ngc5921.demo.clean.image',box='126,128,130,129',chans='23')

CASA <14>: xval
  Out[14]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [126, 128, 0, 23],
 'data': array([[ 0.00938627,  0.01487772],
       [ 0.00955847,  0.01688832],
       [ 0.00696965,  0.01501907],
       [ 0.00460964,  0.01220793],
       [ 0.00358087,  0.00990202]]),
 'mask': array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]], dtype=bool),
 'trc': [130, 129, 0, 23],
 'unit': 'Jy/beam'}

CASA <15>: print xval['data'][0][1]
0.0148777160794
```

In this example, a rectangular box was extracted, and you can see the order in the array and how to address specific elements.

  



***





#### Math Operations / Statistics 

Deriving image statistics and performing mathematical operations on images

Mathematical operations on images are typically completed using the CASA task [**immath**](global-task-list.ipynb#task_immath), and image statistics may be derived using the CASA tasks [**imstat**](global-task-list.ipynb#task_imstat) and [**imdev**](global-task-list.ipynb#task_imdev). Here, we give an overview of how these tasks are used.



#####  Mathematical Operations on Images (**immath**)

The CASA task **immath** is useful for performing mathematical operations on images and on specific channels within images, including e.g. addition or subtraction of two cubes, squaring an image, computing a spectral index, and determining polarization angles and intensities. The inputs are:

```
#immath :: Perform math operations on images
imagename           =         ''        #a list of input images
mode                = 'evalexpr'        #mode for math operation (evalexpr, spix, pola, poli)
     expr           =         ''        #Mathematical expression using images
     varnames       =         ''        #a list of variable names to use with the image files

outfile             = 'immath_results.im' #File where the output is saved
mask                =         ''        #Mask to use. Default is none.
region              =         ''        #Region selection. 
                                        #Default is to use the full image.
box                 =         ''        #Rectangular region to
                                        #select in direction plane.
                                        #Default is to use the
                                        #entire direction plane.
chans               =         ''        #Channels to use. 
                                        #Default is to use all channels.
stokes              =         ''        #Stokes planes to use. 
                                        #Default is to use all Stokes planes.
imagemd             =         ''        #An image name from which metadata should be copied. The input
                                        #can be either an image listed under imagename or any other
                                        #image on disk. Leaving this parameter unset may copy header
                                        #metadata from any of the input images, which
                                        #one is not guaranteed.
```

<div class="alert alert-warning">
**Alert:** **immath** does not convert any brightness units, e.g. from Jy/beam to K or vice versa. The user is responsible for making sure the images are consistent with the values in the header and the image. It is not advisable to mix input images that are in different units or have different beam sizes.
</div>

The *imagename* parameter must be given the name of a single image as a string (e.g. *imagename=\'image1.im\'*) or the names of multiple images in a list of strings (e.g. *imagename=\[\'image1.im\', \'image2.im\'\]* ). The **immath** task outputs an image file, and the name of the output file is specified using the *outfile* parameter.

The *mode* parameter selects what **immath** is to do. The default, *mode=\'evalexpr\'*, allows the user to specify a mathematical operation to execute on the input images through the *expr* sub-parameter. The mathematical expression is specified in *expr* as a Lattice Expression Language (LEL) string (see the [page on LEL strings](imaging.ipynb#lattice-expression-language-lel)). The standard usage for *mode=\'evalexpr\'* is to input a list of images into the *imagename* parameter, and then refer to them in the *expr* subparameter in LEL by the names IM0, IM1, \....  For example,

```
immath(imagename=['image1.im','image2.im'],expr='IM0-IM1',outfile='ImageDiff.im')
```

would subtract the second image given from the first.

For the special modes \'*spix\'*, \'*pola\'*, and \'*poli\'*, the images required for the operation may need to be listed in *imagename* in a particular order. See examples of usage for polarization data below, paying particular attention to posted alerts.

The mathematical expression can be computed on the entire image cube, or on selected regions and image planes, which can be specified through the *mask*, *region*, *box*, *chans*, and *stokes* parameters. Mask specification is done using the *mask* parameter which can optionally contain an on-the-fly mask expression (in [LEL](imaging.ipynb#lattice-expression-language-lel)) or point to an image with a pixel mask. In some cases, one may like to use a flat image (e.g. a moment image) mask applied to an entire cube. The *stretch=True* subparameter in *mask* allows one to expand the mask to all planes (i.e. channels or Stokes planes) of the cube. Region selection can also be carried out through the *region* parameter (see the pages on [Region Files](imaging.ipynb#region-files) and [Region File Format](imaging.ipynb#region-file-format)) and *box* parameter, while image plane selection is controlled by *chans* and *stokes* parameters.

The image metadata in the output file is adopted from another image, which can be specified through the *imagemd* parameter. In *imagemd*, input the name of the image from which the metadata should be copied and used for the output image. If left blank, the task may pick any of the input image headers, so it is better to define this parameter. In fact, the image specified in *imagemd* can be any image, even an image that is not part of the calculations in **immath**.

Detailed examples of **immath** usage are given below.

#####  Examples for **immath**

In the following, we show a examples of **immath** usage. Note that the image names in the *expr* are assumed to refer to existing image files in the current working directory.

 

###### Simple math

Select a single plane (channel 22) of the 3-D cube:

```
immath(imagename='ngc5921.demo.cleanimg.image',
       expr='IM0',chans='22',
       outfile='ngc5921.demo.chan22.image')
```

Double all values in our image:

```
immath(imagename=['ngc5921.demo.chan22.image'],
       expr='IM0*2.0',
       outfile='ngc5921.demo.chan22double.image' )
```

Square all values in our image:

```
immath(imagename=['ngc5921.demo.chan22.image'],
       expr='IM0^2',
       outfile='ngc5921.demo.chan22squared.image' )
```

<div class="alert alert-info">
**NOTE**: The units in the output image are still claimed to be "Jy/beam", i.e. **immath** will not correctly scale the units in the image for non-linear cases like this. Beware!
</div>

Subtract our image containing channel 22 from the original 3-D cube.  Note that in this example, the 2-D plane (channel 22) is extended into the third dimension, so that che channel 22 image is subtracted from each plane in the 3-D cube:

```
immath(imagename=['ngc5921.demo.cleanimg.image','ngc5921.demo.chan22.image'],
       expr='IM0-IM1',
       outfile='ngc5921.demo.sub22.image')
```

Divide an image by another, with a threshold on one of the images:

```
immath(imagename=['ngc5921.demo.cleanimg.image','ngc5921.demo.chan22.image'],
       expr='IM0/IM1[IM1>0.008]',
       outfile='ngc5921.demo.div22.image')
```

You can do other mathematical operations on an image (e.g. trigonometric functions), as well as use scalar results from an image (e.g. *max, min, median, mean, variance*) in **immath**. You also have access to constants such as *e()* and *pi()*. As an example, the following expression uses the *sine* function, square root (sqrt), a scalar function (max), and the constant *pi* :

```
immath(imagename=['ngc5921.demo.chan22.image','ngc5921.demo.chan22squared.image'],
       expr='sin(float(pi())*IM0/sqrt(max(IM1)))',
       outfile='ngc5921.demo.chan22sine.image')
```

<div class="alert alert-info">
**NOTE**: Once again, the units in the output image are still claimed to be "Jy/beam", i.e. **immath** will not correctly scale the units in the image for non-linear cases like this. Beware!
</div>

 

###### Region and Channel Selection

Select and save a region including the inner 1/4 of an image for channels 0 through 9 (*chans=\'\<10\'*) and channels 40, 42, and 44:

```
default('immath')
imagename=['ngc5921.demo.cleanimg.image']
expr='IM0'
region='box[[64pix,64pix],[192pix,192pix]]'
chans='<10;40,42,44'
outfile='ngc5921.demo.inner.image'
immath()
```

If more than one channel is specified in the *chans* parameter, then the output image will contain multiple channels spanning the range from the lowest channel specified to the highest. In the example above, the output image will span channels 0 through 44, for a total of 45 channels. The channels that were not selected (in this case, channels 10 through 39 and channels 41 and 43) will be masked in the output cube. If we had set *chans=\'40,42,44\'* then there would be 5 output channels corresponding to channels 40, 41, 42, 43, and 44 of the MS with 41 and 43 masked.

Note that the *chans* syntax allows the operators \'\<\', \'\<=\', \'\>\', and \'\>\'. For example, the following two inputs select the same channels.

```
chans = '<17,>79'
chans = '<=16,>=80'
```


###### Polarization manipulation

Extract each Stokes plane from a cube into an individual image:

```
default('immath')
imagename = '3C129BC.clean.image'
outfile='3C129BC.I'; expr='IM0'; stokes='I'; immath();
outfile='3C129BC.Q'; expr='IM0'; stokes='Q'; immath();
outfile='3C129BC.U'; expr='IM0'; stokes='U'; immath();
outfile='3C129BC.V'; expr='IM0'; stokes='V'; immath();
```

Extract linearly polarized intensity and polarization position angle images:

```
immath(stokes='', outfile='3C129BC.P', mode='poli',
       imagename=['3C129BC.Q','3C129BC.U'], sigma='0.0mJy/beam');
immath(stokes='', outfile='3C129BC.X', mode='pola',
       imagename=['3C129BC.Q','3C129BC.U'], sigma='0.0mJy/beam');
```

<div class="alert alert-warning">
**ALERT:** For *mode='pola'* you MUST call as a function as in this example (giving the parameters as arguments) or **immath** will fail.
</div>

Create a fractional linear polarization image:

```
default( 'immath')
imagename = ['3C129BC.I','3C129BC.Q','3C129BC.U']
outfile='3C129BC.fractional_linpol'
expr='sqrt((IM1^2 + IM2^2)/IM0^2)'
stokes=''
immath()
```

Create a polarized intensity image:

```
default( 'immath')
imagename = ['3C129BC.Q','3C129BC.U','3C129BC.V']
outfile='3C129BC.pol_intensity'
expr='sqrt(IM0^2 + IM1^2 + IM2^2)'
stokes=''
immath()
```

Toolkit Tricks: The following uses the toolkit. You can make a complex linear polarization (Q+iU) image using the **imagepol** tool:

```
  #Make an imagepol tool and open the clean image
  potool = casac.homefinder.find_home_by_name('imagepolHome')
  po = potool.create()
  po.open('3C129BC.clean.image')
  #Use complexlinpol to make a Q+iU image
  po.complexlinpol('3C129BC.cmplxlinpol')
  po.close()
```

You can now display this in the **viewer**, in particular overlay this over the intensity raster with the intensity contours. When you load the image, use the LEL:

```
  '3C129BC.cmplxlinpol'['3C129BC.P'>0.0001]
```

which is entered into the LEL box at the bottom of the Load Data menu.

 

###### Using Masks in **immath**

The *mask* parameter is used inside **immath** to apply a mask to all the images used in *expr* before calculations are done (if you are curious, it uses the **ia.subimage** tool method to make virtual images that are then input in the LEL to the **ia.imagecalc** method).For example, let's assume that we have made a single channel image using **clean:**

```
default('clean')
 
vis = 'ngc5921.demo.src.split.ms.contsub'
imagename = 'ngc5921.demo.chan22.cleanimg'
mode = 'channel'
nchan = 1
start = 22
step = 1
 
field = ''
spw = ''
imsize = [256,256]
cell = [15.,15.]
psfalg = 'clark'
gain = 0.1
niter = 6000
threshold='8.0mJy'
weighting = 'briggs'
rmode = 'norm'
robust = 0.5

mask = [108,108,148,148]
 
clean()
```

There is now a file ngc5921.demo.chan22.cleanimg.mask that is an image with values 1.0 inside the cleanbox region and 0.0 outside. We can use this to mask the clean image:

```
default('immath')
imagename = 'ngc5921.demo.chan22.cleanimg.image'
expr='IM0'
mask='"ngc5921.demo.chan22.cleanimg.mask">0.5'
outfile='ngc5921.demo.chan22.cleanimg.imasked'
immath()
```

Toolkit Tricks: Note that there are also pixel masks that can be contained in each image. These are Boolean masks, and are implicitly used in the calculation for each image in *expr*. If you want to use the mask in a different image not in *expr*, try it in *mask*:

```
#First make a pixel mask inside ngc5921.demo.chan22.cleanimg.mask
ia.open('ngc5921.demo.chan22.cleanimg.mask')
ia.calcmask('"ngc5921.demo.chan22.cleanimg.mask">0.5')
ia.summary()
ia.close()
#There is now a 'mask0' mask in this image as reported by the summary

#Now apply this pixel mask in immath
default('immath')
imagename='ngc5921.demo.chan22.cleanimg.image'
expr='IM0'
mask='mask(ngc5921.demo.chan22.cleanimg.mask)'
outfile='ngc5921.demo.chan22.cleanimg.imasked1'
immath()
```

Note that nominally the axes of the mask must be congruent to the axes of the images in *expr*. However, one exception is that the image in mask can have fewer axes (but not axes that exist but are of the wrong lengths). In this case, **immath** will extend the missing axes to cover the range in the images in *expr*. Thus, you can apply a mask made from a single channel to a whole cube.

```
#drop degenerate stokes and freq axes from mask image
ia.open('ngc5921.demo.chan22.cleanimg.mask')
im2 = ia.subimage(outfile='ngc5921.demo.chan22.cleanimg.mymask',dropdeg=True)
im2.summary()
im2.close()
ia.close()
#mymask has only RA and Dec axes

#Now apply this mask to the whole cube
default('immath')
imagename='ngc5921.demo.cleanimg.image'
expr='IM0'
mask='"ngc5921.demo.chan22.cleanimg.mymask">0.5'
outfile='ngc5921.demo.cleanimg.imasked'
immath()
```




##### Computing Image Statistics (**imstat**)

The **imstat** task will calculate statistics on a region of an image and return the results as a value in a Python dictionary. The inputs are:

```
#imstat :: Displays statistical information from an image or image region
imagename           =         ''        #Name of the input image.
axes                =         -1        #List of axes to evaluate statistics over. Default is
                                        #all axes.
region              =         ''        #Image Region or name. Use Viewer.
box                 =         ''        #Select one or more box regions.
chans               =         ''        #Select the channel(spectral) range. 
stokes              =         ''        #Stokes params to image (I,IV,IQU,IQUV). Default '' =>
                                        #include all
listit              =       True        #Print stats and bounding box to logger?
verbose             =      False        #Print additional messages to logger?
mask                =         ''        #Mask to use. Default is none.
logfile             =         ''        #Name of file to write fit results.
algorithm           =  'classic'        #Algorithm to use. Supported values are 'chauvenet',
                                        #'classic', 'fit-half', and 'hinges-fences'. Minimum
                                        #match is supported.
     clmethod       =     'auto'        #Method to use for calculating classical statistics.
                                        #Supported methods are 'auto', 'tiled', and
                                        #'framework'. Ignored if algorithm is not 'classic'.
```

Area selection can be done using *region* and *mask* parameters. Plane selection is controlled by *chans* and *stokes*. The parameter *axes* will select the dimensions that the statistics are calculated over. Typical data cubes have axes like: RA axis 0, DEC axis 1, Velocity axis 2. So, e.g. *axes=\[0,1\]* would be the most common setting to calculate statistics per spectral channel.A typical output of **imstat** on a cube with *axes=\[0,1\]* and *algorithm=\'classic\'* (default) looks like:

```python
No region specified. Using full positional plane.
Using all spectral channels.
Using polarizations ALL
Determining stats for image IRC10216_HC3N.cube_r0.5.image
Set region from supplied region record
Statistics calculated using Classic algorithm
Regions ---
         -- bottom-left corner (pixel) [blc]:  [0, 0, 0, 0]
         -- top-right corner (pixel) [trc]:    [299, 299, 0, 63]
         -- bottom-left corner (world) [blcf]: 09:48:01.492, +13.15.40.658, I, 3.63994e+10Hz
         -- top-right corner (world) [trcf]:   09:47:53.299, +13.17.40.258, I, 3.63915e+10Hz
No region specified. Using full positional plane.
Using all spectral channels.
Using polarizations ALL
Selected bounding box :
    [0, 0, 0, 0] to [299, 299, 0, 63]  (09:48:01.492, +13.15.40.658, I, 3.63994e+10Hz to 09:47:53.299, +13.17.40.258, I, 3.63915e+10Hz)
#Frequency  Frequency(Plane) Npts          Sum           Mean          Rms           Std dev       Minimum       Maximum     
  3.63993552e+10                  0  9.000000e+04  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  3.63992302e+10                  1  9.000000e+04  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  3.63991052e+10                  2  9.000000e+04  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  3.63989802e+10                  3  9.000000e+04  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  3.63988551e+10                  4  9.000000e+04  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  3.63987301e+10                  5  9.000000e+04  6.069948e-01  6.744386e-06  1.534640e-03  1.534634e-03 -6.355108e-03  6.166496e-03
  3.63986051e+10                  6  9.000000e+04  2.711720e-01  3.013023e-06  1.538071e-03  1.538077e-03 -6.165663e-03  5.862981e-03
  3.63984801e+10                  7  9.000000e+04  2.501259e-01  2.779177e-06  1.578049e-03  1.578056e-03 -6.771976e-03  6.272645e-03
  3.63983551e+10                  8  9.000000e+04 -3.706732e-01 -4.118591e-06  1.607191e-03  1.607194e-03 -8.871284e-03  6.591001e-03
```

where the header information provides the specifications of the data that were selected followed by the table with the frequency values of the planes, the plane numbers, Npts (the number of pixels per plane), and the Sum, Median, RMS, Standard deviations, Minimum, and Maximum of the pixel values for each plane. Similar output is provided when the data is averaged over different axes. The logger output can also be written into or appended to a log file for further processing elsewhere (*logfile* parameter).**imstat** has access to different statistics algorithms. Most of them represent different ways on how to treat distributions that are not Gaussian, in particular to eliminate outlier values from the statistics. Available algorithms are CLASSIC, where all unmasked pixels are used, FIT-HALF, where one (good) half of the distribution is being mirrored across a central value, HINGES-FENCES, where the inner quartiles plus a 'fence' data portion is being used, and CHAUVENET, which includes values based on the number of standard deviations from the mean. For more information, see the inline help of the **imstat** task.

#####  Using the task return value

The contents of the return value of **imstat** are in a Python dictionary of key-value sets. For example,

```
xstat = imstat()
```

will assign this to the Python variable xstat. The keys for xstat are outlined on the **imstat** page.For example, an **imstat** call might be

```
 default('imstat')
 imagename = 'ngc5921.demo.cleanimg.image'  #The NGC5921 image cube
 box       = '108,108,148,148'              #20 pixels around the center
 chans     = '21'                           #channel 21

 xstat = imstat()
```

In the terminal window, **imstat** reports:

```python
Statistics on  ngc5921.usecase.clean.image

Region ---
   -- bottom-left corner (pixel) [blc]: [108, 108, 0, 21]
   -- top-right corner (pixel) [trc]:   [148, 148, 0, 21]
   -- bottom-left corner (world) [blcf]: 15:22:20.076, +04.58.59.981, I, 1.41332e+09Hz
   -- top-right corner( world) [trcf]: 15:21:39.919, +05.08.59.981, I, 1.41332e+09Hz

Values --
   -- flux [flux]:              0.111799236126
   -- number of points [npts]:  1681.0
   -- maximum value [max]:      0.029451508075
   -- minimum value [min]:     -0.00612453464419
   -- position of max value (pixel) [maxpos]:  [124, 131, 0, 21]
   -- position of min value (pixel) [minpos]:  [142, 110, 0, 21]
   -- position of max value (world) [maxposf]: 15:22:04.016, +05.04.44.999, I, 1.41332e+09Hz
   -- position of min value (world) [minposf]: 15:21:45.947, +04.59.29.990, I, 1.41332e+09Hz
   -- Sum of pixel values [sum]: 1.32267159822
   -- Sum of squared pixel values [sumsq]: 0.0284534543692
   
Statistics ---
   -- Mean of the pixel values [mean]:       0.000786836167885
   -- Standard deviation of the Mean [sigma]: 0.00403944306904
   -- Root mean square [rms]:               0.00411418313161
   -- Median of the pixel values [median]:     0.000137259965413
   -- Median of the deviations [medabsdevmed]:       0.00152346317191
   -- Quartile [quartile]:                       0.00305395200849
```

The return value in xstat is

```python
CASA <152>: xstat
  Out[152]:
{'blc': array([108, 108,   0,  21]),
 'blcf': '15:22:20.076, +04.58.59.981, I, 1.41332e+09Hz',
 'flux': array([ 0.11179924]),
 'max': array([ 0.02945151]),
 'maxpos': array([124, 131,   0,  21]),
 'maxposf': '15:22:04.016, +05.04.44.999, I, 1.41332e+09Hz',
 'mean': array([ 0.00078684]),
 'medabsdevmed': array([ 0.00152346]),
 'median': array([ 0.00013726]),
 'min': array([-0.00612453]),
 'minpos': array([142, 110,   0,  21]),
 'minposf': '15:21:45.947, +04.59.29.990, I, 1.41332e+09Hz',
 'npts': array([ 1681.]),
 'quartile': array([ 0.00305395]),
 'rms': array([ 0.00411418]),
 'sigma': array([ 0.00403944]),
 'sum': array([ 1.3226716]),
 'sumsq': array([ 0.02845345]),
 'trc': array([148, 148,   0,  21]),
 'trcf': '15:21:39.919, +05.08.59.981, I, 1.41332e+09Hz'}
```

<div class="alert alert-warning">
**ALERT:** The return dictionary currently includes NumPy array values, which have to be accessed by an array index to get the array value. To access these dictionary elements, use the standard Python dictionary syntax, e.g. [xstat[<key string>][<array index>]]
</div>

For example, to extract the standard deviation as a number

```
mystddev = xstat['sigma'][0]
print 'Sigma = '+str(xstat['sigma'][0])
```

#####  Examples for **imstat**

To extract statistics for an image:

```
xstat = imstat('b1608.demo.clean2.image')
#Printing out some of these
  print 'Max   = '+str(xstat['max'][0])
  print 'Sigma = '+str(xstat['sigma'][0])
#results:
#Max   = 0.016796965152
#Sigma = 0.00033631979385
```

In a box around the brightest component:

```
xstat_A = imstat('b1608.demo.clean2.image',box='124,125,132,133')
#Printing out some of these
  print 'Comp A Max Flux = '+str(xstat_A['max'][0])
  print 'Comp A Max X,Y  = ('+str(xstat_A['maxpos'][0])+','+str(xstat_A['maxpos'][1])+')'
#results:
#Comp A Max Flux = 0.016796965152
#Comp A Max X,Y  = (128,129)
```

 



##### Computing a *Deviation* Image (**imdev**)

The **imdev** task produces an output image whose value in each pixel represents the \"error\" or \"deviation\" in the input image at the corresponding pixel. The output image has the same dimensions and coordinate system as the input image, or as the selected region of the input image. The inputs are:

```
#imdev :: Create an image that can represent the statistical deviations of the input image.
imagename          =          ''        #Input image name
outfile            =          ''        #Output image file name. If left blank (the default), no
                                        #image is written but a new image tool referencing
                                        #the collapsed image is returned.
region             =          ''        #Region selection. Default is to use the full image.
box                =          ''        #Rectangular region(s) to select in direction plane.
                                        #Default is to use the entire direction plane.
chans              =          ''        #Channels to use. Default is to use all channels.
stokes             =          ''        #Stokes planes to use. Default is to use all Stokes planes.
mask               =          ''        #Mask to use. Default setting is none.
overwrite          =       False        #Overwrite (unprompted) pre-existing output file? Ignored
                                        #if "outfile" is left blank.
grid               =      [1, 1]        #x,y grid spacing. Array of exactly two positive integers.
anchor             =       'ref'        #x,y anchor pixel location. Either "ref" to use the image
                                        #exactly two integers.
xlength            =      '1pix'        #Either x coordinate length of box, or diameter of circle.
                                        #Circle is used if ylength is a reference pixel or an
                                        #empty string.
ylength            =      '1pix'        #y coordinate length of box. Use a circle if ylength is
                                        #an empty string.
interp             =     'cubic'        #Interpolation algorithm to use. Can be "nearest", "linear",
                                        #"cubic", or "lanczos". Minimum match supported.
stattype           =     'sigma'        #Statistic to compute. See below for the list of supported
                                        #statistics.
statalg            =   'classic'        #Statistics computation algorithm to use. Supported values
                                        #are "chauvenet" and "classic". Minimum match is supported.
```

Area selection can be done using the *region* and *mask* parameters. Plane selection is controlled by the *chans* and *stokes* parameters. Statistics are computed spatially: a deviation image is computed independently for each channel/Stokes plane. If the *outfile* parameter is left blank, the task returns an image tool referencing the resulting image; otherwise the resulting image is written to disk.

The statistic to be computed is selected using the *stattype* parameter. Allowed statistics are:

<div>

```
iqr                      inner quartile range (q3 - q1)
max                      maximum
mean                     mean
medabsdevmed, madm       median absolute deviation from the median
median                   median
min                      minimum
npts                     number of points
q1                       first quartile
q3                       third quartile
rms                      rms
sigma, std               standard deviation
sumsq                    sum of squares
sum                      sum
var                      variance
xmadm                    median absolute deviation from the median multipied by x, where x is the
                         reciprocal of Phi^-1(3/4),where Phi^-1 is the reciprocal of the quantile
                         function. Numerically, x = 1.482602218505602. See, eg,
                         https://en.wikipedia.org/wiki/Median_absolute_deviation#Relation_to_standard_deviation
```

</div>

The chosen statistic is calculated around a set of grid points (pixels) across the input image with grid spacing specified by the *grid* parameter. The size and shape of the region used to compute the statistic at each grid point is specified by the *xlength* and *ylength* parameters. If *ylength* is an empty string, then the region used is a circle centered on each grid point with diameter provided by *xlength*. Otherwise, a rectangular region with dimensions of *xlength* by *ylength* is used. These two parameters may be specified as valid quantities with recognized units (e.g., \"4arcsec\" or \"4pix\"). They may also be specified as numerical values, in which case the unit is assumed to be pixels.

The chosen statistic is calculated at every grid point in the input image, and the result is reflected at the corresponding pixel of the output image. Values at all other pixels in the output image are determined by interpolating across the grid points using the interpolation scheme given by the input parameter *interp*. The *statalg* parameter specifies the algorithm for the statistics computation. Available algorithms are CLASSIC, where all unmasked pixels are used, and CHAUVENET, which includes values based on the number of standard deviations from the mean.

##### Examples for **imdev**

Compute a \"standard deviation\" image using grid-spacing of 4 arcsec in the X direction and 5 arcsec in the Y direction, with linear interpolation to compute values at non-grid-pixels. Compute the standard deviation in a box of 20 x 25 arcsec.

<div>

```
imdev("my.image", "std.image", grid=[4,5], xlength="20arcsec", ylength="25arcsec",
       stattype="sigma", interp="linear", statalg="classic")
```

</div>

Compute an image showing median absolute deviation (MAD) across the image, with MAD converted to an equivalent RMS value. Anchor the grid at a specific pixel \[1000,1000\] with grid-spacing of 10 pixels, and use circles of diameter 30 pixels for the statistical computation. Calculate the statistic using the z-score/Chauvenet algorithm by fixing the maximum z-score to determine outliers to 5. Use cubic interpolation to determine the value at non-grid-pixels. Have the task return a pointer to the output image.

<div>

```
myim = imdev("my.image", anchor=[1000,1000], grid=[10,10], xlength=30, ylength='',
       stattype="xmadm", interp="cubic", statalg="chauvenet", zscore=5)
```

</div>



***





#### Image Selection Parameters 

Select parts of an image with the box, chans, and stokes parameters

Many of the image analysis tasks contain a set of parameters that can be used to define a sub-section of an image that the task works on. This includes selection in the spatial coordinates, typically RA/DEC or GLON/GLAT (or image axes 0,1) which can be defined by the *box* parameter. Spectral selection can be achieved by the *chans* parameter (typically axis 3) and Stokes selection with the *stokes* parameter (typically axis 2). These parameters are described below and are a quick way to select sub-images. For more complicated selections, we recommend the usage of the [CASA region file](imaging.ipynb#region-files) ([CRTF file](imaging.ipynb#region-file-format)).    

 

##### Region Selection (*box*)

Direction (e.g. RA, Dec) areal selection in the image analysis tasks is controlled by the *box* parameter or through the *region* parameter. Note that one should either specify a region (recommended) or any of *box*/*chans*/*stokes* parameters. Specifying both at the same time will give priority to the command line inputs in \'chans\' and \'stokes\' but will keep the region file specification for the spatial region selection.

The *box* parameter selects spatial rectangular areas:

```
box        =      ''   #Select one or more box regions
```

E.g. for right ascension and declination. Boxes are specified by their bottom-left corner (blc) and top-right corner (trc) as follows: blcx, blcy, trcx, trcy. At the moment, CASA only accepts pixel values. The default (empty string) is to select all pixels of an image.

Example:

```
box='0,0,50,50'
```

selects a square with 50 pixels on the side starting at 0.

```
box='10,20,30,40,100,100,150,150'
```

selects two regions at a time. The first and the last four values mark the two boxes, following (blcx1, blcy1, trcx1, trcy1, blcx2, blcy2, trcx2, trcy2), with b/t = bottom/top and l/r = left/right.

<div class="alert alert-warning">
**ALERT:** Note that one should either specify a region (recommended) or any of *box*/*chans*/*stokes*. If both are specified, the following rules apply:
          * If the *region* parameter is specified as a python dictionary (e.g. such as various **rg** tool methods return), a binary region file, or a region-in-image, it is not permissible to specify any of the *box*, *chans*, or *stokes* parameters.
          * If the *region* parameter is specified to be a CRTF file name, or a CRTF region string, then the resulting *region* and *box* selection is the union of the box specification with any regions in the CRTF file/string. This is the equivalent of translating the box specification into the equivalent "box" CRTF specification and prepending that specification to the specified CRTF file/string in the *region* parameter.
</div>

##### Plane Selection (*chans*, *stokes*)

The channel, frequency, or velocity plane(s) of the image is chosen using the *chans* parameter:

```
chans      =      ''   #Select the channel(spectral) range
```

Using channel numbers, it is possible to also set ranges, e.g.**

```
chans='0,3,4,8'          #select channels 0, 3, 4, 8
chans='3~20;50,51'       #channels 3 to 20 and 50 and 51
chans='<10;>=55'         #channels 0 to 9 and 55 and greater (inclusively)*
*
```

Sometimes, as in the **immoments** task, the channel/plane selection is generalized to work on more than one axis type. In this case, the *planes* parameter is used. This behaves like *chans* in syntax.

*chans* can also be set in the CASA region format to allow settings in frequency and velocity, e.g.

```
 chans=('range=[-50km/s,50km/s], restfreq=100GHz, frame=LSRK')
```

This example would even define a new velocity system independent of the one in the image itself. If the rest frequency and velocity frame within the image are being used, the latter two entries are not needed. The parentheses are needed when the call is in a single command.

A frequency selection looks like:

```
chans=('range=[100GHz,100.125GHz]')
```

The polarization plane(s) of the image is chosen with the *stokes* parameter:

```
stokes     =      ''   #Stokes params to image (I,IV,IQU,IQUV)
```

*stokes* parameters to image. Best practice is to separate the stokes parameters by commas, but this is not essential if no ambiguity exists. Options are: \'I\',\'Q\',\'U\',\'V\', \'I\',\'IV\',\'QU\',\'IQUV\', \'RR\',\'RL\',\'LR\',\'LL\', \'XX\',\'YX\',\'XY\',\'YY\',\...

Examples:

```
stokes='IQUV';  
stokes='I,Q'
stokes='RR,RL'
```

<div class="alert alert-warning">
**ALERT:** For image analysis tasks and tool methods which also accept the *region* parameter, the following rules apply if both the *chans* and *region* parameters are simultaneously specified:
    * If the *region* parameter is specified as a python dictionary (e.g. such as various **rg** tool methods return), a binary region file, or a region-in-image, it is not permissable to specify any of the *box*, *chans*, or *stokes* parameters.
    * If the *region* parameter is specified to be a CRTF file name, or a CRTF region string, it is only permissable to specify *chans* if that specification can be represented as a single contiguous channel range. In that case, the *chans* specification overrides any global or per-region range specification in the CRTF file/string, and is used as the global spectral range selection for all regions in the CRTF file/string.
</div>

 



***





#### Region Files 

CASA Region Text Files usage and definition

##### Regions (*region*)

The *region* parameter points to a CASA region which can be directly specified or listed in a ImageRegion file. An ImageRegion file can be created with the CASA viewer's region manager, or directly using the [CASA region file (crtf) syntax](imaging.ipynb#region-file-format). Typically ImageRegion files will have the suffix \".crtf\" for CASA Region Text Format.

For example:

```
region='circle[[18h12m24s, -23d11m00s], 2.3arcsec]'
```

or

```
region='myimage.im.crtf'
```

to specify a region file. For the most part, the *region* parameter in tasks only accepts strings (e.g. file names, region shape descriptions) while the *region* parameter in **ia** tool methods only accepts python region dictionaries (e.g. produced using the **rg** tool).

 

<div class="alert alert-warning">
**Alert:** When both the *region* parameter and any of *box*/*chans*/*stokes* are specified simultaneously, the task may perform unwanted selections. While it is safest to only specify one of these (sets of) parameters, the following rules apply:
</div>

For image analysis tasks and tool methods which also accept the box, chans, and/or stokes parameters, the following rules apply if the region parameter is specified:

-   If region is specified as a python dictionary (eg such as various rg tool methods return), a binary region file, or a region-in-image, then it is not permissable to specify any of the box, chans, or stokes parameters.
-   If the region parameter is specified to be a CRTF file name, or a CRTF region    string, the following rules apply:    -   If box is specified, the resulting selection is the union of the box specification with any regions in the CRTF file/string. This is the equivalent of translating the box specification into the equivalent \"box\" CRTF specification and prepending that specification to the specified CRTF file/string in the region parameter.
    -   If chans is specified, it must be able to be represented as a single contiguous range of channels. In this case, the chans specification overrides any global or per-region range specification in the CRTF file/string, and is used as the global spectral range selection for all regions in the CRTF file/string.
    -   If stokes is specified, this specification overrides any global or per-region corr specification in the CRTF file/string, and is used as the global correlation selection for all regions in the CRTF file/string.

<div class="alert alert-info">
**NOTE:** The CASA image analysis tasks will determine how a region is projected on a pixel image. The current CASA definition is that when the center of a pixel is inside the region, the full pixel is considered to be included in the region.  If the center of the pixel is outside the region, the full pixel will be excluded. Note that the CASA viewer behavior is not entirely consistent and for rectangles it assumes that *any* fractional pixel coverage will include the entire pixel. For other supported shapes (ellipses and polygons), however, ithe viewer adheres to the 'center of pixel' definition, consistent with the image analysis tools and tasks. 

For purely single-pixel work regions may not necessarily be the best choice and alternate methods may be preferable to using regions, eg. **ia.topixel**, **ia.toworld**, **ia.pixelvalue**.
</div>

<div class="alert alert-warning">
**ALERT:** Some region file specifications are not recognized by the viewer, the viewer only supports rectangles (box), ellipses, and polygons.
</div>

 

 

 



***





#### Region File Format 

Syntax for CASA CRTF Region Files

The CASA region file format provides a flexible, easily edited set of region definitions which are accepted across CASA tasks. Region files may be written by hand or using the CASA **viewer**.

<div class="alert alert-warning">
**ALERT**: Whereas the region format is supported by all the data processing tasks, the viewer implementation is still limited to rectangles, ellipses, and some markers.
</div>

For a file to be recognized as a valid CASA region text file, the first line must contain the string:

    #CRTF

\"CRTF\" stands for \"CASA Region Text Format\". One may also include an optional version number at the end of the string, so it reads \#CRTFv0; this indicates the version of the format definition.

Region files have two different kinds of definitions, \"regions\" and \"annotations\", each of which is one line long. To indicate an annotation, a line must begin with \"ann\". Lines that begin with the comment character (\#) are not considered for processing or display.

The second line of a file may define global parameters that are to be used for all regions and annotations in that file, in which case the line starts with the word \"global\". The parameters set here may also be overridden by keywords in a specific line, in which case the keywords pertain only to that one line.

<div class="alert alert-info">
**NOTE:** All regions are considered by tasks. They will be displayed by visualization tasks as well as used to create masks, etc., as appropriate. Annotations are used by display tasks, and are for visual reference only.
</div>

Some tasks, like **clean**, require that a region cannot be entirely outside the image.

 



##### Region Definitions

All regions lines will follow this general arrangement:

    {shape} {additional parameter=value pairs}

The possible parameter/value pairs are described in more detail below. Note that most parameters beyond the shape and its coordinates can be defined globally.

Possible units for coordinates are:

-   *sexagesimal*, e.g. 18h12m24s for right ascension or -03.47.27.1 for declination
-   *decimal degrees*, e.g. 140.0342deg for both RA and Dec
-   *radians*, e.g. 2.37666rad for both RA and Dec
-   *pixels*, e.g. 204pix

Possible units of length are:

-   *degrees*, e.g. 23deg
-   *arcminutes*, e.g. 23arcmin
-   *arcseconds*, e.g. 23arcsec
-   *radians*, e.g. 0.00035rad
-   *pixels*, e.g. 23pix

*Units must always be included when defining a region.* 

<div class="alert alert-info">
**NOTE:** The CASA image analysis tasks will determine how a region is projected on a pixel image. The current CASA definition is that when the center of a pixel is inside the region, the full pixel is considered to be included in the region.  If the center of the pixel is outside the region, the full pixel will be excluded. Note that the CASA viewer behavior is not entirely consistent and for rectangles it assumes that *any* fractional pixel coverage will include the entire pixel. For other supported shapes (ellipses and polygons), however, ithe viewer adheres to the 'center of pixel' definition, consistent with the image analysis tools and tasks. 

For purely single-pixel work regions may not necessarily be the best choice and alternate methods may be preferable to using regions, eg. **ia.topixel**, **ia.toworld**, **ia.pixelvalue**.
</div>

 



##### Allowed Shapes

-   **Rectangular box**; the two coordinates are two opposite corners:

```{=html}
<!-- -->
```
    box[[x1, y1], [x2, y2]]

-   **Center box**; \[x, y\] define the center point of the box and \[x_width, y_width\] the width of the sides:

```{=html}
<!-- -->
```
    centerbox[[x, y], [x_width, y_width]]

-   **Rotated box**; \[x, y\] define the center point of the box; \[x_width, y_width\] the width of the sides; rotang the rotation angle:

```{=html}
<!-- -->
```
    rotbox[[x, y], [x_width, y_width], rotang]

-   **Polygon**; there could be many \[x, y\] corners. If parts of the polygon overlap, then the pixels in that overlapping region are taken into account. Note that the last point will connect with the first point to close the polygon:

```{=html}
<!-- -->
```
    poly[[x1, y1], [x2, y2], [x3, y3], ...]

-   **Circle**; center of the circle \[x,y\], r is the radius:

```{=html}
<!-- -->
```
    circle[[x, y], r]

-   **Annulus**; center of the circle is \[x, y\], \[r1, r2\] are inner and outer radii:

```{=html}
<!-- -->
```
    annulus[[x, y], [r1, r2]]

-   **Ellipse**; center of the ellipse is \[x, y\]; semi-major and semi-minor axes are \[bmaj, bmin\]; position angle of the major axis is pa:

```{=html}
<!-- -->
```
    ellipse[[x, y], [bmaj, bmin], pa]

 



##### Annotation Definitions

In addition to the definitions for regions, above, the following are always treated as annotations:

-   **Line**; coordinates define the end points of the line:

```{=html}
<!-- -->
```
    line[[x1, y1], [x2, y2]]

-   **Vector**; coordinates define end points; second coordinate pair is location of tip of arrow:

```{=html}
<!-- -->
```
    vector[[x1, y1], [x2, y2]]

-   **Text**; coordinates define leftmost point of text string:

```{=html}
<!-- -->
```
    text[[x, y], ’my text’]

-   **Symbol**; coordinates define location of symbol (see [below](#fonts-and-symbols) for a list of allowed symbols):

```{=html}
<!-- -->
```
    symbol[[x, y], {symbol}]

 



##### Global Definitions

Definitions to be used throughout the region file are placed on a line beginning with \'global\', usually at the top of the file. These definitions may also be used on any individual region or annotation line; in this case, the value defined on that line will override the predefined global (but only for that line). If a 'global' line occurs later in the file, subsequent lines will obey those definitions.

-   *Coordinate reference frame*:
    -   Possible values: J2000, B1950, B1950_VLA, GALACTIC,        HADEC, AZEL, AZELSW, AZELNE, ECLIPTIC, MECLIPTIC, TECLIPTIC, SUPERGAL, ICRS
    -   Default: image value

```{=html}
<!-- -->
```
    coord = J2000

Frequency/velocity axis:

-   Possible values: REST, LSRK, LSRD, BARY, GEO, TOPO, GALACTO, LGROUP, CMB
-   Default: image value

```{=html}
<!-- -->
```
    frame=TOPO

-   Frequency/velocity range:
    -   Possible units: GHz, MHz, kHz, km/s, Hz, channel, chan (=channel)
    -   Default: image range

```{=html}
<!-- -->
```
    range=[min, max]

-   Correlation axis:
    -   Possible values: I, Q, U, V, RR, RL, LR, LL, XX, XY, YX, YY, RX, RY, LX, LY, XR, XL, YR, YL, PP, PQ, QP, QQ, RCircular, LCircular, Linear, Ptotal, Plinear, PFtotal, PFlinear, Pangle
    -   Default: all planes present in image

```{=html}
<!-- -->
```
    corr=[X, Y]

-   Velocity calculation:
    -   Possible values: RADIO, OPTICAL, Z, BETA, GAMMA
    -   Default: image value

```{=html}
<!-- -->
```
    veltype=RADIO

-   Rest frequency:
    -   Default: image value

```{=html}
<!-- -->
```
    restfreq=1.42GHz

-   Line characteristics:
    -   Possible values: any line style recognized by matplotlib: '-'=solid, '\--'=dashed, ':'=dotted
    -   Default:

```{=html}
<!-- -->
```
    linewidth=1
    linestyle=’-’

-   Symbol characteristics:
    -   Symbol size and thickness:

```{=html}
<!-- -->
```
    symsize = 1
    symthick = 1

-   Region, symbol, and text color:
    -   Possible values: any color recognized by matplotlib, including hex values
    -   Default:

```{=html}
<!-- -->
```
    color=green
    color=red

-   Text font characteristics:
    -   Possible values: see [below](#allowed-fonts)
    -   'usetex' is a boolean parameter that determines whether or not the text line should be interpreted as LaTeX, and would require working LaTeX, dvipng, and Ghostscript installations (equivalent to the text.usetex parameter in matplotlib).

```{=html}
<!-- -->
```
    font=Helvetica
    fontsize=10pt 
    fontstyle=bold
    usetex=True/False

-   Label position:
    -   Possible values: 'left', 'right', 'top', 'bottom'
    -   Default: 'top'

```{=html}
<!-- -->
```
    labelpos=’right’

-   Label color:
    -   Default: color of associated region.
    -   Allowed values: same as values for region colors.

```{=html}
<!-- -->
```
    labelcolor=’green’

-   Label offset:
    -   Default: \[0,0\].
    -   Allowed values: any positive or negative number, in units of pixels.

```{=html}
<!-- -->
```
    labeloff=[1, 1]

 



##### Allowed Additional Parameters

These must be defined per region line:

-   *Labels*: text label for a region; should be placed so text does not overlap with region boundary

```{=html}
<!-- -->
```
    label=’string’

-   *\"OR/NOT\" operators*: A \"+\" at the beginning of a line will flag it with a boolean \"OR\" (default), and a \"-\" will flag it with a boolean \"NOT\". Overlapping regions will be treated according to their sequence in the file; i.e., ((((entireImage OR line1) OR line2) NOT line3) OR line4). This allows some flexibility in building \"non-standard\" regions. Note that a task (e.g., clean) will still consider all lines: if one wishes to remove a region from consideration, it should be commented out (\"\#\").
-   Default: OR (+)

 



##### Examples

A file with both global definitions and per-line definitions:

    #CRTFv0
    global coord=B1950_VLA, frame=BARY, corr=[I, Q], color=blue

    # A simple circle region:
    circle[[18h12m24s, -23d11m00s], 2.3arcsec]

    # A box region, this one only for annotation:
    ann box[[140.0342deg, -12.34243deg], [140.0360deg, -12.34320deg]]

    # A rotated box region, for a particular range of velocities:
    rotbox[[12h01m34.1s, 12d23m33s], [3arcmin, 1arcmin], 12deg], range=[-1240km/s, 1240km/s]

    # An annular region, overriding some of the global defaults:
    annulus[[17h51m03.2s, -45d17m50s], [4.12deg, 0.10deg]], corr=[I,Q,U,V], color=red, label=’My label here’

    # Cuts an ellipse out of the previous regions, but only for Q and a particular frequency range:
    -ellipse[[17:51:03.2, -45.17.50], [1.34deg, 0.25deg], 45rad], range=[1.420GHz, 1.421GHz], corr=[Q], color=green, label=’Removed this’

    # A diamond marker, in J2000 coordinates:
    symbol[[32.1423deg, 12.1412deg], D], linewidth=2, coord=J2000, symsize=2

 



##### Fonts and Symbols

##### Allowed Symbols

  -------------------------------------------------------------------- -----------------------------------------------------
  \'.\'                                                                point marker 
  \',\'                                  pixel marker
  \'o\'                                  circle marker
  \'v\'                                  triangle_down marker
  \'\^\'                                 triangle_up marker
  \'\<\'   triangle_left marker
  \'\>\'   triangle_right marker
  \'1\'                                  tri_down marker
  \'2\'                                  tri_up marker
  \'3\'                                  tri_left marker
  \'4\'                                  tri_right marker
  \'s\'                                  square marker
  \'p\'                                  pentagon marker
  \'\*\'                                 star marker
  \'h\'                                  hexagon1 marker
  \'H\'                                  hexagon2 marker
  \'+\'                                  plus marker
  \'x\'                                  x marker
  \'D\'                                  diamond marker
  \'d\'                                  thin_diamond marker
  \'\|\'                                 vline marker
  \'\_\'                                 hline marker
  -------------------------------------------------------------------- -----------------------------------------------------


##### Allowed Fonts

###### Allowed Fonts for Linux

\"Century Schoolbook L\", \"Console\", \"Courier\", \"Courier 10 Pitch\", \"Cursor\", \"David CLM\", \"DejaVu LGC Sans\", \"DejaVu LGC Sans Condensed\", \"DejaVu LGC Sans Light\", \"DejaVu LGC Sans Mono\", \"DejaVu LGC Serif\", \"DejaVu LGC Serif Condensed\", \"Dingbats\", \"Drugulin CLM\", \"East Syriac Adiabene\", \"Ellinia CLM\", \"Estrangelo Antioch\", \"Estrangelo Edessa\", \"Estrangelo Nisibin\", \"Estrangelo Nisibin Outline\", \"Estrangelo Talada\", \"Fangsong ti\", \"Fixed \[Sony\]\", \"Fixed \[Eten\]\", \"Fixed \[Misc\]\", \"Fixed \[MNKANAME\]\", \"Frank Ruehl CLM\", \"fxd\", \"Goha-Tibeb Zemen\", \"goth_p\", \"Gothic \[Shinonome\]\", \"Gothic \[mplus\]\", \"hlv\", \"hlvw\", \"KacstArt\", \"KacstBook\", \"KacstDecorative\", \"KacstDigital\", \"KacstFarsi\", \"KacstLetter\", \"KacstPoster\", \"KacstQura\", \"KacstQuraFixed\", \"KacstQuran\", \"KacstTitle\", \"KacstTitleL\", \"Liberation Mono\", \"Liberation Sans\", \"Liberation Serif\", \"LKLUG\", \"Lohit Bengali\", \"Lohit Gujarati\", \"Lohit Hindi\", \"Lohit Kannada\", \"Lohit Malayalam\", \"Lohit Oriya\", \"Lohit Punjabi\", \"Lohit Tamil\", \"Lohit Telugu\", \"LucidaTypewriter\", \"Luxi Mono\", \"Luxi Sans\", \"Luxi Serif\", \"Marumoji\", \"Miriam CLM\", \"Miriam Mono CLM\", \"MiscFixed\", \"Monospace\", \"Nachlieli CLM\", \"Nimbus Mono L\", \"Nimbus Roman No9 L\", \"Nimbus Sans L\", \"Nimbus Sans L Condensed\", \"PakTypeNaqsh\", \"PakTypeTehreer\", \"qub\", \"Sans Serif\", \"Sazanami Gothic\", \"Sazanami Mincho\", \"Serif\", \"Serto Batnan\", \"Serto Jerusalem\", \"Serto Jerusalem Outline\", \"Serto Mardin\", \"Standard Symbols L\", \"sys\", \"URW Bookman L\", \"URW Chancery L\", \"URW Gothic L\", \"URW Palladio L\", \"Utopia\", \"Yehuda CLM\"

###### Allowed Fonts for MacOS X

\"Abadi MT Condensed Light\", \"Adobe Caslon Pro\", \"Adobe Garamond Pro\", \"Al Bayan\", \"American Typewriter\", \"Andale Mono\", \"Apple Braille\", \"Apple Chancery\", \"Apple LiGothic\", \"Apple LiSung\", \"Apple Symbols\", \"AppleGothic\", \"AppleMyungjo\", \"Arial\", \"Arial Black\", \"Arial Hebrew\", \"Arial Narrow\", \"Arial Rounded MT Bold\", \"Arial Unicode MS\", \"Arno Pro\", \"Ayuthaya\", \"Baghdad\", \"Baskerville\", \"Baskerville Old Face\", \"Batang\", \"Bauhaus 93\", \"Bell Gothic Std\", \"Bell MT\", \"Bernard MT Condensed\", \"BiauKai\", \"Bickham Script Pro\", \"Big Caslon\", \"Birch Std\", \"Blackoak Std\", \"Book Antiqua\", \"Bookman Old Style\", \"Bookshelf Symbol 7\", \"Braggadocio\", \"Britannic Bold\", \"Brush Script MT\", \"Brush Script Std\", \"Calibri\", \"Calisto MT\", \"Cambria\", \"Candara\", \"Century\", \"Century Gothic\", \"Century Schoolbook\", \"Chalkboard\", \"Chalkduster\", \"Chaparral Pro\", \"Charcoal CY\", \"Charlemagne Std\", \"Cochin\", \"Colonna MT\", \"Comic Sans MS\", \"Consolas\", \"Constantia\", \"Cooper Black\", \"Cooper Std\", \"Copperplate\", \"Copperplate Gothic Bold\", \"Copperplate Gothic Light\", \"Corbel\", \"Corsiva Hebrew\", \"Courier\", \"Courier New\", \"Curlz MT\", \"DecoType Naskh\", \"Desdemona\", \"Devanagari MT\", \"Didot\", \"Eccentric Std\", \"Edwardian Script ITC\", \"Engravers MT\", \"Euphemia UCAS\", \"Eurostile\", \"Footlight MT Light\", \"Franklin Gothic Book\", \"Franklin Gothic Medium\", \"Futura\", \"Garamond\", \"Garamond Premier Pro\", \"GB18030 Bitmap\", \"Geeza Pro\", \"Geneva\", \"Geneva CY\", \"Georgia\", \"Giddyup Std\", \"Gill Sans\", \"Gill Sans MT\", \"Gill Sans Ultra Bold\", \"Gloucester MT Extra Condensed\", \"Goudy Old Style\", \"Gujarati MT\", \"Gulim\", \"GungSeo\", \"Gurmukhi MT\", \"Haettenschweiler\", \"Harrington\", \"HeadLineA\", \"Hei\", \"Heiti SC\", \"Heiti TC\", \"Helvetica\", \"Helvetica CY\", \"Helvetica Neue\", \"Herculanum\" \"Hiragino Kaku Gothic Pro\", \"Hiragino Kaku Gothic ProN\", \"Hiragino Kaku Gothic Std\", \"Hiragino Kaku Gothic StdN\", \"Hiragino Maru Gothic Pro\", \"Hiragino Maru Gothic ProN\", \"Hiragino Mincho Pro\", \"Hiragino Mincho ProN\", \"Hiragino Sans GB\", \"Hobo Std\", \"Hoefler Text\", \"Impact\", \"Imprint MT Shadow\", \"InaiMathi\", \"Kai\", \"Kailasa\", \"Kino MT\", \"Kokonor\", \"Kozuka Gothic Pro\", \"Kozuka Mincho Pro\", \"Krungthep\", \"KufiStandardGK\", \"Letter Gothic Std\", \"LiHei Pro\", \"LiSong Pro\", \"Lithos Pro\", \"Lucida Blackletter\", \"Lucida Bright\", \"Lucida Calligraphy\", \"Lucida Console\", \"Lucida Fax\", \"Lucida Grande\", \"Lucida Handwriting\", \"Lucida Sans\", \"Lucida Sans Typewriter\", \"Lucida Sans Unicode\", \"Marker Felt\", \"Marlett\", \"Matura MT Script Capitals\", \"Meiryo\", \"Menlo\", \"Mesquite Std\", \"Microsoft Sans Serif\", \"Minion Pro\", \"Mistral\", \"Modern No. 20\", \"Monaco\", \"Monotype Corsiva\", \"Monotype Sorts\", \"MS Gothic\", \"MS Mincho\", \"MS PGothic\", \"MS PMincho\", \"MS Reference Sans Serif\", \"MS Reference Specialty\", \"Mshtakan\", \"MT Extra\", \"Myriad Pro\", \"Nadeem\", \"New Peninim MT\", \"News Gothic MT\", \"Nueva Std\", \"OCR A Std\", \"Onyx\", \"Optima\", \"Orator Std\", \"Osaka\", \"Papyrus\", \"PCMyungjo\", \"Perpetua\", \"Perpetua Titling MT\", \"PilGi\", \"Plantagenet Cherokee\", \"Playbill\", \"PMingLiU\", \"Poplar Std\", \"Prestige Elite Std\", \"Raanana\", \"Rockwell\", \"Rockwell Extra Bold\", \"Rosewood Std\", \"Sathu\", \"Silom\", \"SimSun\", \"Skia\", \"Stencil\", \"Stencil Std\", \"STFangsong\", \"STHeiti\", \"STKaiti\", \"STSong\", \"Symbol\", \"Tahoma\", \"Tekton Pro\", \"Thonburi\", \"Times\", \"Times New Roman\", \"Trajan Pro\", \"Trebuchet MS\", \"Tw Cen MT\", \"Verdana\", \"Webdings\", \"Wide Latin\", \"Wingdings\", \"Wingdings 2\", \"Wingdings 3\", \"Zapf Dingbats\", \"Zapfino\"

 



***





#### Image Masks 

Pixel by pixel selection via image masks

A mask can be used to define whether part of an image is used or not. There are different options for masks:

-   an image cube with Boolean True/False values: They usually live inside image cubes and are automatically applied to the data. More than one mask may exist in a cube. The task **makemask** (see [below](#image-mask-handling--makemask-)) can be used to access and manipulate internal Boolean masks via the image.im:mask syntax. 
-   an image cube with zero and non-zero values: They are their own image cubes and are applied to other image cubes when needed.
-   an LEL string for a condition.

 



##### Masks (*mask parameter*)

Using image cubes is useful to mask on a pixel by pixel basis, where False and zeros mark masked (excluded) pixels (see also \'[LEL masks](imaging.ipynb#lel-masks)\'). Both versions can be converted into each other with the task **makemask**. Some analysis tasks show an optional *stretch* parameter which is useful, e.g., to expand a single plane mask to an entire cube along the spectral axis.

To use a different zero/non-zero mask (in this case *\'*myimage.mask*\'*), the parameter can be set like the following:

```
mask='mask(myimage.mask)'
```

The default boolean masks inside images will also be respected with the above syntax.

But remember that an image can have multiple Boolean masks, so to use the mask2 in an image, set the parameter as: 

```
mask='mask(myimage.mask:mask2)'
```

using the syntax where the mask is specified after the colon. To see what masks are present in your image, use the **makemask** task.

An [LEL string](#lattice-expressions--expr-) can be an on-the-fly (OTF) mask expression or refer to an image pixel mask.

Note that the mask file supplied in the *mask* parameter must have the same shape, same number of axes, and same axes length as the images supplied in the *expr* parameter, with one exception. The mask may be missing some of the axes --- if this is the case then the mask will be expanded along these axes to become the same shape.

The following example uses the mask from file ngc5921.clean.cleanbox.mask :

```
mask='mask(ngc5921.clean.cleanbox.mask)'
```

Here, the mask is calculated to be all pixels with values larger than 0.5:

```
mask='"ngc5921.clean.cleanbox.mask">0.5'
```

Because it is an LEL expression, care must be taken to properly escape characters which LEL views as special. For details, see the [LEL document](imaging.ipynb#lattice-expression-language). As an example, specifying

```
mask = "3clean_mask.im" (WILL FAIL)
```

will cause the image analysis application to fail, because the image name begins with a number. The solution is to escape the name properly, e.g.:

```
mask = "'3clean_mask.im'"
```

 



##### Image Mask Handling (**makemask**) 

**makemask** facilitates the handling of image masks in CASA. As mentioned above, there are two basic mask formats: 1) one or more Boolean masks stored internally in the image file, and 2) images with zero and non-zero image values. **makemask** looks like:

```
#makemask :: Makes and manipulates image masks
mode                =     'list'        #Mask method (list, copy,expand,delete,setdefaultmask)
     inpimage       =         ''        #Name of input image.
```

To distinguish between Boolean internal masks and zero/non-zero images, **makemask** uses the syntax galaxy.image:mask0 for Boolean masks within an image: in this example, the Boolean mask mask0 within the image galaxy.image*.* Without the colon separator, the image itself will be treated as a zero/non-zero mask.*mode=\'list\'* lists all the internal Boolean masks that are present in an image. The default masks can be set with *mode=\'setdefaultmask\'* and they can be deleted with the *mode=\'delete\'*. The default mask is used when an image is displayed in the viewer and is used in all analysis tasks.*mode=\'copy\'* lets a user copy a Boolean mask from one image to another image, or to write out the Boolean mask as a zero/non-zero image. The latter format is very useful when different masks are combined or manipulated. All the image analysis tools, in particular **immath** are applicable for such zero/non-zero masks as they act like normal images. **makemask** will always attempt to regrid the input mask to the output image.In addition *mode=\'copy\'* can be used to merge masks and also accepts regions. E.g. to create a mask from a CASA region file, the input would look like:

```
#makemask :: Makes and manipulates image masks
mode                =     'copy'        #Mask method (list, copy,expand,delete,setdefaultmask)
     inpimage       = 'inputimage.im'   #Name of input image.
     inpmask        = 'region.crtf'    #mask(s) to be processed: image masks,T/F internal masks
                                        #(Need to include parent image names),regions(for copy mode)
     output         = 'imagemask.im'    #Name of output mask (imagename or imagename:internal_maskname)
     overwrite      =      False        #overwrite output if exists?
```

*mode=\'expand\'* furthermore expands a mask in the spectral domain. It regrids first then stretches the edge channels. E.g. a one plane continuum image would be stretched to all planes of a data cube.

 



***





#### Image Analysis Tools 

Using the toolkit for image analysis.



##### Summary

The CASA image analysis module contains an image analysis tool with numerous methods, as well as several higher level tasks. The tasks free users from the burden of resource management, and offer what many consider to be a more user-friendly interface available via the input \<taskname\> CASA command. In many cases, image analysis tasks are really just simple wrappers around analogous tool methods (e.g., the **imcollapse** task is just a relatively simple wrapper around the **ia.collapse**() tool method call), although in some cases, such as with the **imregrid** task, the mapping is not as simple, and much more goes on \"under the hood\" of a task.

 

##### Overview of Image Analysis Tool Functionality

At the heart of the image analysis module is the image analysis tool. An image analysis tool provides access to CASA images. Currently only single-precision, floating-point CASA images are supported by all methods in the image analysis tool and complex-valued images are supported by many, but not all, methods.

The default, global image analysis tool is named **ia**. New, initially-unattached image analysis tools can be created via

```
my_new_ia = iatool()
```

Image analysis tools also provide direct (native) access to FITS and Miriad images, although such access is read-only. These foreign formats to CASA format. For optimum processing speed, it is highly recommended to convert foreign formats to CASA images.

It is important to note that many methods return new image analysis tools that are attached to an image that method has created. Even if one does not intend on using this returned tool, it is important to capture it and run **ia.done**() on it or it will continue to use resources unnecessarily, e.g.

```
new_image_tool = ia.collapse("my_collapsed.im")
#do things with new_image_tool and then run done() on it
new_image_tool.done()
```

###### Tool Manipulation

-   **ia.close**(): Detach tool from image and perform required clean up.
-   **ia.done**(): Detach tool from image and perform required clean up and optionally removed attached image.
-   **ia.isopen**(): Determines if there is an image attached to the tool.
-   **ia.newimage**(): Create a new image analysis tool using an image.
-   **ia.newimagefromarray**(): Create a new image analysis tool from a numpy array.
-   **ia.newimagefromfile**(): Create a new image analysis tool using an image.
-   **ia.newimagefromfits**(): Create a new image analysis tool using a FITS image.
-   **ia.newimagefromimage**(): Create a new image analysis tool using an image.
-   **ia.newimagefromshape**(): Create a new image analysis tool using an image shape.
-   **ia.open**(): Attach the image analysis tool to the specified image.
-   **ia.type**(): Tool type. Always returns \'image\'.

###### FITS Conversion

There is functionality to interconvert between CASA images and FITS files. There is also native access to FITS files.

-   **ia.fromfits**(): Convert a FITS image file to a CASA image
-   **ia.tofits**(): Convert a CASA image to a FITS file.

###### ImageCreation

There are various ways to create CASA images from various data structures.

-   **ia.fromarray**(): Create a CASA image from a numpy array of pixel values.
-   **ia.fromshape**(): Create a CASA image of a specified shape.
-   **ia.maketestimage**(): Create a test image from a FITS file in the CASA data repository.

###### Image Destruction

-   **ia.remove**(): Delete the attached image from disk.
-   **ia.removefile**(): Delete the specified image from disk.

###### Image Interrogation

Various metadata and pixel data can be interrogated.

-   **ia.beamarea**(): Get the image synthesized beam area.
-   **ia.boundingbox**(): Get the bounding rectangular box which circumscribes the specified region.
-   **ia.brightnessunit**(): Get the image brightness unit.
-   **ia.commonbeam**(): For an image with multiple beams, compute the size of the smallest beam that circumscribes all of the image\'s beams.
-   **ia.getchunk**(): Get pixel or mask values from (a specified rectangular region of) an image.
-   **ia.getregion**(): Get pixel or mask values from a specified region of an image.
-   **ia.haslock**(): Determines if the image has a lock associated with it.
-   **ia.history**(): Get the history information from an image.
-   **ia.miscinfo**(): Retrieve \"miscellaneous\" metadata associated with an image.
-   **ia.name**(): Get the image name.
-   **ia.pixelvalue**(): Get the pixel and mask values at a specified location of an image.
-   **ia.restoringbeam**(): Get information about the synthesized beam(s) of an image.
-   **ia.shape**(): Get image shape.
-   **ia.summary**(): Get various metadata of an image.

###### Manipulation of Image Metadata

-   **ia.lock**(): Acquire a lock on the attached image.
-   **ia.rename**(): Rename the image.
-   **ia.rotatebeam**(): Rotate the synthesized beam(s) of an image through a specified angle.
-   **ia.setbrightnessunit**(): Set image brightness unit.
-   **ia.sethistory**(): Add history records to an image.
-   **ia.setmiscinfo**(): Set image miscellaneous metadata.
-   **ia.setrestoringbeam**(): Set image synthesized beam(s).
-   **ia.unlock**(): Release the image lock.

###### Manipulation of Image Pixel and Pixel Mask Values

-   **ia.calc**(): Replace the pixel values in the attached image with the values determined from the specified LEL expression.
-   **ia.calcmask**(): Compute a pixel mask based on an LEL expression.
-   **ia.insert**(): Insert the pixel values of another image into an image.
-   **ia.maskhandler**(): Manipulate image pixel masks.
-   **ia.modify**(): Modify an image using a model specified by a component list.
-   **ia.putchunk**(): Set pixel values (in a specified rectrangular region) of an image.
-   **ia.putregion**(): Set pixel values in a specified region of an image.
-   **ia.replacemaskedpixels**(): Set masked pixel to a specified value.
-   **ia.set**(): Set pixel or mask values.

###### Operations on Images

Various operations can be performed on images which result in new images.

-   **ia.addnoise**(): Add noise to an image.
-   **ia.boxcar**(): Boxcar smooth an image along a specified axis.
-   **ia.decimate**(): Remove planes of an image.
-   **ia.collapse**(): Collapse image along specified axis, computing aggregate function of pixels along that axis.
-   **ia.convolve**(): Convolve an image with an array or with another image.
-   **ia.continuumsub**(): Subtract continuum emission in a spectral line image.
-   **ia.convolve2d**(): Convolve an image with a two-dimensional kernel.
-   **ia.crop**(): Crop pixels from the edge of an image.
-   **ia.fft**(): Fast Fourier Transform (FFT) the image.
-   **ia.hanning**(): Hanning smooth an image along a specified axis.
-   **ia.imagecalc**(): Create an image from an LEL expression.
-   **ia.imageconcat**(): Concatenate multiple images along a specified axis.
-   **ia.makecomplex**(): Create a complex-valued image from two float-valued images representing the real and imaginary values.
-   **ia.pad**(): Pad the edges of an image with pixels.
-   **ia.pv**(): Create a position-velocity image.
-   **ia.pbcor**(): Construct a primary beam corrected image.
-   **ia.rebin**(): Rebin pixel values by specified factors.
-   **ia.regrid**(): Regrid an image to a specified coordinate system.
-   **ia.rotate**(): Rotate the direction coordinate of an image.
-   **ia.sepconvolve**(): Convolve an image with a separable kernel.
-   **ia.subimage**(): Create an image by specifying a region of an image.
-   **ia.transpose**(): Transpose an image.

###### Image Analysis

-   **ia.convertflux**(): Interconvert between peak intensity and flux density for a specified Gaussian source.
-   **ia.decompose**(): Decompose complex source into individual two dimensional models.
-   **ia.deconvolvecomponentlist**(): Deconvolve a component list from the restoring beam.
-   **ia.findsources**(): Find strong point sources in an image.
-   **ia.fitcomponents**(): Fit two-dimensional models to the direction plane(s) of an image.
-   **ia.fitprofile**(): Fit one-dimensional models along an axis image.
-   **ia.histograms**(): Compute histograms from the pixel values of an image.
-   **ia.maxfit**(): Find maximum value in the direction coordinate and do a simple parabolic fit.
-   **ia.moments**(): Compute moments of an image.
-   **ia.statistics**(): Compute image statistics using various algorithms.
-   **ia.twopointcorrelation**(): compute two point autocorrelation functions from the image

###### Image Coordinates

The coordinate system of an image can be manipulated. Specific coordinate system values can be directly manipulated using the CASA coordinate system tool.

-   **ia.adddegaxes**(): Add degenerate axes to an image\'s coordinate system.
-   **ia.coordmeasures**(): Convert from pixel to world coordinates, and return as a measure.
-   **ia.coordsys**(): Retrieve the image coordinate system as a CASA coordinate system tool.
-   **ia.setcoordsys**(): Replace the image\'s coordinate system with another.
-   **ia.topixel**(): Convert from world to pixel coordinates.
-   **ia.toworld**(): Convert from pixel to world coordinates.

######  Miscellaneous

-   **ia.makearray**(): Create a numpy array of specified shape and value.

 

##### Overview of Image Analysis Tasks

###### FITS Conversion

-   **exportfits**: Convert a CASA image to a FITS image.
-   **importfits**: Convert a FITS image to a CASA image.

###### Interrogation and Manipulation of Image Metadata

-   **imhead**: Summarize, interrogate, and modify image metadata
-   **imhistory**: List and append records to image history.

###### Operations on Images

Various operations can be performed on images which result in new images.

-   **imcollapse**: Collapse image along specified axis, computing aggregate function of pixels along that axis.
-   **imcontsub**: Subtract continuum emission in a spectral line image.
-   **immath**: Perform mathematical operations upon images.
-   **immoments**: Compute image moments.
-   **impbcor**: Construct a primary beam corrected image.
-   **impv**: Create a position-velocity image.
-   **imrebin**: Rebin pixel values by specified factors.
-   **imregrid**: Regrid an image to a specfied coordinate system.
-   **imsmooth**: Perform various two-dimensional convolutions.
-   **imsubimage**: Create an image by specifying a region of an image.
-   **imtrans**: Transpose an image.
-   **specsmooth**: Perform various one-dimensional convolutions.

###### Image Analysis

-   **imfit**: Fit two-dimensional models to the direction plane(s) of an image.
-   **imstat**: Compute image statistics using various algorithms.
-   **imval**: Interrogate pixel values.
-   **rmfit**: Compute rotation measure.
-   **specfit**: Fit one-dimensional models along a specified axis of an image.
-   **specflux**: Report spectral profile and calculate spectral flux over a user-specified region.
-   **spxfit**: Fit spectral index models along a specified axis of an image.

 



##### General

A persistent CASA image is stored on disk. Several files and subdirectories containing the image pixel data, mask data, and metadata are stored in a directory. The name of that directory is the name of the image.To access an existing persistent image, use the **ia.open**() method:

```
ia.open("my.im")
```

When you are finished with the image, it is important to close the tool so it no longer uses system resources:

```
ia.close()
```

It is also possible to create temporary images, which, if small enough, are stored completely in memory and destroyed when the user is finished with them. Creating such images is usually accomplished by running one of the image creation methods, and leaving the name of the output image blank (this is usually the default). So, for example, to create an image of a specified shape, one might run:

```
ia.fromshape(shape=[20,20,20])
```

As with persistent images, it is important to close the image analysis tool when finished with temporary images. In this case, the temporary image will be destroyed.

Persistent images can, in principle, be stored in a variety of ways. For example, the image could be stored row by row; this is the way that most older generation packages store images. It makes for very fast row by row access, but very slow in other directions (e.g. extract all the profiles along the third axis of an image). A CASA image is stored with what is called tiling. This means that small multi-dimensional chunks (a tile) are stored sequentially. It means that row by row access is a little slower, but access speed is essentially the same in all directions.

Here are some simple examples using image tools.

```
#access the CASA "test" FITS image and write it to a CASA image named "zz"
ia.maketestimage('zz',overwrite=true)
```

```
#print a summary to the logger and capture the summary metadata in variable "summary"
summary = ia.summary()
```

```
#evaluate image statistics and save the stats info to a variable called "stats"
stats = ia.statistics()
```

```
#create a rectangular region using the rg tool
box = rg.box([10,10], [50,50])
```

```
#create a subimage of that region, and name the resulting image "zz2"
#capture the new image tool attached to "zz2" in the variable "im2"
im2 = ia.subimage('zz2', box, overwrite=true)
```

```
#get statistics for zz2 and store the results in the variable "stats2"
stats2 = im2.statistics()
```

```
print "CLEANING UP OLD zz2.amp/zz2.phase IF THEY EXIST. IGNORE WARNINGS!"
ia.removefile('zz2.amp')
ia.removefile('zz2.phase')
#FFT subimage and store amp and phase
im2.fft(amp='zz2.amp',phase='zz2.phase')
```

```
#close image tools
im2.close()
ia.close()
```

 



##### Foreign Images

The image analysis tool also provides native, read-only access to some foreign image formats. Presently, these are FITS (Float, Double, Short and Long pixel values are supported) and Miriad. This means that you don\'t have to convert the file to native CASA format in order to access the image. For example:

```
#Assumes environment variable is set
pathname = os.environ.get("CASAPATH")
pathname = pathname.split()[0]
datapath1 = pathname + "/data/demo/Images/imagetestimage.fits"
#Access FITS image
ia.open(datapath1)
ia.close()
#Access Miriad image
ia.open('im.mir')
ia.close()
#create a new image tool attached to the FITS image
ims = ia.newimagefromimage(infile=datapath1)
#create a region record representing the inner quarter of an image
innerquarter=rg.box([0.25,0.25],[0.75,0.75],frac=true)
#create a subimage of the inner quarter of the FITS image
subim = ims.subimage(region=innerquarter)
#done with the tools, release resources
ia.close()
ims.close()
```

In general, any parameter to a task or a tool method which accepts an image name will support CASA, FITS, or Miriad images.

There are some performance penalties of which you should be aware. First, because CASA images are tiled (see above), performance is the same regardless of how the images are accessed. In contrast, FITS and Miriad images are not tiled. This means that the performance when accessing these types of images will be poorer for certain operations. e.g., extracting a profile along the third axis of an image. Second, for FITS images, masked values are indicated via a \"magic value\'\'. This means that the mask is worked out on the fly every time the image is accessed.

If you find performance is poor or if you want a writable image, then use appropriate tool methods to convert the foreign format image to a CASA image.

 



##### Virtual Images

It is possible to have an image analysis tool that is not associated with a single persistent image; these are called \"virtual\'\' images. For example, with **ia.imagecalc**(), one can create an expression which may contain many images. You can write the result of the expression to a persistent image, but if you wish, you can also just maintain the expression, evaluating it each time it is needed - nothing is ever written out to disk in this case. There are other image methods like this (the documentation for each one explains what it does). The rules are:

-   If you specify the *outfile* or equivalent parameter, then the output image is always persistent with the name specified.
-   If you leave the *outfile* or equivalent parameter unset, then if possible, a virtual image will be created. Sometimes this virtual image will be an expression as in the example above (i.e. it references other images) or a temporary image in memory, or a temporary image on disk. The **ia.summary**() method will list the type of image. When you **ia.close**() that image tool, the virtual image will be destroyed.
-   If you leave the *outfile* or equivalent parameter unset, and the called method cannot create a virtual image, it will create a persistent image with a name of its choice (sometimes input plus function name).
-   A virtual image can always be written to disk as a persistent image with the **ia.subimage**() method.

 



##### Coordinate Systems

An image contains a coordinate system. A coordinate system tool is used to manipulate a coordinate system. An image tool allows you to recover the coordinate system into a coordinate system tool via the **ia.coordsys**() method. You can set a new image coordinate system with the **ia.setcoordsys**() method.

You can do some basic world to pixel and vice versa coordinate transformations via the image tool **ia.topixel**(), **ia.toworld**(), and **ia.coordmeasures**() methods.

 



##### Lattice Expression Language (LEL)

LEL allows you to create mathematical expressions involving images. For example, add the corresponding pixel values of two images, or multiply the miniumum value of one image by the square root of the pixel values of another image. The LEL syntax is quite rich and is described in detail on the [Lattice Expression Language](imaging.ipynb#lattice-expression-language-lel) pages.

<div class="alert alert-info">
**IMPORTANT NOTE**: Image names which contain "special" characters (eg, "+", "-", etc) must be properly escaped. See the *Lattice names* subsection of the *Expressions* section in the aforementioned document for details.
</div>

To produce an image that is the result of an LEL computation, use the **ia.calc**() or **ia.imagecalc**() image analysis tool methods. Here are some examples.

In this example the image analysis tool is attached to the persistent image named \"zz\". This image\'s name is used in an LEL expression which adds the pixel values of that image to the sine of the pixel values of that image (for trigonometric LEL functions, pixel values are taken to be in radians). Note that the **ia.calc**() method overwrites the pixel values of the attached image with the values computed by the LEL expression. To create a new image without overwriting the pixel values of the image associated with the image tool, use the **ia.imagecalc**() method.

```
ia.maketestimage('zz', overwrite=true)
#Make the minimum value zero
ia.calc('zz + min(zz)')
ia.close()
```

This example demonstrates ways of dealing with image names which have special characters.

```
ia.maketestimage("test-im", overwrite=true)
#escape special characters using a ""
im1 = ia.imagecalc(pixels='test-im + 5')
#or surround the entire image name with quotes
im2 = ia.imagecalc(pixels='"test-im" + 5')
#or
im3 = ia.imagecalc(pixels="'test-im' + 5")
im1.close()
im2.close()
im3.close()
ia.close()
```

 



##### Region Selection

A region designates a subset of pixels in the image in which one is interested. The region is selected based on coordinate information. Such a selection complements on-the-fly masks in which pixels are selected based on a mathematical expression which is tested against their values (see below). Regions may be specified in several ways. The region manager tool (default **rg**) has several methods for generating regions. These methods generally return a dictionary representation of a region which can be used as input for the *region* parameter in various image analysis tool methods and tasks. A region can also be specified by the *box*/*chans*/*stokes* selection parameters in tasks and tool methods which accept them. Regions can also be specified in a special format known as CASA region text format. This format allows for specifying of various region shapes and spectral and polarization extents. This specification can be placed in a file, and in this case, the *region* parameter can be set to the name of that file and the region information will be extracted. Alternatively, the *region* parameter can be set directly to the CRTF specification. The complete CRTF specification can be found in the \"[Region File Format](imaging.ipynb#region-file-format)\" section.

 



##### Pixel Masks

A pixel mask is a set of boolean values which have a one-to-one correspondence with image pixels. A value of True indicates that pixel is \"good\" (i.e., should be used in computations), while a value of False indicates that pixel is \"bad\". For example, blanked pixels in a FITS image are treated as \"bad\" by CASA. When such a file is imported into a CASA image, a pixel mask is created to reflect the badness of blanked pixels in the FITS image. For persistent CASA images, pixel masks are stored in the same directory in which other image information is stored.

If an image does not have a pixel mask associated with it, all of its pixels are treated as good by CASA.

A CASA image may contain any number of pixel masks and these masks can be managed via the **ia.maskhandler**() image analysis tool method. If an image contains multiple pixel masks, only a maximum of one mask will be used during a run of a task or tool method. This pixel mask is known as the \"default\" pixel mask. The default pixel mask can be set by running **ia.maskhandler**(*set=\"pixelmaskname\"*). You can also indicate that none of the image pixel masks should be applied by running **ia.maskhandler**(*set=\"\"*). In this case, all pixels are considered to be good. Pixel masks can also be viewed in the output of the **ia.summary**() image analysis tool method and **imhead** task output.

The **ia.putregion**() image analysis tool method run with *usemask=True* can be used to change the values of the default pixel mask. The image analysis tool method **ia.set**() can also be used to set the values of the default pixel mask. The image analysis tool method **ia.calcmask**() can be used to create a new pixel mask based on a boolean LEL expression.

 



##### On The Fly Pixel Masks

Most image analysis tool methods and tasks accept a parameter named *mask*, which represents an OTF (on-the-fly) pixel mask that is computed for use by only that tool method or task (the exception being the **ia.calcmask**() image analysis tool method in which case a persistent pixel mask is attached to the image; see previous section). This parameter may be specified in one of two ways:

1.  As an LEL boolean expression, or
2.  as a single image name, in which case, pixel values \>= 0.5 are treated as True (good) values, and all others are treated as False.

If the image has a default pixel mask, the mask used in the computation is the logical AND of the OTF pixel mask and default pixel mask. For example:

```
ia.maketestimage('zz', overwrite=true)
#create default pixel mask for which only positive valued pixels are good
ia.calcmask("zz>0")
#compute statistics by specifying an OTF mask, which gets ANDed with
#the default pixel mask, effectively making only pixels with values between 0 and 1 "good"
#for the statistics computation
stats = ia.statistics(mask="zz < 1")
ia.close()
```

The mask expression must in general conform in shape and coordinates with the input image.

A useful LEL function to use in conjunction with the *mask* parameter is **indexin**(). This enables the user to specify a mask based upon selected pixel coordinates or indices rather than image values. For example:

```
ia.fromshape(shape=[20])
#only pixels in the specified planes along the specified axis are considered good.
#prints [False False False False True True True True True True False False False False True False False False True True]
print ia.getregion(mask='indexin(0, [4:9, 14, 18:19])',getmask=true)
ia.close()
```

 



##### Regions As Pixel Masks

Regions, which have previously been discussed, are just another form of an OTF pixel mask, and in fact, if one specifies the *region* and *mask* parameters simultaneously, and the associated image also has a default pixel mask, all these three types of pixel masks are just ANDed together to form the pixel mask that is used in the resulting computation. One can even convert a region specification into a persistent pixel mask by specifying the *region* parameter in e.g., the **ia.fromimage**() image analysis tool method. The created image will have a default pixel mask that is a representation of the region specified (if the initial image had a default pixel mask, then that will be ANDed with the region specification to form the default pixel mask of the resulting image).

 



***





#### Lattice Expression Language 



***





##### Lattice Expression Language 

Introduction to LEL

##### Lattice Expression Language

###### Introduction 

The Lattice Expression Language (LEL) makes it possible to do arithmetic on lattices (in particular on images \[which are just lattices plus coordinates\]). An expression can be seen as a lattice (or image) in itself. It can be used in any operation where a normal image is used.

To summarize, the following functionality is supported:

-   The common mathematical, comparison, and relational [operators](imaging.ipynb#lel-expressions).
-   An extensive list of mathematical and logical [functions](imaging.ipynb#lel-expressions).
-   Mixed [data type](imaging.ipynb#lel-expressions) arithmetic and automatic data type promotion.
-   Support of image [masks](imaging.ipynb#lel-masks).
-   [Masking](imaging.ipynb#lel-masks) using boolean expressions.
-   Handling of [masks](imaging.ipynb#lel-masks) in an expression.
-   Support of image [regions](imaging.ipynb#lel-regions).
-   Interface from both Python and C++.

The first section explains the syntax. The last sections show the interface to LEL using Python or C++. At the end some [examples](imaging.ipynb#lel-examples) are given. If you like, you can go straight to the examples and hopefully immediately be able to do some basic things.

LEL operates on lattices, which are a generalization of arrays. As said above a particular type of lattice is an image; they will be used most often. Because lattices can be very large and usually reside on disk, an expression is only evaluated when a chunk of its data is requested. This is similar to reading only the requested chunk of data from a disk file.

LEL is quite efficient and can therefore be used well in C++ and Python code. Note however, that it can never be as efficient as carefully constructed C++ code.



***





##### LEL Expressions 

LEL Expressions in detail

\--CASA Developer\--

A LEL expression can be as simple or as complex as one likes using the standard arithmetic, comparison, and logical operators. Parentheses can be used to group subexpressions. The operands in an expression can be lattices, constants, functions, and condition masks. 

> $lat1 + 10$
>
> $lat1 + 2 * max(lat2,1)$
>
> $amp(lat1, lat2)$
>
> $lat1 + mean(img[region1])$
>
> $lat1 + mean(lat2[lat2>5 \unicode{x20}\unicode{x20}\unicode{x26}\unicode{x26}\unicode{x20}\unicode{x20} lat2<10])$

The last example shows how a boolean expression can be used to form a mask on a lattice. Only the pixels fulfilling the boolean condition will be used when calculating the mean. In general the result of a LEL expression is a lattice, but it can be a scalar too. If is is a scalar, it will be handled correctly by C++ and Python functions using it as the source in, say, an assignment to another lattice. LEL fully supports masks. In most cases the mask of a subexpression is formed by AND-ing the masks of its operands. It is fully explained in a [later section](imaging.ipynb#lel-masks).

LEL supports the following data types:

> Bool
>
> Float - single precision real (which includes integers)
>
> Double - double precision real
>
> Complex - single precision complex
>
> DComplex - double precision complex

All these data types can be used for scalars and lattices.LEL will do automatic data type promotion when needed. E.g. when a Double and a Complex are used in an operation, they will be promoted to DComplex. It is also possible to promote explicitly using the conversion functions (FLOAT, DOUBLE, COMPLEX and DCOMPLEX). These functions can also be used to demote a data type (e.g. convert from Double to Float), which can sometimes be useful for better performance.

Region is a specific data type. It indicates a region of any type (in pixel or world coordinates, relative, fractional). A region can only be applied to a lattice subexpression using operator `[]`.

##### Constants

Scalar constants of the various data types can be formed as follows (which is similar to Python):

-   A Bool constant can be given as True or False.

```{=html}
<!-- -->
```
-   A Float constant can be any integer or floating-point number. For example:

> $3$
>
> $3.14$
>
> $3.14e-2$

-   A Double constant is a floating-point number using a D for the exponent. One can also use the `DOUBLE` function. For example:

> $1d2$
>
> $3.14d-2$
>
> $double(2)$

-   The imaginary part of a Complex or DComplex constant is formed by a Float or Double constant immediately followed by a lowercase i. A full complex constant is formed by adding another constant as the real part. For example:

> $1.5 + 2i$
>
> $2i+1.5$ is identical

Note that a full complex constant has to be enclosed in parentheses when, say, a multiplication is performed on it. For example:

> $2 * (1.5+2i)$

The functions `pi()` and `e()` should be used to specify the constants pi and e. Note that they form a Double constant, so when using for example pi with a Float lattice, it could make a lot of sense to convert pi to a Float. Otherwise the lattice is converted to a Double, which is time-consuming. However, one may have very valid reasons to convert to Double, e.g. to ensure that the calculations are accurate enough.

##### Operators

The following operators can be used (with their normal meaning and precedence):

> Unary + and -

> Can not be used with Bool operands.

> Unary !

> Logical NOT operator. Can only be used with Bool operands.

> For a region it forms the complement.

> Binary \^, \*, /, %, +, and -

> \% is the modulo operator. E.g. `3%1.4` results in `0.2` and `-10%3` results in `-1`.

> \^ is the power operator.

> All operators are left-associative, except \^ which is right-associative; thus `2`\^`1`\^`2` results in `2`.

> Operator % can only be used for real operands, while the others can be used for real and complex operands.

> Operator - can also be used for regions. It forms the difference of the left and right operand.

> ==, ! =, \>, \> =, \<,  and \< =

> For Bool operands only = = and ! = can be used. A Bool operand cannot be compared with a numeric operand. The comparison operators use the norm for complex values.

> && and \| \|  && and \|\|

> Logical AND and OR operator.

> These operators can only be used with Bool operands. When used on a region && forms the intersection, while \| \| forms the union.

> The precedence order is:

> \^

> unary `+, -, !``*, /, %`

> `+, -` 

> `= = ,! = , > , > = , < , < =`

> `&&`

> `| |`

Note that \^ has a higher precedence than the unary operators.`For example, -3`\^`2` results in `-9`.

The operands of these operators can be 2 scalars, 2 lattices, or a lattice and a scalar. When 2 lattices are used, they should in principle conform; i.e. they should have the same shape and coordinates. However, LEL will try if it can extend one lattice to make it conformant with the other. It can do that if both lattices have coordinates and if one lattice is a true subset of the other (thus if one lattice has all the coordinate axes of the other lattice and if those axes have the same length or have length 1). If so, LEL will add missing axes and/or stretch axes with length 1.

##### Functions

In the following tables the function names are shown in uppercase, while the result and argument types are shown in lowercase. Note, however, that function names are case-insensitive. All functions can have scalar and/or lattice arguments.When a function can have multiple arguments (e.g. atan2), the operands are automatically promoted where needed.

###### Mathematical functions

Several functions can operate on real or complex arguments. The data types of such arguments are given as \'numeric\'.

> `Double PI()`

Returns the value of pi.

> `Double E()`

Returns the value of e.

> `numeric SIN(numeric)`

> `numeric SINH(numeric)`

> `real ASIN(real)`

> `numeric COS(numeric)`

> `numeric COSH(numeric)`

> `real ACOS(real)`

> `real TAN(real)`

> `real TANH(real)`

> `real ATAN(real)`

> `real ATAN2(real y, real x)`

Returns `ATAN(y/x)` in correct quadrant.

> `numeric EXP(numeric)`

> `numeric LOG(numeric)`

Natural logarithm.

> `numeric LOG10(numeric)`

> `numeric POW(numeric, numeric)`

The same as operator \^.

> `numeric SQRT(numeric)`

> `complex COMPLEX(real, real)`

Create a complex number from two reals.

> `complex CONJ(complex)`

> `real REAL(numeric)`

Real value itself or real part of a complex number.

> `real IMAG(complex)`

Imaginary part of a complex number.

> `real NORM(numeric)`

> `real ABS(numeric), real AMPLITUDE(numeric)`

`B`oth find the amplitude of a complex number. If the numeric argument is real, imaginary part zero is assumed.

> `real ARG(complex), real PHASE(complex)`

`B`oth find the phase of a complex number.

> `numeric MIN(numeric, numeric)`

> `numeric MAX(numeric, numeric)`

> `Float SIGN(real)`

Returns -1 for a negative value, 0 for zero, 1 for a positive value.

> `real ROUND(real)`

Rounds the absolute value of the number. E.g. `ROUND(-1.6) = -2`.

> `real FLOOR(real)`

Works towards negative infinity. E.g. `FLOOR(-1.2) = -2`

> `real CEIL(real)`

Works towards positive infinity.

> `real FMOD(real, real)`

The same as operator %.

Note that the trigonometric functions need their arguments in radians.

###### Scalar result functions

The result of these functions is a scalar.

> `double `$NELEMENTS(anytype)$

> Return number of elements in a lattice (1 for a scalar).

> `double `$NDIM(anytype)$

> Return dimensionality of a lattice (0 for a scalar).

> `double `$LENGTH(anytype, real axis)$

> Return length of a lattice axis (returns 1 for a scalar or if axis exceeds number of axes). Axis number is 1-relative.

> `Bool `$ANY(Bool)$

> Is any element true?

> `Bool `$ALL(Bool)$

> Are all elements true?

> `Double `$NTRUE(Bool)$

> Number of true elements.

> `Double `$NFALSE(Bool)$

> Number of false elements.

> `numeric `$SUM(numeric)$

> Return sum of all elements.

> `numeric `$MIN(numeric)$

> Return minimum of all elements.

> `numeric `$MAX(numeric)$
>
> Return maximum of all elements.

> `real `$MEDIAN(real)$
>
> Return median of a lattice. For smallish lattices (max. 512\*512 elements) the median can be found in 1 pass. Other lattices usually require 2 passes.

> `real `$FRACTILE(real,float)$

> Return the fractile of a lattice at the fraction given by the second argument. A fraction of 0.5 is the same as the median. The fraction has to be between 0 and 1. For smallish lattices (max. 512\*512 elements) the fractile can be found in 1 pass. Other lattices usually require 2 passes.

> `real `$FRACTILERANGE(real,float,float)$

> Return the range between the fractiles at the fraction given by the second and third argument. The fractions have to be between 0 and 1 and the second fraction has to be greater than the first one. The second fraction is optional and defaults to `1-fraction1`. Thus:

> $FRACTILERANGE(lat, 0.1)$

> ``$FRACTILERANGE(lat, 0.1, 0.9)$

> ``$FRACTILE(lat,0.9) - FRACTILE(lat,0.1)$

> are equal, be it that the last one is about twice as slow. For smallish lattices (max. 512\*512 elements) the fractile range can be found in 1 pass. Other lattices usually require 2 passes.

> `numeric MEAN(numeric)`

> Return mean of all elements.

> `numeric VARIANCE(numeric)`

> Return variance. 

> (`sum((a(i) - mean(a))**2) / (nelements(a) - 1)`). All calculations are done in double precision.

> `numeric STDDEV(numeric)`

> Return standard deviation (the square root of the variance).

> `real AVDEV(numeric)`

> Return average deviation.

> (`sum(abs(a(i) - mean(a))) / nelements(a)`). All calculations are done in double precision. 

###### Miscellaneous functions

> `numeric `$REBIN(numeric,[f1,f2,...])$

> Rebins the image using the given (integer) factors. It averages the pixels in each bin with shape \[f1,f2,\...\]. Masked-off pixels are not taken into account. If all pixels in a bin are masked off, the resulting pixel will be masked off. The length of the factor list \[f1,f2,\...\] has to match the dimensionality of the image. The factors do not need to divide the axes lengths evenly. Each factor can be a literal value, but it can also be any expression resulting in a real scalar value. For instance, for a 3-dimensional image:
>
> $rebin(lat,[2,2,1])$
>
> will halve the size of axis 1 and 2.

> `real `$AMP(real,real)$

> It returns the square root of the quadrature sum of the two arguments. Thus:

> $amp(lat1,lat2)$

> gives $\sqrt{{lat}_1^2 + {lat}_2^2}$

> This can be used to form, for example, (biased) polarized intensity images when lat1 and lat2 are Stokes Q and U images.

> `real `$PA(real,real)$

> It returns a \`\`position angle\'\' (in degrees) from the two lattices. That is,

> $pa(lat1,lat2)$

> gives $180/\pi*atan2(lat1, lat2)/2$

> This can be used to form, for example, linear polarization position angle images when lat1 and lat2 are Stokes Q and U images, respectively.

> `real `$SPECTRALINDEX(real,real)$

> It returns a the spectral index made from the two lattices. That is,

> $log(s1/s2) / log(f1/f2)$

> where s1 and s2 are the source fluxes in the lattices and f1 and f2 are the frequencies of the spectral axes of both lattices. Similar to e.g. operator + the lattices do not need to have the same shape. One can be extended/stretched as needed.

> `anytype `$VALUE(anytype)$

> It returns the argument without its possible mask, thus it removes the mask from the argument. The section about [mask handling](imaging.ipynb#lel-masks) discusses it in more detail.

> `Bool `$MASK(anytype)$

> It returns the mask of the argument. The section about [mask handling](imaging.ipynb#lel-masks) discusses it in more detail.

> `Bool `$ISNAN(anytype)$

> It tests lattice elements on a NaN value and sets the corresponding output element to T if so; otherwise to F.

> `anytype REPLACE(anytype, anytype)`

> The first argument has to be a lattice (expression). The optional second argument can be a scalar or a lattice (expression). It defaults to 0. The result of the function is a copy of the first argument, where each masked-off element in the first argument is replaced by the corresponding element in the second argument. The result\'s mask is a copy of the mask of the first argument.
>
> $replace (lat1, 0)$
>
> $replace (lat1, lat2)$
>
> The first example replaces each masked-off element in `lat1` by 0. The second example replaces it by the corresponding element in `lat2`. A possible mask of `lat2` is not used.

> ``$anytype IIF(Bool, anytype, anytype)$

> The first argument is a boolean expression. If an element in it is true, the corresponding element from the second argument is taken, otherwise from the third argument. It is similar to the ternary `?:` construct in C++. E.g.

> $iif (lat1>0, lat1, 0)$ same as $max(lat1,0)$

> $iif (sum(lat1)>0, lat1, lat2)$

> The examples shows that scalars and lattices can be freely mixed. When all arguments are scalars, the result is a scalar. Otherwise the result is a lattice. Note that the mask of the result is formed by combining the mask of the arguments in an appropriate way as explained in the section about [mask handling](imaging.ipynb#lel-masks).

> ``$Bool INDEXIN(real axis, set indices)$

> The first argument is a 1-relative axis number. The second argument is a set of indices. It returns a Bool array telling for each lattice element if the index of the given axis is contained in the set of indices.

> The 1-relative indices have to be given as elements with integer values enclosed in square brackets and separated by commas. Each element can be a single index, an index range as `start:end`, or a strided index range as `start:end:stride`. The elements do not need to be ordered, but in a range start must be \< = end. For example:

> $image[indexin(2, [3,4:8,10:20:2])]$

> masks `image` such that only the pixels with an index 3, 4, 5, 6, 7, 8, 10, 12, 14, 16, 18, or 20 on the second axis are set to True.

> The following special syntax exists for this function.

> $INDEXi IN set$

> where `i` is the axis number. So the example above can also be written as:

> $image[index2 in [3,4:8,10:20:2]]$

> Negated versions of this function exist as:

> $INDEXNOTIN(axis, set)$

> $INDEXi NOT IN set$

###### Conversion functions

> ``$Float FLOAT(real)$

> Convert to single precision.

> ``$Double DOUBLE(real)$

> Convert to double precision.

> ``$Complex COMPLEX(numeric)$

> Convert to single precision complex. If the argument is real, the imaginary part is set to 0.

> ``$DComplex DCOMPLEX(numeric)$

> Convert to double precision complex. If the argument is real, the imaginary part is set to 0.

> ``$Bool BOOLEAN(region)$

> Convert to boolean. This can be useful to convert a region to a boolean lattice. Only a region in pixel coordinates can be converted, so in practice only an image mask can be converted.

Note that, where necessary, up-conversions are done automatically. Usually it may only be needed to do a down-conversion (e.g. Double to Float).

###### Lattice names

When a lattice (e.g. an image) is used in an expression, its name has to be given. The name can be given directly if it consists of the characters `-.$~ `and alphanumeric characters.

If the name contains other characters or if it is a reserved word (currently only T and F are reserved), it has to be escaped. Escaping can be done by preceeding the special characters with a backslash or by enclosing the string in single or double quotes. E.g.

      ~/myimage.data
      ~/myimage.data\-old
      '~/myimage.data-old'

 



***





##### LEL Masks 

Image Masks, Lattice Condition Masks, and Mask Handling.

\--CASA Developer\--

###### Access to Image Masks

A boolean mask associated with an image indicates whether a pixel is good (mask value True) or bad (mask value False). If the mask value is bad, then the image pixel is not used for computation (e.g. when finding the mean of the image).

An image can have zero (all pixels are good) or more masks. One mask can be designated as the default mask. By default it will be applied to the image (from Python, designation of the default mask is handled by the **ia.maskhandler** method of the Image tool).

When using LEL, the basic behaviour is that the default mask is used. However, by qualifying the image name with a suffix string, it is possible to specify that no mask or another mask should be used. The suffix is a colon followed by the word `nomask` or the name of the alternative mask.

      myimage.data
      myimage.data:nomask
      'myimage.data:othermask'

 The first example uses the default mask (if the image has one). The second example uses no mask (thus all pixels are designated good) and the third example uses mask `othermask`.

Note that if the image name is enclosed in quotes, the mask name should be enclosed too. It means that a colon cannot be part of an image name.

 It is also possible to use a mask from another image like

      myimage.data:nomask[myotherimage::othermask]

This syntax is explained in the section describing [regions](imaging.ipynb#lel-regions).

###### Lattice Condition Mask

We have seen in the previous section that lattices (in this case images) can have an associated mask. These masks are stored with the image - they are persistent.

It is also possible to create transient masks when a LEL expression is executed (dawn, usually). This is done with the operator `[]` and a boolean expression. For example,

      sum( lat1[lat1<5 && lat1>10] )

creates a mask for `lat1` indicating that only its elements fulfilling the boolean condition should be taken into account in the `sum` function. Note that the mask is local to that part of the expression. So in the expression

      sum( lat1[lat1<5 && lat1>10] ) + sum(lat1)

the second `sum` function takes all elements into account. Masking can also be applied to more complex expressions and it is recursive.

      (lat1+lat2)[lat3<lat4]
      sum( lat1[lat1<5][lat1>10] )
      (lat1 + lat2[lat3<lat4]) [lat1<5]

The first example applies the mask generated by the `[]` operator to the expression `lat1+lat2`. The second example shows the recursion (which ANDs the masks). It is effectively a (slower) implementation of the first example in this subsection. In the last example, the expression inside the parentheses is only evaluated where the condition `[lat1<5]` is true and the resulting expression has a mask associated with it.

Please note that it is possible to select pixels on an axis by means of the function `INDEXIN` (or by the `INDEXi IN` expression) as shown in the previous section about [miscellaneous functions](imaging.ipynb#lel-expressions).

###### Mask Handling

As explained in the previous subsections, lattices can have a mask. Examples are a [mask](imaging.ipynb#lel-masks) of good pixels in an image, a mask created using a [boolean condition](imaging.ipynb#lel-expressions) and the operator `[]`, or a mask defining a [region](imaging.ipynb#lel-regions) within its bounding box.A pixel is bad when the image has a mask and when the mask value for that pixel is False. Functions like `max` ignore the bad pixels.Note that in a MeasurementSet a False mask value indicates a good visibility. Alas this is a historically grown distinction in radio-astronomy.Image masks are combined and propagated throughout an expression. E.g. when two lattices are added, the mask of the result is formed by and-ing the masks of the two lattices. That is, the resultant mask is True where the mask of lattice one is true AND the mask of lattice 2 is True. Otherwise, the resultant mask is False.In general the mask of a subexpression is formed by and-ing the masks of the operands. This is true for e.g. +, \*, `atan2`, etc.. However, there are a few special cases:

-   The mask created by `operator[condition]` is formed by and-ing the condition result, the mask of the result, and the mask of the subexpression where the condition is applied to. For example, suppose `lat1` and `lat2` each have a mask. Then in

          sum( lat1[lat2<5] )

    the `sum` function will only sum those elements for which the mask of `lat1` and `lat2` is valid and for which the condition is true. 

-   The logical AND operator forms the resultant mask by looking at the result and the masks of the operands. 

          lat1[lat1<0 && lat2>0]

    Let us say both `lat1` and `lat2` have masks. The operand `lat1<0` is true if the mask of `lat1` is true and the operand evaluates to true, otherwise it is false. Apply the same rule to the operand `lat2 > 0`. The AND operator gives true if the left and right operands are both true. If the left operand is false, the right operand is no longer relevant. It is, in fact, 3-valued logic with the values true, false, and undefined. 

    Thus, the full expression generates a lattice with a mask. The mask is true when the condition in the `[]` operator is true, and false otherwise. The values of the output lattice are only defined where its mask is true. 

-   The logical OR operator works the same as the AND operator. If an operand has a true value the other operand can be ignored.

-   The mask of the result of the `replace` function is a copy of the mask of its first operand. The mask of the second operand is not used at all.

-   The `iif` function has three operands. Depending on the condition, an element from the second or third operand has to be taken. The resultant mask is formed by the mask of the condition and-ed with the appropriate elements from the masks of the second or third operand.

-   The `value` function returns the value without a mask, thus it removes the mask from a value. It has the same effect as the `image:nomask` construction discussed above. However, the `value` function is more general, because it can also be applied to a subexpression.

-   The `mask` function returns the mask of a value. The returned value is a boolean lattice and has no mask itself. When the value has no mask, it returns a mask consisting of all True values. When applied to an image, it returns its default mask.

Consider the following more or less equivalent examples:

       value(image1)[mask(image2)]
       image1:nomask[mask(image2)]
       image1:nomask[image2::mask0]

The first two use the default mask of `image2` as the mask for `image1`.The latter uses `mask0` of `image2` as the mask for `image1`. It is equivalent to the first two examples if `mask0` is the default mask of `image2`.It is possible that the entire mask of a subexpression is false. For example, if the mean of such a subexpression is taken, the result is undefined. This is fully supported by LEL, because a scalar value also has a mask associated with it. One can see a masked-off scalar as a lattice with an all false mask. Hence an operation involving an undefined scalar results in an undefined scalar. The following functions act as described below on fully masked-off lattices:

-   MEDIAN, MEAN, VARIANCE, STDDEV, AVDEV, MIN, MAX    result in an undefined scalar:
-   NELEMENTS, NTRUE, NFALSE, SUM    result in a scalar with value 0.
-   ANY    results in a scalar with value F.
-   ALL    results in a scalar with value T.
-   LENGTH, NDIM    ignore the mask because only the shape of the lattice matters.

You should also be aware that if you remove a mask from an image, the values of the image that were previously masked bad may have values that are meaningless.

###### Mask Storage

In many of the expressions we have looked at in the examples, a mask has been generated. What happens to this mask and indeed the values of the expression depends upon the implementation. If for example, the function you are invoking with LEL writes out the result, then both the mask and result will be stored. On the other hand, it is possible to just use LEL expressions but never write out the results to disk. In this case, no data or mask is written to disk. You can read more about this in the [interface](imaging.ipynb#lel-interface) section.



***





##### LEL Regions 

Regions of interest

\--CASA Developer\--

A region-of-interest generally specifies a portion of a lattice which you are interested in for some astronomical purpose (e.g. what is the flux density of this source). Quite a rich variety of regions are supported in CASA. There are simple regions like a box or a polygon, and compound regions like unions and intersections. Regions may contain their own \`\`region masks\'\'. For example, with a 2-d polygon, the region is defined by the vertices, the bounding box and a mask which says whether a pixel inside the bounding box is inside of the polygon or outside of the polygon.

In addition, although masks and regions are used somewhat differently by the user, a mask is really a special kind of region; they are implemented with the same underlying code.

Like masks, regions can be persistently stored in image. Within Python, regions can be created using the various methods of the rg tool. Regions can also be defined in plain text files (see [Region File Format](imaging.ipynb#region-file-format)).

We saw in the previous section how the condition operator `[]` could be used to generate masks with logical expressions. This operator has a further talent. A region of any type can be applied to a lattice with the `[]` operator. You can think of the region as also effectively being a logical expression. The only difference with what we have seen before is that it results in a lattice with the shape of the region\'s bounding box. If the lattice or the region (as in the polygon above) has a mask, they are and-ed to form the result\'s mask.

All types of regions supported in CASA can be used, thus:

-   regions in pixel or world coordinates
-   in absolute, relative and/or fractional units
-   basic regions box, ellipsoid, and polygon
-   compound regions union, intersection, difference, and complement.
-   extension of a region or group of regions to higher dimensions
-   masks

The documentation in the classes LCRegion, LCSlicer, and WCRegion gives you more information about the various regions.

At this moment a region can not be defined in LEL itself. It is only possible to use regions predefined in an image or another table.

A predefined region can be used by specifying its name. There are three ways to specify a region name:

1.  `tablename::regionname` The region is looked up in the given table (which will usually be an image) in which it is stored.
2.  `::regionname` The region is looked up in the last table used in the expression.
3.  `regionname` Is usually equivalent to above. However, there is no syntactical difference between the name of a region and a lattice/image. Therefore LEL will first try if the name represents a lattice or image. If not, the name is supposed to be a region name. The prefix `::` in the previous way tells that the name should only be looked up as a region. 

Examples are

      myimage.data[reg1]
      (myimage.data - otherimage)[::reg1]
      (myimage.data - otherimage)[myimage.data::reg1]
      myimage.data:nomask[myotherimage::othermask]

In the first example region `reg1` is looked up in image `myimage.data`. It is assumed that `reg1` is not the name of an image or lattice. It results in a lattice whose shape is the shape of the bounding box of the region. The mask of the result is the and of the region mask and the lattice mask.

In the second example it is stated explicitly that `reg1` is a region by using the :: syntax. The region is looked up in `otherimage`, because that is the last table used in the expression. The result is a lattice with the shape of the bounding box of the region.

In the third example the region is looked up in `myimage.data`. Note that the this and the previous example also show that a region can be applied to a subexpression.

In the fourth example we have been very cunning. We have taken advantage of the fact that masks are special sorts of regions. We have told the image `myimage.data` not to apply any of its own masks. We have then used the `[]` operator to generate a mask from the mask stored in a different image, `myotherimage`. This effectively applies the mask from one image to another. Apart from copying the mask, this is the only way to do this.

Unions, intersections, differences and complements of regions can be generated and stored (in C++ and Python). However, it is also possible to form a union, etc. in LEL itself. However, that can only be done if the regions have the same type (i.e. both in world or in pixel coordinates).The following operators can be used:

-   `reg1 || reg2` to form the union.
-   `reg1 && reg2` to form the intersection.
-   `reg1 - reg2` to form the difference.
-   `!reg1` to form the complement.

The normal CASA rules are used when a region is applied:

-   A region in world or relative coordinates can only be applied to an image (or a subexpression resulting in an image). Otherwise there is no way to convert it to absolute pixel coordinates.
-   The axes of a region in world coordinates have to be axes in the image (subexpression). However, the region can have fewer axes.
-   If a region has fewer axes than the image or lattice the region is automatically extended to the full image by taking the full length of the missing axes.



***





##### LEL Optimization 

Performance issues in LEL.

\--CASA Developer\--

When giving a LEL expression, it is important to keep an eye on performance issues.

###### LEL itself will do some optimization:

-   As said in the introduction a LEL expression is evaluated in chunks. However, a scalar subexpression is executed only once when getting the first chunk. E.g. in

          lat1 + mean(lat2)

    the subexpression `mean(lat2)` is executed only once and not over and over again when the user gets chunks. 

-   Often the exponent 2 is used in the `pow` function (or operator `^`). This is optimized by using multiplication instead of using the system pow function. 

-   When LEL finds a [masked-off scalar](imaging.ipynb#lel-expressions) in a subexpression, it does not evaluate the other operand. Instead it sets the result immediately to a masked-off scalar. Exceptions are the operators AND and OR and function `iif`, because their masks depend on the operand values.

###### The user can optimize by specifying the expression carefully.

-   It is strongly recommended to combine scalars into a subexpression to avoid unnecessary scalar-lattice operations. E.g.

          2 * lat1 * pi()

    should be written as

          lat1 * (2 * pi())
          #or
          2 * pi() * lat1

    because in that way the scalars form a scalar subexpression which is calculated only once. Note that the subexpression parentheses are needed in the first case, because multiplications are done from left to right.    In the future LEL will be optimized to shuffle the operands when possible and needed.

-   It is important to be careful with the automatic data type promotion of single precision lattices. Several scalar functions (e.g. pi) produce a double precision value, so using `pi` with a single precision lattice causes the lattice to be promoted to double precision. If accuracy allows it, it is much better to convert `pi` to single precision. E.g. assume `lat1` and `lat2` are single precision lattices.

          atan2(lat1,lat2) + pi()/2

    The result of `atan2` is single precision, because both operands are single precision. However, `pi` is double precision, so the result of `atan2` is promoted to double precision to make the addition possible. Specifying the expression as:

          atan2(lat1,lat2) + float(pi())/2

    avoids that (expensive) data type promotion. 

-   `POW(LAT,2)` or `LAT``^``2` is faster than `LAT*LAT`, because it accesses lattice `LAT` only once.

-   `SQRT(LAT)` is faster than `LAT``^``0.5` or `POW(LAT,0.5)` 

-   `POW(U,2) + POW(V,2) < 1000``^``2` is considerably faster than    `SQRT(SQUARE(U) + SQUARE(V)) < 1000`, because it avoids the `SQRT` function.

-   LEL can be used with disk-based lattices and/or memory-based lattices. When used with memory-based lattices it is better to make subexpressions the first operand in another subexpression or a function. E.g.    `lat1*lat2 + lat3`    is better than    `lat3 + lat1*lat2`    The reason is that in the first case no copy needs to be made of the lattice data which already reside in memory. All LEL operators and functions try to reference the data of their latter operands instead of making a copy.    In general this optimization does not apply to LEL expression. However, when using the true [C++ interface](imaging.ipynb#lel-interface) to classes like `LatticeExprNode`, one can easily use memory-based lattices. In that case it can be advantageous to pay attention to this optimization.



***





##### LEL Interface 

Python and C++ interfaces to LEL

\--CASA Developer\--

There are two interfaces to LEL. One is from Python and the other from C++. It depends upon your needs which one you access. Most high level users of CASA will access LEL only via the Python interface.

 

###### Simple String Expressions

 The **ia.imagecalc** method evaluates the expression and stores the result and mask in an output image. If you specify the output image name, it is written to a disk file of that name. If you don\'t give it, the output image is never written out; it is evaluated every time an action (like **ia.statistics**) is requested.

    im = ia.imagecalc(outfile='outimage', pixels='inimage1+inimage2');
    im.statistics();

The first command creates an image file `outimage` filling it with the sum of the input images. The second command does statistics on that new image. Writing it as

    im = ia.imagecalc(pixels='inimage1+inimage2');
    im.statistics();

would do the same with the exception of creating the output image. Instead the created image is transient; it only lives as an expression and each time it is used the expression is evaluated.

We can use the method **ia.calc** on an already existing image. Thus

     ia.open('ngc1213');
     ia.calc('ngc1213^2');

would replace the pixels by the square of their value in the opened image.

Sometimes you need to double quote the file names in your expression. For example, if the images reside in a different directory as in this example.

    im = ia.imagecalc ('"dir1/im1" + "/nfs/data/im2"');

###### C++ interface

This consists of 2 parts.

1\. The function `command` in Images/ImageExprParse.h can be used to execute a LEL command. The result is a LatticeExprNode object. This example does the same as the Python one shown above. E.g.

      LatticeExprNode seltab1 = ImageExprParse::command
      ("imagein1 + imagein2");

2\. The other interface is a true C++ interface having the advantage that C++ variables can be used. Class LatticeExprNode contains functions to form an expression. The same operators and functions as in the command interface are available. For example:

      Float clipValue = 10;
      PagedImage<Float> image("imagein");
      LatticeExpr<Float> expr(min(image,clipValue));

forms an expression to clip the image. Note that the expression is written as a normal C++ expression. The overloaded operators and functions in class LatticeExprNode ensure that the expression is formed in the correct way. Note that a `LatticeExprNode` object is usually automatically converted to a templated `LatticeExpr` object, which makes it possible to use it as a normal `Lattice`. So far the expression is only formed, but not evaluated. Evaluation is only done when the expression is used in an operation, e.g. as the source of the copy operation shown below.

      PagedImage<Float> imout("imageout");
      imout.copyData (expr);

 



***





##### LEL Examples 

Examples for LEL in C++ and Glish.

\--CASA Developer\--

The following examples show some LEL expressions (equally valid in C++ or Glish).

Note that LEL is readonly; i.e. it does not change any value in the images given. A function in the `image` client has to be used to do something with the result (e.g. storing in another image).

-   `lat1+lat2` \-- adds 2 lattices
-   `mean(myimage:nomask) -- `results in a scalar value giving the mean of the image. No mask is used for the image, thus all pixels are used. The scalar value can be used as a lattice. E.g. it can be used as the source in the `image` function `replacemaskedpixels` to set all masked-off elements of a lattice to the mean.
-   `complex(lat1,lat2) -- `results in a complex lattice formed by `lat1` as the real part and `lat2` as the imaginary part.
-   `min(lat1, 2*mean(lat1)) -- `results in a lattice where `lat1` is clipped at twice its mean value.
-   `min(myimage, 2*mean(mymage[myregion])) -- `results in an image where `myimage` is clipped at twice the mean value of region `myregion` in the image..
-   `lat1[lat1>2*min(lat1)]` \-- results in a lattice with a mask. Only the pixels greater than twice the minimum are valid.
-   `replace(lat1)`  \-- results in a lattice where each masked-off element in `lat1` is replaced by 0.
-   `iif(lat1<mean(lat1),lat1*2,lat1/2) -- `results in a lattice where the elements less than the mean are doubled and the elements greater or equal to the mean are divided by 2.

Here follows a sample Glish session showing some of the LEL capabilities and how Glish variables can be used in LEL.

    duw01> glish -l image.g
    - a := array(1:50,5,10)              # make some data
    - global im1 := imagefromarray('im1', a);   # fill an image with it
    - im1.shape()
    [5 10]
    - local pixels, mask
    - im1.getregion(pixels, mask);       # get pixels and mask
    - mask[1,1] := F                     # set some mask elements to False
    - mask[3,4] := F
    - im1.putregion(mask=mask);          # put new mask back
    - global reg:=drm.box([1,1],[4,4]);  # a box region
    - im2 := imagecalc(pixels='$im1[$reg]')     # read-only image applying region
    - local pixels2, mask2
    - im2.getregion(pixels2, mask2);     # get the pixels and mask
    - print pixels2
    [[1:4,]
        1 6 11 16
        2 7 12 17
        3 8 13 18
        4 9 14 19] 
    - print mask2
    [[1:4,]
        F T T T
        T T T T
        T T T F
        T T T T] 
    - im1.replacemaskedpixels ('mean(im2)'); # replace masked-off values
    - im1.getregion (pixels2, mask2);        # by mean of masked-on in im2
    - print pixels2
    [[1:5,]
        10.0714283 6  11 16         21 26 31 36 41 46
        2          7  12 17         22 27 32 37 42 47
        3          8  13 10.0714283 23 28 33 38 43 48
        4          9  14 19         24 29 34 39 44 49
        5          10 15 20         25 30 35 40 45 50]



***





### CARTA 

Visualization and analysis with the CARTA

CARTA is a new visualization software available to general users who wish to try visualizing image products outside the CASA Viewer.

CARTA is the *Cube Analysis and Rendering Tool for Astronomy*, a new image visualization and analysis tool designed for the [ALMA](https://www.almaobservatory.org/en/home/), the [VLA](https://science.nrao.edu/facilities/vla), and the [SKA pathfinders](https://www.skatelescope.org/). As the image size increases drastically with modern telescopes in recent years, viewing an image with a local image viewer or with a remote image viewer via the ssh protocol becomes less efficient. The mission of CARTA is to provide usability and scalability for the future by utilizing modern web technologies and computing parallelization.The official CARTA website and download instructions can be found [here](https://cartavis.github.io/).

<div class="alert alert-warning">
**WARNING: C**ARTA currenly only contains a subset of the functionality that is available in the CASA Viewer. Eventually, CARTA is expected to replace the CASA Viewer as the standard software to visualize ALMA and VLA imaging. However, additional functionality needs to be added to CARTA over the coming cycles before it should be regarded as a replacement for the CASA Viewer.
</div>

Several feature where CARTA v.1.3 is already superior to the CASA Viewer are:

-   CARTA shows better performance, especially noticeable when handling large image cubes.
-   CARTA launches in a web browser for remote display options.
-   CARTA can display Stokes wedges.
-   CARTA does not suffer from problems that the CASA Viewer has with properly displaying headers
-   CARTA allows users to flexibly modify and safe the layout.
-   CARTA supports the HDF5 image format.

Subsequent releases of CARTA will continue to enhance CARTA's performance. For a full overview of the current and upcoming features, see the official [CARTA website](https://cartavis.github.io/).

*CARTA is developed by the Academia Sinica Institute of Astronomy and Astrophysics ([ASIAA](https://www.asiaa.sinica.edu.tw/)), the Inter-University Institute for Data Intensive Astronomy ([IDIA](https://www.idia.ac.za/)), the National Radio Astronomy Observatory ([NRAO](https://science.nrao.edu/)), and the Department of Physics, [University of Alberta.](https://www.ualberta.ca/physics/index.html)*

There are currently no items in this folder.